In [1]:
class StateMachineError(Exception): pass
class EventNotFound(StateMachineError, ValueError): pass
class CallbackNotFound(StateMachineError, ValueError): pass

class EventRegistry():
    def __init__(self):
        self.events = {}
    def create_event(self, ev):
        self.events[ev] = []
    def _get_handlers(self, ev):
        handlers = self.events.get(ev)
        if not handlers:
            raise EventNotFound(ev)
        return handlers
    def register_event(self, ev, callback):
        handlers = self._get_handlers(ev)
        handlers.append(callback)
    def unregister_event(self, ev, callback):
        handlers = self._get_handlers(ev)
        try:
            handlers.remove(callback)
        except ValueError:
            raise CallbackNotFound("%r not found for %s" % (ev, callback)) from None
            
    def callback_event(self, ev, *args):
        handlers = self._get_handlers(ev)
        for cb in handlers:
            try:
                cb(*args)
            except TypeError as e:
                print("Callback %r failed: %s" % (cb, str(e)))
    
    

In [36]:
import tkinter as tk
import tkinter.ttk as ttk
from collections import OrderedDict
import json
import time
import copy

class ScrollFrame(tk.Frame):
    def __init__(self, master, height=100, width=100, yscroll=True, xscroll=False):
        super().__init__(master, relief=tk.GROOVE, width=width, height=height)
        self.width = width
        self.height = height
        self.canvas = tk.Canvas(self)
        self.inner_frame = tk.Frame(self.canvas) #, width=width, height=height)
        
        if yscroll:
            self.scrollbar = tk.Scrollbar(self, orient="vertical", command=self.canvas.yview)
            self.canvas.configure(yscrollcommand=self.scrollbar.set)
        else:
            self.scrollbar = None
            
        if xscroll:
            self.xscrollbar = tk.Scrollbar(self, orient="horizontal", command=self.canvas.xview)
            self.canvas.configure(xscrollcommand=self.xscrollbar.set)
        else:
            self.xscrollbar = None
        
        self.window = self.canvas.create_window((0,0), window=self.inner_frame, anchor='nw')
        self.inner_frame.bind("<Configure>", self._on_frame_configure)
        self.canvas.bind("<MouseWheel>", self.on_mouse_scroll)
        self.items = []
        
    def add_item(self, item):
        self.items.append(item)
    
    def create_widget(self, klass, bind_scroll=True, *args, **kw):
        w = klass(self.inner_frame, *args, **kw)
        self.add_item(w)
        if bind_scroll:
            w.bind("<MouseWheel>", self.on_mouse_scroll)
        return w
    
    def _configure_widgets(self):
        
        if self.scrollbar:
            self.scrollbar.pack(side="right", fill=tk.Y)
        if self.xscrollbar:
            self.xscrollbar.pack(side="bottom", fill=tk.X)
        self.canvas.pack(side="left", fill=tk.BOTH)
        #self.canvas.configure(width=self.width-self.scrollbar.winfo_width(), height=self.height-self.xscrollbar.winfo_height())
        #self.inner_frame.pack_configure(fill=tk.X)
        #self.canvas.configure(width=self.width-self.scrollbar.winfo_width())
        #for it in self.items:
            #it.pack(anchor='w', expand=True, fill=tk.X)

    def pack(self, **kw):
        super().pack(**kw)
        self._configure_widgets()
    
    def grid(self, **kw):
        super().grid(**kw)
        self._configure_widgets()            
            
    def _on_frame_configure(self, e):
        self.canvas.configure(scrollregion=self.canvas.bbox("all"), width=self.width, height=self.height)
        
    def on_mouse_scroll(self, e):
        self.canvas.yview_scroll(-int(e.delta/120), "units")


In [10]:
_trenddata_15min_agitation_json = '{\n  "max":[[1482454778130,33.129],[1482454781130,33.632],[1482454784130,34.212],[1482454787130,34.502],[1482454790130,34.661],[1482454793130,34.931],[1482454796130,35.297],[1482454799130,35.299],[1482454802130,35.505],[1482454805130,35.648],[1482454808130,35.635],[1482454811130,35.639],[1482454814130,35.618],[1482454817130,36.019],[1482454820130,36.030],[1482454823130,35.998],[1482454826130,35.996],[1482454829130,36.005],[1482454832130,36.008],[1482454835130,35.998],[1482454838130,35.991],[1482454841130,35.993],[1482454844130,36.005],[1482454847130,36.018],[1482454850130,36.025],[1482454853130,36.008],[1482454856130,35.989],[1482454859130,36.012],[1482454862130,36.023],[1482454865130,36.007],[1482454868130,36.030],[1482454871130,36.020],[1482454874130,36.014],[1482454877130,36.027],[1482454880130,36.016],[1482454883130,35.999],[1482454886130,36.024],[1482454889130,36.042],[1482454892130,36.023],[1482454895130,36.022],[1482454898130,36.008],[1482454901130,36.003],[1482454904130,35.991],[1482454907130,36.021],[1482454910130,36.015],[1482454913130,36.027],[1482454916130,36.029],[1482454919130,36.006],[1482454922130,36.016],[1482454925130,35.997],[1482454928130,36.028],[1482454931130,35.994],[1482454934130,36.002],[1482454937130,36.024],[1482454940130,36.000],[1482454943130,36.030],[1482454946130,36.016],[1482454949130,36.018],[1482454952130,36.036],[1482454955130,36.019],[1482454958130,36.016],[1482454961130,35.995],[1482454964130,36.012],[1482454967130,36.011],[1482454970130,36.024],[1482454973130,36.006],[1482454976130,36.021],[1482454979130,36.026],[1482454982130,36.002],[1482454985130,36.011],[1482454988130,36.032],[1482454991130,36.021],[1482454994130,36.013],[1482454997130,36.015],[1482455000130,36.008],[1482455003130,36.030],[1482455006130,36.012],[1482455009130,36.022],[1482455012130,36.012],[1482455015130,36.016],[1482455018130,36.039],[1482455021130,36.014],[1482455024130,36.023],[1482455027130,36.009],[1482455030130,36.028],[1482455033130,36.030],[1482455036130,35.989],[1482455039130,36.020],[1482455042130,36.010],[1482455045130,36.004],[1482455048130,36.017],[1482455051130,36.005],[1482455054130,35.999],[1482455057130,36.004],[1482455060130,35.997],[1482455063130,36.019],[1482455066130,35.999],[1482455069130,36.008],[1482455072130,36.011],[1482455075130,36.013],[1482455078130,35.994],[1482455081130,36.004],[1482455084130,35.988],[1482455087130,36.018],[1482455090130,36.006],[1482455093130,36.027],[1482455096130,36.024],[1482455099130,36.021],[1482455102130,35.996],[1482455105130,36.017],[1482455108130,36.021],[1482455111130,36.013],[1482455114130,36.004],[1482455117130,36.014],[1482455120130,36.016],[1482455123130,36.020],[1482455126130,36.005],[1482455129130,36.011],[1482455132130,35.994],[1482455135130,36.012],[1482455138130,35.999],[1482455141130,36.003],[1482455144130,36.006],[1482455147130,36.041],[1482455150130,36.023],[1482455153130,36.000],[1482455156130,36.040],[1482455159130,36.019],[1482455162130,36.024],[1482455165130,36.020],[1482455168130,36.026],[1482455171130,36.027],[1482455174130,36.010],[1482455177130,36.037],[1482455180130,36.032],[1482455183130,36.032],[1482455186130,36.025],[1482455189130,36.022],[1482455192130,36.006],[1482455195130,36.002],[1482455198130,36.012],[1482455201130,36.016],[1482455204130,35.997],[1482455207130,36.036],[1482455210130,36.033],[1482455213130,36.015],[1482455216130,36.007],[1482455219130,35.993],[1482455222130,36.025],[1482455225130,36.028],[1482455228130,36.007],[1482455231130,36.015],[1482455234130,36.004],[1482455237130,36.014],[1482455240130,36.030],[1482455243130,36.007],[1482455246130,36.012],[1482455249130,36.006],[1482455252130,36.039],[1482455255130,36.009],[1482455258130,36.048],[1482455261130,36.015],[1482455264130,36.025],[1482455267130,36.023],[1482455270130,36.034],[1482455273130,36.027],[1482455276130,36.013],[1482455279130,36.015],[1482455282130,36.020],[1482455285130,36.003],[1482455288130,36.017],[1482455291130,35.988],[1482455294130,36.005],[1482455297130,36.012],[1482455300130,36.026],[1482455303130,36.020],[1482455306130,36.011],[1482455309130,36.019],[1482455312130,36.007],[1482455315130,36.022],[1482455318130,36.018],[1482455321130,36.014],[1482455324130,36.005],[1482455327130,36.035],[1482455330130,36.021],[1482455333130,36.021],[1482455336130,36.019],[1482455339130,36.012],[1482455342130,36.003],[1482455345130,36.009],[1482455348130,36.040],[1482455351130,36.004],[1482455354130,36.009],[1482455357130,36.027],[1482455360130,36.009],[1482455363130,36.017],[1482455366130,36.022],[1482455369130,36.006],[1482455372130,36.003],[1482455375130,36.019],[1482455378130,36.035],[1482455381130,36.002],[1482455384130,36.004],[1482455387130,36.024],[1482455390130,36.020],[1482455393130,36.004],[1482455396130,36.034],[1482455399130,36.014],[1482455402130,36.004],[1482455405130,36.009],[1482455408130,36.018],[1482455411130,36.007],[1482455414130,36.038],[1482455417130,36.030],[1482455420130,36.007],[1482455423130,36.029],[1482455426130,36.015],[1482455429130,36.019],[1482455432130,36.003],[1482455435130,35.999],[1482455438130,36.000],[1482455441130,36.022],[1482455444130,36.003],[1482455447130,36.001],[1482455450130,36.008],[1482455453130,36.030],[1482455456130,36.032],[1482455459130,36.015],[1482455462130,35.988],[1482455465130,36.027],[1482455468130,36.023],[1482455471130,36.010],[1482455474130,36.016],[1482455477130,36.037],[1482455480130,36.006],[1482455483130,36.022],[1482455486130,36.022],[1482455489130,36.015],[1482455492130,36.023],[1482455495130,36.014],[1482455498130,36.032],[1482455501130,36.035],[1482455504130,36.017],[1482455507130,36.022],[1482455510130,36.025],[1482455513130,36.017],[1482455516130,36.006],[1482455519130,36.022],[1482455522130,35.998],[1482455525130,36.036],[1482455528130,36.023],[1482455531130,36.010],[1482455534130,36.024],[1482455537130,36.018],[1482455540130,36.035],[1482455543130,36.021],[1482455546130,36.023],[1482455549130,36.017],[1482455552130,36.010],[1482455555130,36.032],[1482455558130,36.016],[1482455561130,36.009],[1482455564130,36.032],[1482455567130,36.019],[1482455570130,36.048],[1482455573130,36.024],[1482455576130,36.027],[1482455579130,36.017],[1482455582130,36.013],[1482455585130,36.023],[1482455588130,36.017],[1482455591130,36.016],[1482455594130,36.023],[1482455597130,36.035],[1482455600130,36.021],[1482455603130,36.026],[1482455606130,36.032],[1482455609130,36.018],[1482455612130,36.019],[1482455615130,36.014],[1482455618130,36.023],[1482455621130,36.012],[1482455624130,36.003],[1482455627130,36.021],[1482455630130,36.013],[1482455633130,36.010],[1482455636130,36.031],[1482455639130,36.022],[1482455642130,36.029],[1482455645130,36.002],[1482455648130,36.024],[1482455651130,36.008],[1482455654130,36.022],[1482455657130,36.006],[1482455660130,36.005],[1482455663130,36.029],[1482455666130,36.026],[1482455669130,36.007],[1482455672130,36.025],[1482455675130,36.022]],\n  "last":[[1482454778130,32.828],[1482454781130,33.312],[1482454784130,33.746],[1482454787130,34.267],[1482454790130,34.548],[1482454793130,34.925],[1482454796130,35.006],[1482454799130,35.278],[1482454802130,35.293],[1482454805130,35.648],[1482454808130,35.614],[1482454811130,35.601],[1482454814130,35.584],[1482454817130,35.670],[1482454820130,36.030],[1482454823130,35.981],[1482454826130,35.973],[1482454829130,35.979],[1482454832130,36.008],[1482454835130,35.998],[1482454838130,35.991],[1482454841130,35.965],[1482454844130,35.965],[1482454847130,35.975],[1482454850130,36.025],[1482454853130,36.008],[1482454856130,35.969],[1482454859130,35.976],[1482454862130,35.985],[1482454865130,36.007],[1482454868130,35.998],[1482454871130,35.988],[1482454874130,36.009],[1482454877130,35.998],[1482454880130,35.989],[1482454883130,35.978],[1482454886130,36.003],[1482454889130,36.042],[1482454892130,36.006],[1482454895130,35.991],[1482454898130,35.990],[1482454901130,36.000],[1482454904130,35.972],[1482454907130,35.991],[1482454910130,36.008],[1482454913130,35.989],[1482454916130,36.029],[1482454919130,36.006],[1482454922130,35.997],[1482454925130,35.988],[1482454928130,35.978],[1482454931130,35.978],[1482454934130,36.002],[1482454937130,36.024],[1482454940130,35.985],[1482454943130,35.989],[1482454946130,35.996],[1482454949130,36.018],[1482454952130,36.036],[1482454955130,35.974],[1482454958130,35.973],[1482454961130,35.986],[1482454964130,36.012],[1482454967130,36.007],[1482454970130,36.016],[1482454973130,35.994],[1482454976130,35.974],[1482454979130,36.018],[1482454982130,35.997],[1482454985130,35.989],[1482454988130,35.990],[1482454991130,35.994],[1482454994130,35.992],[1482454997130,36.015],[1482455000130,36.008],[1482455003130,36.006],[1482455006130,35.980],[1482455009130,36.012],[1482455012130,36.012],[1482455015130,36.008],[1482455018130,35.974],[1482455021130,35.999],[1482455024130,35.977],[1482455027130,36.009],[1482455030130,36.019],[1482455033130,36.009],[1482455036130,35.984],[1482455039130,35.992],[1482455042130,36.010],[1482455045130,36.004],[1482455048130,35.978],[1482455051130,35.990],[1482455054130,35.987],[1482455057130,36.004],[1482455060130,35.997],[1482455063130,36.003],[1482455066130,35.996],[1482455069130,35.989],[1482455072130,36.011],[1482455075130,36.013],[1482455078130,35.991],[1482455081130,35.965],[1482455084130,35.968],[1482455087130,36.018],[1482455090130,35.980],[1482455093130,36.027],[1482455096130,35.998],[1482455099130,35.977],[1482455102130,35.982],[1482455105130,36.017],[1482455108130,36.021],[1482455111130,35.983],[1482455114130,35.985],[1482455117130,35.984],[1482455120130,36.016],[1482455123130,36.020],[1482455126130,35.971],[1482455129130,35.985],[1482455132130,35.987],[1482455135130,36.005],[1482455138130,35.999],[1482455141130,35.988],[1482455144130,35.977],[1482455147130,36.011],[1482455150130,35.976],[1482455153130,35.976],[1482455156130,36.008],[1482455159130,36.019],[1482455162130,36.024],[1482455165130,35.971],[1482455168130,35.997],[1482455171130,36.008],[1482455174130,36.004],[1482455177130,36.014],[1482455180130,35.982],[1482455183130,36.013],[1482455186130,35.990],[1482455189130,36.007],[1482455192130,36.000],[1482455195130,35.981],[1482455198130,35.991],[1482455201130,36.011],[1482455204130,35.997],[1482455207130,36.036],[1482455210130,35.986],[1482455213130,36.003],[1482455216130,35.992],[1482455219130,35.988],[1482455222130,36.025],[1482455225130,35.975],[1482455228130,35.988],[1482455231130,36.001],[1482455234130,36.004],[1482455237130,36.014],[1482455240130,36.023],[1482455243130,35.996],[1482455246130,35.991],[1482455249130,35.998],[1482455252130,36.032],[1482455255130,36.006],[1482455258130,35.993],[1482455261130,35.983],[1482455264130,36.005],[1482455267130,36.023],[1482455270130,36.004],[1482455273130,35.997],[1482455276130,35.981],[1482455279130,36.005],[1482455282130,36.020],[1482455285130,36.002],[1482455288130,35.984],[1482455291130,35.979],[1482455294130,35.987],[1482455297130,36.012],[1482455300130,36.009],[1482455303130,36.008],[1482455306130,35.987],[1482455309130,35.988],[1482455312130,36.007],[1482455315130,36.022],[1482455318130,35.989],[1482455321130,35.988],[1482455324130,35.981],[1482455327130,36.011],[1482455330130,36.021],[1482455333130,36.011],[1482455336130,35.989],[1482455339130,35.991],[1482455342130,36.003],[1482455345130,35.981],[1482455348130,36.022],[1482455351130,35.980],[1482455354130,36.000],[1482455357130,36.027],[1482455360130,36.006],[1482455363130,36.017],[1482455366130,35.977],[1482455369130,35.990],[1482455372130,36.000],[1482455375130,36.019],[1482455378130,36.035],[1482455381130,36.002],[1482455384130,35.981],[1482455387130,36.009],[1482455390130,36.020],[1482455393130,36.004],[1482455396130,35.987],[1482455399130,36.010],[1482455402130,35.984],[1482455405130,36.009],[1482455408130,35.995],[1482455411130,36.007],[1482455414130,35.995],[1482455417130,36.011],[1482455420130,36.002],[1482455423130,35.996],[1482455426130,36.009],[1482455429130,36.012],[1482455432130,36.002],[1482455435130,35.980],[1482455438130,35.970],[1482455441130,36.022],[1482455444130,36.002],[1482455447130,36.001],[1482455450130,36.003],[1482455453130,36.005],[1482455456130,36.014],[1482455459130,36.015],[1482455462130,35.988],[1482455465130,35.988],[1482455468130,35.995],[1482455471130,35.994],[1482455474130,36.016],[1482455477130,36.011],[1482455480130,36.006],[1482455483130,35.974],[1482455486130,36.001],[1482455489130,36.015],[1482455492130,36.023],[1482455495130,35.956],[1482455498130,36.012],[1482455501130,35.996],[1482455504130,36.017],[1482455507130,36.022],[1482455510130,36.011],[1482455513130,35.998],[1482455516130,35.990],[1482455519130,36.022],[1482455522130,35.998],[1482455525130,36.026],[1482455528130,36.012],[1482455531130,35.984],[1482455534130,36.024],[1482455537130,36.018],[1482455540130,36.029],[1482455543130,35.987],[1482455546130,35.995],[1482455549130,35.996],[1482455552130,36.010],[1482455555130,35.993],[1482455558130,36.016],[1482455561130,35.977],[1482455564130,36.013],[1482455567130,36.019],[1482455570130,36.021],[1482455573130,36.024],[1482455576130,35.984],[1482455579130,35.994],[1482455582130,36.013],[1482455585130,36.018],[1482455588130,36.004],[1482455591130,35.999],[1482455594130,35.988],[1482455597130,36.029],[1482455600130,36.021],[1482455603130,36.014],[1482455606130,36.014],[1482455609130,36.006],[1482455612130,36.004],[1482455615130,36.014],[1482455618130,36.023],[1482455621130,36.012],[1482455624130,35.981],[1482455627130,36.010],[1482455630130,36.013],[1482455633130,36.010],[1482455636130,35.956],[1482455639130,36.013],[1482455642130,36.007],[1482455645130,36.002],[1482455648130,36.016],[1482455651130,36.008],[1482455654130,35.980],[1482455657130,35.996],[1482455660130,36.001],[1482455663130,36.029],[1482455666130,36.026],[1482455669130,35.998],[1482455672130,35.988],[1482455675130,36.022]],\n  "min":[[1482454778130,32.828],[1482454781130,33.312],[1482454784130,33.746],[1482454787130,34.267],[1482454790130,34.526],[1482454793130,34.891],[1482454796130,35.006],[1482454799130,35.268],[1482454802130,35.280],[1482454805130,35.615],[1482454808130,35.614],[1482454811130,35.601],[1482454814130,35.584],[1482454817130,35.670],[1482454820130,35.985],[1482454823130,35.980],[1482454826130,35.973],[1482454829130,35.979],[1482454832130,35.976],[1482454835130,35.982],[1482454838130,35.985],[1482454841130,35.965],[1482454844130,35.965],[1482454847130,35.975],[1482454850130,35.981],[1482454853130,35.981],[1482454856130,35.951],[1482454859130,35.976],[1482454862130,35.973],[1482454865130,35.960],[1482454868130,35.996],[1482454871130,35.976],[1482454874130,36.009],[1482454877130,35.998],[1482454880130,35.989],[1482454883130,35.978],[1482454886130,35.999],[1482454889130,35.996],[1482454892130,36.006],[1482454895130,35.991],[1482454898130,35.971],[1482454901130,35.959],[1482454904130,35.962],[1482454907130,35.991],[1482454910130,36.000],[1482454913130,35.987],[1482454916130,35.951],[1482454919130,35.962],[1482454922130,35.985],[1482454925130,35.968],[1482454928130,35.978],[1482454931130,35.978],[1482454934130,35.983],[1482454937130,35.991],[1482454940130,35.967],[1482454943130,35.989],[1482454946130,35.984],[1482454949130,35.978],[1482454952130,36.020],[1482454955130,35.974],[1482454958130,35.973],[1482454961130,35.977],[1482454964130,35.985],[1482454967130,35.997],[1482454970130,36.002],[1482454973130,35.994],[1482454976130,35.974],[1482454979130,36.018],[1482454982130,35.990],[1482454985130,35.973],[1482454988130,35.990],[1482454991130,35.968],[1482454994130,35.992],[1482454997130,35.986],[1482455000130,35.962],[1482455003130,35.993],[1482455006130,35.980],[1482455009130,35.998],[1482455012130,35.969],[1482455015130,35.980],[1482455018130,35.974],[1482455021130,35.966],[1482455024130,35.977],[1482455027130,35.984],[1482455030130,36.019],[1482455033130,36.009],[1482455036130,35.974],[1482455039130,35.992],[1482455042130,35.972],[1482455045130,35.981],[1482455048130,35.978],[1482455051130,35.985],[1482455054130,35.984],[1482455057130,35.975],[1482455060130,35.980],[1482455063130,36.003],[1482455066130,35.995],[1482455069130,35.989],[1482455072130,35.977],[1482455075130,35.981],[1482455078130,35.969],[1482455081130,35.965],[1482455084130,35.968],[1482455087130,35.964],[1482455090130,35.976],[1482455093130,35.994],[1482455096130,35.998],[1482455099130,35.977],[1482455102130,35.982],[1482455105130,35.976],[1482455108130,36.004],[1482455111130,35.975],[1482455114130,35.969],[1482455117130,35.984],[1482455120130,35.991],[1482455123130,35.989],[1482455126130,35.971],[1482455129130,35.985],[1482455132130,35.978],[1482455135130,36.005],[1482455138130,35.965],[1482455141130,35.983],[1482455144130,35.977],[1482455147130,36.011],[1482455150130,35.976],[1482455153130,35.976],[1482455156130,36.008],[1482455159130,35.973],[1482455162130,35.996],[1482455165130,35.971],[1482455168130,35.979],[1482455171130,35.982],[1482455174130,35.978],[1482455177130,36.000],[1482455180130,35.982],[1482455183130,35.991],[1482455186130,35.990],[1482455189130,35.972],[1482455192130,35.968],[1482455195130,35.981],[1482455198130,35.986],[1482455201130,35.977],[1482455204130,35.957],[1482455207130,35.986],[1482455210130,35.986],[1482455213130,35.990],[1482455216130,35.971],[1482455219130,35.965],[1482455222130,35.978],[1482455225130,35.975],[1482455228130,35.979],[1482455231130,35.995],[1482455234130,35.978],[1482455237130,35.987],[1482455240130,36.007],[1482455243130,35.996],[1482455246130,35.984],[1482455249130,35.987],[1482455252130,36.002],[1482455255130,35.967],[1482455258130,35.993],[1482455261130,35.957],[1482455264130,36.005],[1482455267130,35.986],[1482455270130,35.995],[1482455273130,35.997],[1482455276130,35.977],[1482455279130,35.979],[1482455282130,35.987],[1482455285130,35.986],[1482455288130,35.976],[1482455291130,35.969],[1482455294130,35.987],[1482455297130,35.982],[1482455300130,36.009],[1482455303130,35.979],[1482455306130,35.969],[1482455309130,35.988],[1482455312130,35.982],[1482455315130,35.994],[1482455318130,35.989],[1482455321130,35.988],[1482455324130,35.973],[1482455327130,35.988],[1482455330130,35.984],[1482455333130,36.004],[1482455336130,35.989],[1482455339130,35.976],[1482455342130,35.969],[1482455345130,35.981],[1482455348130,36.004],[1482455351130,35.980],[1482455354130,35.997],[1482455357130,35.984],[1482455360130,35.983],[1482455363130,35.972],[1482455366130,35.977],[1482455369130,35.989],[1482455372130,35.993],[1482455375130,35.984],[1482455378130,36.009],[1482455381130,35.986],[1482455384130,35.981],[1482455387130,35.994],[1482455390130,35.989],[1482455393130,35.975],[1482455396130,35.987],[1482455399130,35.979],[1482455402130,35.973],[1482455405130,35.974],[1482455408130,35.987],[1482455411130,35.967],[1482455414130,35.995],[1482455417130,35.977],[1482455420130,36.002],[1482455423130,35.996],[1482455426130,36.002],[1482455429130,35.997],[1482455432130,35.977],[1482455435130,35.980],[1482455438130,35.970],[1482455441130,35.982],[1482455444130,35.967],[1482455447130,35.980],[1482455450130,36.003],[1482455453130,36.002],[1482455456130,36.000],[1482455459130,35.961],[1482455462130,35.960],[1482455465130,35.988],[1482455468130,35.995],[1482455471130,35.992],[1482455474130,35.973],[1482455477130,36.003],[1482455480130,35.985],[1482455483130,35.974],[1482455486130,36.001],[1482455489130,35.980],[1482455492130,35.987],[1482455495130,35.956],[1482455498130,35.987],[1482455501130,35.996],[1482455504130,35.997],[1482455507130,35.988],[1482455510130,35.982],[1482455513130,35.998],[1482455516130,35.990],[1482455519130,35.996],[1482455522130,35.979],[1482455525130,35.998],[1482455528130,35.996],[1482455531130,35.984],[1482455534130,35.969],[1482455537130,35.998],[1482455540130,36.011],[1482455543130,35.985],[1482455546130,35.994],[1482455549130,35.996],[1482455552130,35.980],[1482455555130,35.965],[1482455558130,35.981],[1482455561130,35.977],[1482455564130,36.013],[1482455567130,35.997],[1482455570130,36.021],[1482455573130,35.986],[1482455576130,35.984],[1482455579130,35.994],[1482455582130,35.985],[1482455585130,36.013],[1482455588130,35.984],[1482455591130,35.999],[1482455594130,35.988],[1482455597130,36.014],[1482455600130,35.995],[1482455603130,35.997],[1482455606130,36.014],[1482455609130,35.993],[1482455612130,36.002],[1482455615130,35.977],[1482455618130,35.997],[1482455621130,35.995],[1482455624130,35.981],[1482455627130,35.995],[1482455630130,35.980],[1482455633130,35.982],[1482455636130,35.956],[1482455639130,35.993],[1482455642130,35.998],[1482455645130,35.983],[1482455648130,36.014],[1482455651130,35.991],[1482455654130,35.980],[1482455657130,35.993],[1482455660130,35.990],[1482455663130,35.992],[1482455666130,36.011],[1482455669130,35.998],[1482455672130,35.988],[1482455675130,36.022]]\n}'
_trenddata_15min_do_json = '{\n  "max":[[1482454778463,-0.24525],[1482454781463,-0.24597],[1482454784463,-0.24519],[1482454787463,-0.24699],[1482454790463,-0.24605],[1482454793463,-0.24575],[1482454796463,-0.24556],[1482454799463,-0.24508],[1482454802463,-0.24740],[1482454805463,-0.24625],[1482454808463,-0.24737],[1482454811463,-0.24675],[1482454814463,-0.24455],[1482454817463,-0.24693],[1482454820463,-0.24663],[1482454823463,-0.24646],[1482454826463,-0.24611],[1482454829463,-0.24655],[1482454832463,-0.24743],[1482454835463,-0.24536],[1482454838463,-0.24533],[1482454841463,-0.24578],[1482454844463,-0.24745],[1482454847463,-0.24642],[1482454850463,-0.24659],[1482454853463,-0.24745],[1482454856463,-0.24539],[1482454859463,-0.24751],[1482454862463,-0.24574],[1482454865463,-0.24540],[1482454868463,-0.24677],[1482454871463,-0.24762],[1482454874463,-0.24645],[1482454877463,-0.24512],[1482454880463,-0.24808],[1482454883463,-0.24607],[1482454886463,-0.24758],[1482454889463,-0.24646],[1482454892463,-0.24459],[1482454895463,-0.24659],[1482454898463,-0.24738],[1482454901463,-0.24574],[1482454904463,-0.24615],[1482454907463,-0.24677],[1482454910463,-0.24728],[1482454913463,-0.24711],[1482454916463,-0.24683],[1482454919463,-0.24508],[1482454922463,-0.24694],[1482454925463,-0.24686],[1482454928463,-0.24663],[1482454931463,-0.24623],[1482454934463,-0.24761],[1482454937463,-0.24543],[1482454940463,-0.24704],[1482454943463,-0.24683],[1482454946463,-0.24458],[1482454949463,-0.24733],[1482454952463,-0.24677],[1482454955463,-0.24525],[1482454958463,-0.24651],[1482454961463,-0.24690],[1482454964463,-0.24553],[1482454967463,-0.24780],[1482454970463,-0.24731],[1482454973463,-0.24525],[1482454976463,-0.24649],[1482454979463,-0.24583],[1482454982463,-0.24623],[1482454985463,-0.24615],[1482454988463,-0.24631],[1482454991463,-0.24625],[1482454994463,-0.24655],[1482454997463,-0.24551],[1482455000463,-0.24679],[1482455003463,-0.24645],[1482455006463,-0.24643],[1482455009463,-0.24697],[1482455012463,-0.24665],[1482455015463,-0.24549],[1482455018463,-0.24676],[1482455021463,-0.24526],[1482455024463,-0.24652],[1482455027463,-0.24733],[1482455030463,-0.24727],[1482455033463,-0.24689],[1482455036463,-0.24522],[1482455039463,-0.24690],[1482455042463,-0.24498],[1482455045463,-0.24560],[1482455048463,-0.24581],[1482455051463,-0.24609],[1482455054463,-0.24792],[1482455057463,-0.24581],[1482455060463,-0.24614],[1482455063463,-0.24711],[1482455066463,-0.24694],[1482455069463,-0.24666],[1482455072463,-0.24474],[1482455075463,-0.24554],[1482455078463,-0.24609],[1482455081463,-0.24530],[1482455084463,-0.24623],[1482455087463,-0.24603],[1482455090463,-0.24553],[1482455093463,-0.24480],[1482455096463,-0.24432],[1482455099463,-0.24364],[1482455102463,-0.24594],[1482455105463,-0.24557],[1482455108463,-0.24748],[1482455111463,-0.24570],[1482455114463,-0.24577],[1482455117463,-0.24522],[1482455120463,-0.24411],[1482455123463,-0.24583],[1482455126463,-0.24605],[1482455129463,-0.24641],[1482455132463,-0.24503],[1482455135463,-0.24737],[1482455138463,-0.24529],[1482455141463,-0.24673],[1482455144463,-0.24618],[1482455147463,-0.24627],[1482455150463,-0.24448],[1482455153463,-0.24786],[1482455156463,-0.24676],[1482455159463,-0.24696],[1482455162463,-0.24593],[1482455165463,-0.24683],[1482455168463,-0.24584],[1482455171463,-0.24676],[1482455174463,-0.24544],[1482455177463,-0.24581],[1482455180463,-0.24489],[1482455183463,-0.24687],[1482455186463,-0.24645],[1482455189463,-0.24873],[1482455192463,-0.24751],[1482455195463,-0.24543],[1482455198463,-0.24649],[1482455201463,-0.24676],[1482455204463,-0.24713],[1482455207463,-0.24508],[1482455210463,-0.24574],[1482455213463,-0.24625],[1482455216463,-0.24641],[1482455219463,-0.24704],[1482455222463,-0.24584],[1482455225463,-0.24566],[1482455228463,-0.24597],[1482455231463,-0.24643],[1482455234463,-0.24533],[1482455237463,-0.24748],[1482455240463,-0.24799],[1482455243463,-0.24595],[1482455246463,-0.24659],[1482455249463,-0.24690],[1482455252463,-0.24679],[1482455255463,-0.24662],[1482455258463,-0.24560],[1482455261463,-0.24556],[1482455264463,-0.24601],[1482455267463,-0.24465],[1482455270463,-0.24619],[1482455273463,-0.24571],[1482455276463,-0.24387],[1482455279463,-0.24537],[1482455282463,-0.24593],[1482455285463,-0.24617],[1482455288463,-0.24707],[1482455291463,-0.24593],[1482455294463,-0.24598],[1482455297463,-0.24502],[1482455300463,-0.24540],[1482455303463,-0.24651],[1482455306463,-0.24659],[1482455309463,-0.24639],[1482455312463,-0.24764],[1482455315463,-0.24728],[1482455318463,-0.24685],[1482455321463,-0.24726],[1482455324463,-0.24734],[1482455327463,-0.24635],[1482455330463,-0.24561],[1482455333463,-0.24530],[1482455336463,-0.24532],[1482455339463,-0.24651],[1482455342463,-0.24597],[1482455345463,-0.24597],[1482455348463,-0.24358],[1482455351463,-0.24649],[1482455354463,-0.24642],[1482455357463,-0.24516],[1482455360463,-0.24570],[1482455363463,-0.24471],[1482455366463,-0.24543],[1482455369463,-0.24659],[1482455372463,-0.24603],[1482455375463,-0.24530],[1482455378463,-0.24631],[1482455381463,-0.24438],[1482455384463,-0.24523],[1482455387463,-0.24551],[1482455390463,-0.24495],[1482455393463,-0.24747],[1482455396463,-0.24503],[1482455399463,-0.24659],[1482455402463,-0.24527],[1482455405463,-0.24537],[1482455408463,-0.24743],[1482455411463,-0.24618],[1482455414463,-0.24489],[1482455417463,-0.24485],[1482455420463,-0.24559],[1482455423463,-0.24601],[1482455426463,-0.24690],[1482455429463,-0.24721],[1482455432463,-0.24629],[1482455435463,-0.24607],[1482455438463,-0.24601],[1482455441463,-0.24660],[1482455444463,-0.24625],[1482455447463,-0.24618],[1482455450463,-0.24669],[1482455453463,-0.24502],[1482455456463,-0.24452],[1482455459463,-0.24508],[1482455462463,-0.24578],[1482455465463,-0.24516],[1482455468463,-0.24813],[1482455471463,-0.24651],[1482455474463,-0.24534],[1482455477463,-0.24584],[1482455480463,-0.24578],[1482455483463,-0.24754],[1482455486463,-0.24748],[1482455489463,-0.24578],[1482455492463,-0.24604],[1482455495463,-0.24527],[1482455498463,-0.24593],[1482455501463,-0.24541],[1482455504463,-0.24744],[1482455507463,-0.24499],[1482455510463,-0.24617],[1482455513463,-0.24625],[1482455516463,-0.24515],[1482455519463,-0.24468],[1482455522463,-0.24578],[1482455525463,-0.24466],[1482455528463,-0.24710],[1482455531463,-0.24575],[1482455534463,-0.24628],[1482455537463,-0.24800],[1482455540463,-0.24547],[1482455543463,-0.24599],[1482455546463,-0.24619],[1482455549463,-0.24804],[1482455552463,-0.24430],[1482455555463,-0.24522],[1482455558463,-0.24539],[1482455561463,-0.24622],[1482455564463,-0.24622],[1482455567463,-0.24505],[1482455570463,-0.24638],[1482455573463,-0.24653],[1482455576463,-0.24516],[1482455579463,-0.24496],[1482455582463,-0.24547],[1482455585463,-0.24492],[1482455588463,-0.24795],[1482455591463,-0.24573],[1482455594463,-0.24502],[1482455597463,-0.24628],[1482455600463,-0.24608],[1482455603463,-0.24550],[1482455606463,-0.24627],[1482455609463,-0.24598],[1482455612463,-0.24510],[1482455615463,-0.24544],[1482455618463,-0.24659],[1482455621463,-0.24657],[1482455624463,-0.24843],[1482455627463,-0.24493],[1482455630463,-0.24776],[1482455633463,-0.24522],[1482455636463,-0.24525],[1482455639463,-0.24584],[1482455642463,-0.24629],[1482455645463,-0.24540],[1482455648463,-0.24587],[1482455651463,-0.24546],[1482455654463,-0.24752],[1482455657463,-0.24403],[1482455660463,-0.24486],[1482455663463,-0.24462],[1482455666463,-0.24583],[1482455669463,-0.24718],[1482455672463,-0.24603],[1482455675463,-0.24599]],\n  "last":[[1482454778463,-0.24770],[1482454781463,-0.24597],[1482454784463,-0.24607],[1482454787463,-0.24747],[1482454790463,-0.24605],[1482454793463,-0.24808],[1482454796463,-0.24632],[1482454799463,-0.24798],[1482454802463,-0.24847],[1482454805463,-0.24625],[1482454808463,-0.24782],[1482454811463,-0.24675],[1482454814463,-0.24583],[1482454817463,-0.24741],[1482454820463,-0.24959],[1482454823463,-0.24667],[1482454826463,-0.24874],[1482454829463,-0.24798],[1482454832463,-0.24853],[1482454835463,-0.24536],[1482454838463,-0.24533],[1482454841463,-0.24649],[1482454844463,-0.24800],[1482454847463,-0.24651],[1482454850463,-0.24659],[1482454853463,-0.24846],[1482454856463,-0.24672],[1482454859463,-0.24751],[1482454862463,-0.24782],[1482454865463,-0.24721],[1482454868463,-0.24677],[1482454871463,-0.24883],[1482454874463,-0.24677],[1482454877463,-0.24694],[1482454880463,-0.24808],[1482454883463,-0.24718],[1482454886463,-0.24826],[1482454889463,-0.24836],[1482454892463,-0.24778],[1482454895463,-0.24895],[1482454898463,-0.24860],[1482454901463,-0.24804],[1482454904463,-0.24880],[1482454907463,-0.24796],[1482454910463,-0.24805],[1482454913463,-0.24747],[1482454916463,-0.24714],[1482454919463,-0.24508],[1482454922463,-0.24694],[1482454925463,-0.24734],[1482454928463,-0.24724],[1482454931463,-0.24623],[1482454934463,-0.24761],[1482454937463,-0.24544],[1482454940463,-0.24858],[1482454943463,-0.24683],[1482454946463,-0.24657],[1482454949463,-0.24789],[1482454952463,-0.24699],[1482454955463,-0.24525],[1482454958463,-0.24752],[1482454961463,-0.24737],[1482454964463,-0.24553],[1482454967463,-0.24867],[1482454970463,-0.24823],[1482454973463,-0.24768],[1482454976463,-0.24898],[1482454979463,-0.24735],[1482454982463,-0.24969],[1482454985463,-0.24805],[1482454988463,-0.24631],[1482454991463,-0.24733],[1482454994463,-0.24750],[1482454997463,-0.24551],[1482455000463,-0.24798],[1482455003463,-0.24645],[1482455006463,-0.24643],[1482455009463,-0.24743],[1482455012463,-0.24665],[1482455015463,-0.24768],[1482455018463,-0.24744],[1482455021463,-0.24526],[1482455024463,-0.24675],[1482455027463,-0.24832],[1482455030463,-0.24731],[1482455033463,-0.24796],[1482455036463,-0.24818],[1482455039463,-0.24832],[1482455042463,-0.24842],[1482455045463,-0.24560],[1482455048463,-0.24819],[1482455051463,-0.24609],[1482455054463,-0.24813],[1482455057463,-0.24581],[1482455060463,-0.24655],[1482455063463,-0.24711],[1482455066463,-0.24758],[1482455069463,-0.24743],[1482455072463,-0.24741],[1482455075463,-0.24706],[1482455078463,-0.24609],[1482455081463,-0.24802],[1482455084463,-0.24623],[1482455087463,-0.24609],[1482455090463,-0.24585],[1482455093463,-0.24480],[1482455096463,-0.24432],[1482455099463,-0.24364],[1482455102463,-0.24860],[1482455105463,-0.24866],[1482455108463,-0.24748],[1482455111463,-0.24774],[1482455114463,-0.24615],[1482455117463,-0.24608],[1482455120463,-0.24829],[1482455123463,-0.24583],[1482455126463,-0.24751],[1482455129463,-0.24641],[1482455132463,-0.24503],[1482455135463,-0.24837],[1482455138463,-0.24639],[1482455141463,-0.24673],[1482455144463,-0.24618],[1482455147463,-0.24663],[1482455150463,-0.24726],[1482455153463,-0.24949],[1482455156463,-0.24768],[1482455159463,-0.24696],[1482455162463,-0.24607],[1482455165463,-0.24683],[1482455168463,-0.24584],[1482455171463,-0.24833],[1482455174463,-0.24544],[1482455177463,-0.24788],[1482455180463,-0.24771],[1482455183463,-0.24740],[1482455186463,-0.24652],[1482455189463,-0.24873],[1482455192463,-0.24751],[1482455195463,-0.24543],[1482455198463,-0.24690],[1482455201463,-0.24900],[1482455204463,-0.24918],[1482455207463,-0.24520],[1482455210463,-0.24574],[1482455213463,-0.24680],[1482455216463,-0.24788],[1482455219463,-0.24704],[1482455222463,-0.24730],[1482455225463,-0.24709],[1482455228463,-0.24597],[1482455231463,-0.24733],[1482455234463,-0.24533],[1482455237463,-0.24847],[1482455240463,-0.24843],[1482455243463,-0.24595],[1482455246463,-0.24784],[1482455249463,-0.24694],[1482455252463,-0.24782],[1482455255463,-0.24662],[1482455258463,-0.24772],[1482455261463,-0.24556],[1482455264463,-0.24785],[1482455267463,-0.24465],[1482455270463,-0.24619],[1482455273463,-0.24571],[1482455276463,-0.24387],[1482455279463,-0.24727],[1482455282463,-0.24681],[1482455285463,-0.24617],[1482455288463,-0.24707],[1482455291463,-0.24860],[1482455294463,-0.24598],[1482455297463,-0.24713],[1482455300463,-0.24750],[1482455303463,-0.24655],[1482455306463,-0.24713],[1482455309463,-0.24810],[1482455312463,-0.24764],[1482455315463,-0.24832],[1482455318463,-0.24730],[1482455321463,-0.24856],[1482455324463,-0.24805],[1482455327463,-0.24635],[1482455330463,-0.24973],[1482455333463,-0.24764],[1482455336463,-0.24534],[1482455339463,-0.24651],[1482455342463,-0.24871],[1482455345463,-0.24597],[1482455348463,-0.24358],[1482455351463,-0.24711],[1482455354463,-0.24791],[1482455357463,-0.24823],[1482455360463,-0.24853],[1482455363463,-0.24471],[1482455366463,-0.24771],[1482455369463,-0.24659],[1482455372463,-0.24603],[1482455375463,-0.24917],[1482455378463,-0.24808],[1482455381463,-0.24438],[1482455384463,-0.24962],[1482455387463,-0.24551],[1482455390463,-0.24880],[1482455393463,-0.24747],[1482455396463,-0.24566],[1482455399463,-0.24659],[1482455402463,-0.24662],[1482455405463,-0.24537],[1482455408463,-0.24823],[1482455411463,-0.24675],[1482455414463,-0.24489],[1482455417463,-0.24659],[1482455420463,-0.24559],[1482455423463,-0.24601],[1482455426463,-0.24829],[1482455429463,-0.24830],[1482455432463,-0.24677],[1482455435463,-0.24607],[1482455438463,-0.24601],[1482455441463,-0.24730],[1482455444463,-0.24625],[1482455447463,-0.24976],[1482455450463,-0.24784],[1482455453463,-0.24707],[1482455456463,-0.24452],[1482455459463,-0.24508],[1482455462463,-0.24578],[1482455465463,-0.24516],[1482455468463,-0.24856],[1482455471463,-0.24717],[1482455474463,-0.24534],[1482455477463,-0.24584],[1482455480463,-0.24578],[1482455483463,-0.24895],[1482455486463,-0.24785],[1482455489463,-0.24578],[1482455492463,-0.24871],[1482455495463,-0.24569],[1482455498463,-0.24593],[1482455501463,-0.24619],[1482455504463,-0.24828],[1482455507463,-0.24564],[1482455510463,-0.24617],[1482455513463,-0.24625],[1482455516463,-0.24764],[1482455519463,-0.24703],[1482455522463,-0.24870],[1482455525463,-0.24516],[1482455528463,-0.24710],[1482455531463,-0.24657],[1482455534463,-0.24908],[1482455537463,-0.24805],[1482455540463,-0.24547],[1482455543463,-0.25009],[1482455546463,-0.24931],[1482455549463,-0.24850],[1482455552463,-0.24557],[1482455555463,-0.24643],[1482455558463,-0.24908],[1482455561463,-0.24622],[1482455564463,-0.24660],[1482455567463,-0.24876],[1482455570463,-0.24800],[1482455573463,-0.24653],[1482455576463,-0.24894],[1482455579463,-0.24496],[1482455582463,-0.24547],[1482455585463,-0.24680],[1482455588463,-0.24795],[1482455591463,-0.24573],[1482455594463,-0.24726],[1482455597463,-0.24641],[1482455600463,-0.24608],[1482455603463,-0.24566],[1482455606463,-0.25044],[1482455609463,-0.24842],[1482455612463,-0.24765],[1482455615463,-0.24868],[1482455618463,-0.24659],[1482455621463,-0.24673],[1482455624463,-0.24844],[1482455627463,-0.24854],[1482455630463,-0.24829],[1482455633463,-0.24935],[1482455636463,-0.24710],[1482455639463,-0.24849],[1482455642463,-0.24629],[1482455645463,-0.24738],[1482455648463,-0.24587],[1482455651463,-0.24546],[1482455654463,-0.24924],[1482455657463,-0.25035],[1482455660463,-0.24693],[1482455663463,-0.24462],[1482455666463,-0.24786],[1482455669463,-0.24718],[1482455672463,-0.24706],[1482455675463,-0.24599]],\n  "min":[[1482454778463,-0.24770],[1482454781463,-0.24670],[1482454784463,-0.24607],[1482454787463,-0.24810],[1482454790463,-0.24836],[1482454793463,-0.24808],[1482454796463,-0.24840],[1482454799463,-0.24798],[1482454802463,-0.24917],[1482454805463,-0.24798],[1482454808463,-0.25072],[1482454811463,-0.24762],[1482454814463,-0.25063],[1482454817463,-0.24744],[1482454820463,-0.24959],[1482454823463,-0.24747],[1482454826463,-0.24874],[1482454829463,-0.24798],[1482454832463,-0.24853],[1482454835463,-0.24727],[1482454838463,-0.24852],[1482454841463,-0.24649],[1482454844463,-0.24800],[1482454847463,-0.24679],[1482454850463,-0.24758],[1482454853463,-0.24946],[1482454856463,-0.24672],[1482454859463,-0.24842],[1482454862463,-0.24782],[1482454865463,-0.24844],[1482454868463,-0.24767],[1482454871463,-0.24883],[1482454874463,-0.24694],[1482454877463,-0.24694],[1482454880463,-0.24839],[1482454883463,-0.24781],[1482454886463,-0.24826],[1482454889463,-0.24897],[1482454892463,-0.24778],[1482454895463,-0.25044],[1482454898463,-0.24880],[1482454901463,-0.24804],[1482454904463,-0.24880],[1482454907463,-0.24796],[1482454910463,-0.24805],[1482454913463,-0.24952],[1482454916463,-0.24717],[1482454919463,-0.24800],[1482454922463,-0.24788],[1482454925463,-0.25007],[1482454928463,-0.24883],[1482454931463,-0.24826],[1482454934463,-0.24895],[1482454937463,-0.24727],[1482454940463,-0.25030],[1482454943463,-0.24884],[1482454946463,-0.24690],[1482454949463,-0.24789],[1482454952463,-0.24707],[1482454955463,-0.24847],[1482454958463,-0.24752],[1482454961463,-0.24768],[1482454964463,-0.24874],[1482454967463,-0.24867],[1482454970463,-0.24860],[1482454973463,-0.25009],[1482454976463,-0.24898],[1482454979463,-0.24867],[1482454982463,-0.24969],[1482454985463,-0.24805],[1482454988463,-0.24737],[1482454991463,-0.24810],[1482454994463,-0.24832],[1482454997463,-0.24828],[1482455000463,-0.24798],[1482455003463,-0.24813],[1482455006463,-0.24780],[1482455009463,-0.24878],[1482455012463,-0.24754],[1482455015463,-0.24884],[1482455018463,-0.24744],[1482455021463,-0.24761],[1482455024463,-0.24873],[1482455027463,-0.24922],[1482455030463,-0.24870],[1482455033463,-0.24815],[1482455036463,-0.24826],[1482455039463,-0.24832],[1482455042463,-0.24842],[1482455045463,-0.24847],[1482455048463,-0.24819],[1482455051463,-0.24870],[1482455054463,-0.24894],[1482455057463,-0.24758],[1482455060463,-0.24816],[1482455063463,-0.24983],[1482455066463,-0.24781],[1482455069463,-0.24743],[1482455072463,-0.24935],[1482455075463,-0.24706],[1482455078463,-0.24775],[1482455081463,-0.24802],[1482455084463,-0.24878],[1482455087463,-0.24828],[1482455090463,-0.24938],[1482455093463,-0.24642],[1482455096463,-0.24642],[1482455099463,-0.24819],[1482455102463,-0.24860],[1482455105463,-0.24866],[1482455108463,-0.24955],[1482455111463,-0.24908],[1482455114463,-0.24744],[1482455117463,-0.24798],[1482455120463,-0.24829],[1482455123463,-0.24891],[1482455126463,-0.24832],[1482455129463,-0.24809],[1482455132463,-0.24607],[1482455135463,-0.24870],[1482455138463,-0.24738],[1482455141463,-0.24832],[1482455144463,-0.24784],[1482455147463,-0.24696],[1482455150463,-0.24726],[1482455153463,-0.24949],[1482455156463,-0.24823],[1482455159463,-0.24910],[1482455162463,-0.24638],[1482455165463,-0.24800],[1482455168463,-0.24720],[1482455171463,-0.24877],[1482455174463,-0.24894],[1482455177463,-0.24789],[1482455180463,-0.24771],[1482455183463,-0.24750],[1482455186463,-0.24681],[1482455189463,-0.24914],[1482455192463,-0.24917],[1482455195463,-0.24741],[1482455198463,-0.24834],[1482455201463,-0.24900],[1482455204463,-0.24918],[1482455207463,-0.24796],[1482455210463,-0.24696],[1482455213463,-0.24737],[1482455216463,-0.24788],[1482455219463,-0.24837],[1482455222463,-0.24730],[1482455225463,-0.24781],[1482455228463,-0.24646],[1482455231463,-0.24733],[1482455234463,-0.24873],[1482455237463,-0.24871],[1482455240463,-0.24892],[1482455243463,-0.24758],[1482455246463,-0.24784],[1482455249463,-0.24969],[1482455252463,-0.24931],[1482455255463,-0.24728],[1482455258463,-0.24772],[1482455261463,-0.24874],[1482455264463,-0.24785],[1482455267463,-0.24932],[1482455270463,-0.24870],[1482455273463,-0.24778],[1482455276463,-0.24901],[1482455279463,-0.24727],[1482455282463,-0.24681],[1482455285463,-0.24856],[1482455288463,-0.24826],[1482455291463,-0.24990],[1482455294463,-0.24791],[1482455297463,-0.24713],[1482455300463,-0.24820],[1482455303463,-0.24892],[1482455306463,-0.24965],[1482455309463,-0.24810],[1482455312463,-0.24953],[1482455315463,-0.24832],[1482455318463,-0.24908],[1482455321463,-0.24856],[1482455324463,-0.24805],[1482455327463,-0.24727],[1482455330463,-0.24973],[1482455333463,-0.24764],[1482455336463,-0.24735],[1482455339463,-0.24823],[1482455342463,-0.24871],[1482455345463,-0.24830],[1482455348463,-0.24795],[1482455351463,-0.24931],[1482455354463,-0.24791],[1482455357463,-0.24823],[1482455360463,-0.24853],[1482455363463,-0.24608],[1482455366463,-0.24771],[1482455369463,-0.24881],[1482455372463,-0.24755],[1482455375463,-0.24917],[1482455378463,-0.24914],[1482455381463,-0.24922],[1482455384463,-0.24962],[1482455387463,-0.24965],[1482455390463,-0.24880],[1482455393463,-0.24914],[1482455396463,-0.24707],[1482455399463,-0.24718],[1482455402463,-0.24720],[1482455405463,-0.24891],[1482455408463,-0.24823],[1482455411463,-0.24748],[1482455414463,-0.24747],[1482455417463,-0.24826],[1482455420463,-0.24910],[1482455423463,-0.24697],[1482455426463,-0.24829],[1482455429463,-0.24928],[1482455432463,-0.24677],[1482455435463,-0.24837],[1482455438463,-0.24834],[1482455441463,-0.24730],[1482455444463,-0.25006],[1482455447463,-0.24976],[1482455450463,-0.24973],[1482455453463,-0.24866],[1482455456463,-0.24912],[1482455459463,-0.24657],[1482455462463,-0.24815],[1482455465463,-0.24846],[1482455468463,-0.24856],[1482455471463,-0.24717],[1482455474463,-0.24744],[1482455477463,-0.24720],[1482455480463,-0.24764],[1482455483463,-0.24895],[1482455486463,-0.24895],[1482455489463,-0.24794],[1482455492463,-0.24871],[1482455495463,-0.24623],[1482455498463,-0.24799],[1482455501463,-0.24830],[1482455504463,-0.24927],[1482455507463,-0.24573],[1482455510463,-0.24958],[1482455513463,-0.24723],[1482455516463,-0.24764],[1482455519463,-0.24703],[1482455522463,-0.24870],[1482455525463,-0.24770],[1482455528463,-0.24750],[1482455531463,-0.24704],[1482455534463,-0.24908],[1482455537463,-0.24806],[1482455540463,-0.24868],[1482455543463,-0.25009],[1482455546463,-0.24931],[1482455549463,-0.24850],[1482455552463,-0.24718],[1482455555463,-0.24643],[1482455558463,-0.24908],[1482455561463,-0.24844],[1482455564463,-0.24660],[1482455567463,-0.24876],[1482455570463,-0.24800],[1482455573463,-0.24795],[1482455576463,-0.24894],[1482455579463,-0.24717],[1482455582463,-0.24890],[1482455585463,-0.24922],[1482455588463,-0.24856],[1482455591463,-0.24953],[1482455594463,-0.24837],[1482455597463,-0.24641],[1482455600463,-0.24905],[1482455603463,-0.24728],[1482455606463,-0.25044],[1482455609463,-0.24842],[1482455612463,-0.24765],[1482455615463,-0.24878],[1482455618463,-0.24942],[1482455621463,-0.24874],[1482455624463,-0.24945],[1482455627463,-0.24854],[1482455630463,-0.24862],[1482455633463,-0.24935],[1482455636463,-0.24806],[1482455639463,-0.24849],[1482455642463,-0.24645],[1482455645463,-0.24886],[1482455648463,-0.24864],[1482455651463,-0.24963],[1482455654463,-0.24924],[1482455657463,-0.25035],[1482455660463,-0.24873],[1482455663463,-0.24604],[1482455666463,-0.24786],[1482455669463,-0.24918],[1482455672463,-0.24752],[1482455675463,-0.24599]]\n}'
_trenddata_15min_ph_json = '{\n  "max":[[1482454778764,0.036825],[1482454781764,0.036785],[1482454784764,0.036680],[1482454787764,0.036660],[1482454790764,0.036692],[1482454793764,0.036704],[1482454796764,0.036547],[1482454799764,0.036632],[1482454802764,0.036604],[1482454805764,0.036648],[1482454808764,0.036467],[1482454811764,0.036706],[1482454814764,0.037427],[1482454817764,0.037079],[1482454820764,0.036831],[1482454823764,0.036680],[1482454826764,0.036626],[1482454829764,0.036721],[1482454832764,0.036508],[1482454835764,0.036652],[1482454838764,0.036773],[1482454841764,0.036734],[1482454844764,0.036618],[1482454847764,0.036646],[1482454850764,0.036636],[1482454853764,0.036659],[1482454856764,0.036779],[1482454859764,0.036608],[1482454862764,0.036601],[1482454865764,0.036647],[1482454868764,0.036574],[1482454871764,0.036600],[1482454874764,0.036725],[1482454877764,0.036725],[1482454880764,0.036584],[1482454883764,0.036594],[1482454886764,0.036547],[1482454889764,0.036504],[1482454892764,0.036675],[1482454895764,0.036591],[1482454898764,0.036577],[1482454901764,0.036688],[1482454904764,0.036648],[1482454907764,0.036569],[1482454910764,0.036713],[1482454913764,0.036685],[1482454916764,0.036771],[1482454919764,0.036739],[1482454922764,0.036707],[1482454925764,0.036714],[1482454928764,0.036528],[1482454931764,0.036631],[1482454934764,0.036551],[1482454937764,0.036624],[1482454940764,0.036605],[1482454943764,0.036840],[1482454946764,0.036591],[1482454949764,0.036568],[1482454952764,0.036688],[1482454955764,0.036709],[1482454958764,0.036710],[1482454961764,0.036686],[1482454964764,0.036689],[1482454967764,0.036750],[1482454970764,0.036556],[1482454973764,0.036719],[1482454976764,0.036664],[1482454979764,0.036633],[1482454982764,0.036685],[1482454985764,0.036596],[1482454988764,0.036758],[1482454991764,0.036622],[1482454994764,0.036628],[1482454997764,0.036659],[1482455000764,0.036679],[1482455003764,0.036848],[1482455006764,0.036789],[1482455009764,0.036606],[1482455012764,0.036916],[1482455015764,0.036655],[1482455018764,0.036547],[1482455021764,0.036767],[1482455024764,0.036822],[1482455027764,0.036633],[1482455030764,0.036738],[1482455033764,0.036627],[1482455036764,0.036675],[1482455039764,0.036765],[1482455042764,0.036783],[1482455045764,0.036947],[1482455048764,0.036983],[1482455051764,0.036758],[1482455054764,0.036663],[1482455057764,0.036628],[1482455060764,0.036803],[1482455063764,0.036901],[1482455066764,0.037239],[1482455069764,0.037098],[1482455072764,0.036624],[1482455075764,0.036937],[1482455078764,0.037694],[1482455081764,0.036763],[1482455084764,0.037248],[1482455087764,0.037056],[1482455090764,0.036960],[1482455093764,0.037652],[1482455096764,0.037559],[1482455099764,0.037030],[1482455102764,0.037865],[1482455105764,0.037532],[1482455108764,0.037818],[1482455111764,0.036413],[1482455114764,0.037030],[1482455117764,0.038098],[1482455120764,0.037754],[1482455123764,0.037664],[1482455126764,0.037788],[1482455129764,0.037005],[1482455132764,0.037843],[1482455135764,0.037184],[1482455138764,0.037131],[1482455141764,0.037244],[1482455144764,0.037124],[1482455147764,0.036914],[1482455150764,0.037263],[1482455153764,0.036947],[1482455156764,0.037837],[1482455159764,0.036918],[1482455162764,0.037337],[1482455165764,0.036948],[1482455168764,0.037241],[1482455171764,0.037001],[1482455174764,0.037227],[1482455177764,0.037249],[1482455180764,0.037193],[1482455183764,0.036886],[1482455186764,0.037525],[1482455189764,0.036959],[1482455192764,0.037087],[1482455195764,0.036886],[1482455198764,0.036948],[1482455201764,0.036878],[1482455204764,0.036896],[1482455207764,0.037070],[1482455210764,0.036795],[1482455213764,0.037061],[1482455216764,0.036879],[1482455219764,0.036824],[1482455222764,0.037249],[1482455225764,0.036932],[1482455228764,0.037250],[1482455231764,0.037480],[1482455234764,0.037001],[1482455237764,0.037170],[1482455240764,0.036949],[1482455243764,0.037660],[1482455246764,0.037037],[1482455249764,0.037628],[1482455252764,0.037036],[1482455255764,0.037127],[1482455258764,0.037464],[1482455261764,0.037558],[1482455264764,0.037828],[1482455267764,0.037472],[1482455270764,0.037602],[1482455273764,0.036982],[1482455276764,0.037390],[1482455279764,0.037251],[1482455282764,0.037179],[1482455285764,0.037770],[1482455288764,0.037984],[1482455291764,0.037299],[1482455294764,0.037019],[1482455297764,0.037325],[1482455300764,0.037256],[1482455303764,0.037903],[1482455306764,0.037379],[1482455309764,0.037056],[1482455312764,0.037399],[1482455315764,0.037137],[1482455318764,0.037089],[1482455321764,0.037253],[1482455324764,0.037704],[1482455327764,0.036907],[1482455330764,0.037064],[1482455333764,0.037395],[1482455336764,0.037219],[1482455339764,0.037209],[1482455342764,0.037193],[1482455345764,0.037140],[1482455348764,0.037236],[1482455351764,0.037118],[1482455354764,0.037188],[1482455357764,0.037143],[1482455360764,0.037196],[1482455363764,0.038118],[1482455366764,0.037747],[1482455369764,0.037973],[1482455372764,0.037520],[1482455375764,0.037100],[1482455378764,0.037332],[1482455381764,0.037265],[1482455384764,0.037163],[1482455387764,0.037279],[1482455390764,0.037286],[1482455393764,0.037620],[1482455396764,0.037792],[1482455399764,0.037459],[1482455402764,0.037514],[1482455405764,0.037041],[1482455408764,0.037775],[1482455411764,0.037738],[1482455414764,0.037006],[1482455417764,0.037118],[1482455420764,0.038009],[1482455423764,0.037355],[1482455426764,0.037294],[1482455429764,0.037102],[1482455432764,0.037434],[1482455435764,0.037376],[1482455438764,0.037161],[1482455441764,0.037394],[1482455444764,0.037250],[1482455447764,0.037036],[1482455450764,0.037129],[1482455453764,0.037382],[1482455456764,0.037380],[1482455459764,0.037364],[1482455462764,0.037304],[1482455465764,0.037377],[1482455468764,0.037145],[1482455471764,0.037219],[1482455474764,0.037369],[1482455477764,0.037299],[1482455480764,0.037265],[1482455483764,0.037185],[1482455486764,0.037167],[1482455489764,0.037209],[1482455492764,0.037339],[1482455495764,0.037255],[1482455498764,0.037388],[1482455501764,0.037383],[1482455504764,0.037259],[1482455507764,0.037303],[1482455510764,0.037459],[1482455513764,0.037307],[1482455516764,0.037347],[1482455519764,0.037267],[1482455522764,0.037234],[1482455525764,0.037397],[1482455528764,0.037417],[1482455531764,0.037386],[1482455534764,0.037492],[1482455537764,0.037117],[1482455540764,0.037293],[1482455543764,0.037318],[1482455546764,0.037327],[1482455549764,0.037057],[1482455552764,0.037438],[1482455555764,0.037342],[1482455558764,0.037496],[1482455561764,0.037300],[1482455564764,0.037495],[1482455567764,0.037381],[1482455570764,0.037436],[1482455573764,0.037072],[1482455576764,0.037285],[1482455579764,0.037345],[1482455582764,0.037317],[1482455585764,0.037408],[1482455588764,0.037272],[1482455591764,0.037327],[1482455594764,0.037361],[1482455597764,0.037442],[1482455600764,0.037373],[1482455603764,0.037361],[1482455606764,0.037367],[1482455609764,0.037610],[1482455612764,0.037531],[1482455615764,0.037370],[1482455618764,0.037385],[1482455621764,0.037356],[1482455624764,0.037230],[1482455627764,0.037449],[1482455630764,0.037371],[1482455633764,0.037291],[1482455636764,0.037443],[1482455639764,0.037425],[1482455642764,0.037453],[1482455645764,0.037378],[1482455648764,0.037503],[1482455651764,0.037393],[1482455654764,0.037669],[1482455657764,0.037383],[1482455660764,0.037513],[1482455663764,0.037617],[1482455666764,0.037285],[1482455669764,0.037496],[1482455672764,0.037444],[1482455675764,0.037541]],\n  "last":[[1482454778764,0.036519],[1482454781764,0.036785],[1482454784764,0.036680],[1482454787764,0.036450],[1482454790764,0.036532],[1482454793764,0.036479],[1482454796764,0.036495],[1482454799764,0.036632],[1482454802764,0.036571],[1482454805764,0.036648],[1482454808764,0.036375],[1482454811764,0.036425],[1482454814764,0.037427],[1482454817764,0.036554],[1482454820764,0.036562],[1482454823764,0.036680],[1482454826764,0.036531],[1482454829764,0.036357],[1482454832764,0.036364],[1482454835764,0.036518],[1482454838764,0.036773],[1482454841764,0.036640],[1482454844764,0.036450],[1482454847764,0.036646],[1482454850764,0.036636],[1482454853764,0.036659],[1482454856764,0.036588],[1482454859764,0.036608],[1482454862764,0.036546],[1482454865764,0.036647],[1482454868764,0.036515],[1482454871764,0.036417],[1482454874764,0.036677],[1482454877764,0.036725],[1482454880764,0.036433],[1482454883764,0.036508],[1482454886764,0.036547],[1482454889764,0.036336],[1482454892764,0.036535],[1482454895764,0.036513],[1482454898764,0.036572],[1482454901764,0.036510],[1482454904764,0.036462],[1482454907764,0.036500],[1482454910764,0.036617],[1482454913764,0.036604],[1482454916764,0.036605],[1482454919764,0.036628],[1482454922764,0.036707],[1482454925764,0.036714],[1482454928764,0.036520],[1482454931764,0.036602],[1482454934764,0.036551],[1482454937764,0.036622],[1482454940764,0.036478],[1482454943764,0.036840],[1482454946764,0.036573],[1482454949764,0.036568],[1482454952764,0.036569],[1482454955764,0.036709],[1482454958764,0.036710],[1482454961764,0.036686],[1482454964764,0.036689],[1482454967764,0.036454],[1482454970764,0.036486],[1482454973764,0.036473],[1482454976764,0.036498],[1482454979764,0.036609],[1482454982764,0.036397],[1482454985764,0.036586],[1482454988764,0.036758],[1482454991764,0.036419],[1482454994764,0.036511],[1482454997764,0.036630],[1482455000764,0.036386],[1482455003764,0.036689],[1482455006764,0.036665],[1482455009764,0.036599],[1482455012764,0.036916],[1482455015764,0.036402],[1482455018764,0.036439],[1482455021764,0.036767],[1482455024764,0.036626],[1482455027764,0.036633],[1482455030764,0.036681],[1482455033764,0.036627],[1482455036764,0.036531],[1482455039764,0.036765],[1482455042764,0.036453],[1482455045764,0.036947],[1482455048764,0.036584],[1482455051764,0.036758],[1482455054764,0.036663],[1482455057764,0.036628],[1482455060764,0.036803],[1482455063764,0.036600],[1482455066764,0.036590],[1482455069764,0.037098],[1482455072764,0.036327],[1482455075764,0.036695],[1482455078764,0.036541],[1482455081764,0.035953],[1482455084764,0.036880],[1482455087764,0.036829],[1482455090764,0.036638],[1482455093764,0.037302],[1482455096764,0.036842],[1482455099764,0.036592],[1482455102764,0.036620],[1482455105764,0.036911],[1482455108764,0.037248],[1482455111764,0.036090],[1482455114764,0.036417],[1482455117764,0.037210],[1482455120764,0.037754],[1482455123764,0.036957],[1482455126764,0.036850],[1482455129764,0.037005],[1482455132764,0.037843],[1482455135764,0.037184],[1482455138764,0.037131],[1482455141764,0.036628],[1482455144764,0.037124],[1482455147764,0.036861],[1482455150764,0.036899],[1482455153764,0.036877],[1482455156764,0.037258],[1482455159764,0.036918],[1482455162764,0.037337],[1482455165764,0.036370],[1482455168764,0.036685],[1482455171764,0.036421],[1482455174764,0.036985],[1482455177764,0.036916],[1482455180764,0.036968],[1482455183764,0.036814],[1482455186764,0.037206],[1482455189764,0.036719],[1482455192764,0.037052],[1482455195764,0.036438],[1482455198764,0.036948],[1482455201764,0.036656],[1482455204764,0.036651],[1482455207764,0.036785],[1482455210764,0.036795],[1482455213764,0.036755],[1482455216764,0.036804],[1482455219764,0.036810],[1482455222764,0.036907],[1482455225764,0.036747],[1482455228764,0.036921],[1482455231764,0.037210],[1482455234764,0.036943],[1482455237764,0.036770],[1482455240764,0.036460],[1482455243764,0.037208],[1482455246764,0.036757],[1482455249764,0.036961],[1482455252764,0.036495],[1482455255764,0.037127],[1482455258764,0.036985],[1482455261764,0.037013],[1482455264764,0.036609],[1482455267764,0.037472],[1482455270764,0.036922],[1482455273764,0.036982],[1482455276764,0.037390],[1482455279764,0.037251],[1482455282764,0.037179],[1482455285764,0.036719],[1482455288764,0.037649],[1482455291764,0.037299],[1482455294764,0.036872],[1482455297764,0.036958],[1482455300764,0.036916],[1482455303764,0.037903],[1482455306764,0.036989],[1482455309764,0.036965],[1482455312764,0.037094],[1482455315764,0.037137],[1482455318764,0.037077],[1482455321764,0.036783],[1482455324764,0.036738],[1482455327764,0.036881],[1482455330764,0.037030],[1482455333764,0.037395],[1482455336764,0.037098],[1482455339764,0.036994],[1482455342764,0.037110],[1482455345764,0.037022],[1482455348764,0.037236],[1482455351764,0.037118],[1482455354764,0.036909],[1482455357764,0.036983],[1482455360764,0.036756],[1482455363764,0.036993],[1482455366764,0.037708],[1482455369764,0.037973],[1482455372764,0.037520],[1482455375764,0.036918],[1482455378764,0.037144],[1482455381764,0.037265],[1482455384764,0.036922],[1482455387764,0.037095],[1482455390764,0.037211],[1482455393764,0.036997],[1482455396764,0.037792],[1482455399764,0.036926],[1482455402764,0.037514],[1482455405764,0.037041],[1482455408764,0.037020],[1482455411764,0.037738],[1482455414764,0.036907],[1482455417764,0.037118],[1482455420764,0.037025],[1482455423764,0.036949],[1482455426764,0.036829],[1482455429764,0.037102],[1482455432764,0.037065],[1482455435764,0.037274],[1482455438764,0.037161],[1482455441764,0.037266],[1482455444764,0.037143],[1482455447764,0.037028],[1482455450764,0.037010],[1482455453764,0.037233],[1482455456764,0.037320],[1482455459764,0.037228],[1482455462764,0.037304],[1482455465764,0.037377],[1482455468764,0.036857],[1482455471764,0.036910],[1482455474764,0.037124],[1482455477764,0.037195],[1482455480764,0.037265],[1482455483764,0.036904],[1482455486764,0.037167],[1482455489764,0.037109],[1482455492764,0.036993],[1482455495764,0.037155],[1482455498764,0.037313],[1482455501764,0.037337],[1482455504764,0.037172],[1482455507764,0.037192],[1482455510764,0.037459],[1482455513764,0.036959],[1482455516764,0.037258],[1482455519764,0.037150],[1482455522764,0.037098],[1482455525764,0.037317],[1482455528764,0.037266],[1482455531764,0.037386],[1482455534764,0.037118],[1482455537764,0.037084],[1482455540764,0.037293],[1482455543764,0.036953],[1482455546764,0.037065],[1482455549764,0.037041],[1482455552764,0.037372],[1482455555764,0.037318],[1482455558764,0.037190],[1482455561764,0.037300],[1482455564764,0.037317],[1482455567764,0.037001],[1482455570764,0.037156],[1482455573764,0.037072],[1482455576764,0.036962],[1482455579764,0.037345],[1482455582764,0.037317],[1482455585764,0.037295],[1482455588764,0.037134],[1482455591764,0.037272],[1482455594764,0.037361],[1482455597764,0.037442],[1482455600764,0.037321],[1482455603764,0.037361],[1482455606764,0.037166],[1482455609764,0.036953],[1482455612764,0.037305],[1482455615764,0.037350],[1482455618764,0.037385],[1482455621764,0.037248],[1482455624764,0.037204],[1482455627764,0.037178],[1482455630764,0.037198],[1482455633764,0.037183],[1482455636764,0.037386],[1482455639764,0.037190],[1482455642764,0.037441],[1482455645764,0.037319],[1482455648764,0.037356],[1482455651764,0.037393],[1482455654764,0.037065],[1482455657764,0.037174],[1482455660764,0.037429],[1482455663764,0.037344],[1482455666764,0.037180],[1482455669764,0.037496],[1482455672764,0.037190],[1482455675764,0.037541]],\n  "min":[[1482454778764,0.036519],[1482454781764,0.036514],[1482454784764,0.036510],[1482454787764,0.036450],[1482454790764,0.036464],[1482454793764,0.036479],[1482454796764,0.036495],[1482454799764,0.036446],[1482454802764,0.036502],[1482454805764,0.036419],[1482454808764,0.036375],[1482454811764,0.036133],[1482454814764,0.036450],[1482454817764,0.036554],[1482454820764,0.036562],[1482454823764,0.036391],[1482454826764,0.036487],[1482454829764,0.036357],[1482454832764,0.036364],[1482454835764,0.036518],[1482454838764,0.036421],[1482454841764,0.036628],[1482454844764,0.036450],[1482454847764,0.036527],[1482454850764,0.036480],[1482454853764,0.036480],[1482454856764,0.036572],[1482454859764,0.036409],[1482454862764,0.036463],[1482454865764,0.036594],[1482454868764,0.036515],[1482454871764,0.036417],[1482454874764,0.036394],[1482454877764,0.036630],[1482454880764,0.036387],[1482454883764,0.036508],[1482454886764,0.036418],[1482454889764,0.036336],[1482454892764,0.036535],[1482454895764,0.036411],[1482454898764,0.036496],[1482454901764,0.036510],[1482454904764,0.036462],[1482454907764,0.036426],[1482454910764,0.036617],[1482454913764,0.036426],[1482454916764,0.036605],[1482454919764,0.036521],[1482454922764,0.036317],[1482454925764,0.036439],[1482454928764,0.036458],[1482454931764,0.036505],[1482454934764,0.036396],[1482454937764,0.036433],[1482454940764,0.036389],[1482454943764,0.036530],[1482454946764,0.036528],[1482454949764,0.036479],[1482454952764,0.036569],[1482454955764,0.036524],[1482454958764,0.036509],[1482454961764,0.036565],[1482454964764,0.036573],[1482454967764,0.036454],[1482454970764,0.036486],[1482454973764,0.036441],[1482454976764,0.036498],[1482454979764,0.036498],[1482454982764,0.036397],[1482454985764,0.036528],[1482454988764,0.036605],[1482454991764,0.036419],[1482454994764,0.036437],[1482454997764,0.036439],[1482455000764,0.036386],[1482455003764,0.036591],[1482455006764,0.036580],[1482455009764,0.036547],[1482455012764,0.036539],[1482455015764,0.036402],[1482455018764,0.036439],[1482455021764,0.036590],[1482455024764,0.036615],[1482455027764,0.036561],[1482455030764,0.036619],[1482455033764,0.036452],[1482455036764,0.036531],[1482455039764,0.036593],[1482455042764,0.036453],[1482455045764,0.036556],[1482455048764,0.036584],[1482455051764,0.036483],[1482455054764,0.036537],[1482455057764,0.036551],[1482455060764,0.036689],[1482455063764,0.036357],[1482455066764,0.036576],[1482455069764,0.036494],[1482455072764,0.036327],[1482455075764,0.036695],[1482455078764,0.036541],[1482455081764,0.035953],[1482455084764,0.036691],[1482455087764,0.036582],[1482455090764,0.036602],[1482455093764,0.037155],[1482455096764,0.036581],[1482455099764,0.036592],[1482455102764,0.036620],[1482455105764,0.036911],[1482455108764,0.036479],[1482455111764,0.036090],[1482455114764,0.036417],[1482455117764,0.036740],[1482455120764,0.037153],[1482455123764,0.036619],[1482455126764,0.036850],[1482455129764,0.036606],[1482455132764,0.036965],[1482455135764,0.036830],[1482455138764,0.036968],[1482455141764,0.036628],[1482455144764,0.036742],[1482455147764,0.036578],[1482455150764,0.036899],[1482455153764,0.036541],[1482455156764,0.036810],[1482455159764,0.036733],[1482455162764,0.036659],[1482455165764,0.036370],[1482455168764,0.036685],[1482455171764,0.036421],[1482455174764,0.036866],[1482455177764,0.036916],[1482455180764,0.036340],[1482455183764,0.036678],[1482455186764,0.036928],[1482455189764,0.036719],[1482455192764,0.036621],[1482455195764,0.036306],[1482455198764,0.036783],[1482455201764,0.036656],[1482455204764,0.036651],[1482455207764,0.036785],[1482455210764,0.036725],[1482455213764,0.036755],[1482455216764,0.036773],[1482455219764,0.036690],[1482455222764,0.036804],[1482455225764,0.036740],[1482455228764,0.036855],[1482455231764,0.036903],[1482455234764,0.036717],[1482455237764,0.036770],[1482455240764,0.036460],[1482455243764,0.037208],[1482455246764,0.036712],[1482455249764,0.036916],[1482455252764,0.036495],[1482455255764,0.036551],[1482455258764,0.036985],[1482455261764,0.036891],[1482455264764,0.036609],[1482455267764,0.036736],[1482455270764,0.036763],[1482455273764,0.036916],[1482455276764,0.036938],[1482455279764,0.037024],[1482455282764,0.036317],[1482455285764,0.036719],[1482455288764,0.036780],[1482455291764,0.036947],[1482455294764,0.036872],[1482455297764,0.036958],[1482455300764,0.036764],[1482455303764,0.036632],[1482455306764,0.036934],[1482455309764,0.036801],[1482455312764,0.037015],[1482455315764,0.036309],[1482455318764,0.036913],[1482455321764,0.036503],[1482455324764,0.036738],[1482455327764,0.036540],[1482455330764,0.036803],[1482455333764,0.036860],[1482455336764,0.036921],[1482455339764,0.036899],[1482455342764,0.036776],[1482455345764,0.036622],[1482455348764,0.036992],[1482455351764,0.036845],[1482455354764,0.036909],[1482455357764,0.036974],[1482455360764,0.036115],[1482455363764,0.036973],[1482455366764,0.037487],[1482455369764,0.036842],[1482455372764,0.037167],[1482455375764,0.036918],[1482455378764,0.037032],[1482455381764,0.036941],[1482455384764,0.036922],[1482455387764,0.036911],[1482455390764,0.037073],[1482455393764,0.036831],[1482455396764,0.037246],[1482455399764,0.036867],[1482455402764,0.037044],[1482455405764,0.036776],[1482455408764,0.037020],[1482455411764,0.036395],[1482455414764,0.036907],[1482455417764,0.036629],[1482455420764,0.037025],[1482455423764,0.036949],[1482455426764,0.036829],[1482455429764,0.036860],[1482455432764,0.037065],[1482455435764,0.037040],[1482455438764,0.037025],[1482455441764,0.037083],[1482455444764,0.036817],[1482455447764,0.036764],[1482455450764,0.037010],[1482455453764,0.037233],[1482455456764,0.037212],[1482455459764,0.037093],[1482455462764,0.036991],[1482455465764,0.036813],[1482455468764,0.036857],[1482455471764,0.036910],[1482455474764,0.037066],[1482455477764,0.037168],[1482455480764,0.037001],[1482455483764,0.036904],[1482455486764,0.036999],[1482455489764,0.037016],[1482455492764,0.036993],[1482455495764,0.037155],[1482455498764,0.036928],[1482455501764,0.037313],[1482455504764,0.037172],[1482455507764,0.037192],[1482455510764,0.037061],[1482455513764,0.036959],[1482455516764,0.037197],[1482455519764,0.037143],[1482455522764,0.037098],[1482455525764,0.037165],[1482455528764,0.037266],[1482455531764,0.037317],[1482455534764,0.037118],[1482455537764,0.037084],[1482455540764,0.037164],[1482455543764,0.036953],[1482455546764,0.037065],[1482455549764,0.036993],[1482455552764,0.037263],[1482455555764,0.037298],[1482455558764,0.037190],[1482455561764,0.037108],[1482455564764,0.037152],[1482455567764,0.037001],[1482455570764,0.037156],[1482455573764,0.036926],[1482455576764,0.036962],[1482455579764,0.037220],[1482455582764,0.037026],[1482455585764,0.037295],[1482455588764,0.037031],[1482455591764,0.037206],[1482455594764,0.037066],[1482455597764,0.037316],[1482455600764,0.037137],[1482455603764,0.037229],[1482455606764,0.037166],[1482455609764,0.036953],[1482455612764,0.037305],[1482455615764,0.037141],[1482455618764,0.037103],[1482455621764,0.037163],[1482455624764,0.037037],[1482455627764,0.037172],[1482455630764,0.037198],[1482455633764,0.037183],[1482455636764,0.037386],[1482455639764,0.037190],[1482455642764,0.037395],[1482455645764,0.037247],[1482455648764,0.037311],[1482455651764,0.037164],[1482455654764,0.037065],[1482455657764,0.037174],[1482455660764,0.037212],[1482455663764,0.037273],[1482455666764,0.037180],[1482455669764,0.037158],[1482455672764,0.037190],[1482455675764,0.037541]]\n}'
_trenddata_15min_temperature_json = '{\n  "max":[[1482454779068,20.278],[1482454782068,20.278],[1482454785068,20.278],[1482454788068,20.279],[1482454791068,20.278],[1482454794068,20.276],[1482454797068,20.274],[1482454800068,20.276],[1482454803068,20.277],[1482454806068,20.278],[1482454809068,20.277],[1482454812068,20.277],[1482454815068,20.276],[1482454818068,20.277],[1482454821068,20.276],[1482454824068,20.275],[1482454827068,20.275],[1482454830068,20.277],[1482454833068,20.277],[1482454836068,20.277],[1482454839068,20.277],[1482454842068,20.276],[1482454845068,20.276],[1482454848068,20.277],[1482454851068,20.277],[1482454854068,20.278],[1482454857068,20.278],[1482454860068,20.277],[1482454863068,20.278],[1482454866068,20.276],[1482454869068,20.275],[1482454872068,20.274],[1482454875068,20.276],[1482454878068,20.276],[1482454881068,20.277],[1482454884068,20.278],[1482454887068,20.278],[1482454890068,20.277],[1482454893068,20.278],[1482454896068,20.278],[1482454899068,20.278],[1482454902068,20.278],[1482454905068,20.278],[1482454908068,20.277],[1482454911068,20.277],[1482454914068,20.277],[1482454917068,20.276],[1482454920068,20.276],[1482454923068,20.277],[1482454926068,20.279],[1482454929068,20.278],[1482454932068,20.277],[1482454935068,20.278],[1482454938068,20.279],[1482454941068,20.279],[1482454944068,20.278],[1482454947068,20.278],[1482454950068,20.278],[1482454953068,20.277],[1482454956068,20.278],[1482454959068,20.278],[1482454962068,20.279],[1482454965068,20.280],[1482454968068,20.280],[1482454971068,20.280],[1482454974068,20.279],[1482454977068,20.278],[1482454980068,20.278],[1482454983068,20.278],[1482454986068,20.278],[1482454989068,20.278],[1482454992068,20.277],[1482454995068,20.276],[1482454998068,20.276],[1482455001068,20.276],[1482455004068,20.280],[1482455007068,20.281],[1482455010068,20.282],[1482455013068,20.281],[1482455016068,20.279],[1482455019068,20.277],[1482455022068,20.279],[1482455025068,20.280],[1482455028068,20.282],[1482455031068,20.282],[1482455034068,20.280],[1482455037068,20.281],[1482455040068,20.282],[1482455043068,20.282],[1482455046068,20.282],[1482455049068,20.281],[1482455052068,20.281],[1482455055068,20.281],[1482455058068,20.281],[1482455061068,20.281],[1482455064068,20.282],[1482455067068,20.283],[1482455070068,20.283],[1482455073068,20.282],[1482455076068,20.283],[1482455079068,20.283],[1482455082068,20.284],[1482455085068,20.284],[1482455088068,20.282],[1482455091068,20.281],[1482455094068,20.281],[1482455097068,20.282],[1482455100068,20.283],[1482455103068,20.284],[1482455106068,20.285],[1482455109068,20.286],[1482455112068,20.286],[1482455115068,20.285],[1482455118068,20.283],[1482455121068,20.283],[1482455124068,20.283],[1482455127068,20.284],[1482455130068,20.286],[1482455133068,20.288],[1482455136068,20.288],[1482455139068,20.289],[1482455142068,20.288],[1482455145068,20.287],[1482455148068,20.286],[1482455151068,20.286],[1482455154068,20.286],[1482455157068,20.287],[1482455160068,20.287],[1482455163068,20.286],[1482455166068,20.286],[1482455169068,20.286],[1482455172068,20.287],[1482455175068,20.288],[1482455178068,20.288],[1482455181068,20.289],[1482455184068,20.289],[1482455187068,20.291],[1482455190068,20.290],[1482455193068,20.288],[1482455196068,20.286],[1482455199068,20.286],[1482455202068,20.286],[1482455205068,20.288],[1482455208068,20.288],[1482455211068,20.288],[1482455214068,20.288],[1482455217068,20.287],[1482455220068,20.286],[1482455223068,20.288],[1482455226068,20.289],[1482455229068,20.290],[1482455232068,20.289],[1482455235068,20.288],[1482455238068,20.287],[1482455241068,20.289],[1482455244068,20.290],[1482455247068,20.290],[1482455250068,20.289],[1482455253068,20.289],[1482455256068,20.288],[1482455259068,20.291],[1482455262068,20.291],[1482455265068,20.292],[1482455268068,20.291],[1482455271068,20.291],[1482455274068,20.290],[1482455277068,20.290],[1482455280068,20.290],[1482455283068,20.291],[1482455286068,20.291],[1482455289068,20.291],[1482455292068,20.292],[1482455295068,20.293],[1482455298068,20.293],[1482455301068,20.294],[1482455304068,20.294],[1482455307068,20.293],[1482455310068,20.293],[1482455313068,20.293],[1482455316068,20.293],[1482455319068,20.293],[1482455322068,20.294],[1482455325068,20.293],[1482455328068,20.294],[1482455331068,20.294],[1482455334068,20.294],[1482455337068,20.294],[1482455340068,20.295],[1482455343068,20.296],[1482455346068,20.296],[1482455349068,20.297],[1482455352068,20.296],[1482455355068,20.294],[1482455358068,20.294],[1482455361068,20.294],[1482455364068,20.296],[1482455367068,20.297],[1482455370068,20.297],[1482455373068,20.297],[1482455376068,20.297],[1482455379068,20.298],[1482455382068,20.299],[1482455385068,20.299],[1482455388068,20.297],[1482455391068,20.298],[1482455394068,20.299],[1482455397068,20.300],[1482455400068,20.300],[1482455403068,20.300],[1482455406068,20.299],[1482455409068,20.299],[1482455412068,20.297],[1482455415068,20.297],[1482455418068,20.296],[1482455421068,20.297],[1482455424068,20.298],[1482455427068,20.299],[1482455430068,20.300],[1482455433068,20.302],[1482455436068,20.302],[1482455439068,20.301],[1482455442068,20.300],[1482455445068,20.300],[1482455448068,20.300],[1482455451068,20.302],[1482455454068,20.302],[1482455457068,20.302],[1482455460068,20.302],[1482455463068,20.300],[1482455466068,20.298],[1482455469068,20.299],[1482455472068,20.299],[1482455475068,20.302],[1482455478068,20.303],[1482455481068,20.301],[1482455484068,20.301],[1482455487068,20.301],[1482455490068,20.301],[1482455493068,20.303],[1482455496068,20.304],[1482455499068,20.304],[1482455502068,20.304],[1482455505068,20.304],[1482455508068,20.304],[1482455511068,20.304],[1482455514068,20.302],[1482455517068,20.302],[1482455520068,20.303],[1482455523068,20.303],[1482455526068,20.305],[1482455529068,20.305],[1482455532068,20.305],[1482455535068,20.305],[1482455538068,20.304],[1482455541068,20.305],[1482455544068,20.304],[1482455547068,20.304],[1482455550068,20.303],[1482455553068,20.305],[1482455556068,20.306],[1482455559068,20.308],[1482455562068,20.308],[1482455565068,20.306],[1482455568068,20.306],[1482455571068,20.305],[1482455574068,20.305],[1482455577068,20.304],[1482455580068,20.304],[1482455583068,20.306],[1482455586068,20.308],[1482455589068,20.308],[1482455592068,20.307],[1482455595068,20.306],[1482455598068,20.306],[1482455601068,20.306],[1482455604068,20.306],[1482455607068,20.306],[1482455610068,20.305],[1482455613068,20.306],[1482455616068,20.307],[1482455619068,20.306],[1482455622068,20.306],[1482455625068,20.306],[1482455628068,20.307],[1482455631068,20.308],[1482455634068,20.309],[1482455637068,20.307],[1482455640068,20.310],[1482455643068,20.309],[1482455646068,20.311],[1482455649068,20.311],[1482455652068,20.310],[1482455655068,20.310],[1482455658068,20.310],[1482455661068,20.311],[1482455664068,20.310],[1482455667068,20.309],[1482455670068,20.309],[1482455673068,20.310],[1482455676068,20.310]],\n  "last":[[1482454779068,20.278],[1482454782068,20.277],[1482454785068,20.278],[1482454788068,20.278],[1482454791068,20.278],[1482454794068,20.276],[1482454797068,20.274],[1482454800068,20.275],[1482454803068,20.276],[1482454806068,20.278],[1482454809068,20.277],[1482454812068,20.276],[1482454815068,20.276],[1482454818068,20.276],[1482454821068,20.275],[1482454824068,20.274],[1482454827068,20.274],[1482454830068,20.276],[1482454833068,20.276],[1482454836068,20.277],[1482454839068,20.277],[1482454842068,20.276],[1482454845068,20.276],[1482454848068,20.277],[1482454851068,20.276],[1482454854068,20.277],[1482454857068,20.278],[1482454860068,20.277],[1482454863068,20.278],[1482454866068,20.276],[1482454869068,20.275],[1482454872068,20.274],[1482454875068,20.274],[1482454878068,20.276],[1482454881068,20.275],[1482454884068,20.277],[1482454887068,20.277],[1482454890068,20.277],[1482454893068,20.278],[1482454896068,20.278],[1482454899068,20.278],[1482454902068,20.278],[1482454905068,20.278],[1482454908068,20.277],[1482454911068,20.277],[1482454914068,20.277],[1482454917068,20.276],[1482454920068,20.276],[1482454923068,20.277],[1482454926068,20.279],[1482454929068,20.278],[1482454932068,20.277],[1482454935068,20.277],[1482454938068,20.278],[1482454941068,20.279],[1482454944068,20.278],[1482454947068,20.277],[1482454950068,20.278],[1482454953068,20.276],[1482454956068,20.278],[1482454959068,20.277],[1482454962068,20.278],[1482454965068,20.279],[1482454968068,20.280],[1482454971068,20.280],[1482454974068,20.279],[1482454977068,20.278],[1482454980068,20.278],[1482454983068,20.278],[1482454986068,20.278],[1482454989068,20.278],[1482454992068,20.277],[1482454995068,20.276],[1482454998068,20.276],[1482455001068,20.276],[1482455004068,20.278],[1482455007068,20.281],[1482455010068,20.281],[1482455013068,20.281],[1482455016068,20.279],[1482455019068,20.277],[1482455022068,20.277],[1482455025068,20.279],[1482455028068,20.281],[1482455031068,20.282],[1482455034068,20.280],[1482455037068,20.281],[1482455040068,20.281],[1482455043068,20.281],[1482455046068,20.282],[1482455049068,20.281],[1482455052068,20.281],[1482455055068,20.280],[1482455058068,20.280],[1482455061068,20.280],[1482455064068,20.282],[1482455067068,20.282],[1482455070068,20.283],[1482455073068,20.282],[1482455076068,20.282],[1482455079068,20.282],[1482455082068,20.282],[1482455085068,20.284],[1482455088068,20.282],[1482455091068,20.281],[1482455094068,20.281],[1482455097068,20.281],[1482455100068,20.283],[1482455103068,20.283],[1482455106068,20.284],[1482455109068,20.285],[1482455112068,20.286],[1482455115068,20.285],[1482455118068,20.283],[1482455121068,20.283],[1482455124068,20.282],[1482455127068,20.283],[1482455130068,20.285],[1482455133068,20.287],[1482455136068,20.288],[1482455139068,20.288],[1482455142068,20.287],[1482455145068,20.287],[1482455148068,20.286],[1482455151068,20.285],[1482455154068,20.286],[1482455157068,20.285],[1482455160068,20.286],[1482455163068,20.286],[1482455166068,20.286],[1482455169068,20.286],[1482455172068,20.285],[1482455175068,20.287],[1482455178068,20.288],[1482455181068,20.289],[1482455184068,20.289],[1482455187068,20.290],[1482455190068,20.290],[1482455193068,20.288],[1482455196068,20.286],[1482455199068,20.285],[1482455202068,20.286],[1482455205068,20.287],[1482455208068,20.288],[1482455211068,20.288],[1482455214068,20.287],[1482455217068,20.287],[1482455220068,20.286],[1482455223068,20.287],[1482455226068,20.288],[1482455229068,20.289],[1482455232068,20.289],[1482455235068,20.288],[1482455238068,20.287],[1482455241068,20.287],[1482455244068,20.289],[1482455247068,20.290],[1482455250068,20.289],[1482455253068,20.289],[1482455256068,20.288],[1482455259068,20.288],[1482455262068,20.291],[1482455265068,20.291],[1482455268068,20.291],[1482455271068,20.291],[1482455274068,20.290],[1482455277068,20.290],[1482455280068,20.290],[1482455283068,20.290],[1482455286068,20.291],[1482455289068,20.290],[1482455292068,20.291],[1482455295068,20.292],[1482455298068,20.292],[1482455301068,20.293],[1482455304068,20.294],[1482455307068,20.293],[1482455310068,20.292],[1482455313068,20.292],[1482455316068,20.293],[1482455319068,20.293],[1482455322068,20.293],[1482455325068,20.293],[1482455328068,20.294],[1482455331068,20.294],[1482455334068,20.293],[1482455337068,20.294],[1482455340068,20.294],[1482455343068,20.296],[1482455346068,20.296],[1482455349068,20.297],[1482455352068,20.296],[1482455355068,20.294],[1482455358068,20.294],[1482455361068,20.294],[1482455364068,20.295],[1482455367068,20.297],[1482455370068,20.297],[1482455373068,20.296],[1482455376068,20.296],[1482455379068,20.298],[1482455382068,20.298],[1482455385068,20.299],[1482455388068,20.297],[1482455391068,20.297],[1482455394068,20.298],[1482455397068,20.298],[1482455400068,20.300],[1482455403068,20.300],[1482455406068,20.299],[1482455409068,20.299],[1482455412068,20.297],[1482455415068,20.297],[1482455418068,20.296],[1482455421068,20.296],[1482455424068,20.296],[1482455427068,20.298],[1482455430068,20.300],[1482455433068,20.301],[1482455436068,20.302],[1482455439068,20.301],[1482455442068,20.300],[1482455445068,20.299],[1482455448068,20.299],[1482455451068,20.301],[1482455454068,20.302],[1482455457068,20.302],[1482455460068,20.302],[1482455463068,20.300],[1482455466068,20.298],[1482455469068,20.297],[1482455472068,20.299],[1482455475068,20.301],[1482455478068,20.303],[1482455481068,20.301],[1482455484068,20.301],[1482455487068,20.301],[1482455490068,20.301],[1482455493068,20.302],[1482455496068,20.303],[1482455499068,20.304],[1482455502068,20.303],[1482455505068,20.303],[1482455508068,20.304],[1482455511068,20.304],[1482455514068,20.302],[1482455517068,20.302],[1482455520068,20.301],[1482455523068,20.303],[1482455526068,20.304],[1482455529068,20.305],[1482455532068,20.304],[1482455535068,20.305],[1482455538068,20.304],[1482455541068,20.305],[1482455544068,20.303],[1482455547068,20.304],[1482455550068,20.303],[1482455553068,20.304],[1482455556068,20.305],[1482455559068,20.307],[1482455562068,20.308],[1482455565068,20.306],[1482455568068,20.306],[1482455571068,20.305],[1482455574068,20.305],[1482455577068,20.304],[1482455580068,20.304],[1482455583068,20.305],[1482455586068,20.307],[1482455589068,20.308],[1482455592068,20.307],[1482455595068,20.306],[1482455598068,20.306],[1482455601068,20.305],[1482455604068,20.306],[1482455607068,20.305],[1482455610068,20.305],[1482455613068,20.305],[1482455616068,20.306],[1482455619068,20.305],[1482455622068,20.306],[1482455625068,20.304],[1482455628068,20.305],[1482455631068,20.306],[1482455634068,20.309],[1482455637068,20.307],[1482455640068,20.308],[1482455643068,20.309],[1482455646068,20.309],[1482455649068,20.311],[1482455652068,20.310],[1482455655068,20.310],[1482455658068,20.309],[1482455661068,20.311],[1482455664068,20.310],[1482455667068,20.308],[1482455670068,20.309],[1482455673068,20.310],[1482455676068,20.309]],\n  "min":[[1482454779068,20.277],[1482454782068,20.277],[1482454785068,20.277],[1482454788068,20.278],[1482454791068,20.277],[1482454794068,20.275],[1482454797068,20.273],[1482454800068,20.275],[1482454803068,20.276],[1482454806068,20.278],[1482454809068,20.276],[1482454812068,20.276],[1482454815068,20.276],[1482454818068,20.276],[1482454821068,20.275],[1482454824068,20.274],[1482454827068,20.274],[1482454830068,20.276],[1482454833068,20.276],[1482454836068,20.277],[1482454839068,20.277],[1482454842068,20.275],[1482454845068,20.276],[1482454848068,20.277],[1482454851068,20.276],[1482454854068,20.277],[1482454857068,20.277],[1482454860068,20.277],[1482454863068,20.277],[1482454866068,20.274],[1482454869068,20.274],[1482454872068,20.273],[1482454875068,20.274],[1482454878068,20.276],[1482454881068,20.275],[1482454884068,20.277],[1482454887068,20.277],[1482454890068,20.277],[1482454893068,20.277],[1482454896068,20.278],[1482454899068,20.278],[1482454902068,20.278],[1482454905068,20.277],[1482454908068,20.277],[1482454911068,20.276],[1482454914068,20.276],[1482454917068,20.276],[1482454920068,20.276],[1482454923068,20.277],[1482454926068,20.278],[1482454929068,20.277],[1482454932068,20.277],[1482454935068,20.277],[1482454938068,20.278],[1482454941068,20.279],[1482454944068,20.277],[1482454947068,20.277],[1482454950068,20.277],[1482454953068,20.276],[1482454956068,20.277],[1482454959068,20.277],[1482454962068,20.278],[1482454965068,20.279],[1482454968068,20.280],[1482454971068,20.279],[1482454974068,20.278],[1482454977068,20.277],[1482454980068,20.278],[1482454983068,20.278],[1482454986068,20.278],[1482454989068,20.277],[1482454992068,20.277],[1482454995068,20.275],[1482454998068,20.276],[1482455001068,20.276],[1482455004068,20.278],[1482455007068,20.281],[1482455010068,20.281],[1482455013068,20.279],[1482455016068,20.278],[1482455019068,20.277],[1482455022068,20.277],[1482455025068,20.279],[1482455028068,20.281],[1482455031068,20.282],[1482455034068,20.279],[1482455037068,20.280],[1482455040068,20.281],[1482455043068,20.281],[1482455046068,20.281],[1482455049068,20.281],[1482455052068,20.280],[1482455055068,20.280],[1482455058068,20.280],[1482455061068,20.280],[1482455064068,20.282],[1482455067068,20.282],[1482455070068,20.283],[1482455073068,20.282],[1482455076068,20.282],[1482455079068,20.282],[1482455082068,20.282],[1482455085068,20.283],[1482455088068,20.281],[1482455091068,20.281],[1482455094068,20.280],[1482455097068,20.281],[1482455100068,20.282],[1482455103068,20.283],[1482455106068,20.284],[1482455109068,20.285],[1482455112068,20.285],[1482455115068,20.284],[1482455118068,20.282],[1482455121068,20.283],[1482455124068,20.282],[1482455127068,20.283],[1482455130068,20.285],[1482455133068,20.287],[1482455136068,20.288],[1482455139068,20.288],[1482455142068,20.287],[1482455145068,20.286],[1482455148068,20.285],[1482455151068,20.285],[1482455154068,20.285],[1482455157068,20.285],[1482455160068,20.286],[1482455163068,20.286],[1482455166068,20.286],[1482455169068,20.285],[1482455172068,20.285],[1482455175068,20.287],[1482455178068,20.287],[1482455181068,20.289],[1482455184068,20.289],[1482455187068,20.290],[1482455190068,20.288],[1482455193068,20.287],[1482455196068,20.285],[1482455199068,20.285],[1482455202068,20.285],[1482455205068,20.287],[1482455208068,20.288],[1482455211068,20.288],[1482455214068,20.287],[1482455217068,20.286],[1482455220068,20.285],[1482455223068,20.287],[1482455226068,20.288],[1482455229068,20.289],[1482455232068,20.288],[1482455235068,20.286],[1482455238068,20.287],[1482455241068,20.287],[1482455244068,20.289],[1482455247068,20.290],[1482455250068,20.288],[1482455253068,20.288],[1482455256068,20.288],[1482455259068,20.288],[1482455262068,20.290],[1482455265068,20.291],[1482455268068,20.291],[1482455271068,20.291],[1482455274068,20.290],[1482455277068,20.290],[1482455280068,20.289],[1482455283068,20.289],[1482455286068,20.290],[1482455289068,20.290],[1482455292068,20.291],[1482455295068,20.292],[1482455298068,20.292],[1482455301068,20.293],[1482455304068,20.293],[1482455307068,20.293],[1482455310068,20.292],[1482455313068,20.292],[1482455316068,20.292],[1482455319068,20.293],[1482455322068,20.293],[1482455325068,20.293],[1482455328068,20.294],[1482455331068,20.292],[1482455334068,20.293],[1482455337068,20.294],[1482455340068,20.294],[1482455343068,20.296],[1482455346068,20.296],[1482455349068,20.297],[1482455352068,20.294],[1482455355068,20.294],[1482455358068,20.294],[1482455361068,20.294],[1482455364068,20.295],[1482455367068,20.297],[1482455370068,20.297],[1482455373068,20.296],[1482455376068,20.296],[1482455379068,20.298],[1482455382068,20.298],[1482455385068,20.297],[1482455388068,20.296],[1482455391068,20.297],[1482455394068,20.298],[1482455397068,20.298],[1482455400068,20.300],[1482455403068,20.299],[1482455406068,20.298],[1482455409068,20.298],[1482455412068,20.296],[1482455415068,20.296],[1482455418068,20.295],[1482455421068,20.296],[1482455424068,20.296],[1482455427068,20.298],[1482455430068,20.300],[1482455433068,20.301],[1482455436068,20.301],[1482455439068,20.300],[1482455442068,20.299],[1482455445068,20.299],[1482455448068,20.299],[1482455451068,20.301],[1482455454068,20.302],[1482455457068,20.302],[1482455460068,20.300],[1482455463068,20.298],[1482455466068,20.297],[1482455469068,20.297],[1482455472068,20.299],[1482455475068,20.301],[1482455478068,20.301],[1482455481068,20.301],[1482455484068,20.300],[1482455487068,20.300],[1482455490068,20.301],[1482455493068,20.302],[1482455496068,20.303],[1482455499068,20.303],[1482455502068,20.303],[1482455505068,20.303],[1482455508068,20.304],[1482455511068,20.303],[1482455514068,20.302],[1482455517068,20.300],[1482455520068,20.301],[1482455523068,20.303],[1482455526068,20.304],[1482455529068,20.304],[1482455532068,20.304],[1482455535068,20.304],[1482455538068,20.304],[1482455541068,20.304],[1482455544068,20.303],[1482455547068,20.304],[1482455550068,20.303],[1482455553068,20.304],[1482455556068,20.305],[1482455559068,20.307],[1482455562068,20.307],[1482455565068,20.306],[1482455568068,20.305],[1482455571068,20.305],[1482455574068,20.305],[1482455577068,20.304],[1482455580068,20.304],[1482455583068,20.305],[1482455586068,20.307],[1482455589068,20.307],[1482455592068,20.306],[1482455595068,20.306],[1482455598068,20.306],[1482455601068,20.305],[1482455604068,20.306],[1482455607068,20.305],[1482455610068,20.305],[1482455613068,20.305],[1482455616068,20.306],[1482455619068,20.305],[1482455622068,20.305],[1482455625068,20.304],[1482455628068,20.305],[1482455631068,20.306],[1482455634068,20.308],[1482455637068,20.307],[1482455640068,20.308],[1482455643068,20.309],[1482455646068,20.309],[1482455649068,20.311],[1482455652068,20.309],[1482455655068,20.310],[1482455658068,20.309],[1482455661068,20.310],[1482455664068,20.309],[1482455667068,20.308],[1482455670068,20.309],[1482455673068,20.309],[1482455676068,20.309]]\n}'
_trenddata_2hr_agitation_json = '{\n  "max":[[1482448500469,36.023],[1482448524469,36.014],[1482448548469,36.028],[1482448572469,29.281],[1482448596469,15.305],[1482448620469,12.419],[1482448644469,12.059],[1482448668469,12.059],[1482448692469,12.060],[1482448716469,12.060],[1482448740469,12.062],[1482448764469,12.060],[1482448788469,12.062],[1482448812469,12.060],[1482448836469,12.060],[1482448860469,12.061],[1482448884469,12.057],[1482448908469,12.060],[1482448932469,12.059],[1482448956469,12.060],[1482448980469,12.060],[1482449004469,12.060],[1482449028469,12.059],[1482449052469,12.058],[1482449076469,12.059],[1482449100469,12.058],[1482449124469,12.059],[1482449148469,12.059],[1482449172469,12.057],[1482449196469,12.061],[1482449220469,12.061],[1482449244469,12.059],[1482449268469,12.060],[1482449292469,12.059],[1482449316469,12.059],[1482449340469,12.060],[1482449364469,12.060],[1482449388469,12.059],[1482449412469,12.060],[1482449436469,12.060],[1482449460469,12.058],[1482449484469,12.059],[1482449508469,12.061],[1482449532469,12.059],[1482449556469,12.059],[1482449580469,12.059],[1482449604469,12.061],[1482449628469,12.057],[1482449652469,12.058],[1482449676469,12.061],[1482449700469,12.059],[1482449724469,12.059],[1482449748469,12.059],[1482449772469,12.062],[1482449796469,12.062],[1482449820469,12.059],[1482449844469,12.059],[1482449868469,12.060],[1482449892469,12.058],[1482449916469,12.058],[1482449940469,12.060],[1482449964469,12.058],[1482449988469,12.061],[1482450012469,12.059],[1482450036469,12.059],[1482450060469,12.059],[1482450084469,12.058],[1482450108469,12.058],[1482450132469,12.060],[1482450156469,12.059],[1482450180469,12.059],[1482450204469,12.059],[1482450228469,12.060],[1482450252469,12.060],[1482450276469,12.058],[1482450300469,12.058],[1482450324469,12.062],[1482450348469,12.057],[1482450372469,12.059],[1482450396469,12.059],[1482450420469,12.060],[1482450444469,12.059],[1482450468469,12.062],[1482450492469,12.060],[1482450516469,12.057],[1482450540469,12.059],[1482450564469,12.060],[1482450588469,12.058],[1482450612469,12.059],[1482450636469,12.060],[1482450660469,12.063],[1482450684469,12.062],[1482450708469,12.056],[1482450732469,12.057],[1482450756469,12.057],[1482450780469,12.060],[1482450804469,12.059],[1482450828469,12.058],[1482450852469,12.057],[1482450876469,12.058],[1482450900469,12.058],[1482450924469,12.058],[1482450948469,12.056],[1482450972469,12.058],[1482450996469,12.058],[1482451020469,12.058],[1482451044469,12.058],[1482451068469,12.060],[1482451092469,12.058],[1482451116469,12.057],[1482451140469,12.056],[1482451164469,12.059],[1482451188469,12.059],[1482451212469,12.059],[1482451236469,12.059],[1482451260469,12.059],[1482451284469,12.059],[1482451308469,12.057],[1482451332469,12.058],[1482451356469,12.057],[1482451380469,12.059],[1482451404469,12.060],[1482451428469,12.057],[1482451452469,12.058],[1482451476469,12.057],[1482451500469,12.059],[1482451524469,12.057],[1482451548469,12.058],[1482451572469,12.059],[1482451596469,12.059],[1482451620469,12.060],[1482451644469,12.058],[1482451668469,12.060],[1482451692469,12.058],[1482451716469,12.059],[1482451740469,12.059],[1482451764469,12.058],[1482451788469,12.057],[1482451812469,12.058],[1482451836469,12.059],[1482451860469,12.059],[1482451884469,12.060],[1482451908469,12.058],[1482451932469,12.057],[1482451956469,12.060],[1482451980469,12.060],[1482452004469,12.060],[1482452028469,12.063],[1482452052469,12.058],[1482452076469,12.059],[1482452100469,12.059],[1482452124469,12.059],[1482452148469,12.061],[1482452172469,12.059],[1482452196469,12.059],[1482452220469,12.061],[1482452244469,12.060],[1482452268469,12.063],[1482452292469,12.060],[1482452316469,12.060],[1482452340469,12.061],[1482452364469,12.055],[1482452388469,12.056],[1482452412469,12.059],[1482452436469,12.061],[1482452460469,12.061],[1482452484469,12.057],[1482452508469,12.059],[1482452532469,12.058],[1482452556469,12.059],[1482452580469,12.060],[1482452604469,12.058],[1482452628469,12.059],[1482452652469,12.058],[1482452676469,12.057],[1482452700469,12.059],[1482452724469,12.059],[1482452748469,12.058],[1482452772469,12.057],[1482452796469,12.057],[1482452820469,12.059],[1482452844469,12.058],[1482452868469,12.060],[1482452892469,12.056],[1482452916469,12.062],[1482452940469,12.058],[1482452964469,12.057],[1482452988469,12.058],[1482453012469,12.056],[1482453036469,12.059],[1482453060469,12.059],[1482453084469,12.058],[1482453108469,12.058],[1482453132469,12.058],[1482453156469,12.058],[1482453180469,12.059],[1482453204469,12.057],[1482453228469,12.061],[1482453252469,12.058],[1482453276469,12.058],[1482453300469,12.058],[1482453324469,12.057],[1482453348469,12.058],[1482453372469,12.059],[1482453396469,4.4177],[1482453420469,0.0000],[1482453444469,0.0000],[1482453468469,0.0000],[1482453492469,0.0000],[1482453516469,0.0000],[1482453540469,0.0000],[1482453564469,0.0000],[1482453588469,0.0000],[1482453612469,0.0000],[1482453636469,0.0000],[1482453660469,0.0000],[1482453684469,0.0000],[1482453708469,0.0000],[1482453732469,0.0000],[1482453756469,0.0000],[1482453780469,0.0000],[1482453804469,0.0000],[1482453828469,0.0000],[1482453852469,0.0000],[1482453876469,0.0000],[1482453900469,0.0000],[1482453924469,0.0000],[1482453948469,0.0000],[1482453972469,0.0000],[1482453996469,0.0000],[1482454020469,0.0000],[1482454044469,0.0000],[1482454068469,0.0000],[1482454092469,0.0000],[1482454116469,5.6161],[1482454140469,8.4473],[1482454164469,8.2551],[1482454188469,8.0707],[1482454212469,8.0729],[1482454236469,8.0715],[1482454260469,8.0742],[1482454284469,0.0000],[1482454308469,0.0000],[1482454332469,10.924],[1482454356469,12.056],[1482454380469,12.058],[1482454404469,12.056],[1482454428469,12.058],[1482454452469,12.057],[1482454476469,12.056],[1482454500469,0.0000],[1482454524469,3.6576],[1482454548469,18.175],[1482454572469,19.668],[1482454596469,20.039],[1482454620469,20.040],[1482454644469,20.042],[1482454668469,20.044],[1482454692469,22.355],[1482454716469,0.0000],[1482454740469,0.0000],[1482454764469,29.426],[1482454788469,34.661],[1482454812469,35.648],[1482454836469,36.030],[1482454860469,36.025],[1482454884469,36.030],[1482454908469,36.042],[1482454932469,36.029],[1482454956469,36.036],[1482454980469,36.026],[1482455004469,36.032],[1482455028469,36.039],[1482455052469,36.030],[1482455076469,36.019],[1482455100469,36.027],[1482455124469,36.021],[1482455148469,36.041],[1482455172469,36.040],[1482455196469,36.037],[1482455220469,36.036],[1482455244469,36.030],[1482455268469,36.048],[1482455292469,36.027],[1482455316469,36.026],[1482455340469,36.035],[1482455364469,36.040],[1482455388469,36.035],[1482455412469,36.038],[1482455436469,36.030],[1482455460469,36.032],[1482455484469,36.037],[1482455508469,36.035],[1482455532469,36.036],[1482455556469,36.035],[1482455580469,36.048],[1482455604469,36.035],[1482455628469,36.023],[1482455652469,36.031],[1482455676469,36.031]],\n  "last":[[1482448500469,36.012],[1482448524469,35.990],[1482448548469,35.984],[1482448572469,29.281],[1482448596469,15.305],[1482448620469,12.419],[1482448644469,12.046],[1482448668469,12.045],[1482448692469,12.047],[1482448716469,12.060],[1482448740469,11.908],[1482448764469,11.937],[1482448788469,12.044],[1482448812469,12.048],[1482448836469,12.060],[1482448860469,11.843],[1482448884469,12.057],[1482448908469,12.048],[1482448932469,12.049],[1482448956469,11.996],[1482448980469,11.851],[1482449004469,11.923],[1482449028469,12.047],[1482449052469,12.049],[1482449076469,11.906],[1482449100469,12.058],[1482449124469,12.059],[1482449148469,12.047],[1482449172469,11.886],[1482449196469,12.057],[1482449220469,12.061],[1482449244469,12.059],[1482449268469,12.045],[1482449292469,11.833],[1482449316469,11.967],[1482449340469,12.060],[1482449364469,12.047],[1482449388469,11.856],[1482449412469,11.861],[1482449436469,12.060],[1482449460469,12.055],[1482449484469,12.059],[1482449508469,11.811],[1482449532469,12.049],[1482449556469,12.045],[1482449580469,12.059],[1482449604469,11.940],[1482449628469,11.978],[1482449652469,12.046],[1482449676469,12.058],[1482449700469,12.059],[1482449724469,11.837],[1482449748469,12.048],[1482449772469,12.043],[1482449796469,11.911],[1482449820469,11.899],[1482449844469,12.055],[1482449868469,12.047],[1482449892469,12.049],[1482449916469,12.058],[1482449940469,11.835],[1482449964469,12.056],[1482449988469,12.049],[1482450012469,12.048],[1482450036469,11.843],[1482450060469,11.988],[1482450084469,12.058],[1482450108469,12.046],[1482450132469,12.051],[1482450156469,11.828],[1482450180469,12.059],[1482450204469,12.057],[1482450228469,12.049],[1482450252469,11.918],[1482450276469,12.027],[1482450300469,12.058],[1482450324469,12.059],[1482450348469,12.050],[1482450372469,12.047],[1482450396469,11.898],[1482450420469,12.003],[1482450444469,12.046],[1482450468469,12.048],[1482450492469,12.047],[1482450516469,11.908],[1482450540469,11.983],[1482450564469,12.058],[1482450588469,12.046],[1482450612469,12.047],[1482450636469,11.926],[1482450660469,12.059],[1482450684469,12.052],[1482450708469,12.000],[1482450732469,11.884],[1482450756469,12.056],[1482450780469,12.057],[1482450804469,12.058],[1482450828469,11.815],[1482450852469,12.046],[1482450876469,12.055],[1482450900469,12.056],[1482450924469,12.035],[1482450948469,11.892],[1482450972469,11.834],[1482450996469,12.056],[1482451020469,12.058],[1482451044469,12.057],[1482451068469,11.854],[1482451092469,11.938],[1482451116469,12.056],[1482451140469,12.055],[1482451164469,12.057],[1482451188469,11.849],[1482451212469,12.050],[1482451236469,12.056],[1482451260469,11.896],[1482451284469,12.014],[1482451308469,12.046],[1482451332469,12.047],[1482451356469,12.057],[1482451380469,11.864],[1482451404469,11.917],[1482451428469,12.049],[1482451452469,12.046],[1482451476469,12.057],[1482451500469,11.908],[1482451524469,12.003],[1482451548469,12.046],[1482451572469,12.048],[1482451596469,12.059],[1482451620469,12.057],[1482451644469,12.056],[1482451668469,11.813],[1482451692469,12.048],[1482451716469,12.057],[1482451740469,12.058],[1482451764469,11.941],[1482451788469,11.899],[1482451812469,12.045],[1482451836469,12.058],[1482451860469,12.059],[1482451884469,11.845],[1482451908469,12.048],[1482451932469,12.047],[1482451956469,12.049],[1482451980469,12.058],[1482452004469,11.858],[1482452028469,11.858],[1482452052469,12.046],[1482452076469,12.058],[1482452100469,12.056],[1482452124469,11.820],[1482452148469,12.050],[1482452172469,12.050],[1482452196469,12.059],[1482452220469,11.885],[1482452244469,12.040],[1482452268469,12.047],[1482452292469,12.049],[1482452316469,11.853],[1482452340469,11.910],[1482452364469,12.047],[1482452388469,12.045],[1482452412469,12.009],[1482452436469,11.838],[1482452460469,12.061],[1482452484469,12.057],[1482452508469,12.046],[1482452532469,12.045],[1482452556469,11.840],[1482452580469,12.057],[1482452604469,12.047],[1482452628469,12.045],[1482452652469,11.827],[1482452676469,12.057],[1482452700469,12.057],[1482452724469,12.059],[1482452748469,11.995],[1482452772469,11.885],[1482452796469,12.057],[1482452820469,12.057],[1482452844469,11.954],[1482452868469,11.828],[1482452892469,12.048],[1482452916469,12.055],[1482452940469,12.058],[1482452964469,11.988],[1482452988469,11.993],[1482453012469,12.047],[1482453036469,12.049],[1482453060469,11.904],[1482453084469,11.896],[1482453108469,12.048],[1482453132469,12.046],[1482453156469,12.057],[1482453180469,12.055],[1482453204469,12.046],[1482453228469,11.959],[1482453252469,11.997],[1482453276469,12.058],[1482453300469,12.057],[1482453324469,11.838],[1482453348469,12.048],[1482453372469,12.047],[1482453396469,4.4177],[1482453420469,0.0000],[1482453444469,0.0000],[1482453468469,0.0000],[1482453492469,0.0000],[1482453516469,0.0000],[1482453540469,0.0000],[1482453564469,0.0000],[1482453588469,0.0000],[1482453612469,0.0000],[1482453636469,0.0000],[1482453660469,0.0000],[1482453684469,0.0000],[1482453708469,0.0000],[1482453732469,0.0000],[1482453756469,0.0000],[1482453780469,0.0000],[1482453804469,0.0000],[1482453828469,0.0000],[1482453852469,0.0000],[1482453876469,0.0000],[1482453900469,0.0000],[1482453924469,0.0000],[1482453948469,0.0000],[1482453972469,0.0000],[1482453996469,0.0000],[1482454020469,0.0000],[1482454044469,0.0000],[1482454068469,0.0000],[1482454092469,0.0000],[1482454116469,0.0000],[1482454140469,5.6161],[1482454164469,8.2551],[1482454188469,8.0702],[1482454212469,7.8104],[1482454236469,8.0649],[1482454260469,7.8077],[1482454284469,0.0000],[1482454308469,0.0000],[1482454332469,0.0000],[1482454356469,10.924],[1482454380469,12.044],[1482454404469,12.052],[1482454428469,12.058],[1482454452469,12.046],[1482454476469,12.044],[1482454500469,0.0000],[1482454524469,0.0000],[1482454548469,3.6576],[1482454572469,18.175],[1482454596469,19.655],[1482454620469,20.014],[1482454644469,20.040],[1482454668469,19.981],[1482454692469,20.044],[1482454716469,0.0000],[1482454740469,0.0000],[1482454764469,0.0000],[1482454788469,29.851],[1482454812469,34.925],[1482454836469,35.670],[1482454860469,35.965],[1482454884469,36.007],[1482454908469,36.042],[1482454932469,35.989],[1482454956469,36.024],[1482454980469,35.986],[1482455004469,35.989],[1482455028469,36.012],[1482455052469,36.009],[1482455076469,36.004],[1482455100469,35.965],[1482455124469,36.017],[1482455148469,35.985],[1482455172469,35.976],[1482455196469,36.014],[1482455220469,36.011],[1482455244469,35.975],[1482455268469,35.998],[1482455292469,35.997],[1482455316469,36.012],[1482455340469,35.988],[1482455364469,35.981],[1482455388469,35.990],[1482455412469,36.004],[1482455436469,36.011],[1482455460469,36.022],[1482455484469,35.988],[1482455508469,36.015],[1482455532469,35.998],[1482455556469,36.018],[1482455580469,35.977],[1482455604469,36.018],[1482455628469,36.006],[1482455652469,36.010],[1482455676469,35.996]],\n  "min":[[1482448500469,35.958],[1482448524469,35.953],[1482448548469,29.281],[1482448572469,15.305],[1482448596469,12.432],[1482448620469,12.042],[1482448644469,12.045],[1482448668469,12.045],[1482448692469,11.854],[1482448716469,11.806],[1482448740469,11.908],[1482448764469,11.832],[1482448788469,11.850],[1482448812469,11.807],[1482448836469,11.843],[1482448860469,11.831],[1482448884469,11.838],[1482448908469,11.842],[1482448932469,11.849],[1482448956469,11.842],[1482448980469,11.851],[1482449004469,11.830],[1482449028469,11.815],[1482449052469,11.818],[1482449076469,11.906],[1482449100469,11.827],[1482449124469,11.839],[1482449148469,11.831],[1482449172469,11.886],[1482449196469,11.873],[1482449220469,11.837],[1482449244469,11.801],[1482449268469,11.833],[1482449292469,11.833],[1482449316469,11.809],[1482449340469,11.842],[1482449364469,11.856],[1482449388469,11.856],[1482449412469,11.842],[1482449436469,11.818],[1482449460469,11.858],[1482449484469,11.811],[1482449508469,11.811],[1482449532469,11.819],[1482449556469,11.808],[1482449580469,11.940],[1482449604469,11.873],[1482449628469,11.821],[1482449652469,11.880],[1482449676469,11.811],[1482449700469,11.889],[1482449724469,11.837],[1482449748469,11.803],[1482449772469,11.823],[1482449796469,11.899],[1482449820469,11.835],[1482449844469,11.859],[1482449868469,11.866],[1482449892469,11.828],[1482449916469,11.855],[1482449940469,11.835],[1482449964469,11.840],[1482449988469,11.810],[1482450012469,11.843],[1482450036469,11.843],[1482450060469,11.843],[1482450084469,11.849],[1482450108469,11.840],[1482450132469,11.828],[1482450156469,11.821],[1482450180469,11.831],[1482450204469,11.842],[1482450228469,11.838],[1482450252469,11.918],[1482450276469,11.850],[1482450300469,11.859],[1482450324469,11.870],[1482450348469,11.839],[1482450372469,11.850],[1482450396469,11.898],[1482450420469,11.840],[1482450444469,11.850],[1482450468469,11.864],[1482450492469,11.831],[1482450516469,11.908],[1482450540469,11.814],[1482450564469,11.858],[1482450588469,11.818],[1482450612469,11.926],[1482450636469,11.832],[1482450660469,11.822],[1482450684469,11.827],[1482450708469,12.000],[1482450732469,11.863],[1482450756469,11.820],[1482450780469,11.855],[1482450804469,11.810],[1482450828469,11.815],[1482450852469,11.862],[1482450876469,11.830],[1482450900469,11.883],[1482450924469,11.873],[1482450948469,11.892],[1482450972469,11.834],[1482450996469,11.816],[1482451020469,11.828],[1482451044469,11.854],[1482451068469,11.854],[1482451092469,11.840],[1482451116469,11.831],[1482451140469,11.831],[1482451164469,11.849],[1482451188469,11.823],[1482451212469,11.808],[1482451236469,11.846],[1482451260469,11.896],[1482451284469,11.827],[1482451308469,11.838],[1482451332469,11.821],[1482451356469,11.864],[1482451380469,11.864],[1482451404469,11.820],[1482451428469,11.872],[1482451452469,11.851],[1482451476469,11.837],[1482451500469,11.908],[1482451524469,11.849],[1482451548469,11.859],[1482451572469,11.842],[1482451596469,11.875],[1482451620469,11.857],[1482451644469,11.813],[1482451668469,11.813],[1482451692469,11.864],[1482451716469,11.828],[1482451740469,11.824],[1482451764469,11.899],[1482451788469,11.841],[1482451812469,11.836],[1482451836469,11.810],[1482451860469,11.913],[1482451884469,11.842],[1482451908469,11.838],[1482451932469,11.873],[1482451956469,11.833],[1482451980469,11.858],[1482452004469,11.858],[1482452028469,11.820],[1482452052469,11.853],[1482452076469,11.825],[1482452100469,11.822],[1482452124469,11.820],[1482452148469,11.829],[1482452172469,11.841],[1482452196469,11.807],[1482452220469,11.885],[1482452244469,11.838],[1482452268469,11.815],[1482452292469,11.842],[1482452316469,11.853],[1482452340469,11.855],[1482452364469,11.801],[1482452388469,11.856],[1482452412469,11.838],[1482452436469,11.838],[1482452460469,11.825],[1482452484469,11.854],[1482452508469,11.854],[1482452532469,11.840],[1482452556469,11.840],[1482452580469,11.791],[1482452604469,11.837],[1482452628469,11.865],[1482452652469,11.827],[1482452676469,11.851],[1482452700469,11.832],[1482452724469,11.828],[1482452748469,11.995],[1482452772469,11.852],[1482452796469,11.822],[1482452820469,11.851],[1482452844469,11.828],[1482452868469,11.828],[1482452892469,11.839],[1482452916469,11.829],[1482452940469,11.840],[1482452964469,11.988],[1482452988469,11.827],[1482453012469,11.835],[1482453036469,11.853],[1482453060469,11.896],[1482453084469,11.851],[1482453108469,11.831],[1482453132469,11.856],[1482453156469,11.882],[1482453180469,11.842],[1482453204469,11.959],[1482453228469,11.867],[1482453252469,11.827],[1482453276469,11.840],[1482453300469,11.838],[1482453324469,11.838],[1482453348469,11.821],[1482453372469,5.1833],[1482453396469,0.0000],[1482453420469,0.0000],[1482453444469,0.0000],[1482453468469,0.0000],[1482453492469,0.0000],[1482453516469,0.0000],[1482453540469,0.0000],[1482453564469,0.0000],[1482453588469,0.0000],[1482453612469,0.0000],[1482453636469,0.0000],[1482453660469,0.0000],[1482453684469,0.0000],[1482453708469,0.0000],[1482453732469,0.0000],[1482453756469,0.0000],[1482453780469,0.0000],[1482453804469,0.0000],[1482453828469,0.0000],[1482453852469,0.0000],[1482453876469,0.0000],[1482453900469,0.0000],[1482453924469,0.0000],[1482453948469,0.0000],[1482453972469,0.0000],[1482453996469,0.0000],[1482454020469,0.0000],[1482454044469,0.0000],[1482454068469,0.0000],[1482454092469,0.0000],[1482454116469,0.0000],[1482454140469,5.6161],[1482454164469,8.0657],[1482454188469,7.9202],[1482454212469,7.8104],[1482454236469,7.8077],[1482454260469,0.0000],[1482454284469,0.0000],[1482454308469,0.0000],[1482454332469,0.0000],[1482454356469,10.924],[1482454380469,12.041],[1482454404469,11.855],[1482454428469,11.876],[1482454452469,11.826],[1482454476469,0.0000],[1482454500469,0.0000],[1482454524469,0.0000],[1482454548469,3.6576],[1482454572469,18.175],[1482454596469,19.655],[1482454620469,20.010],[1482454644469,19.901],[1482454668469,19.869],[1482454692469,0.0000],[1482454716469,0.0000],[1482454740469,0.0000],[1482454764469,0.0000],[1482454788469,29.851],[1482454812469,34.891],[1482454836469,35.670],[1482454860469,35.951],[1482454884469,35.960],[1482454908469,35.959],[1482454932469,35.951],[1482454956469,35.967],[1482454980469,35.974],[1482455004469,35.962],[1482455028469,35.966],[1482455052469,35.972],[1482455076469,35.969],[1482455100469,35.964],[1482455124469,35.969],[1482455148469,35.965],[1482455172469,35.971],[1482455196469,35.968],[1482455220469,35.957],[1482455244469,35.975],[1482455268469,35.957],[1482455292469,35.969],[1482455316469,35.969],[1482455340469,35.969],[1482455364469,35.972],[1482455388469,35.981],[1482455412469,35.967],[1482455436469,35.970],[1482455460469,35.960],[1482455484469,35.973],[1482455508469,35.956],[1482455532469,35.969],[1482455556469,35.965],[1482455580469,35.977],[1482455604469,35.984],[1482455628469,35.977],[1482455652469,35.956],[1482455676469,35.988]]\n}'
_trenddata_2hr_do_json = '{\n  "max":[[1482448500772,-0.24413],[1482448524772,-0.24413],[1482448548772,-0.24430],[1482448572772,-0.24474],[1482448596772,-0.24569],[1482448620772,-0.24502],[1482448644772,-0.24567],[1482448668772,-0.24456],[1482448692772,-0.24512],[1482448716772,-0.24355],[1482448740772,-0.24458],[1482448764772,-0.24471],[1482448788772,-0.24452],[1482448812772,-0.24571],[1482448836772,-0.24498],[1482448860772,-0.24502],[1482448884772,-0.24441],[1482448908772,-0.24495],[1482448932772,-0.24359],[1482448956772,-0.24496],[1482448980772,-0.24469],[1482449004772,-0.24377],[1482449028772,-0.24438],[1482449052772,-0.24445],[1482449076772,-0.24462],[1482449100772,-0.24505],[1482449124772,-0.24502],[1482449148772,-0.24305],[1482449172772,-0.24456],[1482449196772,-0.24314],[1482449220772,-0.24536],[1482449244772,-0.24451],[1482449268772,-0.24427],[1482449292772,-0.24550],[1482449316772,-0.24508],[1482449340772,-0.24355],[1482449364772,-0.24427],[1482449388772,-0.24564],[1482449412772,-0.24445],[1482449436772,-0.24333],[1482449460772,-0.24379],[1482449484772,-0.24353],[1482449508772,-0.24305],[1482449532772,-0.24451],[1482449556772,-0.24446],[1482449580772,-0.24435],[1482449604772,-0.24424],[1482449628772,-0.24451],[1482449652772,-0.24418],[1482449676772,-0.24407],[1482449700772,-0.24427],[1482449724772,-0.24459],[1482449748772,-0.24488],[1482449772772,-0.24111],[1482449796772,-0.24397],[1482449820772,-0.24519],[1482449844772,-0.24526],[1482449868772,-0.24621],[1482449892772,-0.24541],[1482449916772,-0.24597],[1482449940772,-0.24462],[1482449964772,-0.24468],[1482449988772,-0.24250],[1482450012772,-0.24540],[1482450036772,-0.24503],[1482450060772,-0.24506],[1482450084772,-0.24468],[1482450108772,-0.24500],[1482450132772,-0.24571],[1482450156772,-0.24444],[1482450180772,-0.24166],[1482450204772,-0.24363],[1482450228772,-0.24376],[1482450252772,-0.23944],[1482450276772,-0.24060],[1482450300772,-0.24029],[1482450324772,-0.24466],[1482450348772,-0.24445],[1482450372772,-0.22990],[1482450396772,-0.24366],[1482450420772,-0.24124],[1482450444772,-0.24418],[1482450468772,-0.23984],[1482450492772,-0.24359],[1482450516772,-0.23638],[1482450540772,-0.24059],[1482450564772,-0.24072],[1482450588772,-0.24181],[1482450612772,-0.24302],[1482450636772,-0.24435],[1482450660772,-0.24390],[1482450684772,-0.24607],[1482450708772,-0.24539],[1482450732772,-0.24393],[1482450756772,-0.23938],[1482450780772,-0.24496],[1482450804772,-0.24328],[1482450828772,-0.24492],[1482450852772,-0.24258],[1482450876772,-0.24362],[1482450900772,-0.24469],[1482450924772,-0.24166],[1482450948772,-0.24329],[1482450972772,-0.23584],[1482450996772,-0.23623],[1482451020772,-0.23475],[1482451044772,-0.24139],[1482451068772,-0.23583],[1482451092772,-0.23960],[1482451116772,-0.24437],[1482451140772,-0.23838],[1482451164772,-0.24424],[1482451188772,-0.24302],[1482451212772,-0.23917],[1482451236772,-0.24096],[1482451260772,-0.23737],[1482451284772,-0.24461],[1482451308772,-0.23638],[1482451332772,-0.23410],[1482451356772,-0.23604],[1482451380772,-0.24386],[1482451404772,-0.24466],[1482451428772,-0.24498],[1482451452772,-0.24464],[1482451476772,-0.24475],[1482451500772,-0.24475],[1482451524772,-0.24442],[1482451548772,-0.24480],[1482451572772,-0.24428],[1482451596772,-0.24529],[1482451620772,-0.24294],[1482451644772,-0.24343],[1482451668772,-0.24485],[1482451692772,-0.24322],[1482451716772,-0.24495],[1482451740772,-0.24554],[1482451764772,-0.24349],[1482451788772,-0.24519],[1482451812772,-0.24560],[1482451836772,-0.24493],[1482451860772,-0.24267],[1482451884772,-0.24422],[1482451908772,-0.24500],[1482451932772,-0.24490],[1482451956772,-0.24438],[1482451980772,-0.24551],[1482452004772,-0.24513],[1482452028772,-0.24551],[1482452052772,-0.24526],[1482452076772,-0.24434],[1482452100772,-0.24503],[1482452124772,-0.24508],[1482452148772,-0.24560],[1482452172772,-0.24468],[1482452196772,-0.24563],[1482452220772,-0.24517],[1482452244772,-0.24546],[1482452268772,-0.24475],[1482452292772,-0.24499],[1482452316772,-0.24578],[1482452340772,-0.24508],[1482452364772,-0.24569],[1482452388772,-0.24561],[1482452412772,-0.24553],[1482452436772,-0.24601],[1482452460772,-0.24556],[1482452484772,-0.24588],[1482452508772,-0.24601],[1482452532772,-0.24551],[1482452556772,-0.24554],[1482452580772,-0.24622],[1482452604772,-0.24539],[1482452628772,-0.24539],[1482452652772,-0.24580],[1482452676772,-0.24561],[1482452700772,-0.24603],[1482452724772,-0.24599],[1482452748772,-0.24536],[1482452772772,-0.24561],[1482452796772,-0.24506],[1482452820772,-0.24588],[1482452844772,-0.24591],[1482452868772,-0.24563],[1482452892772,-0.24612],[1482452916772,-0.24530],[1482452940772,-0.24561],[1482452964772,-0.24506],[1482452988772,-0.24583],[1482453012772,-0.24464],[1482453036772,-0.24614],[1482453060772,-0.24594],[1482453084772,-0.24583],[1482453108772,-0.24530],[1482453132772,-0.24513],[1482453156772,-0.24603],[1482453180772,-0.24523],[1482453204772,-0.24554],[1482453228772,-0.24570],[1482453252772,-0.24553],[1482453276772,-0.24543],[1482453300772,-0.24570],[1482453324772,-0.24604],[1482453348772,-0.24561],[1482453372772,-0.24437],[1482453396772,-0.24625],[1482453420772,-0.24563],[1482453444772,-0.24617],[1482453468772,-0.24593],[1482453492772,-0.24590],[1482453516772,-0.24475],[1482453540772,-0.24653],[1482453564772,-0.24621],[1482453588772,-0.24549],[1482453612772,-0.24570],[1482453636772,-0.24523],[1482453660772,-0.24594],[1482453684772,-0.24587],[1482453708772,-0.24607],[1482453732772,-0.24578],[1482453756772,-0.24618],[1482453780772,-0.24632],[1482453804772,-0.24587],[1482453828772,-0.24583],[1482453852772,-0.24537],[1482453876772,-0.24512],[1482453900772,-0.24580],[1482453924772,-0.24631],[1482453948772,-0.24478],[1482453972772,-0.24621],[1482453996772,-0.24591],[1482454020772,-0.24526],[1482454044772,-0.24635],[1482454068772,-0.24567],[1482454092772,-0.24554],[1482454116772,-0.24603],[1482454140772,-0.24569],[1482454164772,-0.24584],[1482454188772,-0.24510],[1482454212772,-0.24559],[1482454236772,-0.24583],[1482454260772,-0.24485],[1482454284772,-0.24525],[1482454308772,-0.24593],[1482454332772,-0.24557],[1482454356772,-0.24580],[1482454380772,-0.24601],[1482454404772,-0.24539],[1482454428772,-0.24553],[1482454452772,-0.24527],[1482454476772,-0.24393],[1482454500772,-0.24561],[1482454524772,-0.24512],[1482454548772,-0.24459],[1482454572772,-0.24580],[1482454596772,-0.24585],[1482454620772,-0.24553],[1482454644772,-0.24544],[1482454668772,-0.24517],[1482454692772,-0.24495],[1482454716772,-0.24541],[1482454740772,-0.24396],[1482454764772,-0.24499],[1482454788772,-0.24490],[1482454812772,-0.24455],[1482454836772,-0.24533],[1482454860772,-0.24539],[1482454884772,-0.24512],[1482454908772,-0.24459],[1482454932772,-0.24508],[1482454956772,-0.24458],[1482454980772,-0.24525],[1482455004772,-0.24551],[1482455028772,-0.24526],[1482455052772,-0.24498],[1482455076772,-0.24474],[1482455100772,-0.24364],[1482455124772,-0.24411],[1482455148772,-0.24448],[1482455172772,-0.24544],[1482455196772,-0.24489],[1482455220772,-0.24508],[1482455244772,-0.24533],[1482455268772,-0.24465],[1482455292772,-0.24387],[1482455316772,-0.24502],[1482455340772,-0.24530],[1482455364772,-0.24358],[1482455388772,-0.24438],[1482455412772,-0.24489],[1482455436772,-0.24485],[1482455460772,-0.24452],[1482455484772,-0.24516],[1482455508772,-0.24499],[1482455532772,-0.24466],[1482455556772,-0.24430],[1482455580772,-0.24496],[1482455604772,-0.24492],[1482455628772,-0.24493],[1482455652772,-0.24522],[1482455676772,-0.24403]],\n  "last":[[1482448500772,-0.24976],[1482448524772,-0.24700],[1482448548772,-0.24656],[1482448572772,-0.24474],[1482448596772,-0.24598],[1482448620772,-0.24665],[1482448644772,-0.24734],[1482448668772,-0.24757],[1482448692772,-0.24591],[1482448716772,-0.24643],[1482448740772,-0.24744],[1482448764772,-0.24941],[1482448788772,-0.24764],[1482448812772,-0.24571],[1482448836772,-0.24968],[1482448860772,-0.24892],[1482448884772,-0.24873],[1482448908772,-0.24757],[1482448932772,-0.24918],[1482448956772,-0.24828],[1482448980772,-0.24829],[1482449004772,-0.24553],[1482449028772,-0.24884],[1482449052772,-0.24697],[1482449076772,-0.24995],[1482449100772,-0.24975],[1482449124772,-0.24615],[1482449148772,-0.24703],[1482449172772,-0.24730],[1482449196772,-0.24897],[1482449220772,-0.24842],[1482449244772,-0.24955],[1482449268772,-0.24921],[1482449292772,-0.24776],[1482449316772,-0.25033],[1482449340772,-0.24581],[1482449364772,-0.24925],[1482449388772,-0.24632],[1482449412772,-0.24902],[1482449436772,-0.24792],[1482449460772,-0.24813],[1482449484772,-0.24834],[1482449508772,-0.24791],[1482449532772,-0.24451],[1482449556772,-0.24718],[1482449580772,-0.24619],[1482449604772,-0.24697],[1482449628772,-0.24707],[1482449652772,-0.24866],[1482449676772,-0.24754],[1482449700772,-0.24860],[1482449724772,-0.24757],[1482449748772,-0.24762],[1482449772772,-0.24510],[1482449796772,-0.24717],[1482449820772,-0.24625],[1482449844772,-0.24900],[1482449868772,-0.24717],[1482449892772,-0.24680],[1482449916772,-0.24816],[1482449940772,-0.24791],[1482449964772,-0.24583],[1482449988772,-0.24826],[1482450012772,-0.24911],[1482450036772,-0.24631],[1482450060772,-0.24877],[1482450084772,-0.24598],[1482450108772,-0.24945],[1482450132772,-0.24676],[1482450156772,-0.24730],[1482450180772,-0.24452],[1482450204772,-0.24799],[1482450228772,-0.24813],[1482450252772,-0.25071],[1482450276772,-0.24452],[1482450300772,-0.24594],[1482450324772,-0.24466],[1482450348772,-0.25582],[1482450372772,-0.24887],[1482450396772,-0.24825],[1482450420772,-0.25162],[1482450444772,-0.25102],[1482450468772,-0.24860],[1482450492772,-0.24744],[1482450516772,-0.25445],[1482450540772,-0.24333],[1482450564772,-0.24697],[1482450588772,-0.24819],[1482450612772,-0.24693],[1482450636772,-0.24605],[1482450660772,-0.24475],[1482450684772,-0.24840],[1482450708772,-0.24539],[1482450732772,-0.24697],[1482450756772,-0.25041],[1482450780772,-0.24569],[1482450804772,-0.24810],[1482450828772,-0.24798],[1482450852772,-0.24557],[1482450876772,-0.24569],[1482450900772,-0.24667],[1482450924772,-0.24816],[1482450948772,-0.25053],[1482450972772,-0.25106],[1482450996772,-0.24192],[1482451020772,-0.24844],[1482451044772,-0.24424],[1482451068772,-0.24352],[1482451092772,-0.25471],[1482451116772,-0.24912],[1482451140772,-0.24775],[1482451164772,-0.24757],[1482451188772,-0.24675],[1482451212772,-0.24770],[1482451236772,-0.24456],[1482451260772,-0.24442],[1482451284772,-0.25098],[1482451308772,-0.23638],[1482451332772,-0.24911],[1482451356772,-0.23988],[1482451380772,-0.24517],[1482451404772,-0.24990],[1482451428772,-0.24663],[1482451452772,-0.24689],[1482451476772,-0.24657],[1482451500772,-0.24796],[1482451524772,-0.24734],[1482451548772,-0.24795],[1482451572772,-0.24496],[1482451596772,-0.25005],[1482451620772,-0.24648],[1482451644772,-0.24571],[1482451668772,-0.24690],[1482451692772,-0.24516],[1482451716772,-0.24495],[1482451740772,-0.24758],[1482451764772,-0.24519],[1482451788772,-0.24607],[1482451812772,-0.24622],[1482451836772,-0.24857],[1482451860772,-0.24849],[1482451884772,-0.24770],[1482451908772,-0.24638],[1482451932772,-0.24604],[1482451956772,-0.24932],[1482451980772,-0.24642],[1482452004772,-0.24622],[1482452028772,-0.24902],[1482452052772,-0.24949],[1482452076772,-0.24709],[1482452100772,-0.24795],[1482452124772,-0.24679],[1482452148772,-0.24815],[1482452172772,-0.24556],[1482452196772,-0.24614],[1482452220772,-0.24517],[1482452244772,-0.24699],[1482452268772,-0.24808],[1482452292772,-0.24663],[1482452316772,-0.24857],[1482452340772,-0.24593],[1482452364772,-0.24706],[1482452388772,-0.24604],[1482452412772,-0.24665],[1482452436772,-0.24782],[1482452460772,-0.24676],[1482452484772,-0.24676],[1482452508772,-0.24789],[1482452532772,-0.24760],[1482452556772,-0.24791],[1482452580772,-0.24730],[1482452604772,-0.24757],[1482452628772,-0.24693],[1482452652772,-0.24612],[1482452676772,-0.24628],[1482452700772,-0.24686],[1482452724772,-0.24639],[1482452748772,-0.24750],[1482452772772,-0.24816],[1482452796772,-0.24506],[1482452820772,-0.24707],[1482452844772,-0.24758],[1482452868772,-0.24825],[1482452892772,-0.24730],[1482452916772,-0.24740],[1482452940772,-0.24636],[1482452964772,-0.24804],[1482452988772,-0.24789],[1482453012772,-0.24776],[1482453036772,-0.24737],[1482453060772,-0.24727],[1482453084772,-0.24747],[1482453108772,-0.24854],[1482453132772,-0.24860],[1482453156772,-0.24636],[1482453180772,-0.24687],[1482453204772,-0.24618],[1482453228772,-0.24699],[1482453252772,-0.24638],[1482453276772,-0.24543],[1482453300772,-0.24641],[1482453324772,-0.24747],[1482453348772,-0.24649],[1482453372772,-0.24726],[1482453396772,-0.24743],[1482453420772,-0.24774],[1482453444772,-0.24767],[1482453468772,-0.24713],[1482453492772,-0.24607],[1482453516772,-0.24890],[1482453540772,-0.24752],[1482453564772,-0.24667],[1482453588772,-0.24770],[1482453612772,-0.24663],[1482453636772,-0.24546],[1482453660772,-0.24840],[1482453684772,-0.24874],[1482453708772,-0.24628],[1482453732772,-0.24745],[1482453756772,-0.24679],[1482453780772,-0.24694],[1482453804772,-0.24680],[1482453828772,-0.24646],[1482453852772,-0.24770],[1482453876772,-0.24570],[1482453900772,-0.24686],[1482453924772,-0.24755],[1482453948772,-0.24750],[1482453972772,-0.24757],[1482453996772,-0.24966],[1482454020772,-0.24788],[1482454044772,-0.24689],[1482454068772,-0.24691],[1482454092772,-0.24770],[1482454116772,-0.24914],[1482454140772,-0.24639],[1482454164772,-0.24591],[1482454188772,-0.24747],[1482454212772,-0.24815],[1482454236772,-0.24877],[1482454260772,-0.24713],[1482454284772,-0.24828],[1482454308772,-0.24615],[1482454332772,-0.24673],[1482454356772,-0.24733],[1482454380772,-0.24614],[1482454404772,-0.24587],[1482454428772,-0.24755],[1482454452772,-0.24627],[1482454476772,-0.24770],[1482454500772,-0.24997],[1482454524772,-0.24703],[1482454548772,-0.24459],[1482454572772,-0.24612],[1482454596772,-0.24629],[1482454620772,-0.24580],[1482454644772,-0.24784],[1482454668772,-0.24522],[1482454692772,-0.24785],[1482454716772,-0.24788],[1482454740772,-0.24396],[1482454764772,-0.24724],[1482454788772,-0.24816],[1482454812772,-0.24808],[1482454836772,-0.24741],[1482454860772,-0.24649],[1482454884772,-0.24721],[1482454908772,-0.24836],[1482454932772,-0.24747],[1482454956772,-0.24544],[1482454980772,-0.24737],[1482455004772,-0.24805],[1482455028772,-0.24743],[1482455052772,-0.24796],[1482455076772,-0.24581],[1482455100772,-0.24802],[1482455124772,-0.24866],[1482455148772,-0.24641],[1482455172772,-0.24949],[1482455196772,-0.24788],[1482455220772,-0.24900],[1482455244772,-0.24709],[1482455268772,-0.24694],[1482455292772,-0.24571],[1482455316772,-0.24713],[1482455340772,-0.24856],[1482455364772,-0.24597],[1482455388772,-0.24659],[1482455412772,-0.24747],[1482455436772,-0.24659],[1482455460772,-0.24730],[1482455484772,-0.24516],[1482455508772,-0.24578],[1482455532772,-0.24625],[1482455556772,-0.24805],[1482455580772,-0.24622],[1482455604772,-0.24680],[1482455628772,-0.24842],[1482455652772,-0.24935],[1482455676772,-0.25035]],\n  "min":[[1482448500772,-0.25162],[1482448524772,-0.24941],[1482448548772,-0.24939],[1482448572772,-0.24887],[1482448596772,-0.25027],[1482448620772,-0.24996],[1482448644772,-0.25010],[1482448668772,-0.25039],[1482448692772,-0.24922],[1482448716772,-0.24934],[1482448740772,-0.25139],[1482448764772,-0.25092],[1482448788772,-0.25116],[1482448812772,-0.24939],[1482448836772,-0.25064],[1482448860772,-0.24976],[1482448884772,-0.25002],[1482448908772,-0.25057],[1482448932772,-0.24944],[1482448956772,-0.24941],[1482448980772,-0.24971],[1482449004772,-0.25079],[1482449028772,-0.24990],[1482449052772,-0.25061],[1482449076772,-0.24995],[1482449100772,-0.25167],[1482449124772,-0.25041],[1482449148772,-0.25101],[1482449172772,-0.24944],[1482449196772,-0.25045],[1482449220772,-0.25147],[1482449244772,-0.24955],[1482449268772,-0.24945],[1482449292772,-0.25125],[1482449316772,-0.25033],[1482449340772,-0.25064],[1482449364772,-0.24992],[1482449388772,-0.24958],[1482449412772,-0.25017],[1482449436772,-0.25085],[1482449460772,-0.25097],[1482449484772,-0.25020],[1482449508772,-0.24990],[1482449532772,-0.25082],[1482449556772,-0.24966],[1482449580772,-0.24939],[1482449604772,-0.24914],[1482449628772,-0.24934],[1482449652772,-0.25027],[1482449676772,-0.24989],[1482449700772,-0.24986],[1482449724772,-0.24966],[1482449748772,-0.25286],[1482449772772,-0.25164],[1482449796772,-0.24992],[1482449820772,-0.24942],[1482449844772,-0.25057],[1482449868772,-0.24955],[1482449892772,-0.25118],[1482449916772,-0.25085],[1482449940772,-0.24953],[1482449964772,-0.24929],[1482449988772,-0.24928],[1482450012772,-0.24986],[1482450036772,-0.24979],[1482450060772,-0.25051],[1482450084772,-0.24958],[1482450108772,-0.24985],[1482450132772,-0.24936],[1482450156772,-0.24912],[1482450180772,-0.25024],[1482450204772,-0.25089],[1482450228772,-0.25033],[1482450252772,-0.25314],[1482450276772,-0.25293],[1482450300772,-0.25557],[1482450324772,-0.25633],[1482450348772,-0.25840],[1482450372772,-0.25122],[1482450396772,-0.25564],[1482450420772,-0.26030],[1482450444772,-0.25262],[1482450468772,-0.25712],[1482450492772,-0.25208],[1482450516772,-0.26010],[1482450540772,-0.25128],[1482450564772,-0.25473],[1482450588772,-0.26422],[1482450612772,-0.25408],[1482450636772,-0.24956],[1482450660772,-0.25037],[1482450684772,-0.25077],[1482450708772,-0.25153],[1482450732772,-0.25622],[1482450756772,-0.25079],[1482450780772,-0.25044],[1482450804772,-0.24929],[1482450828772,-0.25003],[1482450852772,-0.25225],[1482450876772,-0.24959],[1482450900772,-0.25003],[1482450924772,-0.25051],[1482450948772,-0.25905],[1482450972772,-0.25967],[1482450996772,-0.26376],[1482451020772,-0.25465],[1482451044772,-0.25196],[1482451068772,-0.25537],[1482451092772,-0.25600],[1482451116772,-0.26519],[1482451140772,-0.25930],[1482451164772,-0.26212],[1482451188772,-0.24953],[1482451212772,-0.25091],[1482451236772,-0.24925],[1482451260772,-0.25103],[1482451284772,-0.25500],[1482451308772,-0.25521],[1482451332772,-0.25609],[1482451356772,-0.26529],[1482451380772,-0.25040],[1482451404772,-0.25024],[1482451428772,-0.24852],[1482451452772,-0.24931],[1482451476772,-0.24999],[1482451500772,-0.24917],[1482451524772,-0.25031],[1482451548772,-0.24891],[1482451572772,-0.25067],[1482451596772,-0.25007],[1482451620772,-0.24958],[1482451644772,-0.24968],[1482451668772,-0.25006],[1482451692772,-0.25013],[1482451716772,-0.24922],[1482451740772,-0.24897],[1482451764772,-0.24917],[1482451788772,-0.24901],[1482451812772,-0.25190],[1482451836772,-0.24965],[1482451860772,-0.24996],[1482451884772,-0.25084],[1482451908772,-0.25013],[1482451932772,-0.24907],[1482451956772,-0.24946],[1482451980772,-0.25381],[1482452004772,-0.25129],[1482452028772,-0.24972],[1482452052772,-0.24949],[1482452076772,-0.25048],[1482452100772,-0.24934],[1482452124772,-0.24858],[1482452148772,-0.25065],[1482452172772,-0.25006],[1482452196772,-0.24921],[1482452220772,-0.24895],[1482452244772,-0.24993],[1482452268772,-0.24932],[1482452292772,-0.24946],[1482452316772,-0.24904],[1482452340772,-0.24900],[1482452364772,-0.24921],[1482452388772,-0.24924],[1482452412772,-0.24884],[1482452436772,-0.24853],[1482452460772,-0.24972],[1482452484772,-0.24956],[1482452508772,-0.24920],[1482452532772,-0.24867],[1482452556772,-0.24929],[1482452580772,-0.24927],[1482452604772,-0.24914],[1482452628772,-0.24863],[1482452652772,-0.24898],[1482452676772,-0.24894],[1482452700772,-0.24975],[1482452724772,-0.24955],[1482452748772,-0.24839],[1482452772772,-0.24942],[1482452796772,-0.24877],[1482452820772,-0.24936],[1482452844772,-0.24962],[1482452868772,-0.24852],[1482452892772,-0.24880],[1482452916772,-0.24938],[1482452940772,-0.24854],[1482452964772,-0.24856],[1482452988772,-0.24911],[1482453012772,-0.24843],[1482453036772,-0.24921],[1482453060772,-0.25063],[1482453084772,-0.24963],[1482453108772,-0.25020],[1482453132772,-0.24901],[1482453156772,-0.24894],[1482453180772,-0.24860],[1482453204772,-0.24985],[1482453228772,-0.24849],[1482453252772,-0.24982],[1482453276772,-0.24853],[1482453300772,-0.24900],[1482453324772,-0.25005],[1482453348772,-0.24953],[1482453372772,-0.24902],[1482453396772,-0.24910],[1482453420772,-0.24951],[1482453444772,-0.24883],[1482453468772,-0.24973],[1482453492772,-0.24944],[1482453516772,-0.24925],[1482453540772,-0.24901],[1482453564772,-0.24847],[1482453588772,-0.24862],[1482453612772,-0.24891],[1482453636772,-0.24992],[1482453660772,-0.24907],[1482453684772,-0.24902],[1482453708772,-0.24895],[1482453732772,-0.24839],[1482453756772,-0.24888],[1482453780772,-0.25007],[1482453804772,-0.25033],[1482453828772,-0.24918],[1482453852772,-0.24876],[1482453876772,-0.24989],[1482453900772,-0.24951],[1482453924772,-0.25060],[1482453948772,-0.25006],[1482453972772,-0.24973],[1482453996772,-0.24976],[1482454020772,-0.24887],[1482454044772,-0.25013],[1482454068772,-0.24944],[1482454092772,-0.24975],[1482454116772,-0.24914],[1482454140772,-0.24837],[1482454164772,-0.24938],[1482454188772,-0.24799],[1482454212772,-0.24958],[1482454236772,-0.24884],[1482454260772,-0.24892],[1482454284772,-0.24892],[1482454308772,-0.24955],[1482454332772,-0.24927],[1482454356772,-0.24890],[1482454380772,-0.24953],[1482454404772,-0.24966],[1482454428772,-0.24975],[1482454452772,-0.24895],[1482454476772,-0.24887],[1482454500772,-0.24997],[1482454524772,-0.24915],[1482454548772,-0.24966],[1482454572772,-0.24917],[1482454596772,-0.25003],[1482454620772,-0.24918],[1482454644772,-0.24956],[1482454668772,-0.24895],[1482454692772,-0.24944],[1482454716772,-0.25005],[1482454740772,-0.25002],[1482454764772,-0.24860],[1482454788772,-0.24965],[1482454812772,-0.25072],[1482454836772,-0.24959],[1482454860772,-0.24946],[1482454884772,-0.24883],[1482454908772,-0.25044],[1482454932772,-0.25007],[1482454956772,-0.25030],[1482454980772,-0.25009],[1482455004772,-0.24832],[1482455028772,-0.24922],[1482455052772,-0.24894],[1482455076772,-0.24983],[1482455100772,-0.24938],[1482455124772,-0.24955],[1482455148772,-0.24870],[1482455172772,-0.24949],[1482455196772,-0.24917],[1482455220772,-0.24918],[1482455244772,-0.24892],[1482455268772,-0.24969],[1482455292772,-0.24990],[1482455316772,-0.24965],[1482455340772,-0.24973],[1482455364772,-0.24931],[1482455388772,-0.24965],[1482455412772,-0.24914],[1482455436772,-0.24928],[1482455460772,-0.25006],[1482455484772,-0.24895],[1482455508772,-0.24958],[1482455532772,-0.24908],[1482455556772,-0.25009],[1482455580772,-0.24894],[1482455604772,-0.25044],[1482455628772,-0.24945],[1482455652772,-0.24963],[1482455676772,-0.25035]]\n}'
_trenddata_2hr_ph_json = '{\n  "max":[[1482448501073,0.030683],[1482448525073,0.030807],[1482448549073,0.030832],[1482448573073,0.031086],[1482448597073,0.030768],[1482448621073,0.030866],[1482448645073,0.030866],[1482448669073,0.030980],[1482448693073,0.031009],[1482448717073,0.031237],[1482448741073,0.031188],[1482448765073,0.031095],[1482448789073,0.031299],[1482448813073,0.031171],[1482448837073,0.031225],[1482448861073,0.031197],[1482448885073,0.031276],[1482448909073,0.031407],[1482448933073,0.031453],[1482448957073,0.031500],[1482448981073,0.031504],[1482449005073,0.031488],[1482449029073,0.031534],[1482449053073,0.031542],[1482449077073,0.031685],[1482449101073,0.031766],[1482449125073,0.031678],[1482449149073,0.031678],[1482449173073,0.031765],[1482449197073,0.031882],[1482449221073,0.031826],[1482449245073,0.031812],[1482449269073,0.031906],[1482449293073,0.031709],[1482449317073,0.031763],[1482449341073,0.031777],[1482449365073,0.031873],[1482449389073,0.031852],[1482449413073,0.031965],[1482449437073,0.031950],[1482449461073,0.032027],[1482449485073,0.032048],[1482449509073,0.032013],[1482449533073,0.032046],[1482449557073,0.032100],[1482449581073,0.032111],[1482449605073,0.032119],[1482449629073,0.032025],[1482449653073,0.032098],[1482449677073,0.032062],[1482449701073,0.032176],[1482449725073,0.032377],[1482449749073,0.032193],[1482449773073,0.032426],[1482449797073,0.032209],[1482449821073,0.032232],[1482449845073,0.032295],[1482449869073,0.032332],[1482449893073,0.032276],[1482449917073,0.032356],[1482449941073,0.032279],[1482449965073,0.032296],[1482449989073,0.032500],[1482450013073,0.032457],[1482450037073,0.032497],[1482450061073,0.032394],[1482450085073,0.032463],[1482450109073,0.032514],[1482450133073,0.032523],[1482450157073,0.032574],[1482450181073,0.032756],[1482450205073,0.032624],[1482450229073,0.032770],[1482450253073,0.032824],[1482450277073,0.032918],[1482450301073,0.033039],[1482450325073,0.032884],[1482450349073,0.032918],[1482450373073,0.033180],[1482450397073,0.033178],[1482450421073,0.033189],[1482450445073,0.033043],[1482450469073,0.033417],[1482450493073,0.033170],[1482450517073,0.033595],[1482450541073,0.033339],[1482450565073,0.033373],[1482450589073,0.033283],[1482450613073,0.033348],[1482450637073,0.033432],[1482450661073,0.033327],[1482450685073,0.033135],[1482450709073,0.033329],[1482450733073,0.033408],[1482450757073,0.033492],[1482450781073,0.033494],[1482450805073,0.033489],[1482450829073,0.033473],[1482450853073,0.033731],[1482450877073,0.033602],[1482450901073,0.033553],[1482450925073,0.033621],[1482450949073,0.033722],[1482450973073,0.034278],[1482450997073,0.034125],[1482451021073,0.034446],[1482451045073,0.033774],[1482451069073,0.034423],[1482451093073,0.034021],[1482451117073,0.033779],[1482451141073,0.033799],[1482451165073,0.033953],[1482451189073,0.034230],[1482451213073,0.034059],[1482451237073,0.033958],[1482451261073,0.034500],[1482451285073,0.033983],[1482451309073,0.034259],[1482451333073,0.034691],[1482451357073,0.034488],[1482451381073,0.034048],[1482451405073,0.034170],[1482451429073,0.034062],[1482451453073,0.034176],[1482451477073,0.034112],[1482451501073,0.034199],[1482451525073,0.034319],[1482451549073,0.034113],[1482451573073,0.034212],[1482451597073,0.034163],[1482451621073,0.034189],[1482451645073,0.034225],[1482451669073,0.034266],[1482451693073,0.034433],[1482451717073,0.034244],[1482451741073,0.034283],[1482451765073,0.034300],[1482451789073,0.034346],[1482451813073,0.034306],[1482451837073,0.034255],[1482451861073,0.034555],[1482451885073,0.034311],[1482451909073,0.034283],[1482451933073,0.034432],[1482451957073,0.034395],[1482451981073,0.034434],[1482452005073,0.034438],[1482452029073,0.034389],[1482452053073,0.034441],[1482452077073,0.034472],[1482452101073,0.034476],[1482452125073,0.034498],[1482452149073,0.034759],[1482452173073,0.034559],[1482452197073,0.034534],[1482452221073,0.034540],[1482452245073,0.034642],[1482452269073,0.034576],[1482452293073,0.034533],[1482452317073,0.034646],[1482452341073,0.035065],[1482452365073,0.035084],[1482452389073,0.034671],[1482452413073,0.034634],[1482452437073,0.034595],[1482452461073,0.034633],[1482452485073,0.034645],[1482452509073,0.034676],[1482452533073,0.034638],[1482452557073,0.034662],[1482452581073,0.034724],[1482452605073,0.034770],[1482452629073,0.035156],[1482452653073,0.034848],[1482452677073,0.034767],[1482452701073,0.034743],[1482452725073,0.034828],[1482452749073,0.034793],[1482452773073,0.034775],[1482452797073,0.034747],[1482452821073,0.034827],[1482452845073,0.034807],[1482452869073,0.034831],[1482452893073,0.034836],[1482452917073,0.034917],[1482452941073,0.034921],[1482452965073,0.034962],[1482452989073,0.034892],[1482453013073,0.034872],[1482453037073,0.035050],[1482453061073,0.034982],[1482453085073,0.034973],[1482453109073,0.035028],[1482453133073,0.035035],[1482453157073,0.035285],[1482453181073,0.035138],[1482453205073,0.035284],[1482453229073,0.035379],[1482453253073,0.035223],[1482453277073,0.035138],[1482453301073,0.035227],[1482453325073,0.035227],[1482453349073,0.035237],[1482453373073,0.035179],[1482453397073,0.035296],[1482453421073,0.035347],[1482453445073,0.035463],[1482453469073,0.035508],[1482453493073,0.035729],[1482453517073,0.035609],[1482453541073,0.035712],[1482453565073,0.035664],[1482453589073,0.035803],[1482453613073,0.036785],[1482453637073,0.036125],[1482453661073,0.035893],[1482453685073,0.035904],[1482453709073,0.035945],[1482453733073,0.036176],[1482453757073,0.036061],[1482453781073,0.036070],[1482453805073,0.036070],[1482453829073,0.036063],[1482453853073,0.036054],[1482453877073,0.036169],[1482453901073,0.036272],[1482453925073,0.036307],[1482453949073,0.036504],[1482453973073,0.037438],[1482453997073,0.037396],[1482454021073,0.037546],[1482454045073,0.037025],[1482454069073,0.036657],[1482454093073,0.036505],[1482454117073,0.037906],[1482454141073,0.037357],[1482454165073,0.037015],[1482454189073,0.037177],[1482454213073,0.036429],[1482454237073,0.037067],[1482454261073,0.037656],[1482454285073,0.037912],[1482454309073,0.037688],[1482454333073,0.037205],[1482454357073,0.037344],[1482454381073,0.037174],[1482454405073,0.036762],[1482454429073,0.036534],[1482454453073,0.036555],[1482454477073,0.036408],[1482454501073,0.036447],[1482454525073,0.036589],[1482454549073,0.036556],[1482454573073,0.036593],[1482454597073,0.036532],[1482454621073,0.036628],[1482454645073,0.036696],[1482454669073,0.036518],[1482454693073,0.036536],[1482454717073,0.036772],[1482454741073,0.036660],[1482454765073,0.036873],[1482454789073,0.036944],[1482454813073,0.037427],[1482454837073,0.037079],[1482454861073,0.036779],[1482454885073,0.036725],[1482454909073,0.036713],[1482454933073,0.036771],[1482454957073,0.036840],[1482454981073,0.036750],[1482455005073,0.036848],[1482455029073,0.036916],[1482455053073,0.036983],[1482455077073,0.037694],[1482455101073,0.037865],[1482455125073,0.038098],[1482455149073,0.037843],[1482455173073,0.037837],[1482455197073,0.037525],[1482455221073,0.037249],[1482455245073,0.037660],[1482455269073,0.037828],[1482455293073,0.037984],[1482455317073,0.037903],[1482455341073,0.037704],[1482455365073,0.038118],[1482455389073,0.037973],[1482455413073,0.037792],[1482455437073,0.038009],[1482455461073,0.037394],[1482455485073,0.037377],[1482455509073,0.037459],[1482455533073,0.037492],[1482455557073,0.037496],[1482455581073,0.037495],[1482455605073,0.037442],[1482455629073,0.037610],[1482455653073,0.037669],[1482455677073,0.037617]],\n  "last":[[1482448501073,0.030277],[1482448525073,0.030498],[1482448549073,0.030703],[1482448573073,0.031086],[1482448597073,0.030755],[1482448621073,0.030808],[1482448645073,0.030756],[1482448669073,0.030638],[1482448693073,0.030838],[1482448717073,0.030988],[1482448741073,0.030981],[1482448765073,0.030753],[1482448789073,0.030895],[1482448813073,0.031081],[1482448837073,0.030813],[1482448861073,0.030896],[1482448885073,0.030962],[1482448909073,0.031190],[1482448933073,0.031112],[1482448957073,0.031375],[1482448981073,0.031368],[1482449005073,0.031388],[1482449029073,0.031168],[1482449053073,0.031416],[1482449077073,0.031192],[1482449101073,0.031243],[1482449125073,0.031357],[1482449149073,0.031503],[1482449173073,0.031382],[1482449197073,0.031356],[1482449221073,0.031657],[1482449245073,0.031243],[1482449269073,0.031518],[1482449293073,0.031443],[1482449317073,0.031404],[1482449341073,0.031676],[1482449365073,0.031459],[1482449389073,0.031597],[1482449413073,0.031672],[1482449437073,0.031727],[1482449461073,0.031605],[1482449485073,0.031537],[1482449509073,0.031550],[1482449533073,0.031912],[1482449557073,0.031860],[1482449581073,0.032111],[1482449605073,0.031980],[1482449629073,0.032025],[1482449653073,0.031710],[1482449677073,0.031776],[1482449701073,0.031772],[1482449725073,0.031929],[1482449749073,0.032193],[1482449773073,0.032426],[1482449797073,0.032098],[1482449821073,0.032086],[1482449845073,0.032019],[1482449869073,0.032100],[1482449893073,0.032072],[1482449917073,0.031924],[1482449941073,0.032026],[1482449965073,0.032094],[1482449989073,0.032123],[1482450013073,0.032139],[1482450037073,0.032343],[1482450061073,0.032202],[1482450085073,0.032347],[1482450109073,0.032008],[1482450133073,0.032327],[1482450157073,0.032202],[1482450181073,0.032587],[1482450205073,0.032253],[1482450229073,0.032297],[1482450253073,0.032100],[1482450277073,0.032465],[1482450301073,0.032686],[1482450325073,0.032756],[1482450349073,0.032131],[1482450373073,0.032610],[1482450397073,0.032636],[1482450421073,0.032651],[1482450445073,0.032501],[1482450469073,0.032683],[1482450493073,0.032896],[1482450517073,0.032861],[1482450541073,0.033339],[1482450565073,0.033044],[1482450589073,0.032856],[1482450613073,0.032967],[1482450637073,0.033309],[1482450661073,0.033327],[1482450685073,0.032996],[1482450709073,0.033296],[1482450733073,0.033093],[1482450757073,0.033047],[1482450781073,0.033286],[1482450805073,0.033067],[1482450829073,0.033017],[1482450853073,0.033437],[1482450877073,0.033549],[1482450901073,0.033446],[1482450925073,0.033255],[1482450949073,0.033314],[1482450973073,0.033314],[1482450997073,0.033559],[1482451021073,0.033502],[1482451045073,0.033555],[1482451069073,0.033708],[1482451093073,0.033115],[1482451117073,0.033531],[1482451141073,0.033734],[1482451165073,0.033791],[1482451189073,0.033509],[1482451213073,0.033921],[1482451237073,0.033798],[1482451261073,0.033664],[1482451285073,0.033451],[1482451309073,0.034259],[1482451333073,0.033538],[1482451357073,0.034112],[1482451381073,0.033929],[1482451405073,0.033694],[1482451429073,0.033846],[1482451453073,0.033986],[1482451477073,0.033704],[1482451501073,0.033814],[1482451525073,0.033795],[1482451549073,0.033787],[1482451573073,0.033970],[1482451597073,0.033850],[1482451621073,0.034040],[1482451645073,0.034092],[1482451669073,0.034121],[1482451693073,0.034144],[1482451717073,0.034132],[1482451741073,0.034118],[1482451765073,0.034158],[1482451789073,0.034065],[1482451813073,0.034155],[1482451837073,0.033948],[1482451861073,0.034210],[1482451885073,0.034233],[1482451909073,0.034060],[1482451933073,0.034225],[1482451957073,0.034093],[1482451981073,0.034208],[1482452005073,0.034198],[1482452029073,0.034081],[1482452053073,0.034097],[1482452077073,0.034234],[1482452101073,0.034187],[1482452125073,0.034237],[1482452149073,0.034272],[1482452173073,0.034458],[1482452197073,0.034534],[1482452221073,0.034338],[1482452245073,0.034376],[1482452269073,0.034576],[1482452293073,0.034394],[1482452317073,0.034306],[1482452341073,0.034534],[1482452365073,0.034202],[1482452389073,0.034550],[1482452413073,0.034634],[1482452437073,0.034406],[1482452461073,0.034465],[1482452485073,0.034537],[1482452509073,0.034526],[1482452533073,0.034610],[1482452557073,0.034489],[1482452581073,0.034524],[1482452605073,0.034490],[1482452629073,0.034611],[1482452653073,0.034848],[1482452677073,0.034614],[1482452701073,0.034651],[1482452725073,0.034691],[1482452749073,0.034721],[1482452773073,0.034602],[1482452797073,0.034582],[1482452821073,0.034669],[1482452845073,0.034597],[1482452869073,0.034728],[1482452893073,0.034791],[1482452917073,0.034800],[1482452941073,0.034740],[1482452965073,0.034693],[1482452989073,0.034776],[1482453013073,0.034705],[1482453037073,0.034736],[1482453061073,0.034938],[1482453085073,0.034743],[1482453109073,0.034883],[1482453133073,0.034735],[1482453157073,0.034918],[1482453181073,0.034986],[1482453205073,0.035037],[1482453229073,0.035379],[1482453253073,0.035130],[1482453277073,0.035138],[1482453301073,0.035148],[1482453325073,0.035010],[1482453349073,0.035237],[1482453373073,0.035081],[1482453397073,0.035059],[1482453421073,0.035218],[1482453445073,0.035266],[1482453469073,0.035368],[1482453493073,0.035374],[1482453517073,0.035293],[1482453541073,0.035362],[1482453565073,0.035631],[1482453589073,0.035442],[1482453613073,0.035868],[1482453637073,0.035401],[1482453661073,0.035676],[1482453685073,0.035687],[1482453709073,0.035821],[1482453733073,0.035947],[1482453757073,0.035946],[1482453781073,0.035911],[1482453805073,0.035954],[1482453829073,0.035990],[1482453853073,0.035844],[1482453877073,0.035994],[1482453901073,0.036075],[1482453925073,0.036166],[1482453949073,0.036020],[1482453973073,0.036061],[1482453997073,0.036101],[1482454021073,0.036679],[1482454045073,0.036458],[1482454069073,0.036370],[1482454093073,0.036413],[1482454117073,0.036188],[1482454141073,0.036265],[1482454165073,0.036990],[1482454189073,0.036946],[1482454213073,0.036238],[1482454237073,0.035974],[1482454261073,0.037656],[1482454285073,0.036184],[1482454309073,0.036698],[1482454333073,0.036371],[1482454357073,0.036397],[1482454381073,0.035599],[1482454405073,0.036762],[1482454429073,0.036534],[1482454453073,0.036324],[1482454477073,0.036214],[1482454501073,0.036357],[1482454525073,0.036447],[1482454549073,0.036527],[1482454573073,0.036490],[1482454597073,0.036442],[1482454621073,0.036550],[1482454645073,0.036436],[1482454669073,0.036478],[1482454693073,0.036449],[1482454717073,0.036368],[1482454741073,0.036501],[1482454765073,0.036430],[1482454789073,0.036448],[1482454813073,0.036479],[1482454837073,0.036554],[1482454861073,0.036640],[1482454885073,0.036647],[1482454909073,0.036336],[1482454933073,0.036604],[1482454957073,0.036622],[1482454981073,0.036686],[1482455005073,0.036586],[1482455029073,0.036599],[1482455053073,0.036627],[1482455077073,0.036628],[1482455101073,0.035953],[1482455125073,0.036911],[1482455149073,0.037005],[1482455173073,0.036877],[1482455197073,0.036916],[1482455221073,0.036656],[1482455245073,0.036747],[1482455269073,0.036961],[1482455293073,0.036982],[1482455317073,0.036958],[1482455341073,0.036783],[1482455365073,0.037022],[1482455389073,0.037973],[1482455413073,0.036997],[1482455437073,0.037118],[1482455461073,0.037266],[1482455485073,0.037377],[1482455509073,0.037109],[1482455533073,0.036959],[1482455557073,0.037084],[1482455581073,0.037300],[1482455605073,0.037295],[1482455629073,0.036953],[1482455653073,0.037183],[1482455677073,0.037174]],\n  "min":[[1482448501073,0.030228],[1482448525073,0.030282],[1482448549073,0.030321],[1482448573073,0.030022],[1482448597073,0.030286],[1482448621073,0.030406],[1482448645073,0.030488],[1482448669073,0.030505],[1482448693073,0.030596],[1482448717073,0.030726],[1482448741073,0.030625],[1482448765073,0.030615],[1482448789073,0.030731],[1482448813073,0.030771],[1482448837073,0.030786],[1482448861073,0.030780],[1482448885073,0.030909],[1482448909073,0.030904],[1482448933073,0.030976],[1482448957073,0.031005],[1482448981073,0.031063],[1482449005073,0.031107],[1482449029073,0.031123],[1482449053073,0.031128],[1482449077073,0.031192],[1482449101073,0.031173],[1482449125073,0.031131],[1482449149073,0.031194],[1482449173073,0.031260],[1482449197073,0.031205],[1482449221073,0.031290],[1482449245073,0.031243],[1482449269073,0.031320],[1482449293073,0.031368],[1482449317073,0.031359],[1482449341073,0.031309],[1482449365073,0.031323],[1482449389073,0.031423],[1482449413073,0.031484],[1482449437073,0.031425],[1482449461073,0.031546],[1482449485073,0.031482],[1482449509073,0.031442],[1482449533073,0.031518],[1482449557073,0.031511],[1482449581073,0.031652],[1482449605073,0.031589],[1482449629073,0.031730],[1482449653073,0.031653],[1482449677073,0.031639],[1482449701073,0.031745],[1482449725073,0.031824],[1482449749073,0.031764],[1482449773073,0.031761],[1482449797073,0.031830],[1482449821073,0.031775],[1482449845073,0.031838],[1482449869073,0.031834],[1482449893073,0.031915],[1482449917073,0.031800],[1482449941073,0.031842],[1482449965073,0.031891],[1482449989073,0.031910],[1482450013073,0.031982],[1482450037073,0.031876],[1482450061073,0.031889],[1482450085073,0.032057],[1482450109073,0.032008],[1482450133073,0.031951],[1482450157073,0.032150],[1482450181073,0.032221],[1482450205073,0.032099],[1482450229073,0.032182],[1482450253073,0.032100],[1482450277073,0.032279],[1482450301073,0.031911],[1482450325073,0.032229],[1482450349073,0.031794],[1482450373073,0.032471],[1482450397073,0.032263],[1482450421073,0.032363],[1482450445073,0.032458],[1482450469073,0.032506],[1482450493073,0.032596],[1482450517073,0.032624],[1482450541073,0.032686],[1482450565073,0.032310],[1482450589073,0.032468],[1482450613073,0.032436],[1482450637073,0.032681],[1482450661073,0.032930],[1482450685073,0.032738],[1482450709073,0.032665],[1482450733073,0.032905],[1482450757073,0.032933],[1482450781073,0.032944],[1482450805073,0.033008],[1482450829073,0.032954],[1482450853073,0.033020],[1482450877073,0.033111],[1482450901073,0.033051],[1482450925073,0.033102],[1482450949073,0.032678],[1482450973073,0.032809],[1482450997073,0.032534],[1482451021073,0.032784],[1482451045073,0.033247],[1482451069073,0.033061],[1482451093073,0.032833],[1482451117073,0.032850],[1482451141073,0.033123],[1482451165073,0.033065],[1482451189073,0.033509],[1482451213073,0.033427],[1482451237073,0.033379],[1482451261073,0.033384],[1482451285073,0.033244],[1482451309073,0.033331],[1482451333073,0.033350],[1482451357073,0.033473],[1482451381073,0.033640],[1482451405073,0.033576],[1482451429073,0.033679],[1482451453073,0.033695],[1482451477073,0.033651],[1482451501073,0.033720],[1482451525073,0.033664],[1482451549073,0.033701],[1482451573073,0.033614],[1482451597073,0.033763],[1482451621073,0.033795],[1482451645073,0.033796],[1482451669073,0.033798],[1482451693073,0.033814],[1482451717073,0.033835],[1482451741073,0.033868],[1482451765073,0.033909],[1482451789073,0.033943],[1482451813073,0.033751],[1482451837073,0.033948],[1482451861073,0.033947],[1482451885073,0.033841],[1482451909073,0.033929],[1482451933073,0.034001],[1482451957073,0.033999],[1482451981073,0.033890],[1482452005073,0.034000],[1482452029073,0.034026],[1482452053073,0.034062],[1482452077073,0.034126],[1482452101073,0.034139],[1482452125073,0.034132],[1482452149073,0.034149],[1482452173073,0.034123],[1482452197073,0.034164],[1482452221073,0.034112],[1482452245073,0.034315],[1482452269073,0.034245],[1482452293073,0.034227],[1482452317073,0.034254],[1482452341073,0.033703],[1482452365073,0.033988],[1482452389073,0.034220],[1482452413073,0.034289],[1482452437073,0.034292],[1482452461073,0.034310],[1482452485073,0.034382],[1482452509073,0.034366],[1482452533073,0.034358],[1482452557073,0.034367],[1482452581073,0.034388],[1482452605073,0.034245],[1482452629073,0.034236],[1482452653073,0.034406],[1482452677073,0.034460],[1482452701073,0.034418],[1482452725073,0.034463],[1482452749073,0.034526],[1482452773073,0.034442],[1482452797073,0.034418],[1482452821073,0.034540],[1482452845073,0.034542],[1482452869073,0.034678],[1482452893073,0.034559],[1482452917073,0.034662],[1482452941073,0.034609],[1482452965073,0.034639],[1482452989073,0.034665],[1482453013073,0.034613],[1482453037073,0.034624],[1482453061073,0.034671],[1482453085073,0.034737],[1482453109073,0.034519],[1482453133073,0.034735],[1482453157073,0.034720],[1482453181073,0.034784],[1482453205073,0.034781],[1482453229073,0.034735],[1482453253073,0.034873],[1482453277073,0.034875],[1482453301073,0.034948],[1482453325073,0.034935],[1482453349073,0.034926],[1482453373073,0.034926],[1482453397073,0.034951],[1482453421073,0.035088],[1482453445073,0.035085],[1482453469073,0.035199],[1482453493073,0.035164],[1482453517073,0.035293],[1482453541073,0.035338],[1482453565073,0.035433],[1482453589073,0.035431],[1482453613073,0.035459],[1482453637073,0.035401],[1482453661073,0.035560],[1482453685073,0.035605],[1482453709073,0.035715],[1482453733073,0.035743],[1482453757073,0.035742],[1482453781073,0.035830],[1482453805073,0.035779],[1482453829073,0.035773],[1482453853073,0.035816],[1482453877073,0.035877],[1482453901073,0.036024],[1482453925073,0.035992],[1482453949073,0.036011],[1482453973073,0.035149],[1482453997073,0.035434],[1482454021073,0.035775],[1482454045073,0.035775],[1482454069073,0.036235],[1482454093073,0.036310],[1482454117073,0.035719],[1482454141073,0.035636],[1482454165073,0.035743],[1482454189073,0.035824],[1482454213073,0.035975],[1482454237073,0.035597],[1482454261073,0.036020],[1482454285073,0.035993],[1482454309073,0.036062],[1482454333073,0.036102],[1482454357073,0.036062],[1482454381073,0.035599],[1482454405073,0.035965],[1482454429073,0.036121],[1482454453073,0.036210],[1482454477073,0.036146],[1482454501073,0.036163],[1482454525073,0.036216],[1482454549073,0.036279],[1482454573073,0.036134],[1482454597073,0.036222],[1482454621073,0.036314],[1482454645073,0.036245],[1482454669073,0.036195],[1482454693073,0.036281],[1482454717073,0.036284],[1482454741073,0.036322],[1482454765073,0.036337],[1482454789073,0.036448],[1482454813073,0.036133],[1482454837073,0.036357],[1482454861073,0.036409],[1482454885073,0.036387],[1482454909073,0.036336],[1482454933073,0.036317],[1482454957073,0.036389],[1482454981073,0.036397],[1482455005073,0.036386],[1482455029073,0.036402],[1482455053073,0.036452],[1482455077073,0.036327],[1482455101073,0.035953],[1482455125073,0.036090],[1482455149073,0.036578],[1482455173073,0.036370],[1482455197073,0.036306],[1482455221073,0.036651],[1482455245073,0.036460],[1482455269073,0.036495],[1482455293073,0.036317],[1482455317073,0.036309],[1482455341073,0.036503],[1482455365073,0.036115],[1482455389073,0.036842],[1482455413073,0.036395],[1482455437073,0.036629],[1482455461073,0.036764],[1482455485073,0.036813],[1482455509073,0.036928],[1482455533073,0.036959],[1482455557073,0.036953],[1482455581073,0.036926],[1482455605073,0.037031],[1482455629073,0.036953],[1482455653073,0.037065],[1482455677073,0.037158]]\n}'
_trenddata_2hr_temperature_json = '{\n  "max":[[1482448501308,20.024],[1482448525308,20.025],[1482448549308,20.026],[1482448573308,20.028],[1482448597308,20.028],[1482448621308,20.031],[1482448645308,20.033],[1482448669308,20.035],[1482448693308,20.037],[1482448717308,20.040],[1482448741308,20.042],[1482448765308,20.041],[1482448789308,20.042],[1482448813308,20.045],[1482448837308,20.045],[1482448861308,20.047],[1482448885308,20.049],[1482448909308,20.051],[1482448933308,20.054],[1482448957308,20.054],[1482448981308,20.057],[1482449005308,20.058],[1482449029308,20.058],[1482449053308,20.058],[1482449077308,20.060],[1482449101308,20.063],[1482449125308,20.063],[1482449149308,20.065],[1482449173308,20.064],[1482449197308,20.069],[1482449221308,20.069],[1482449245308,20.067],[1482449269308,20.068],[1482449293308,20.068],[1482449317308,20.068],[1482449341308,20.069],[1482449365308,20.070],[1482449389308,20.072],[1482449413308,20.074],[1482449437308,20.074],[1482449461308,20.076],[1482449485308,20.075],[1482449509308,20.075],[1482449533308,20.079],[1482449557308,20.080],[1482449581308,20.082],[1482449605308,20.080],[1482449629308,20.083],[1482449653308,20.083],[1482449677308,20.083],[1482449701308,20.086],[1482449725308,20.088],[1482449749308,20.087],[1482449773308,20.087],[1482449797308,20.088],[1482449821308,20.089],[1482449845308,20.089],[1482449869308,20.089],[1482449893308,20.093],[1482449917308,20.092],[1482449941308,20.091],[1482449965308,20.095],[1482449989308,20.095],[1482450013308,20.096],[1482450037308,20.095],[1482450061308,20.098],[1482450085308,20.097],[1482450109308,20.098],[1482450133308,20.099],[1482450157308,20.101],[1482450181308,20.104],[1482450205308,20.103],[1482450229308,20.106],[1482450253308,20.106],[1482450277308,20.111],[1482450301308,20.110],[1482450325308,20.112],[1482450349308,20.116],[1482450373308,20.117],[1482450397308,20.117],[1482450421308,20.118],[1482450445308,20.118],[1482450469308,20.121],[1482450493308,20.123],[1482450517308,20.123],[1482450541308,20.127],[1482450565308,20.126],[1482450589308,20.130],[1482450613308,20.128],[1482450637308,20.131],[1482450661308,20.133],[1482450685308,20.132],[1482450709308,20.133],[1482450733308,20.138],[1482450757308,20.139],[1482450781308,20.139],[1482450805308,20.139],[1482450829308,20.138],[1482450853308,20.142],[1482450877308,20.143],[1482450901308,20.145],[1482450925308,20.145],[1482450949308,20.146],[1482450973308,20.146],[1482450997308,20.147],[1482451021308,20.148],[1482451045308,20.151],[1482451069308,20.151],[1482451093308,20.152],[1482451117308,20.154],[1482451141308,20.154],[1482451165308,20.155],[1482451189308,20.156],[1482451213308,20.156],[1482451237308,20.157],[1482451261308,20.157],[1482451285308,20.158],[1482451309308,20.159],[1482451333308,20.159],[1482451357308,20.162],[1482451381308,20.164],[1482451405308,20.165],[1482451429308,20.166],[1482451453308,20.166],[1482451477308,20.166],[1482451501308,20.168],[1482451525308,20.169],[1482451549308,20.169],[1482451573308,20.169],[1482451597308,20.170],[1482451621308,20.171],[1482451645308,20.173],[1482451669308,20.173],[1482451693308,20.173],[1482451717308,20.174],[1482451741308,20.176],[1482451765308,20.177],[1482451789308,20.178],[1482451813308,20.178],[1482451837308,20.179],[1482451861308,20.181],[1482451885308,20.179],[1482451909308,20.179],[1482451933308,20.182],[1482451957308,20.182],[1482451981308,20.182],[1482452005308,20.181],[1482452029308,20.181],[1482452053308,20.183],[1482452077308,20.184],[1482452101308,20.185],[1482452125308,20.185],[1482452149308,20.187],[1482452173308,20.186],[1482452197308,20.186],[1482452221308,20.189],[1482452245308,20.191],[1482452269308,20.189],[1482452293308,20.189],[1482452317308,20.192],[1482452341308,20.192],[1482452365308,20.192],[1482452389308,20.192],[1482452413308,20.191],[1482452437308,20.193],[1482452461308,20.193],[1482452485308,20.193],[1482452509308,20.195],[1482452533308,20.195],[1482452557308,20.195],[1482452581308,20.195],[1482452605308,20.195],[1482452629308,20.196],[1482452653308,20.197],[1482452677308,20.198],[1482452701308,20.199],[1482452725308,20.199],[1482452749308,20.199],[1482452773308,20.197],[1482452797308,20.198],[1482452821308,20.200],[1482452845308,20.201],[1482452869308,20.203],[1482452893308,20.202],[1482452917308,20.203],[1482452941308,20.204],[1482452965308,20.204],[1482452989308,20.204],[1482453013308,20.204],[1482453037308,20.209],[1482453061308,20.207],[1482453085308,20.208],[1482453109308,20.209],[1482453133308,20.210],[1482453157308,20.210],[1482453181308,20.212],[1482453205308,20.213],[1482453229308,20.216],[1482453253308,20.215],[1482453277308,20.216],[1482453301308,20.215],[1482453325308,20.215],[1482453349308,20.216],[1482453373308,20.217],[1482453397308,20.220],[1482453421308,20.222],[1482453445308,20.226],[1482453469308,20.229],[1482453493308,20.232],[1482453517308,20.234],[1482453541308,20.236],[1482453565308,20.240],[1482453589308,20.243],[1482453613308,20.244],[1482453637308,20.244],[1482453661308,20.247],[1482453685308,20.247],[1482453709308,20.250],[1482453733308,20.253],[1482453757308,20.253],[1482453781308,20.255],[1482453805308,20.256],[1482453829308,20.254],[1482453853308,20.254],[1482453877308,20.258],[1482453901308,20.263],[1482453925308,20.262],[1482453949308,20.264],[1482453973308,20.264],[1482453997308,20.266],[1482454021308,20.266],[1482454045308,20.268],[1482454069308,20.272],[1482454093308,20.275],[1482454117308,20.270],[1482454141308,20.266],[1482454165308,20.264],[1482454189308,20.263],[1482454213308,20.263],[1482454237308,20.265],[1482454261308,20.263],[1482454285308,20.266],[1482454309308,20.267],[1482454333308,20.267],[1482454357308,20.266],[1482454381308,20.267],[1482454405308,20.267],[1482454429308,20.267],[1482454453308,20.269],[1482454477308,20.267],[1482454501308,20.270],[1482454525308,20.272],[1482454549308,20.272],[1482454573308,20.271],[1482454597308,20.270],[1482454621308,20.274],[1482454645308,20.273],[1482454669308,20.272],[1482454693308,20.271],[1482454717308,20.275],[1482454741308,20.277],[1482454765308,20.282],[1482454789308,20.279],[1482454813308,20.278],[1482454837308,20.277],[1482454861308,20.278],[1482454885308,20.278],[1482454909308,20.278],[1482454933308,20.279],[1482454957308,20.279],[1482454981308,20.280],[1482455005308,20.281],[1482455029308,20.282],[1482455053308,20.282],[1482455077308,20.283],[1482455101308,20.284],[1482455125308,20.286],[1482455149308,20.289],[1482455173308,20.288],[1482455197308,20.291],[1482455221308,20.288],[1482455245308,20.290],[1482455269308,20.292],[1482455293308,20.293],[1482455317308,20.294],[1482455341308,20.296],[1482455365308,20.297],[1482455389308,20.299],[1482455413308,20.300],[1482455437308,20.302],[1482455461308,20.302],[1482455485308,20.303],[1482455509308,20.304],[1482455533308,20.305],[1482455557308,20.308],[1482455581308,20.308],[1482455605308,20.308],[1482455629308,20.308],[1482455653308,20.311],[1482455677308,20.311]],\n  "last":[[1482448501308,20.020],[1482448525308,20.023],[1482448549308,20.025],[1482448573308,20.023],[1482448597308,20.027],[1482448621308,20.027],[1482448645308,20.031],[1482448669308,20.033],[1482448693308,20.033],[1482448717308,20.035],[1482448741308,20.039],[1482448765308,20.038],[1482448789308,20.041],[1482448813308,20.044],[1482448837308,20.043],[1482448861308,20.044],[1482448885308,20.047],[1482448909308,20.048],[1482448933308,20.051],[1482448957308,20.054],[1482448981308,20.054],[1482449005308,20.058],[1482449029308,20.055],[1482449053308,20.056],[1482449077308,20.059],[1482449101308,20.060],[1482449125308,20.060],[1482449149308,20.063],[1482449173308,20.063],[1482449197308,20.062],[1482449221308,20.068],[1482449245308,20.063],[1482449269308,20.067],[1482449293308,20.067],[1482449317308,20.067],[1482449341308,20.067],[1482449365308,20.068],[1482449389308,20.070],[1482449413308,20.073],[1482449437308,20.074],[1482449461308,20.072],[1482449485308,20.074],[1482449509308,20.073],[1482449533308,20.076],[1482449557308,20.075],[1482449581308,20.081],[1482449605308,20.079],[1482449629308,20.078],[1482449653308,20.080],[1482449677308,20.081],[1482449701308,20.082],[1482449725308,20.086],[1482449749308,20.087],[1482449773308,20.086],[1482449797308,20.087],[1482449821308,20.088],[1482449845308,20.087],[1482449869308,20.088],[1482449893308,20.088],[1482449917308,20.090],[1482449941308,20.090],[1482449965308,20.091],[1482449989308,20.095],[1482450013308,20.095],[1482450037308,20.095],[1482450061308,20.094],[1482450085308,20.095],[1482450109308,20.096],[1482450133308,20.098],[1482450157308,20.100],[1482450181308,20.101],[1482450205308,20.102],[1482450229308,20.101],[1482450253308,20.106],[1482450277308,20.107],[1482450301308,20.108],[1482450325308,20.111],[1482450349308,20.111],[1482450373308,20.114],[1482450397308,20.114],[1482450421308,20.117],[1482450445308,20.118],[1482450469308,20.118],[1482450493308,20.120],[1482450517308,20.121],[1482450541308,20.122],[1482450565308,20.125],[1482450589308,20.127],[1482450613308,20.127],[1482450637308,20.126],[1482450661308,20.130],[1482450685308,20.131],[1482450709308,20.132],[1482450733308,20.132],[1482450757308,20.137],[1482450781308,20.139],[1482450805308,20.136],[1482450829308,20.136],[1482450853308,20.137],[1482450877308,20.142],[1482450901308,20.144],[1482450925308,20.141],[1482450949308,20.145],[1482450973308,20.145],[1482450997308,20.146],[1482451021308,20.146],[1482451045308,20.147],[1482451069308,20.150],[1482451093308,20.149],[1482451117308,20.151],[1482451141308,20.154],[1482451165308,20.152],[1482451189308,20.154],[1482451213308,20.156],[1482451237308,20.153],[1482451261308,20.155],[1482451285308,20.154],[1482451309308,20.158],[1482451333308,20.157],[1482451357308,20.159],[1482451381308,20.161],[1482451405308,20.163],[1482451429308,20.164],[1482451453308,20.166],[1482451477308,20.163],[1482451501308,20.167],[1482451525308,20.167],[1482451549308,20.165],[1482451573308,20.166],[1482451597308,20.167],[1482451621308,20.169],[1482451645308,20.171],[1482451669308,20.170],[1482451693308,20.173],[1482451717308,20.173],[1482451741308,20.173],[1482451765308,20.176],[1482451789308,20.176],[1482451813308,20.175],[1482451837308,20.173],[1482451861308,20.178],[1482451885308,20.177],[1482451909308,20.174],[1482451933308,20.179],[1482451957308,20.179],[1482451981308,20.180],[1482452005308,20.180],[1482452029308,20.180],[1482452053308,20.180],[1482452077308,20.182],[1482452101308,20.183],[1482452125308,20.183],[1482452149308,20.184],[1482452173308,20.185],[1482452197308,20.186],[1482452221308,20.184],[1482452245308,20.188],[1482452269308,20.189],[1482452293308,20.186],[1482452317308,20.189],[1482452341308,20.191],[1482452365308,20.191],[1482452389308,20.191],[1482452413308,20.190],[1482452437308,20.191],[1482452461308,20.190],[1482452485308,20.191],[1482452509308,20.190],[1482452533308,20.194],[1482452557308,20.193],[1482452581308,20.195],[1482452605308,20.194],[1482452629308,20.194],[1482452653308,20.194],[1482452677308,20.196],[1482452701308,20.193],[1482452725308,20.199],[1482452749308,20.199],[1482452773308,20.197],[1482452797308,20.194],[1482452821308,20.198],[1482452845308,20.197],[1482452869308,20.200],[1482452893308,20.199],[1482452917308,20.201],[1482452941308,20.202],[1482452965308,20.202],[1482452989308,20.204],[1482453013308,20.203],[1482453037308,20.203],[1482453061308,20.207],[1482453085308,20.206],[1482453109308,20.208],[1482453133308,20.207],[1482453157308,20.207],[1482453181308,20.211],[1482453205308,20.212],[1482453229308,20.214],[1482453253308,20.215],[1482453277308,20.214],[1482453301308,20.212],[1482453325308,20.214],[1482453349308,20.216],[1482453373308,20.214],[1482453397308,20.215],[1482453421308,20.219],[1482453445308,20.222],[1482453469308,20.225],[1482453493308,20.226],[1482453517308,20.231],[1482453541308,20.233],[1482453565308,20.236],[1482453589308,20.237],[1482453613308,20.244],[1482453637308,20.243],[1482453661308,20.245],[1482453685308,20.244],[1482453709308,20.246],[1482453733308,20.250],[1482453757308,20.249],[1482453781308,20.250],[1482453805308,20.252],[1482453829308,20.254],[1482453853308,20.253],[1482453877308,20.254],[1482453901308,20.257],[1482453925308,20.262],[1482453949308,20.262],[1482453973308,20.262],[1482453997308,20.262],[1482454021308,20.263],[1482454045308,20.267],[1482454069308,20.267],[1482454093308,20.272],[1482454117308,20.268],[1482454141308,20.266],[1482454165308,20.263],[1482454189308,20.263],[1482454213308,20.261],[1482454237308,20.263],[1482454261308,20.263],[1482454285308,20.264],[1482454309308,20.264],[1482454333308,20.266],[1482454357308,20.266],[1482454381308,20.262],[1482454405308,20.264],[1482454429308,20.267],[1482454453308,20.265],[1482454477308,20.267],[1482454501308,20.268],[1482454525308,20.268],[1482454549308,20.271],[1482454573308,20.271],[1482454597308,20.270],[1482454621308,20.271],[1482454645308,20.268],[1482454669308,20.272],[1482454693308,20.271],[1482454717308,20.270],[1482454741308,20.274],[1482454765308,20.275],[1482454789308,20.277],[1482454813308,20.276],[1482454837308,20.276],[1482454861308,20.276],[1482454885308,20.276],[1482454909308,20.277],[1482454933308,20.277],[1482454957308,20.278],[1482454981308,20.278],[1482455005308,20.278],[1482455029308,20.281],[1482455053308,20.280],[1482455077308,20.280],[1482455101308,20.282],[1482455125308,20.284],[1482455149308,20.285],[1482455173308,20.286],[1482455197308,20.288],[1482455221308,20.286],[1482455245308,20.288],[1482455269308,20.289],[1482455293308,20.290],[1482455317308,20.292],[1482455341308,20.293],[1482455365308,20.296],[1482455389308,20.297],[1482455413308,20.298],[1482455437308,20.296],[1482455461308,20.300],[1482455485308,20.298],[1482455509308,20.301],[1482455533308,20.302],[1482455557308,20.304],[1482455581308,20.308],[1482455605308,20.307],[1482455629308,20.305],[1482455653308,20.309],[1482455677308,20.309]],\n  "min":[[1482448501308,20.020],[1482448525308,20.023],[1482448549308,20.023],[1482448573308,20.022],[1482448597308,20.025],[1482448621308,20.026],[1482448645308,20.029],[1482448669308,20.032],[1482448693308,20.033],[1482448717308,20.035],[1482448741308,20.038],[1482448765308,20.037],[1482448789308,20.040],[1482448813308,20.041],[1482448837308,20.043],[1482448861308,20.044],[1482448885308,20.047],[1482448909308,20.047],[1482448933308,20.050],[1482448957308,20.050],[1482448981308,20.053],[1482449005308,20.055],[1482449029308,20.055],[1482449053308,20.056],[1482449077308,20.058],[1482449101308,20.060],[1482449125308,20.059],[1482449149308,20.062],[1482449173308,20.060],[1482449197308,20.061],[1482449221308,20.063],[1482449245308,20.062],[1482449269308,20.066],[1482449293308,20.065],[1482449317308,20.065],[1482449341308,20.067],[1482449365308,20.067],[1482449389308,20.068],[1482449413308,20.071],[1482449437308,20.070],[1482449461308,20.072],[1482449485308,20.071],[1482449509308,20.071],[1482449533308,20.073],[1482449557308,20.075],[1482449581308,20.079],[1482449605308,20.078],[1482449629308,20.078],[1482449653308,20.078],[1482449677308,20.080],[1482449701308,20.082],[1482449725308,20.085],[1482449749308,20.083],[1482449773308,20.085],[1482449797308,20.086],[1482449821308,20.087],[1482449845308,20.086],[1482449869308,20.087],[1482449893308,20.087],[1482449917308,20.088],[1482449941308,20.088],[1482449965308,20.090],[1482449989308,20.091],[1482450013308,20.094],[1482450037308,20.093],[1482450061308,20.092],[1482450085308,20.094],[1482450109308,20.096],[1482450133308,20.097],[1482450157308,20.098],[1482450181308,20.100],[1482450205308,20.100],[1482450229308,20.100],[1482450253308,20.103],[1482450277308,20.106],[1482450301308,20.106],[1482450325308,20.107],[1482450349308,20.110],[1482450373308,20.114],[1482450397308,20.113],[1482450421308,20.115],[1482450445308,20.114],[1482450469308,20.118],[1482450493308,20.120],[1482450517308,20.121],[1482450541308,20.122],[1482450565308,20.123],[1482450589308,20.126],[1482450613308,20.125],[1482450637308,20.124],[1482450661308,20.128],[1482450685308,20.127],[1482450709308,20.130],[1482450733308,20.131],[1482450757308,20.132],[1482450781308,20.134],[1482450805308,20.136],[1482450829308,20.135],[1482450853308,20.136],[1482450877308,20.141],[1482450901308,20.140],[1482450925308,20.141],[1482450949308,20.142],[1482450973308,20.142],[1482450997308,20.142],[1482451021308,20.146],[1482451045308,20.147],[1482451069308,20.148],[1482451093308,20.146],[1482451117308,20.150],[1482451141308,20.152],[1482451165308,20.151],[1482451189308,20.153],[1482451213308,20.153],[1482451237308,20.153],[1482451261308,20.153],[1482451285308,20.154],[1482451309308,20.156],[1482451333308,20.156],[1482451357308,20.159],[1482451381308,20.160],[1482451405308,20.161],[1482451429308,20.160],[1482451453308,20.162],[1482451477308,20.162],[1482451501308,20.165],[1482451525308,20.166],[1482451549308,20.165],[1482451573308,20.166],[1482451597308,20.167],[1482451621308,20.168],[1482451645308,20.170],[1482451669308,20.170],[1482451693308,20.170],[1482451717308,20.171],[1482451741308,20.172],[1482451765308,20.173],[1482451789308,20.173],[1482451813308,20.173],[1482451837308,20.172],[1482451861308,20.177],[1482451885308,20.172],[1482451909308,20.174],[1482451933308,20.178],[1482451957308,20.179],[1482451981308,20.180],[1482452005308,20.179],[1482452029308,20.178],[1482452053308,20.180],[1482452077308,20.182],[1482452101308,20.182],[1482452125308,20.181],[1482452149308,20.184],[1482452173308,20.184],[1482452197308,20.182],[1482452221308,20.184],[1482452245308,20.188],[1482452269308,20.186],[1482452293308,20.184],[1482452317308,20.189],[1482452341308,20.187],[1482452365308,20.190],[1482452389308,20.189],[1482452413308,20.188],[1482452437308,20.188],[1482452461308,20.190],[1482452485308,20.190],[1482452509308,20.189],[1482452533308,20.192],[1482452557308,20.191],[1482452581308,20.193],[1482452605308,20.192],[1482452629308,20.192],[1482452653308,20.194],[1482452677308,20.194],[1482452701308,20.192],[1482452725308,20.194],[1482452749308,20.196],[1482452773308,20.195],[1482452797308,20.194],[1482452821308,20.197],[1482452845308,20.196],[1482452869308,20.199],[1482452893308,20.199],[1482452917308,20.201],[1482452941308,20.201],[1482452965308,20.200],[1482452989308,20.202],[1482453013308,20.202],[1482453037308,20.201],[1482453061308,20.203],[1482453085308,20.205],[1482453109308,20.204],[1482453133308,20.206],[1482453157308,20.206],[1482453181308,20.210],[1482453205308,20.210],[1482453229308,20.211],[1482453253308,20.212],[1482453277308,20.212],[1482453301308,20.212],[1482453325308,20.212],[1482453349308,20.214],[1482453373308,20.214],[1482453397308,20.215],[1482453421308,20.219],[1482453445308,20.221],[1482453469308,20.225],[1482453493308,20.225],[1482453517308,20.230],[1482453541308,20.232],[1482453565308,20.234],[1482453589308,20.233],[1482453613308,20.241],[1482453637308,20.241],[1482453661308,20.244],[1482453685308,20.244],[1482453709308,20.246],[1482453733308,20.249],[1482453757308,20.249],[1482453781308,20.250],[1482453805308,20.251],[1482453829308,20.251],[1482453853308,20.251],[1482453877308,20.254],[1482453901308,20.257],[1482453925308,20.259],[1482453949308,20.261],[1482453973308,20.261],[1482453997308,20.261],[1482454021308,20.262],[1482454045308,20.265],[1482454069308,20.266],[1482454093308,20.269],[1482454117308,20.266],[1482454141308,20.262],[1482454165308,20.260],[1482454189308,20.259],[1482454213308,20.261],[1482454237308,20.259],[1482454261308,20.260],[1482454285308,20.263],[1482454309308,20.264],[1482454333308,20.264],[1482454357308,20.262],[1482454381308,20.262],[1482454405308,20.264],[1482454429308,20.265],[1482454453308,20.265],[1482454477308,20.264],[1482454501308,20.266],[1482454525308,20.268],[1482454549308,20.270],[1482454573308,20.267],[1482454597308,20.267],[1482454621308,20.270],[1482454645308,20.266],[1482454669308,20.268],[1482454693308,20.269],[1482454717308,20.270],[1482454741308,20.272],[1482454765308,20.274],[1482454789308,20.277],[1482454813308,20.273],[1482454837308,20.274],[1482454861308,20.275],[1482454885308,20.273],[1482454909308,20.276],[1482454933308,20.276],[1482454957308,20.276],[1482454981308,20.277],[1482455005308,20.275],[1482455029308,20.277],[1482455053308,20.279],[1482455077308,20.280],[1482455101308,20.280],[1482455125308,20.282],[1482455149308,20.285],[1482455173308,20.285],[1482455197308,20.285],[1482455221308,20.285],[1482455245308,20.286],[1482455269308,20.288],[1482455293308,20.289],[1482455317308,20.292],[1482455341308,20.292],[1482455365308,20.294],[1482455389308,20.296],[1482455413308,20.296],[1482455437308,20.295],[1482455461308,20.298],[1482455485308,20.297],[1482455509308,20.301],[1482455533308,20.300],[1482455557308,20.303],[1482455581308,20.304],[1482455605308,20.305],[1482455629308,20.304],[1482455653308,20.307],[1482455677308,20.308]]\n}'
_trenddata_12hr_agitation_json = '{\n  "max":[[1482412621623,0.0000],[1482412765623,0.0000],[1482412909623,0.0000],[1482413053623,0.0000],[1482413197623,0.0000],[1482413341623,0.0000],[1482413485623,0.0000],[1482413629623,0.0000],[1482413773623,0.0000],[1482413917623,0.0000],[1482414061623,0.0000],[1482414205623,0.0000],[1482414349623,0.0000],[1482414493623,0.0000],[1482414637623,0.0000],[1482414781623,0.0000],[1482414925623,0.0000],[1482415069623,0.0000],[1482415213623,0.0000],[1482415357623,0.0000],[1482415501623,0.0000],[1482415645623,0.0000],[1482415789623,0.0000],[1482415933623,0.0000],[1482416077623,0.0000],[1482416221623,0.0000],[1482416365623,0.0000],[1482416509623,0.0000],[1482416653623,0.0000],[1482416797623,0.0000],[1482416941623,0.0000],[1482417085623,0.0000],[1482417229623,0.0000],[1482417373623,0.0000],[1482417517623,0.0000],[1482417661623,0.0000],[1482417805623,0.0000],[1482417949623,0.0000],[1482418093623,0.0000],[1482418237623,0.0000],[1482418381623,0.0000],[1482418525623,0.0000],[1482418669623,0.0000],[1482418813623,0.0000],[1482418957623,0.0000],[1482419101623,0.0000],[1482419245623,0.0000],[1482419389623,0.0000],[1482419533623,0.0000],[1482419677623,0.0000],[1482419821623,0.0000],[1482419965623,0.0000],[1482420109623,0.0000],[1482420253623,0.0000],[1482420397623,0.0000],[1482420541623,0.0000],[1482420685623,0.0000],[1482420829623,0.0000],[1482420973623,0.0000],[1482421117623,0.0000],[1482421261623,0.0000],[1482421405623,0.0000],[1482421549623,0.0000],[1482421693623,0.0000],[1482421837623,0.0000],[1482421981623,0.0000],[1482422125623,0.0000],[1482422269623,0.0000],[1482422413623,0.0000],[1482422557623,0.0000],[1482422701623,0.0000],[1482422845623,0.0000],[1482422989623,0.0000],[1482423133623,0.0000],[1482423277623,0.0000],[1482423421623,0.0000],[1482423565623,0.0000],[1482423709623,0.0000],[1482423853623,0.0000],[1482423997623,0.0000],[1482424141623,0.0000],[1482424285623,0.0000],[1482424429623,0.0000],[1482424573623,0.0000],[1482424717623,0.0000],[1482424861623,0.0000],[1482425005623,0.0000],[1482425149623,0.0000],[1482425293623,0.0000],[1482425437623,0.0000],[1482425581623,0.0000],[1482425725623,0.0000],[1482425869623,0.0000],[1482426013623,0.0000],[1482426157623,0.0000],[1482426301623,0.0000],[1482426445623,0.0000],[1482426589623,0.0000],[1482426733623,0.0000],[1482426877623,0.0000],[1482427021623,0.0000],[1482427165623,0.0000],[1482427309623,0.0000],[1482427453623,0.0000],[1482427597623,0.0000],[1482427741623,0.0000],[1482427885623,0.0000],[1482428029623,0.0000],[1482428173623,0.0000],[1482428317623,0.0000],[1482428461623,0.0000],[1482428605623,0.0000],[1482428749623,0.0000],[1482428893623,0.0000],[1482429037623,0.0000],[1482429181623,0.0000],[1482429325623,0.0000],[1482429469623,0.0000],[1482429613623,0.0000],[1482429757623,0.0000],[1482429901623,0.0000],[1482430045623,0.0000],[1482430189623,0.0000],[1482430333623,0.0000],[1482430477623,0.0000],[1482430621623,0.0000],[1482430765623,0.0000],[1482430909623,0.0000],[1482431053623,0.0000],[1482431197623,0.0000],[1482431341623,0.0000],[1482431485623,0.0000],[1482431629623,0.0000],[1482431773623,0.0000],[1482431917623,0.0000],[1482432061623,0.0000],[1482432205623,0.0000],[1482432349623,0.0000],[1482432493623,0.0000],[1482432637623,0.0000],[1482432781623,0.0000],[1482432925623,0.0000],[1482433069623,0.0000],[1482433213623,0.0000],[1482433357623,0.0000],[1482433501623,0.0000],[1482433645623,0.0000],[1482433789623,0.0000],[1482433933623,0.0000],[1482434077623,0.0000],[1482434221623,0.0000],[1482434365623,0.0000],[1482434509623,0.0000],[1482434653623,0.0000],[1482434797623,0.0000],[1482434941623,0.0000],[1482435085623,0.0000],[1482435229623,0.0000],[1482435373623,0.0000],[1482435517623,0.0000],[1482435661623,0.0000],[1482435805623,0.0000],[1482435949623,0.0000],[1482436093623,0.0000],[1482436237623,0.0000],[1482436381623,0.0000],[1482436525623,0.0000],[1482436669623,0.0000],[1482436813623,0.0000],[1482436957623,0.0000],[1482437101623,0.0000],[1482437245623,0.0000],[1482437389623,0.0000],[1482437533623,0.0000],[1482437677623,0.0000],[1482437821623,0.0000],[1482437965623,0.0000],[1482438109623,0.0000],[1482438253623,0.0000],[1482438397623,0.0000],[1482438541623,0.0000],[1482438685623,0.0000],[1482438829623,0.0000],[1482438973623,0.0000],[1482439117623,0.0000],[1482439261623,0.0000],[1482439405623,0.0000],[1482439549623,0.0000],[1482439693623,0.0000],[1482439837623,0.0000],[1482439981623,0.0000],[1482440125623,0.0000],[1482440269623,0.0000],[1482440413623,0.0000],[1482440557623,0.0000],[1482440701623,0.0000],[1482440845623,0.0000],[1482440989623,0.0000],[1482441133623,0.0000],[1482441277623,0.0000],[1482441421623,0.0000],[1482441565623,0.0000],[1482441709623,0.0000],[1482441853623,0.0000],[1482441997623,0.0000],[1482442141623,0.0000],[1482442285623,0.0000],[1482442429623,0.0000],[1482442573623,0.0000],[1482442717623,0.0000],[1482442861623,0.0000],[1482443005623,0.0000],[1482443149623,0.0000],[1482443293623,0.0000],[1482443437623,0.0000],[1482443581623,0.0000],[1482443725623,0.0000],[1482443869623,0.0000],[1482444013623,16.324],[1482444157623,20.031],[1482444301623,20.032],[1482444445623,20.038],[1482444589623,20.037],[1482444733623,20.039],[1482444877623,20.035],[1482445021623,20.038],[1482445165623,20.037],[1482445309623,20.042],[1482445453623,20.042],[1482445597623,20.043],[1482445741623,20.040],[1482445885623,0.0000],[1482446029623,0.0000],[1482446173623,0.0000],[1482446317623,0.0000],[1482446461623,0.0000],[1482446605623,0.0000],[1482446749623,0.0000],[1482446893623,0.0000],[1482447037623,0.0000],[1482447181623,0.0000],[1482447325623,0.0000],[1482447469623,0.0000],[1482447613623,0.0000],[1482447757623,0.0000],[1482447901623,0.0000],[1482448045623,12.057],[1482448189623,12.058],[1482448333623,35.305],[1482448477623,36.028],[1482448621623,29.281],[1482448765623,12.062],[1482448909623,12.061],[1482449053623,12.060],[1482449197623,12.061],[1482449341623,12.060],[1482449485623,12.061],[1482449629623,12.061],[1482449773623,12.062],[1482449917623,12.061],[1482450061623,12.060],[1482450205623,12.060],[1482450349623,12.062],[1482450493623,12.062],[1482450637623,12.063],[1482450781623,12.060],[1482450925623,12.058],[1482451069623,12.060],[1482451213623,12.059],[1482451357623,12.060],[1482451501623,12.059],[1482451645623,12.060],[1482451789623,12.059],[1482451933623,12.060],[1482452077623,12.063],[1482452221623,12.063],[1482452365623,12.061],[1482452509623,12.061],[1482452653623,12.059],[1482452797623,12.060],[1482452941623,12.062],[1482453085623,12.059],[1482453229623,12.061],[1482453373623,12.059],[1482453517623,0.0000],[1482453661623,0.0000],[1482453805623,0.0000],[1482453949623,0.0000],[1482454093623,8.4473],[1482454237623,8.0742],[1482454381623,12.058],[1482454525623,20.039],[1482454669623,22.355],[1482454813623,36.030],[1482454957623,36.042],[1482455101623,36.041],[1482455245623,36.048],[1482455389623,36.040],[1482455533623,36.048],[1482455677623,36.031]],\n  "last":[[1482412621623,0.0000],[1482412765623,0.0000],[1482412909623,0.0000],[1482413053623,0.0000],[1482413197623,0.0000],[1482413341623,0.0000],[1482413485623,0.0000],[1482413629623,0.0000],[1482413773623,0.0000],[1482413917623,0.0000],[1482414061623,0.0000],[1482414205623,0.0000],[1482414349623,0.0000],[1482414493623,0.0000],[1482414637623,0.0000],[1482414781623,0.0000],[1482414925623,0.0000],[1482415069623,0.0000],[1482415213623,0.0000],[1482415357623,0.0000],[1482415501623,0.0000],[1482415645623,0.0000],[1482415789623,0.0000],[1482415933623,0.0000],[1482416077623,0.0000],[1482416221623,0.0000],[1482416365623,0.0000],[1482416509623,0.0000],[1482416653623,0.0000],[1482416797623,0.0000],[1482416941623,0.0000],[1482417085623,0.0000],[1482417229623,0.0000],[1482417373623,0.0000],[1482417517623,0.0000],[1482417661623,0.0000],[1482417805623,0.0000],[1482417949623,0.0000],[1482418093623,0.0000],[1482418237623,0.0000],[1482418381623,0.0000],[1482418525623,0.0000],[1482418669623,0.0000],[1482418813623,0.0000],[1482418957623,0.0000],[1482419101623,0.0000],[1482419245623,0.0000],[1482419389623,0.0000],[1482419533623,0.0000],[1482419677623,0.0000],[1482419821623,0.0000],[1482419965623,0.0000],[1482420109623,0.0000],[1482420253623,0.0000],[1482420397623,0.0000],[1482420541623,0.0000],[1482420685623,0.0000],[1482420829623,0.0000],[1482420973623,0.0000],[1482421117623,0.0000],[1482421261623,0.0000],[1482421405623,0.0000],[1482421549623,0.0000],[1482421693623,0.0000],[1482421837623,0.0000],[1482421981623,0.0000],[1482422125623,0.0000],[1482422269623,0.0000],[1482422413623,0.0000],[1482422557623,0.0000],[1482422701623,0.0000],[1482422845623,0.0000],[1482422989623,0.0000],[1482423133623,0.0000],[1482423277623,0.0000],[1482423421623,0.0000],[1482423565623,0.0000],[1482423709623,0.0000],[1482423853623,0.0000],[1482423997623,0.0000],[1482424141623,0.0000],[1482424285623,0.0000],[1482424429623,0.0000],[1482424573623,0.0000],[1482424717623,0.0000],[1482424861623,0.0000],[1482425005623,0.0000],[1482425149623,0.0000],[1482425293623,0.0000],[1482425437623,0.0000],[1482425581623,0.0000],[1482425725623,0.0000],[1482425869623,0.0000],[1482426013623,0.0000],[1482426157623,0.0000],[1482426301623,0.0000],[1482426445623,0.0000],[1482426589623,0.0000],[1482426733623,0.0000],[1482426877623,0.0000],[1482427021623,0.0000],[1482427165623,0.0000],[1482427309623,0.0000],[1482427453623,0.0000],[1482427597623,0.0000],[1482427741623,0.0000],[1482427885623,0.0000],[1482428029623,0.0000],[1482428173623,0.0000],[1482428317623,0.0000],[1482428461623,0.0000],[1482428605623,0.0000],[1482428749623,0.0000],[1482428893623,0.0000],[1482429037623,0.0000],[1482429181623,0.0000],[1482429325623,0.0000],[1482429469623,0.0000],[1482429613623,0.0000],[1482429757623,0.0000],[1482429901623,0.0000],[1482430045623,0.0000],[1482430189623,0.0000],[1482430333623,0.0000],[1482430477623,0.0000],[1482430621623,0.0000],[1482430765623,0.0000],[1482430909623,0.0000],[1482431053623,0.0000],[1482431197623,0.0000],[1482431341623,0.0000],[1482431485623,0.0000],[1482431629623,0.0000],[1482431773623,0.0000],[1482431917623,0.0000],[1482432061623,0.0000],[1482432205623,0.0000],[1482432349623,0.0000],[1482432493623,0.0000],[1482432637623,0.0000],[1482432781623,0.0000],[1482432925623,0.0000],[1482433069623,0.0000],[1482433213623,0.0000],[1482433357623,0.0000],[1482433501623,0.0000],[1482433645623,0.0000],[1482433789623,0.0000],[1482433933623,0.0000],[1482434077623,0.0000],[1482434221623,0.0000],[1482434365623,0.0000],[1482434509623,0.0000],[1482434653623,0.0000],[1482434797623,0.0000],[1482434941623,0.0000],[1482435085623,0.0000],[1482435229623,0.0000],[1482435373623,0.0000],[1482435517623,0.0000],[1482435661623,0.0000],[1482435805623,0.0000],[1482435949623,0.0000],[1482436093623,0.0000],[1482436237623,0.0000],[1482436381623,0.0000],[1482436525623,0.0000],[1482436669623,0.0000],[1482436813623,0.0000],[1482436957623,0.0000],[1482437101623,0.0000],[1482437245623,0.0000],[1482437389623,0.0000],[1482437533623,0.0000],[1482437677623,0.0000],[1482437821623,0.0000],[1482437965623,0.0000],[1482438109623,0.0000],[1482438253623,0.0000],[1482438397623,0.0000],[1482438541623,0.0000],[1482438685623,0.0000],[1482438829623,0.0000],[1482438973623,0.0000],[1482439117623,0.0000],[1482439261623,0.0000],[1482439405623,0.0000],[1482439549623,0.0000],[1482439693623,0.0000],[1482439837623,0.0000],[1482439981623,0.0000],[1482440125623,0.0000],[1482440269623,0.0000],[1482440413623,0.0000],[1482440557623,0.0000],[1482440701623,0.0000],[1482440845623,0.0000],[1482440989623,0.0000],[1482441133623,0.0000],[1482441277623,0.0000],[1482441421623,0.0000],[1482441565623,0.0000],[1482441709623,0.0000],[1482441853623,0.0000],[1482441997623,0.0000],[1482442141623,0.0000],[1482442285623,0.0000],[1482442429623,0.0000],[1482442573623,0.0000],[1482442717623,0.0000],[1482442861623,0.0000],[1482443005623,0.0000],[1482443149623,0.0000],[1482443293623,0.0000],[1482443437623,0.0000],[1482443581623,0.0000],[1482443725623,0.0000],[1482443869623,0.0000],[1482444013623,0.0000],[1482444157623,16.985],[1482444301623,20.003],[1482444445623,20.010],[1482444589623,20.006],[1482444733623,20.013],[1482444877623,20.017],[1482445021623,19.865],[1482445165623,20.013],[1482445309623,20.011],[1482445453623,20.012],[1482445597623,20.021],[1482445741623,20.014],[1482445885623,0.0000],[1482446029623,0.0000],[1482446173623,0.0000],[1482446317623,0.0000],[1482446461623,0.0000],[1482446605623,0.0000],[1482446749623,0.0000],[1482446893623,0.0000],[1482447037623,0.0000],[1482447181623,0.0000],[1482447325623,0.0000],[1482447469623,0.0000],[1482447613623,0.0000],[1482447757623,0.0000],[1482447901623,0.0000],[1482448045623,0.0000],[1482448189623,12.055],[1482448333623,0.0000],[1482448477623,35.286],[1482448621623,29.281],[1482448765623,12.060],[1482448909623,11.843],[1482449053623,11.923],[1482449197623,12.047],[1482449341623,11.833],[1482449485623,12.060],[1482449629623,12.059],[1482449773623,11.837],[1482449917623,12.047],[1482450061623,12.048],[1482450205623,11.828],[1482450349623,12.058],[1482450493623,12.046],[1482450637623,12.046],[1482450781623,11.884],[1482450925623,12.055],[1482451069623,12.058],[1482451213623,12.057],[1482451357623,12.046],[1482451501623,12.046],[1482451645623,12.059],[1482451789623,12.058],[1482451933623,11.845],[1482452077623,11.858],[1482452221623,12.050],[1482452365623,11.853],[1482452509623,12.061],[1482452653623,12.047],[1482452797623,11.995],[1482452941623,12.048],[1482453085623,12.049],[1482453229623,12.055],[1482453373623,11.838],[1482453517623,0.0000],[1482453661623,0.0000],[1482453805623,0.0000],[1482453949623,0.0000],[1482454093623,0.0000],[1482454237623,8.0702],[1482454381623,0.0000],[1482454525623,12.044],[1482454669623,20.014],[1482454813623,0.0000],[1482454957623,36.042],[1482455101623,36.009],[1482455245623,36.014],[1482455389623,35.988],[1482455533623,35.988],[1482455677623,36.006]],\n  "min":[[1482412621623,0.0000],[1482412765623,0.0000],[1482412909623,0.0000],[1482413053623,0.0000],[1482413197623,0.0000],[1482413341623,0.0000],[1482413485623,0.0000],[1482413629623,0.0000],[1482413773623,0.0000],[1482413917623,0.0000],[1482414061623,0.0000],[1482414205623,0.0000],[1482414349623,0.0000],[1482414493623,0.0000],[1482414637623,0.0000],[1482414781623,0.0000],[1482414925623,0.0000],[1482415069623,0.0000],[1482415213623,0.0000],[1482415357623,0.0000],[1482415501623,0.0000],[1482415645623,0.0000],[1482415789623,0.0000],[1482415933623,0.0000],[1482416077623,0.0000],[1482416221623,0.0000],[1482416365623,0.0000],[1482416509623,0.0000],[1482416653623,0.0000],[1482416797623,0.0000],[1482416941623,0.0000],[1482417085623,0.0000],[1482417229623,0.0000],[1482417373623,0.0000],[1482417517623,0.0000],[1482417661623,0.0000],[1482417805623,0.0000],[1482417949623,0.0000],[1482418093623,0.0000],[1482418237623,0.0000],[1482418381623,0.0000],[1482418525623,0.0000],[1482418669623,0.0000],[1482418813623,0.0000],[1482418957623,0.0000],[1482419101623,0.0000],[1482419245623,0.0000],[1482419389623,0.0000],[1482419533623,0.0000],[1482419677623,0.0000],[1482419821623,0.0000],[1482419965623,0.0000],[1482420109623,0.0000],[1482420253623,0.0000],[1482420397623,0.0000],[1482420541623,0.0000],[1482420685623,0.0000],[1482420829623,0.0000],[1482420973623,0.0000],[1482421117623,0.0000],[1482421261623,0.0000],[1482421405623,0.0000],[1482421549623,0.0000],[1482421693623,0.0000],[1482421837623,0.0000],[1482421981623,0.0000],[1482422125623,0.0000],[1482422269623,0.0000],[1482422413623,0.0000],[1482422557623,0.0000],[1482422701623,0.0000],[1482422845623,0.0000],[1482422989623,0.0000],[1482423133623,0.0000],[1482423277623,0.0000],[1482423421623,0.0000],[1482423565623,0.0000],[1482423709623,0.0000],[1482423853623,0.0000],[1482423997623,0.0000],[1482424141623,0.0000],[1482424285623,0.0000],[1482424429623,0.0000],[1482424573623,0.0000],[1482424717623,0.0000],[1482424861623,0.0000],[1482425005623,0.0000],[1482425149623,0.0000],[1482425293623,0.0000],[1482425437623,0.0000],[1482425581623,0.0000],[1482425725623,0.0000],[1482425869623,0.0000],[1482426013623,0.0000],[1482426157623,0.0000],[1482426301623,0.0000],[1482426445623,0.0000],[1482426589623,0.0000],[1482426733623,0.0000],[1482426877623,0.0000],[1482427021623,0.0000],[1482427165623,0.0000],[1482427309623,0.0000],[1482427453623,0.0000],[1482427597623,0.0000],[1482427741623,0.0000],[1482427885623,0.0000],[1482428029623,0.0000],[1482428173623,0.0000],[1482428317623,0.0000],[1482428461623,0.0000],[1482428605623,0.0000],[1482428749623,0.0000],[1482428893623,0.0000],[1482429037623,0.0000],[1482429181623,0.0000],[1482429325623,0.0000],[1482429469623,0.0000],[1482429613623,0.0000],[1482429757623,0.0000],[1482429901623,0.0000],[1482430045623,0.0000],[1482430189623,0.0000],[1482430333623,0.0000],[1482430477623,0.0000],[1482430621623,0.0000],[1482430765623,0.0000],[1482430909623,0.0000],[1482431053623,0.0000],[1482431197623,0.0000],[1482431341623,0.0000],[1482431485623,0.0000],[1482431629623,0.0000],[1482431773623,0.0000],[1482431917623,0.0000],[1482432061623,0.0000],[1482432205623,0.0000],[1482432349623,0.0000],[1482432493623,0.0000],[1482432637623,0.0000],[1482432781623,0.0000],[1482432925623,0.0000],[1482433069623,0.0000],[1482433213623,0.0000],[1482433357623,0.0000],[1482433501623,0.0000],[1482433645623,0.0000],[1482433789623,0.0000],[1482433933623,0.0000],[1482434077623,0.0000],[1482434221623,0.0000],[1482434365623,0.0000],[1482434509623,0.0000],[1482434653623,0.0000],[1482434797623,0.0000],[1482434941623,0.0000],[1482435085623,0.0000],[1482435229623,0.0000],[1482435373623,0.0000],[1482435517623,0.0000],[1482435661623,0.0000],[1482435805623,0.0000],[1482435949623,0.0000],[1482436093623,0.0000],[1482436237623,0.0000],[1482436381623,0.0000],[1482436525623,0.0000],[1482436669623,0.0000],[1482436813623,0.0000],[1482436957623,0.0000],[1482437101623,0.0000],[1482437245623,0.0000],[1482437389623,0.0000],[1482437533623,0.0000],[1482437677623,0.0000],[1482437821623,0.0000],[1482437965623,0.0000],[1482438109623,0.0000],[1482438253623,0.0000],[1482438397623,0.0000],[1482438541623,0.0000],[1482438685623,0.0000],[1482438829623,0.0000],[1482438973623,0.0000],[1482439117623,0.0000],[1482439261623,0.0000],[1482439405623,0.0000],[1482439549623,0.0000],[1482439693623,0.0000],[1482439837623,0.0000],[1482439981623,0.0000],[1482440125623,0.0000],[1482440269623,0.0000],[1482440413623,0.0000],[1482440557623,0.0000],[1482440701623,0.0000],[1482440845623,0.0000],[1482440989623,0.0000],[1482441133623,0.0000],[1482441277623,0.0000],[1482441421623,0.0000],[1482441565623,0.0000],[1482441709623,0.0000],[1482441853623,0.0000],[1482441997623,0.0000],[1482442141623,0.0000],[1482442285623,0.0000],[1482442429623,0.0000],[1482442573623,0.0000],[1482442717623,0.0000],[1482442861623,0.0000],[1482443005623,0.0000],[1482443149623,0.0000],[1482443293623,0.0000],[1482443437623,0.0000],[1482443581623,0.0000],[1482443725623,0.0000],[1482443869623,0.0000],[1482444013623,0.0000],[1482444157623,16.985],[1482444301623,19.849],[1482444445623,19.812],[1482444589623,19.843],[1482444733623,19.804],[1482444877623,19.817],[1482445021623,19.826],[1482445165623,19.819],[1482445309623,19.772],[1482445453623,19.835],[1482445597623,19.847],[1482445741623,0.0000],[1482445885623,0.0000],[1482446029623,0.0000],[1482446173623,0.0000],[1482446317623,0.0000],[1482446461623,0.0000],[1482446605623,0.0000],[1482446749623,0.0000],[1482446893623,0.0000],[1482447037623,0.0000],[1482447181623,0.0000],[1482447325623,0.0000],[1482447469623,0.0000],[1482447613623,0.0000],[1482447757623,0.0000],[1482447901623,0.0000],[1482448045623,0.0000],[1482448189623,0.0000],[1482448333623,0.0000],[1482448477623,29.281],[1482448621623,11.854],[1482448765623,11.806],[1482448909623,11.831],[1482449053623,11.815],[1482449197623,11.801],[1482449341623,11.809],[1482449485623,11.808],[1482449629623,11.811],[1482449773623,11.803],[1482449917623,11.810],[1482450061623,11.828],[1482450205623,11.821],[1482450349623,11.839],[1482450493623,11.814],[1482450637623,11.818],[1482450781623,11.810],[1482450925623,11.816],[1482451069623,11.828],[1482451213623,11.808],[1482451357623,11.820],[1482451501623,11.837],[1482451645623,11.813],[1482451789623,11.810],[1482451933623,11.833],[1482452077623,11.820],[1482452221623,11.807],[1482452365623,11.801],[1482452509623,11.791],[1482452653623,11.827],[1482452797623,11.822],[1482452941623,11.827],[1482453085623,11.831],[1482453229623,11.827],[1482453373623,0.0000],[1482453517623,0.0000],[1482453661623,0.0000],[1482453805623,0.0000],[1482453949623,0.0000],[1482454093623,0.0000],[1482454237623,0.0000],[1482454381623,0.0000],[1482454525623,0.0000],[1482454669623,0.0000],[1482454813623,0.0000],[1482454957623,35.951],[1482455101623,35.964],[1482455245623,35.957],[1482455389623,35.960],[1482455533623,35.956],[1482455677623,35.956]]\n}'
_trenddata_12hr_do_json = '{\n  "max":[[1482412621867,-0.24269],[1482412765867,-0.23498],[1482412909867,-0.23921],[1482413053867,-0.24110],[1482413197867,-0.23413],[1482413341867,-0.24417],[1482413485867,-0.24364],[1482413629867,-0.24403],[1482413773867,-0.24485],[1482413917867,-0.24503],[1482414061867,-0.23784],[1482414205867,-0.23913],[1482414349867,-0.22149],[1482414493867,-0.24081],[1482414637867,-0.24529],[1482414781867,-0.24407],[1482414925867,-0.23937],[1482415069867,-0.24271],[1482415213867,-0.23995],[1482415357867,-0.23880],[1482415501867,-0.23199],[1482415645867,-0.23192],[1482415789867,-0.24233],[1482415933867,-0.23058],[1482416077867,-0.22646],[1482416221867,-0.24053],[1482416365867,-0.22927],[1482416509867,-0.23644],[1482416653867,-0.24413],[1482416797867,-0.24384],[1482416941867,-0.23515],[1482417085867,-0.24482],[1482417229867,-0.24356],[1482417373867,-0.23652],[1482417517867,-0.24010],[1482417661867,-0.24394],[1482417805867,-0.23727],[1482417949867,-0.24016],[1482418093867,-0.23593],[1482418237867,-0.24298],[1482418381867,-0.23351],[1482418525867,-0.24043],[1482418669867,-0.23992],[1482418813867,-0.24154],[1482418957867,-0.24016],[1482419101867,-0.23645],[1482419245867,-0.22742],[1482419389867,-0.24019],[1482419533867,-0.24474],[1482419677867,-0.22796],[1482419821867,-0.23056],[1482419965867,-0.23277],[1482420109867,-0.23546],[1482420253867,-0.23450],[1482420397867,-0.24217],[1482420541867,-0.23943],[1482420685867,-0.22789],[1482420829867,-0.22296],[1482420973867,-0.24432],[1482421117867,-0.23882],[1482421261867,-0.23510],[1482421405867,-0.24192],[1482421549867,-0.23885],[1482421693867,-0.23716],[1482421837867,-0.23475],[1482421981867,-0.24162],[1482422125867,-0.23242],[1482422269867,-0.23409],[1482422413867,-0.23872],[1482422557867,-0.24523],[1482422701867,-0.24005],[1482422845867,-0.24406],[1482422989867,-0.24273],[1482423133867,-0.24421],[1482423277867,-0.23996],[1482423421867,-0.23490],[1482423565867,-0.23542],[1482423709867,-0.23072],[1482423853867,-0.22956],[1482423997867,-0.24042],[1482424141867,-0.24333],[1482424285867,-0.23783],[1482424429867,-0.22955],[1482424573867,-0.23662],[1482424717867,-0.23586],[1482424861867,-0.23943],[1482425005867,-0.23709],[1482425149867,-0.23063],[1482425293867,-0.23593],[1482425437867,-0.23311],[1482425581867,-0.24483],[1482425725867,-0.24567],[1482425869867,-0.24546],[1482426013867,-0.23996],[1482426157867,-0.24363],[1482426301867,-0.24132],[1482426445867,-0.23906],[1482426589867,-0.23698],[1482426733867,-0.23114],[1482426877867,-0.23970],[1482427021867,-0.23283],[1482427165867,-0.24005],[1482427309867,-0.24489],[1482427453867,-0.24578],[1482427597867,-0.24105],[1482427741867,-0.24209],[1482427885867,-0.23970],[1482428029867,-0.24422],[1482428173867,-0.23154],[1482428317867,-0.23746],[1482428461867,-0.24243],[1482428605867,-0.24067],[1482428749867,-0.24526],[1482428893867,-0.24495],[1482429037867,-0.23944],[1482429181867,-0.24432],[1482429325867,-0.23627],[1482429469867,-0.24525],[1482429613867,-0.24509],[1482429757867,-0.24115],[1482429901867,-0.23769],[1482430045867,-0.24202],[1482430189867,-0.24513],[1482430333867,-0.24492],[1482430477867,-0.24530],[1482430621867,-0.24335],[1482430765867,-0.23196],[1482430909867,-0.24010],[1482431053867,-0.24124],[1482431197867,-0.24560],[1482431341867,-0.24570],[1482431485867,-0.24541],[1482431629867,-0.24539],[1482431773867,-0.24533],[1482431917867,-0.24505],[1482432061867,-0.23995],[1482432205867,-0.23972],[1482432349867,-0.23798],[1482432493867,-0.24066],[1482432637867,-0.23803],[1482432781867,-0.24175],[1482432925867,-0.23960],[1482433069867,-0.24510],[1482433213867,-0.24430],[1482433357867,-0.24495],[1482433501867,-0.24510],[1482433645867,-0.24401],[1482433789867,-0.24255],[1482433933867,-0.24420],[1482434077867,-0.24469],[1482434221867,-0.24418],[1482434365867,-0.24428],[1482434509867,-0.24076],[1482434653867,-0.23818],[1482434797867,-0.24316],[1482434941867,-0.24299],[1482435085867,-0.24088],[1482435229867,-0.24411],[1482435373867,-0.23909],[1482435517867,-0.23815],[1482435661867,-0.23454],[1482435805867,-0.23644],[1482435949867,-0.24163],[1482436093867,-0.24339],[1482436237867,-0.23996],[1482436381867,-0.24115],[1482436525867,-0.24114],[1482436669867,-0.24432],[1482436813867,-0.23982],[1482436957867,-0.23759],[1482437101867,-0.23483],[1482437245867,-0.23735],[1482437389867,-0.24496],[1482437533867,-0.24434],[1482437677867,-0.24297],[1482437821867,-0.23795],[1482437965867,-0.23787],[1482438109867,-0.23025],[1482438253867,-0.23846],[1482438397867,-0.24389],[1482438541867,-0.23225],[1482438685867,-0.23256],[1482438829867,-0.23926],[1482438973867,-0.23504],[1482439117867,-0.24401],[1482439261867,-0.24086],[1482439405867,-0.23944],[1482439549867,-0.24016],[1482439693867,-0.24428],[1482439837867,-0.24312],[1482439981867,-0.23817],[1482440125867,-0.22725],[1482440269867,-0.24430],[1482440413867,-0.24329],[1482440557867,-0.23807],[1482440701867,-0.23665],[1482440845867,-0.23938],[1482440989867,-0.24049],[1482441133867,-0.24456],[1482441277867,-0.24284],[1482441421867,-0.23715],[1482441565867,-0.22935],[1482441709867,-0.24227],[1482441853867,-0.24374],[1482441997867,-0.24368],[1482442141867,-0.24278],[1482442285867,-0.24070],[1482442429867,-0.23433],[1482442573867,-0.23784],[1482442717867,-0.24411],[1482442861867,-0.22831],[1482443005867,-0.24318],[1482443149867,-0.24332],[1482443293867,-0.23606],[1482443437867,-0.23835],[1482443581867,-0.24249],[1482443725867,-0.24243],[1482443869867,-0.24098],[1482444013867,-0.24352],[1482444157867,-0.24417],[1482444301867,-0.24426],[1482444445867,-0.23607],[1482444589867,-0.22562],[1482444733867,-0.23560],[1482444877867,-0.24178],[1482445021867,-0.24243],[1482445165867,-0.24356],[1482445309867,-0.24318],[1482445453867,-0.24329],[1482445597867,-0.24393],[1482445741867,-0.24417],[1482445885867,-0.24462],[1482446029867,-0.24295],[1482446173867,-0.24424],[1482446317867,-0.24032],[1482446461867,-0.24289],[1482446605867,-0.24339],[1482446749867,-0.24366],[1482446893867,-0.24393],[1482447037867,-0.23764],[1482447181867,-0.24275],[1482447325867,-0.24335],[1482447469867,-0.24390],[1482447613867,-0.24062],[1482447757867,-0.24321],[1482447901867,-0.24324],[1482448045867,-0.23783],[1482448189867,-0.24406],[1482448333867,-0.23004],[1482448477867,-0.24241],[1482448621867,-0.24456],[1482448765867,-0.24355],[1482448909867,-0.24359],[1482449053867,-0.24377],[1482449197867,-0.24305],[1482449341867,-0.24355],[1482449485867,-0.24305],[1482449629867,-0.24407],[1482449773867,-0.24111],[1482449917867,-0.24250],[1482450061867,-0.24468],[1482450205867,-0.23944],[1482450349867,-0.22990],[1482450493867,-0.23638],[1482450637867,-0.24181],[1482450781867,-0.23938],[1482450925867,-0.23584],[1482451069867,-0.23475],[1482451213867,-0.23737],[1482451357867,-0.23410],[1482451501867,-0.24428],[1482451645867,-0.24294],[1482451789867,-0.24267],[1482451933867,-0.24422],[1482452077867,-0.24434],[1482452221867,-0.24468],[1482452365867,-0.24508],[1482452509867,-0.24551],[1482452653867,-0.24539],[1482452797867,-0.24506],[1482452941867,-0.24464],[1482453085867,-0.24513],[1482453229867,-0.24523],[1482453373867,-0.24437],[1482453517867,-0.24475],[1482453661867,-0.24523],[1482453805867,-0.24512],[1482453949867,-0.24478],[1482454093867,-0.24554],[1482454237867,-0.24485],[1482454381867,-0.24527],[1482454525867,-0.24393],[1482454669867,-0.24396],[1482454813867,-0.24455],[1482454957867,-0.24458],[1482455101867,-0.24364],[1482455245867,-0.24387],[1482455389867,-0.24358],[1482455533867,-0.24430],[1482455677867,-0.24403]],\n  "last":[[1482412621867,-0.24786],[1482412765867,-0.24690],[1482412909867,-0.24629],[1482413053867,-0.24704],[1482413197867,-0.24743],[1482413341867,-0.24680],[1482413485867,-0.24642],[1482413629867,-0.24635],[1482413773867,-0.24603],[1482413917867,-0.24718],[1482414061867,-0.25303],[1482414205867,-0.24689],[1482414349867,-0.24611],[1482414493867,-0.24662],[1482414637867,-0.24706],[1482414781867,-0.24656],[1482414925867,-0.24718],[1482415069867,-0.24585],[1482415213867,-0.24690],[1482415357867,-0.24667],[1482415501867,-0.24416],[1482415645867,-0.24632],[1482415789867,-0.24359],[1482415933867,-0.24643],[1482416077867,-0.25201],[1482416221867,-0.24651],[1482416365867,-0.24711],[1482416509867,-0.24812],[1482416653867,-0.24657],[1482416797867,-0.24745],[1482416941867,-0.24731],[1482417085867,-0.24653],[1482417229867,-0.24683],[1482417373867,-0.24721],[1482417517867,-0.24701],[1482417661867,-0.24866],[1482417805867,-0.24563],[1482417949867,-0.24804],[1482418093867,-0.24648],[1482418237867,-0.24623],[1482418381867,-0.24711],[1482418525867,-0.24704],[1482418669867,-0.24657],[1482418813867,-0.24723],[1482418957867,-0.24711],[1482419101867,-0.24681],[1482419245867,-0.24389],[1482419389867,-0.24686],[1482419533867,-0.24683],[1482419677867,-0.24585],[1482419821867,-0.24627],[1482419965867,-0.24643],[1482420109867,-0.24597],[1482420253867,-0.24372],[1482420397867,-0.24544],[1482420541867,-0.24603],[1482420685867,-0.24461],[1482420829867,-0.24540],[1482420973867,-0.24716],[1482421117867,-0.24618],[1482421261867,-0.24667],[1482421405867,-0.24693],[1482421549867,-0.24614],[1482421693867,-0.24608],[1482421837867,-0.24465],[1482421981867,-0.24760],[1482422125867,-0.24717],[1482422269867,-0.24685],[1482422413867,-0.24679],[1482422557867,-0.24578],[1482422701867,-0.24636],[1482422845867,-0.24683],[1482422989867,-0.24595],[1482423133867,-0.24710],[1482423277867,-0.24649],[1482423421867,-0.26289],[1482423565867,-0.25149],[1482423709867,-0.24714],[1482423853867,-0.24818],[1482423997867,-0.24690],[1482424141867,-0.24683],[1482424285867,-0.25013],[1482424429867,-0.24594],[1482424573867,-0.25142],[1482424717867,-0.24554],[1482424861867,-0.24677],[1482425005867,-0.24730],[1482425149867,-0.24709],[1482425293867,-0.24818],[1482425437867,-0.24700],[1482425581867,-0.24760],[1482425725867,-0.24734],[1482425869867,-0.24697],[1482426013867,-0.24645],[1482426157867,-0.24870],[1482426301867,-0.24704],[1482426445867,-0.24672],[1482426589867,-0.24762],[1482426733867,-0.23114],[1482426877867,-0.24844],[1482427021867,-0.24690],[1482427165867,-0.24839],[1482427309867,-0.24704],[1482427453867,-0.24694],[1482427597867,-0.24672],[1482427741867,-0.24721],[1482427885867,-0.24761],[1482428029867,-0.24758],[1482428173867,-0.24665],[1482428317867,-0.24846],[1482428461867,-0.24662],[1482428605867,-0.24758],[1482428749867,-0.24727],[1482428893867,-0.24774],[1482429037867,-0.24810],[1482429181867,-0.24628],[1482429325867,-0.24727],[1482429469867,-0.24622],[1482429613867,-0.24778],[1482429757867,-0.24897],[1482429901867,-0.24641],[1482430045867,-0.24645],[1482430189867,-0.24707],[1482430333867,-0.24677],[1482430477867,-0.24679],[1482430621867,-0.24709],[1482430765867,-0.24348],[1482430909867,-0.24607],[1482431053867,-0.24667],[1482431197867,-0.24781],[1482431341867,-0.24728],[1482431485867,-0.24768],[1482431629867,-0.24908],[1482431773867,-0.24908],[1482431917867,-0.24689],[1482432061867,-0.24646],[1482432205867,-0.24823],[1482432349867,-0.24858],[1482432493867,-0.24747],[1482432637867,-0.24669],[1482432781867,-0.24645],[1482432925867,-0.24703],[1482433069867,-0.24667],[1482433213867,-0.24770],[1482433357867,-0.24611],[1482433501867,-0.24741],[1482433645867,-0.24726],[1482433789867,-0.24703],[1482433933867,-0.24743],[1482434077867,-0.24772],[1482434221867,-0.24685],[1482434365867,-0.24642],[1482434509867,-0.24667],[1482434653867,-0.24726],[1482434797867,-0.24633],[1482434941867,-0.24738],[1482435085867,-0.24713],[1482435229867,-0.24663],[1482435373867,-0.24714],[1482435517867,-0.24619],[1482435661867,-0.24641],[1482435805867,-0.24629],[1482435949867,-0.24615],[1482436093867,-0.24709],[1482436237867,-0.24138],[1482436381867,-0.24764],[1482436525867,-0.24114],[1482436669867,-0.24641],[1482436813867,-0.24603],[1482436957867,-0.24734],[1482437101867,-0.24752],[1482437245867,-0.24652],[1482437389867,-0.24673],[1482437533867,-0.24757],[1482437677867,-0.24754],[1482437821867,-0.24617],[1482437965867,-0.24965],[1482438109867,-0.24641],[1482438253867,-0.24744],[1482438397867,-0.24669],[1482438541867,-0.24523],[1482438685867,-0.24667],[1482438829867,-0.24641],[1482438973867,-0.24740],[1482439117867,-0.24605],[1482439261867,-0.24767],[1482439405867,-0.24520],[1482439549867,-0.24856],[1482439693867,-0.24844],[1482439837867,-0.24642],[1482439981867,-0.24509],[1482440125867,-0.24792],[1482440269867,-0.24839],[1482440413867,-0.24670],[1482440557867,-0.24594],[1482440701867,-0.24648],[1482440845867,-0.24741],[1482440989867,-0.24049],[1482441133867,-0.24795],[1482441277867,-0.24523],[1482441421867,-0.24656],[1482441565867,-0.24846],[1482441709867,-0.24642],[1482441853867,-0.24486],[1482441997867,-0.24830],[1482442141867,-0.24813],[1482442285867,-0.24672],[1482442429867,-0.24730],[1482442573867,-0.24532],[1482442717867,-0.24826],[1482442861867,-0.24724],[1482443005867,-0.24981],[1482443149867,-0.24733],[1482443293867,-0.26678],[1482443437867,-0.24638],[1482443581867,-0.25503],[1482443725867,-0.24533],[1482443869867,-0.24652],[1482444013867,-0.24451],[1482444157867,-0.24660],[1482444301867,-0.24847],[1482444445867,-0.24632],[1482444589867,-0.23347],[1482444733867,-0.24503],[1482444877867,-0.25030],[1482445021867,-0.24587],[1482445165867,-0.24834],[1482445309867,-0.24762],[1482445453867,-0.24486],[1482445597867,-0.24741],[1482445741867,-0.24977],[1482445885867,-0.24666],[1482446029867,-0.24976],[1482446173867,-0.24594],[1482446317867,-0.24847],[1482446461867,-0.24583],[1482446605867,-0.24604],[1482446749867,-0.24642],[1482446893867,-0.24710],[1482447037867,-0.24553],[1482447181867,-0.24781],[1482447325867,-0.24690],[1482447469867,-0.24681],[1482447613867,-0.24843],[1482447757867,-0.24561],[1482447901867,-0.24645],[1482448045867,-0.24931],[1482448189867,-0.24559],[1482448333867,-0.24649],[1482448477867,-0.24604],[1482448621867,-0.24474],[1482448765867,-0.24643],[1482448909867,-0.24892],[1482449053867,-0.24553],[1482449197867,-0.24703],[1482449341867,-0.24776],[1482449485867,-0.24792],[1482449629867,-0.24619],[1482449773867,-0.24757],[1482449917867,-0.24717],[1482450061867,-0.24911],[1482450205867,-0.24730],[1482450349867,-0.24594],[1482450493867,-0.25102],[1482450637867,-0.24819],[1482450781867,-0.24697],[1482450925867,-0.24569],[1482451069867,-0.24844],[1482451213867,-0.24757],[1482451357867,-0.23638],[1482451501867,-0.24689],[1482451645867,-0.25005],[1482451789867,-0.24758],[1482451933867,-0.24770],[1482452077867,-0.24902],[1482452221867,-0.24556],[1482452365867,-0.24857],[1482452509867,-0.24676],[1482452653867,-0.24757],[1482452797867,-0.24750],[1482452941867,-0.24730],[1482453085867,-0.24737],[1482453229867,-0.24687],[1482453373867,-0.24747],[1482453517867,-0.24713],[1482453661867,-0.24663],[1482453805867,-0.24679],[1482453949867,-0.24686],[1482454093867,-0.24689],[1482454237867,-0.24747],[1482454381867,-0.24673],[1482454525867,-0.24770],[1482454669867,-0.24580],[1482454813867,-0.24724],[1482454957867,-0.24836],[1482455101867,-0.24796],[1482455245867,-0.24788],[1482455389867,-0.24856],[1482455533867,-0.24516],[1482455677867,-0.24842]],\n  "min":[[1482412621867,-0.24868],[1482412765867,-0.25241],[1482412909867,-0.25177],[1482413053867,-0.25091],[1482413197867,-0.25827],[1482413341867,-0.25453],[1482413485867,-0.25484],[1482413629867,-0.24938],[1482413773867,-0.24876],[1482413917867,-0.25473],[1482414061867,-0.26276],[1482414205867,-0.25259],[1482414349867,-0.26219],[1482414493867,-0.25269],[1482414637867,-0.25011],[1482414781867,-0.24959],[1482414925867,-0.24874],[1482415069867,-0.24935],[1482415213867,-0.26297],[1482415357867,-0.26329],[1482415501867,-0.26487],[1482415645867,-0.26028],[1482415789867,-0.25183],[1482415933867,-0.26308],[1482416077867,-0.26383],[1482416221867,-0.25404],[1482416365867,-0.25682],[1482416509867,-0.25521],[1482416653867,-0.24955],[1482416797867,-0.25053],[1482416941867,-0.24958],[1482417085867,-0.25147],[1482417229867,-0.25346],[1482417373867,-0.26202],[1482417517867,-0.25496],[1482417661867,-0.25310],[1482417805867,-0.25072],[1482417949867,-0.25258],[1482418093867,-0.25552],[1482418237867,-0.25019],[1482418381867,-0.25210],[1482418525867,-0.25336],[1482418669867,-0.25112],[1482418813867,-0.25732],[1482418957867,-0.25603],[1482419101867,-0.25156],[1482419245867,-0.25867],[1482419389867,-0.24915],[1482419533867,-0.24928],[1482419677867,-0.26741],[1482419821867,-0.25878],[1482419965867,-0.24953],[1482420109867,-0.26530],[1482420253867,-0.25752],[1482420397867,-0.25467],[1482420541867,-0.25180],[1482420685867,-0.25976],[1482420829867,-0.26331],[1482420973867,-0.24881],[1482421117867,-0.25235],[1482421261867,-0.25371],[1482421405867,-0.24951],[1482421549867,-0.25172],[1482421693867,-0.26069],[1482421837867,-0.25869],[1482421981867,-0.24918],[1482422125867,-0.25266],[1482422269867,-0.25491],[1482422413867,-0.26131],[1482422557867,-0.24951],[1482422701867,-0.25193],[1482422845867,-0.24897],[1482422989867,-0.25414],[1482423133867,-0.25514],[1482423277867,-0.25620],[1482423421867,-0.26289],[1482423565867,-0.25383],[1482423709867,-0.26865],[1482423853867,-0.25887],[1482423997867,-0.25186],[1482424141867,-0.25775],[1482424285867,-0.26736],[1482424429867,-0.27434],[1482424573867,-0.26856],[1482424717867,-0.26711],[1482424861867,-0.25320],[1482425005867,-0.25652],[1482425149867,-0.25719],[1482425293867,-0.26032],[1482425437867,-0.25966],[1482425581867,-0.25282],[1482425725867,-0.24997],[1482425869867,-0.24840],[1482426013867,-0.25130],[1482426157867,-0.24935],[1482426301867,-0.27155],[1482426445867,-0.26359],[1482426589867,-0.25991],[1482426733867,-0.25258],[1482426877867,-0.25436],[1482427021867,-0.25770],[1482427165867,-0.25976],[1482427309867,-0.25002],[1482427453867,-0.25123],[1482427597867,-0.26447],[1482427741867,-0.25391],[1482427885867,-0.26328],[1482428029867,-0.25594],[1482428173867,-0.25647],[1482428317867,-0.25354],[1482428461867,-0.25527],[1482428605867,-0.25950],[1482428749867,-0.26893],[1482428893867,-0.25414],[1482429037867,-0.25397],[1482429181867,-0.25477],[1482429325867,-0.25824],[1482429469867,-0.25258],[1482429613867,-0.24975],[1482429757867,-0.25000],[1482429901867,-0.26104],[1482430045867,-0.24920],[1482430189867,-0.25255],[1482430333867,-0.25330],[1482430477867,-0.24922],[1482430621867,-0.25180],[1482430765867,-0.26362],[1482430909867,-0.25504],[1482431053867,-0.25360],[1482431197867,-0.24931],[1482431341867,-0.24975],[1482431485867,-0.24915],[1482431629867,-0.24908],[1482431773867,-0.24908],[1482431917867,-0.25017],[1482432061867,-0.25370],[1482432205867,-0.25776],[1482432349867,-0.25837],[1482432493867,-0.25235],[1482432637867,-0.26430],[1482432781867,-0.24959],[1482432925867,-0.24867],[1482433069867,-0.24948],[1482433213867,-0.24856],[1482433357867,-0.24898],[1482433501867,-0.24857],[1482433645867,-0.24873],[1482433789867,-0.25432],[1482433933867,-0.24966],[1482434077867,-0.24876],[1482434221867,-0.24857],[1482434365867,-0.24837],[1482434509867,-0.24904],[1482434653867,-0.24992],[1482434797867,-0.25216],[1482434941867,-0.24908],[1482435085867,-0.24920],[1482435229867,-0.24815],[1482435373867,-0.25123],[1482435517867,-0.25093],[1482435661867,-0.25256],[1482435805867,-0.25395],[1482435949867,-0.24888],[1482436093867,-0.25026],[1482436237867,-0.25752],[1482436381867,-0.24938],[1482436525867,-0.24890],[1482436669867,-0.24892],[1482436813867,-0.24936],[1482436957867,-0.25571],[1482437101867,-0.24876],[1482437245867,-0.24997],[1482437389867,-0.25003],[1482437533867,-0.25596],[1482437677867,-0.25140],[1482437821867,-0.26079],[1482437965867,-0.25327],[1482438109867,-0.26444],[1482438253867,-0.25391],[1482438397867,-0.24959],[1482438541867,-0.25137],[1482438685867,-0.24959],[1482438829867,-0.25874],[1482438973867,-0.25983],[1482439117867,-0.25632],[1482439261867,-0.26128],[1482439405867,-0.25254],[1482439549867,-0.25067],[1482439693867,-0.25633],[1482439837867,-0.24904],[1482439981867,-0.25184],[1482440125867,-0.25449],[1482440269867,-0.24983],[1482440413867,-0.25426],[1482440557867,-0.25045],[1482440701867,-0.26011],[1482440845867,-0.25270],[1482440989867,-0.25081],[1482441133867,-0.24918],[1482441277867,-0.24948],[1482441421867,-0.25099],[1482441565867,-0.25538],[1482441709867,-0.25858],[1482441853867,-0.25057],[1482441997867,-0.25097],[1482442141867,-0.25426],[1482442285867,-0.26015],[1482442429867,-0.25619],[1482442573867,-0.25147],[1482442717867,-0.25628],[1482442861867,-0.26056],[1482443005867,-0.25432],[1482443149867,-0.26386],[1482443293867,-0.26678],[1482443437867,-0.26633],[1482443581867,-0.25503],[1482443725867,-0.25193],[1482443869867,-0.25694],[1482444013867,-0.25743],[1482444157867,-0.24976],[1482444301867,-0.25097],[1482444445867,-0.25833],[1482444589867,-0.25906],[1482444733867,-0.26321],[1482444877867,-0.26024],[1482445021867,-0.26305],[1482445165867,-0.25227],[1482445309867,-0.25646],[1482445453867,-0.25225],[1482445597867,-0.25714],[1482445741867,-0.25317],[1482445885867,-0.25102],[1482446029867,-0.25714],[1482446173867,-0.25055],[1482446317867,-0.25220],[1482446461867,-0.25030],[1482446605867,-0.25087],[1482446749867,-0.25030],[1482446893867,-0.25031],[1482447037867,-0.25407],[1482447181867,-0.25515],[1482447325867,-0.25370],[1482447469867,-0.25147],[1482447613867,-0.26542],[1482447757867,-0.25439],[1482447901867,-0.26492],[1482448045867,-0.25684],[1482448189867,-0.25087],[1482448333867,-0.25270],[1482448477867,-0.25358],[1482448621867,-0.25039],[1482448765867,-0.25139],[1482448909867,-0.25057],[1482449053867,-0.25167],[1482449197867,-0.25147],[1482449341867,-0.25125],[1482449485867,-0.25097],[1482449629867,-0.25027],[1482449773867,-0.25286],[1482449917867,-0.25118],[1482450061867,-0.25051],[1482450205867,-0.25314],[1482450349867,-0.26030],[1482450493867,-0.26010],[1482450637867,-0.26422],[1482450781867,-0.25622],[1482450925867,-0.26376],[1482451069867,-0.26519],[1482451213867,-0.26212],[1482451357867,-0.26529],[1482451501867,-0.25067],[1482451645867,-0.25013],[1482451789867,-0.25190],[1482451933867,-0.25381],[1482452077867,-0.25065],[1482452221867,-0.25006],[1482452365867,-0.24924],[1482452509867,-0.24972],[1482452653867,-0.24975],[1482452797867,-0.24962],[1482452941867,-0.24938],[1482453085867,-0.25063],[1482453229867,-0.24985],[1482453373867,-0.25005],[1482453517867,-0.24973],[1482453661867,-0.24992],[1482453805867,-0.25033],[1482453949867,-0.25060],[1482454093867,-0.25013],[1482454237867,-0.24958],[1482454381867,-0.24975],[1482454525867,-0.25003],[1482454669867,-0.25005],[1482454813867,-0.25072],[1482454957867,-0.25044],[1482455101867,-0.24983],[1482455245867,-0.24990],[1482455389867,-0.25006],[1482455533867,-0.25044],[1482455677867,-0.25035]]\n}'
_trenddata_12hr_ph_json = '{\n  "max":[[1482412622271,7.2837],[1482412766271,7.2847],[1482412910271,7.2837],[1482413054271,7.2819],[1482413198271,7.2823],[1482413342271,7.2821],[1482413486271,7.2820],[1482413630271,7.2819],[1482413774271,7.2810],[1482413918271,7.2822],[1482414062271,7.2820],[1482414206271,7.2830],[1482414350271,7.2825],[1482414494271,7.2832],[1482414638271,7.2836],[1482414782271,7.2832],[1482414926271,7.2819],[1482415070271,7.2808],[1482415214271,7.2826],[1482415358271,7.2822],[1482415502271,7.2825],[1482415646271,7.2819],[1482415790271,7.2818],[1482415934271,7.2814],[1482416078271,7.2812],[1482416222271,7.2807],[1482416366271,7.2832],[1482416510271,7.2836],[1482416654271,7.2832],[1482416798271,7.2815],[1482416942271,7.2808],[1482417086271,7.2800],[1482417230271,7.2804],[1482417374271,7.2808],[1482417518271,7.2804],[1482417662271,7.2798],[1482417806271,7.2798],[1482417950271,7.2796],[1482418094271,7.2811],[1482418238271,7.2809],[1482418382271,7.2806],[1482418526271,7.2805],[1482418670271,7.2806],[1482418814271,7.2800],[1482418958271,7.2802],[1482419102271,7.2804],[1482419246271,7.2796],[1482419390271,7.2795],[1482419534271,7.2805],[1482419678271,7.2797],[1482419822271,7.2808],[1482419966271,7.2809],[1482420110271,7.2811],[1482420254271,7.2796],[1482420398271,7.2786],[1482420542271,7.2789],[1482420686271,7.2793],[1482420830271,7.2790],[1482420974271,7.2780],[1482421118271,7.2782],[1482421262271,7.2790],[1482421406271,7.2770],[1482421550271,7.2782],[1482421694271,7.2784],[1482421838271,7.2783],[1482421982271,7.2790],[1482422126271,7.2795],[1482422270271,7.2795],[1482422414271,7.2783],[1482422558271,7.2784],[1482422702271,7.2782],[1482422846271,7.2794],[1482422990271,7.2774],[1482423134271,7.2775],[1482423278271,7.2773],[1482423422271,7.2776],[1482423566271,7.2770],[1482423710271,7.2775],[1482423854271,7.2782],[1482423998271,7.2782],[1482424142271,7.2760],[1482424286271,7.2769],[1482424430271,7.2769],[1482424574271,7.2711],[1482424718271,7.2721],[1482424862271,7.2726],[1482425006271,7.2718],[1482425150271,7.2721],[1482425294271,7.2717],[1482425438271,7.2731],[1482425582271,7.2731],[1482425726271,7.2732],[1482425870271,7.2718],[1482426014271,7.2713],[1482426158271,7.2719],[1482426302271,7.2720],[1482426446271,7.2720],[1482426590271,7.2721],[1482426734271,7.2727],[1482426878271,7.2721],[1482427022271,7.2682],[1482427166271,7.2674],[1482427310271,7.2692],[1482427454271,7.2681],[1482427598271,7.2679],[1482427742271,7.2681],[1482427886271,7.2682],[1482428030271,7.2692],[1482428174271,7.2692],[1482428318271,7.2693],[1482428462271,7.2693],[1482428606271,7.2693],[1482428750271,7.2695],[1482428894271,7.2797],[1482429038271,7.2713],[1482429182271,7.2718],[1482429326271,7.2693],[1482429470271,7.2705],[1482429614271,7.2692],[1482429758271,7.2690],[1482429902271,7.2795],[1482430046271,7.2795],[1482430190271,7.2700],[1482430334271,7.2693],[1482430478271,7.2693],[1482430622271,7.2718],[1482430766271,7.2720],[1482430910271,7.2708],[1482431054271,7.2710],[1482431198271,7.2705],[1482431342271,7.2692],[1482431486271,7.2696],[1482431630271,7.2693],[1482431774271,7.2692],[1482431918271,7.2692],[1482432062271,7.2691],[1482432206271,7.2693],[1482432350271,7.2707],[1482432494271,7.2714],[1482432638271,7.2705],[1482432782271,7.2692],[1482432926271,7.2693],[1482433070271,7.2691],[1482433214271,7.2688],[1482433358271,7.2679],[1482433502271,7.2675],[1482433646271,7.2667],[1482433790271,7.2680],[1482433934271,7.2679],[1482434078271,7.2731],[1482434222271,7.2761],[1482434366271,7.2756],[1482434510271,7.2756],[1482434654271,7.2732],[1482434798271,7.2735],[1482434942271,7.2743],[1482435086271,7.2731],[1482435230271,7.2768],[1482435374271,6.7438],[1482435518271,6.0015],[1482435662271,5.0775],[1482435806271,3.9425],[1482435950271,2.5351],[1482436094271,0.73273],[1482436238271,0.017009],[1482436382271,0.017028],[1482436526271,0.017210],[1482436670271,0.017339],[1482436814271,0.017431],[1482436958271,0.017790],[1482437102271,0.017778],[1482437246271,0.017806],[1482437390271,0.017735],[1482437534271,0.017802],[1482437678271,0.017988],[1482437822271,0.018100],[1482437966271,0.018183],[1482438110271,0.018731],[1482438254271,0.018539],[1482438398271,0.018627],[1482438542271,0.018860],[1482438686271,0.018957],[1482438830271,0.019060],[1482438974271,0.019145],[1482439118271,0.019202],[1482439262271,0.019406],[1482439406271,0.019471],[1482439550271,0.019599],[1482439694271,0.019711],[1482439838271,0.019881],[1482439982271,0.019990],[1482440126271,0.020407],[1482440270271,0.020151],[1482440414271,0.020138],[1482440558271,0.020621],[1482440702271,0.020769],[1482440846271,0.021007],[1482440990271,0.020912],[1482441134271,0.020536],[1482441278271,0.020567],[1482441422271,0.021283],[1482441566271,0.022072],[1482441710271,0.021552],[1482441854271,0.021216],[1482441998271,0.021767],[1482442142271,0.021864],[1482442286271,0.022001],[1482442430271,0.022138],[1482442574271,0.022268],[1482442718271,0.021884],[1482442862271,0.023213],[1482443006271,0.022200],[1482443150271,0.022713],[1482443294271,0.022944],[1482443438271,0.023007],[1482443582271,0.023139],[1482443726271,0.023459],[1482443870271,0.023232],[1482444014271,0.023374],[1482444158271,0.024793],[1482444302271,0.025576],[1482444446271,0.026822],[1482444590271,0.027156],[1482444734271,0.027086],[1482444878271,0.027309],[1482445022271,0.027450],[1482445166271,0.027623],[1482445310271,0.027760],[1482445454271,0.027989],[1482445598271,0.028066],[1482445742271,0.028288],[1482445886271,0.028845],[1482446030271,0.029061],[1482446174271,0.032281],[1482446318271,0.032843],[1482446462271,0.030843],[1482446606271,0.030817],[1482446750271,0.030618],[1482446894271,0.030282],[1482447038271,0.031850],[1482447182271,0.030335],[1482447326271,0.030645],[1482447470271,0.031288],[1482447614271,0.030303],[1482447758271,0.030380],[1482447902271,0.030301],[1482448046271,0.030471],[1482448190271,0.030545],[1482448334271,0.031396],[1482448478271,0.030832],[1482448622271,0.031086],[1482448766271,0.031299],[1482448910271,0.031504],[1482449054271,0.031766],[1482449198271,0.031906],[1482449342271,0.031965],[1482449486271,0.032100],[1482449630271,0.032176],[1482449774271,0.032426],[1482449918271,0.032500],[1482450062271,0.032523],[1482450206271,0.032918],[1482450350271,0.033189],[1482450494271,0.033595],[1482450638271,0.033432],[1482450782271,0.033731],[1482450926271,0.034278],[1482451070271,0.034446],[1482451214271,0.034500],[1482451358271,0.034691],[1482451502271,0.034319],[1482451646271,0.034433],[1482451790271,0.034555],[1482451934271,0.034438],[1482452078271,0.034759],[1482452222271,0.034642],[1482452366271,0.035084],[1482452510271,0.034724],[1482452654271,0.035156],[1482452798271,0.034831],[1482452942271,0.034962],[1482453086271,0.035285],[1482453230271,0.035379],[1482453374271,0.035463],[1482453518271,0.035803],[1482453662271,0.036785],[1482453806271,0.036169],[1482453950271,0.037546],[1482454094271,0.037906],[1482454238271,0.037912],[1482454382271,0.037344],[1482454526271,0.036593],[1482454670271,0.036772],[1482454814271,0.037427],[1482454958271,0.036916],[1482455102271,0.038098],[1482455246271,0.037984],[1482455390271,0.038118],[1482455534271,0.037496],[1482455678271,0.037669]],\n  "last":[[1482412622271,7.2795],[1482412766271,7.2818],[1482412910271,7.2808],[1482413054271,7.2792],[1482413198271,7.2798],[1482413342271,7.2818],[1482413486271,7.2794],[1482413630271,7.2807],[1482413774271,7.2780],[1482413918271,7.2803],[1482414062271,7.2805],[1482414206271,7.2803],[1482414350271,7.2811],[1482414494271,7.2798],[1482414638271,7.2806],[1482414782271,7.2820],[1482414926271,7.2793],[1482415070271,7.2794],[1482415214271,7.2807],[1482415358271,7.2804],[1482415502271,7.2798],[1482415646271,7.2815],[1482415790271,7.2794],[1482415934271,7.2794],[1482416078271,7.2795],[1482416222271,7.2794],[1482416366271,7.2801],[1482416510271,7.2815],[1482416654271,7.2796],[1482416798271,7.2784],[1482416942271,7.2781],[1482417086271,7.2781],[1482417230271,7.2782],[1482417374271,7.2795],[1482417518271,7.2781],[1482417662271,7.2781],[1482417806271,7.2772],[1482417950271,7.2792],[1482418094271,7.2781],[1482418238271,7.2794],[1482418382271,7.2806],[1482418526271,7.2768],[1482418670271,7.2778],[1482418814271,7.2777],[1482418958271,7.2770],[1482419102271,7.2781],[1482419246271,7.2787],[1482419390271,7.2765],[1482419534271,7.2794],[1482419678271,7.2794],[1482419822271,7.2794],[1482419966271,7.2794],[1482420110271,7.2785],[1482420254271,7.2794],[1482420398271,7.2774],[1482420542271,7.2756],[1482420686271,7.2770],[1482420830271,7.2782],[1482420974271,7.2756],[1482421118271,7.2768],[1482421262271,7.2763],[1482421406271,7.2756],[1482421550271,7.2744],[1482421694271,7.2764],[1482421838271,7.2756],[1482421982271,7.2753],[1482422126271,7.2786],[1482422270271,7.2788],[1482422414271,7.2759],[1482422558271,7.2756],[1482422702271,7.2769],[1482422846271,7.2768],[1482422990271,7.2756],[1482423134271,7.2761],[1482423278271,7.2757],[1482423422271,7.2738],[1482423566271,7.2745],[1482423710271,7.2747],[1482423854271,7.2770],[1482423998271,7.2762],[1482424142271,7.2748],[1482424286271,7.2751],[1482424430271,7.2756],[1482424574271,7.2698],[1482424718271,7.2705],[1482424862271,7.2705],[1482425006271,7.2713],[1482425150271,7.2705],[1482425294271,7.2691],[1482425438271,7.2698],[1482425582271,7.2718],[1482425726271,7.2717],[1482425870271,7.2706],[1482426014271,7.2692],[1482426158271,7.2693],[1482426302271,7.2705],[1482426446271,7.2706],[1482426590271,7.2704],[1482426734271,7.2723],[1482426878271,7.2704],[1482427022271,7.2665],[1482427166271,7.2654],[1482427310271,7.2654],[1482427454271,7.2666],[1482427598271,7.2668],[1482427742271,7.2667],[1482427886271,7.2657],[1482428030271,7.2679],[1482428174271,7.2668],[1482428318271,7.2673],[1482428462271,7.2679],[1482428606271,7.2677],[1482428750271,7.2683],[1482428894271,7.2687],[1482429038271,7.2700],[1482429182271,7.2705],[1482429326271,7.2672],[1482429470271,7.2684],[1482429614271,7.2666],[1482429758271,7.2679],[1482429902271,7.2677],[1482430046271,7.2795],[1482430190271,7.2671],[1482430334271,7.2680],[1482430478271,7.2667],[1482430622271,7.2692],[1482430766271,7.2706],[1482430910271,7.2692],[1482431054271,7.2690],[1482431198271,7.2679],[1482431342271,7.2679],[1482431486271,7.2691],[1482431630271,7.2688],[1482431774271,7.2668],[1482431918271,7.2675],[1482432062271,7.2687],[1482432206271,7.2675],[1482432350271,7.2666],[1482432494271,7.2679],[1482432638271,7.2695],[1482432782271,7.2667],[1482432926271,7.2671],[1482433070271,7.2665],[1482433214271,7.2679],[1482433358271,7.2655],[1482433502271,7.2653],[1482433646271,7.2652],[1482433790271,7.2646],[1482433934271,7.2665],[1482434078271,7.2678],[1482434222271,7.2717],[1482434366271,7.2744],[1482434510271,7.2742],[1482434654271,7.2719],[1482434798271,7.2718],[1482434942271,7.2726],[1482435086271,7.2715],[1482435230271,7.2717],[1482435374271,6.7438],[1482435518271,6.0015],[1482435662271,5.0775],[1482435806271,3.9425],[1482435950271,2.5351],[1482436094271,0.73273],[1482436238271,0.016717],[1482436382271,0.016828],[1482436526271,0.017171],[1482436670271,0.017114],[1482436814271,0.017348],[1482436958271,0.017312],[1482437102271,0.017253],[1482437246271,0.017588],[1482437390271,0.017410],[1482437534271,0.017589],[1482437678271,0.016895],[1482437822271,0.017736],[1482437966271,0.016991],[1482438110271,0.018018],[1482438254271,0.018096],[1482438398271,0.018406],[1482438542271,0.018299],[1482438686271,0.018652],[1482438830271,0.018725],[1482438974271,0.018939],[1482439118271,0.019047],[1482439262271,0.019027],[1482439406271,0.019217],[1482439550271,0.019309],[1482439694271,0.018458],[1482439838271,0.019245],[1482439982271,0.019093],[1482440126271,0.018980],[1482440270271,0.018962],[1482440414271,0.019969],[1482440558271,0.019228],[1482440702271,0.020046],[1482440846271,0.019990],[1482440990271,0.020288],[1482441134271,0.020074],[1482441278271,0.020274],[1482441422271,0.020370],[1482441566271,0.020305],[1482441710271,0.020727],[1482441854271,0.020929],[1482441998271,0.020923],[1482442142271,0.021376],[1482442286271,0.021020],[1482442430271,0.021272],[1482442574271,0.021638],[1482442718271,0.021432],[1482442862271,0.021628],[1482443006271,0.021807],[1482443150271,0.022038],[1482443294271,0.021178],[1482443438271,0.022277],[1482443582271,0.022365],[1482443726271,0.022685],[1482443870271,0.022803],[1482444014271,0.023066],[1482444158271,0.023146],[1482444302271,0.024569],[1482444446271,0.025539],[1482444590271,0.027045],[1482444734271,0.026247],[1482444878271,0.026699],[1482445022271,0.026845],[1482445166271,0.026991],[1482445310271,0.027507],[1482445454271,0.027503],[1482445598271,0.027731],[1482445742271,0.027836],[1482445886271,0.028547],[1482446030271,0.028517],[1482446174271,0.028737],[1482446318271,0.029313],[1482446462271,0.029595],[1482446606271,0.029626],[1482446750271,0.029717],[1482446894271,0.030024],[1482447038271,0.030066],[1482447182271,0.030167],[1482447326271,0.029933],[1482447470271,0.029891],[1482447614271,0.029839],[1482447758271,0.029875],[1482447902271,0.029896],[1482448046271,0.029868],[1482448190271,0.030283],[1482448334271,0.030476],[1482448478271,0.030350],[1482448622271,0.031086],[1482448766271,0.030988],[1482448910271,0.030896],[1482449054271,0.031388],[1482449198271,0.031503],[1482449342271,0.031443],[1482449486271,0.031727],[1482449630271,0.032111],[1482449774271,0.031929],[1482449918271,0.032100],[1482450062271,0.032139],[1482450206271,0.032202],[1482450350271,0.032686],[1482450494271,0.032501],[1482450638271,0.032856],[1482450782271,0.033093],[1482450926271,0.033549],[1482451070271,0.033502],[1482451214271,0.033791],[1482451358271,0.034259],[1482451502271,0.033986],[1482451646271,0.033850],[1482451790271,0.034118],[1482451934271,0.034233],[1482452078271,0.034081],[1482452222271,0.034458],[1482452366271,0.034306],[1482452510271,0.034465],[1482452654271,0.034490],[1482452798271,0.034721],[1482452942271,0.034791],[1482453086271,0.034736],[1482453230271,0.034986],[1482453374271,0.035010],[1482453518271,0.035368],[1482453662271,0.035868],[1482453806271,0.035946],[1482453950271,0.036075],[1482454094271,0.036458],[1482454238271,0.036946],[1482454382271,0.036371],[1482454526271,0.036214],[1482454670271,0.036550],[1482454814271,0.036430],[1482454958271,0.036336],[1482455102271,0.036627],[1482455246271,0.036916],[1482455390271,0.036783],[1482455534271,0.037377],[1482455678271,0.036953]],\n  "min":[[1482412622271,7.2787],[1482412766271,7.2800],[1482412910271,7.2784],[1482413054271,7.2780],[1482413198271,7.2782],[1482413342271,7.2781],[1482413486271,7.2786],[1482413630271,7.2779],[1482413774271,7.2777],[1482413918271,7.2786],[1482414062271,7.2779],[1482414206271,7.2780],[1482414350271,7.2779],[1482414494271,7.2790],[1482414638271,7.2794],[1482414782271,7.2787],[1482414926271,7.2784],[1482415070271,7.2781],[1482415214271,7.2774],[1482415358271,7.2784],[1482415502271,7.2768],[1482415646271,7.2781],[1482415790271,7.2781],[1482415934271,7.2777],[1482416078271,7.2769],[1482416222271,7.2768],[1482416366271,7.2778],[1482416510271,7.2793],[1482416654271,7.2779],[1482416798271,7.2769],[1482416942271,7.2770],[1482417086271,7.2766],[1482417230271,7.2765],[1482417374271,7.2769],[1482417518271,7.2766],[1482417662271,7.2766],[1482417806271,7.2766],[1482417950271,7.2762],[1482418094271,7.2756],[1482418238271,7.2775],[1482418382271,7.2768],[1482418526271,7.2768],[1482418670271,7.2765],[1482418814271,7.2761],[1482418958271,7.2756],[1482419102271,7.2766],[1482419246271,7.2759],[1482419390271,7.2765],[1482419534271,7.2764],[1482419678271,7.2764],[1482419822271,7.2756],[1482419966271,7.2768],[1482420110271,7.2770],[1482420254271,7.2755],[1482420398271,7.2755],[1482420542271,7.2753],[1482420686271,7.2751],[1482420830271,7.2755],[1482420974271,7.2755],[1482421118271,7.2752],[1482421262271,7.2751],[1482421406271,7.2742],[1482421550271,7.2743],[1482421694271,7.2753],[1482421838271,7.2744],[1482421982271,7.2750],[1482422126271,7.2763],[1482422270271,7.2747],[1482422414271,7.2755],[1482422558271,7.2751],[1482422702271,7.2755],[1482422846271,7.2755],[1482422990271,7.2743],[1482423134271,7.2742],[1482423278271,7.2735],[1482423422271,7.2738],[1482423566271,7.2742],[1482423710271,7.2737],[1482423854271,7.2755],[1482423998271,7.2742],[1482424142271,7.2735],[1482424286271,7.2730],[1482424430271,7.2678],[1482424574271,7.2681],[1482424718271,7.2689],[1482424862271,7.2691],[1482425006271,7.2686],[1482425150271,7.2682],[1482425294271,7.2681],[1482425438271,7.2686],[1482425582271,7.2704],[1482425726271,7.2691],[1482425870271,7.2691],[1482426014271,7.2691],[1482426158271,7.2692],[1482426302271,7.2690],[1482426446271,7.2691],[1482426590271,7.2689],[1482426734271,7.2691],[1482426878271,7.2653],[1482427022271,7.2650],[1482427166271,7.2641],[1482427310271,7.2648],[1482427454271,7.2655],[1482427598271,7.2641],[1482427742271,7.2647],[1482427886271,7.2653],[1482428030271,7.2656],[1482428174271,7.2651],[1482428318271,7.2653],[1482428462271,7.2653],[1482428606271,7.2662],[1482428750271,7.2660],[1482428894271,7.2665],[1482429038271,7.2676],[1482429182271,7.2665],[1482429326271,7.2658],[1482429470271,7.2666],[1482429614271,7.2666],[1482429758271,7.2666],[1482429902271,7.2663],[1482430046271,7.2665],[1482430190271,7.2666],[1482430334271,7.2665],[1482430478271,7.2665],[1482430622271,7.2665],[1482430766271,7.2685],[1482430910271,7.2666],[1482431054271,7.2665],[1482431198271,7.2654],[1482431342271,7.2653],[1482431486271,7.2666],[1482431630271,7.2666],[1482431774271,7.2666],[1482431918271,7.2664],[1482432062271,7.2655],[1482432206271,7.2661],[1482432350271,7.2666],[1482432494271,7.2675],[1482432638271,7.2663],[1482432782271,7.2665],[1482432926271,7.2665],[1482433070271,7.2665],[1482433214271,7.2652],[1482433358271,7.2641],[1482433502271,7.2640],[1482433646271,7.2639],[1482433790271,7.2643],[1482433934271,7.2653],[1482434078271,7.2653],[1482434222271,7.2716],[1482434366271,7.2729],[1482434510271,7.2716],[1482434654271,7.2712],[1482434798271,7.2711],[1482434942271,7.2705],[1482435086271,7.2704],[1482435230271,6.7479],[1482435374271,6.0072],[1482435518271,5.0847],[1482435662271,3.9512],[1482435806271,2.5464],[1482435950271,0.74650],[1482436094271,0.016389],[1482436238271,0.016045],[1482436382271,0.016141],[1482436526271,0.016551],[1482436670271,0.016739],[1482436814271,0.016380],[1482436958271,0.016360],[1482437102271,0.017215],[1482437246271,0.016792],[1482437390271,0.017279],[1482437534271,0.017138],[1482437678271,0.016895],[1482437822271,0.016978],[1482437966271,0.016991],[1482438110271,0.017107],[1482438254271,0.017396],[1482438398271,0.017664],[1482438542271,0.018041],[1482438686271,0.017424],[1482438830271,0.017927],[1482438974271,0.017852],[1482439118271,0.017852],[1482439262271,0.017464],[1482439406271,0.018683],[1482439550271,0.018341],[1482439694271,0.018425],[1482439838271,0.018624],[1482439982271,0.018764],[1482440126271,0.018750],[1482440270271,0.018962],[1482440414271,0.019104],[1482440558271,0.019212],[1482440702271,0.019330],[1482440846271,0.019519],[1482440990271,0.019686],[1482441134271,0.019897],[1482441278271,0.019996],[1482441422271,0.020191],[1482441566271,0.019902],[1482441710271,0.020311],[1482441854271,0.020579],[1482441998271,0.020617],[1482442142271,0.020901],[1482442286271,0.020441],[1482442430271,0.021018],[1482442574271,0.021261],[1482442718271,0.021177],[1482442862271,0.021466],[1482443006271,0.021747],[1482443150271,0.021444],[1482443294271,0.021178],[1482443438271,0.021557],[1482443582271,0.022238],[1482443726271,0.022324],[1482443870271,0.022276],[1482444014271,0.022271],[1482444158271,0.023146],[1482444302271,0.024554],[1482444446271,0.025184],[1482444590271,0.025519],[1482444734271,0.025907],[1482444878271,0.026106],[1482445022271,0.026492],[1482445166271,0.026934],[1482445310271,0.027039],[1482445454271,0.027256],[1482445598271,0.027466],[1482445742271,0.027573],[1482445886271,0.028008],[1482446030271,0.028321],[1482446174271,0.028585],[1482446318271,0.028301],[1482446462271,0.026004],[1482446606271,0.024311],[1482446750271,0.029557],[1482446894271,0.029522],[1482447038271,0.024054],[1482447182271,0.029586],[1482447326271,0.029582],[1482447470271,0.029175],[1482447614271,0.029019],[1482447758271,0.029627],[1482447902271,0.029214],[1482448046271,0.029607],[1482448190271,0.029801],[1482448334271,0.030025],[1482448478271,0.030060],[1482448622271,0.030022],[1482448766271,0.030615],[1482448910271,0.030780],[1482449054271,0.031107],[1482449198271,0.031194],[1482449342271,0.031309],[1482449486271,0.031425],[1482449630271,0.031589],[1482449774271,0.031761],[1482449918271,0.031800],[1482450062271,0.031876],[1482450206271,0.032099],[1482450350271,0.031794],[1482450494271,0.032310],[1482450638271,0.032436],[1482450782271,0.032905],[1482450926271,0.032534],[1482451070271,0.032784],[1482451214271,0.033065],[1482451358271,0.033331],[1482451502271,0.033614],[1482451646271,0.033763],[1482451790271,0.033751],[1482451934271,0.033841],[1482452078271,0.034026],[1482452222271,0.034112],[1482452366271,0.033703],[1482452510271,0.034310],[1482452654271,0.034236],[1482452798271,0.034418],[1482452942271,0.034559],[1482453086271,0.034519],[1482453230271,0.034735],[1482453374271,0.034926],[1482453518271,0.035164],[1482453662271,0.035401],[1482453806271,0.035742],[1482453950271,0.035149],[1482454094271,0.035636],[1482454238271,0.035597],[1482454382271,0.035599],[1482454526271,0.036134],[1482454670271,0.036195],[1482454814271,0.036133],[1482454958271,0.036317],[1482455102271,0.035953],[1482455246271,0.036306],[1482455390271,0.036115],[1482455534271,0.036813],[1482455678271,0.036953]]\n}'
_trenddata_12hr_temperature_json = '{\n  "max":[[1482412622672,19.501],[1482412766672,19.497],[1482412910672,19.492],[1482413054672,19.490],[1482413198672,19.485],[1482413342672,19.482],[1482413486672,19.478],[1482413630672,19.473],[1482413774672,19.470],[1482413918672,19.466],[1482414062672,19.463],[1482414206672,19.458],[1482414350672,19.456],[1482414494672,19.452],[1482414638672,19.449],[1482414782672,19.445],[1482414926672,19.442],[1482415070672,19.438],[1482415214672,19.436],[1482415358672,19.433],[1482415502672,19.429],[1482415646672,19.427],[1482415790672,19.423],[1482415934672,19.422],[1482416078672,19.416],[1482416222672,19.413],[1482416366672,19.409],[1482416510672,19.408],[1482416654672,19.406],[1482416798672,19.400],[1482416942672,19.398],[1482417086672,19.393],[1482417230672,19.391],[1482417374672,19.387],[1482417518672,19.385],[1482417662672,19.382],[1482417806672,19.381],[1482417950672,19.375],[1482418094672,19.374],[1482418238672,19.369],[1482418382672,19.369],[1482418526672,19.365],[1482418670672,19.362],[1482418814672,19.358],[1482418958672,19.356],[1482419102672,19.352],[1482419246672,19.349],[1482419390672,19.347],[1482419534672,19.344],[1482419678672,19.342],[1482419822672,19.339],[1482419966672,19.337],[1482420110672,19.333],[1482420254672,19.331],[1482420398672,19.329],[1482420542672,19.328],[1482420686672,19.326],[1482420830672,19.326],[1482420974672,19.325],[1482421118672,19.322],[1482421262672,19.322],[1482421406672,19.321],[1482421550672,19.318],[1482421694672,19.317],[1482421838672,19.314],[1482421982672,19.314],[1482422126672,19.312],[1482422270672,19.310],[1482422414672,19.308],[1482422558672,19.308],[1482422702672,19.308],[1482422846672,19.306],[1482422990672,19.306],[1482423134672,19.304],[1482423278672,19.304],[1482423422672,19.302],[1482423566672,19.300],[1482423710672,19.302],[1482423854672,19.298],[1482423998672,19.295],[1482424142672,19.294],[1482424286672,19.293],[1482424430672,19.294],[1482424574672,19.295],[1482424718672,19.296],[1482424862672,19.295],[1482425006672,19.296],[1482425150672,19.298],[1482425294672,19.295],[1482425438672,19.290],[1482425582672,19.289],[1482425726672,19.285],[1482425870672,19.283],[1482426014672,19.282],[1482426158672,19.281],[1482426302672,19.279],[1482426446672,19.278],[1482426590672,19.278],[1482426734672,19.276],[1482426878672,19.274],[1482427022672,19.273],[1482427166672,19.272],[1482427310672,19.270],[1482427454672,19.269],[1482427598672,19.269],[1482427742672,19.267],[1482427886672,19.265],[1482428030672,19.263],[1482428174672,19.262],[1482428318672,19.262],[1482428462672,19.260],[1482428606672,19.262],[1482428750672,19.260],[1482428894672,19.259],[1482429038672,19.259],[1482429182672,19.261],[1482429326672,19.260],[1482429470672,19.261],[1482429614672,19.262],[1482429758672,19.261],[1482429902672,19.261],[1482430046672,19.261],[1482430190672,19.260],[1482430334672,19.260],[1482430478672,19.261],[1482430622672,19.261],[1482430766672,19.264],[1482430910672,19.265],[1482431054672,19.267],[1482431198672,19.269],[1482431342672,19.270],[1482431486672,19.270],[1482431630672,19.273],[1482431774672,19.271],[1482431918672,19.273],[1482432062672,19.275],[1482432206672,19.277],[1482432350672,19.282],[1482432494672,19.286],[1482432638672,19.292],[1482432782672,19.292],[1482432926672,19.299],[1482433070672,19.308],[1482433214672,19.309],[1482433358672,19.312],[1482433502672,19.315],[1482433646672,19.318],[1482433790672,19.322],[1482433934672,19.328],[1482434078672,19.331],[1482434222672,19.335],[1482434366672,19.340],[1482434510672,19.344],[1482434654672,19.345],[1482434798672,19.352],[1482434942672,19.356],[1482435086672,19.358],[1482435230672,19.365],[1482435374672,19.370],[1482435518672,19.373],[1482435662672,19.380],[1482435806672,19.384],[1482435950672,19.393],[1482436094672,19.397],[1482436238672,19.402],[1482436382672,19.405],[1482436526672,19.413],[1482436670672,19.418],[1482436814672,19.421],[1482436958672,19.424],[1482437102672,19.429],[1482437246672,19.431],[1482437390672,19.435],[1482437534672,19.439],[1482437678672,19.445],[1482437822672,19.450],[1482437966672,19.453],[1482438110672,19.458],[1482438254672,19.466],[1482438398672,19.470],[1482438542672,19.475],[1482438686672,19.481],[1482438830672,19.491],[1482438974672,19.493],[1482439118672,19.500],[1482439262672,19.504],[1482439406672,19.510],[1482439550672,19.517],[1482439694672,19.524],[1482439838672,19.530],[1482439982672,19.535],[1482440126672,19.541],[1482440270672,19.547],[1482440414672,19.552],[1482440558672,19.560],[1482440702672,19.567],[1482440846672,19.575],[1482440990672,19.582],[1482441134672,19.586],[1482441278672,19.592],[1482441422672,19.600],[1482441566672,19.607],[1482441710672,19.613],[1482441854672,19.617],[1482441998672,19.622],[1482442142672,19.627],[1482442286672,19.635],[1482442430672,19.643],[1482442574672,19.647],[1482442718672,19.654],[1482442862672,19.662],[1482443006672,19.666],[1482443150672,19.674],[1482443294672,19.681],[1482443438672,19.687],[1482443582672,19.692],[1482443726672,19.696],[1482443870672,19.701],[1482444014672,19.715],[1482444158672,19.777],[1482444302672,19.806],[1482444446672,19.826],[1482444590672,19.842],[1482444734672,19.860],[1482444878672,19.871],[1482445022672,19.879],[1482445166672,19.890],[1482445310672,19.896],[1482445454672,19.908],[1482445598672,19.911],[1482445742672,19.923],[1482445886672,19.946],[1482446030672,19.952],[1482446174672,19.967],[1482446318672,19.980],[1482446462672,19.991],[1482446606672,19.994],[1482446750672,20.000],[1482446894672,20.001],[1482447038672,20.002],[1482447182672,20.003],[1482447326672,20.004],[1482447470672,20.005],[1482447614672,20.004],[1482447758672,20.002],[1482447902672,20.004],[1482448046672,20.007],[1482448190672,20.015],[1482448334672,20.020],[1482448478672,20.026],[1482448622672,20.037],[1482448766672,20.045],[1482448910672,20.057],[1482449054672,20.063],[1482449198672,20.069],[1482449342672,20.074],[1482449486672,20.080],[1482449630672,20.086],[1482449774672,20.089],[1482449918672,20.095],[1482450062672,20.099],[1482450206672,20.111],[1482450350672,20.118],[1482450494672,20.127],[1482450638672,20.133],[1482450782672,20.142],[1482450926672,20.147],[1482451070672,20.154],[1482451214672,20.158],[1482451358672,20.166],[1482451502672,20.169],[1482451646672,20.174],[1482451790672,20.181],[1482451934672,20.182],[1482452078672,20.187],[1482452222672,20.191],[1482452366672,20.193],[1482452510672,20.195],[1482452654672,20.199],[1482452798672,20.203],[1482452942672,20.204],[1482453086672,20.210],[1482453230672,20.216],[1482453374672,20.226],[1482453518672,20.243],[1482453662672,20.253],[1482453806672,20.258],[1482453950672,20.266],[1482454094672,20.275],[1482454238672,20.267],[1482454382672,20.269],[1482454526672,20.272],[1482454670672,20.277],[1482454814672,20.282],[1482454958672,20.282],[1482455102672,20.289],[1482455246672,20.294],[1482455390672,20.302],[1482455534672,20.308],[1482455678672,20.311]],\n  "last":[[1482412622672,19.497],[1482412766672,19.495],[1482412910672,19.489],[1482413054672,19.489],[1482413198672,19.484],[1482413342672,19.480],[1482413486672,19.477],[1482413630672,19.472],[1482413774672,19.467],[1482413918672,19.463],[1482414062672,19.460],[1482414206672,19.457],[1482414350672,19.454],[1482414494672,19.452],[1482414638672,19.449],[1482414782672,19.444],[1482414926672,19.439],[1482415070672,19.435],[1482415214672,19.433],[1482415358672,19.430],[1482415502672,19.427],[1482415646672,19.423],[1482415790672,19.419],[1482415934672,19.416],[1482416078672,19.414],[1482416222672,19.413],[1482416366672,19.409],[1482416510672,19.408],[1482416654672,19.406],[1482416798672,19.396],[1482416942672,19.394],[1482417086672,19.392],[1482417230672,19.389],[1482417374672,19.386],[1482417518672,19.381],[1482417662672,19.377],[1482417806672,19.381],[1482417950672,19.373],[1482418094672,19.370],[1482418238672,19.365],[1482418382672,19.366],[1482418526672,19.362],[1482418670672,19.361],[1482418814672,19.358],[1482418958672,19.356],[1482419102672,19.349],[1482419246672,19.347],[1482419390672,19.346],[1482419534672,19.344],[1482419678672,19.339],[1482419822672,19.337],[1482419966672,19.333],[1482420110672,19.333],[1482420254672,19.329],[1482420398672,19.326],[1482420542672,19.328],[1482420686672,19.323],[1482420830672,19.324],[1482420974672,19.323],[1482421118672,19.322],[1482421262672,19.320],[1482421406672,19.314],[1482421550672,19.314],[1482421694672,19.313],[1482421838672,19.312],[1482421982672,19.308],[1482422126672,19.310],[1482422270672,19.308],[1482422414672,19.307],[1482422558672,19.304],[1482422702672,19.305],[1482422846672,19.305],[1482422990672,19.302],[1482423134672,19.303],[1482423278672,19.299],[1482423422672,19.298],[1482423566672,19.299],[1482423710672,19.296],[1482423854672,19.296],[1482423998672,19.293],[1482424142672,19.291],[1482424286672,19.291],[1482424430672,19.289],[1482424574672,19.292],[1482424718672,19.292],[1482424862672,19.292],[1482425006672,19.291],[1482425150672,19.293],[1482425294672,19.290],[1482425438672,19.289],[1482425582672,19.285],[1482425726672,19.284],[1482425870672,19.280],[1482426014672,19.282],[1482426158672,19.276],[1482426302672,19.277],[1482426446672,19.274],[1482426590672,19.276],[1482426734672,19.272],[1482426878672,19.272],[1482427022672,19.267],[1482427166672,19.272],[1482427310672,19.269],[1482427454672,19.269],[1482427598672,19.264],[1482427742672,19.266],[1482427886672,19.264],[1482428030672,19.261],[1482428174672,19.262],[1482428318672,19.260],[1482428462672,19.259],[1482428606672,19.257],[1482428750672,19.257],[1482428894672,19.258],[1482429038672,19.256],[1482429182672,19.258],[1482429326672,19.256],[1482429470672,19.257],[1482429614672,19.259],[1482429758672,19.260],[1482429902672,19.259],[1482430046672,19.259],[1482430190672,19.257],[1482430334672,19.258],[1482430478672,19.256],[1482430622672,19.258],[1482430766672,19.261],[1482430910672,19.263],[1482431054672,19.260],[1482431198672,19.265],[1482431342672,19.266],[1482431486672,19.266],[1482431630672,19.266],[1482431774672,19.271],[1482431918672,19.270],[1482432062672,19.273],[1482432206672,19.273],[1482432350672,19.275],[1482432494672,19.282],[1482432638672,19.283],[1482432782672,19.291],[1482432926672,19.292],[1482433070672,19.299],[1482433214672,19.305],[1482433358672,19.308],[1482433502672,19.308],[1482433646672,19.312],[1482433790672,19.315],[1482433934672,19.319],[1482434078672,19.327],[1482434222672,19.332],[1482434366672,19.331],[1482434510672,19.338],[1482434654672,19.340],[1482434798672,19.345],[1482434942672,19.350],[1482435086672,19.356],[1482435230672,19.358],[1482435374672,19.363],[1482435518672,19.367],[1482435662672,19.370],[1482435806672,19.380],[1482435950672,19.382],[1482436094672,19.391],[1482436238672,19.395],[1482436382672,19.403],[1482436526672,19.403],[1482436670672,19.412],[1482436814672,19.417],[1482436958672,19.420],[1482437102672,19.420],[1482437246672,19.429],[1482437390672,19.427],[1482437534672,19.431],[1482437678672,19.437],[1482437822672,19.441],[1482437966672,19.446],[1482438110672,19.452],[1482438254672,19.458],[1482438398672,19.465],[1482438542672,19.466],[1482438686672,19.474],[1482438830672,19.481],[1482438974672,19.488],[1482439118672,19.493],[1482439262672,19.499],[1482439406672,19.503],[1482439550672,19.511],[1482439694672,19.514],[1482439838672,19.521],[1482439982672,19.529],[1482440126672,19.532],[1482440270672,19.537],[1482440414672,19.546],[1482440558672,19.551],[1482440702672,19.558],[1482440846672,19.565],[1482440990672,19.574],[1482441134672,19.581],[1482441278672,19.587],[1482441422672,19.593],[1482441566672,19.598],[1482441710672,19.605],[1482441854672,19.611],[1482441998672,19.617],[1482442142672,19.617],[1482442286672,19.627],[1482442430672,19.632],[1482442574672,19.641],[1482442718672,19.643],[1482442862672,19.653],[1482443006672,19.660],[1482443150672,19.667],[1482443294672,19.674],[1482443438672,19.675],[1482443582672,19.684],[1482443726672,19.692],[1482443870672,19.695],[1482444014672,19.698],[1482444158672,19.715],[1482444302672,19.778],[1482444446672,19.807],[1482444590672,19.824],[1482444734672,19.838],[1482444878672,19.859],[1482445022672,19.871],[1482445166672,19.880],[1482445310672,19.889],[1482445454672,19.894],[1482445598672,19.904],[1482445742672,19.910],[1482445886672,19.923],[1482446030672,19.944],[1482446174672,19.951],[1482446318672,19.966],[1482446462672,19.980],[1482446606672,19.991],[1482446750672,19.994],[1482446894672,19.997],[1482447038672,20.000],[1482447182672,19.999],[1482447326672,20.000],[1482447470672,20.001],[1482447614672,20.001],[1482447758672,19.998],[1482447902672,20.000],[1482448046672,20.002],[1482448190672,20.006],[1482448334672,20.014],[1482448478672,20.018],[1482448622672,20.023],[1482448766672,20.035],[1482448910672,20.044],[1482449054672,20.058],[1482449198672,20.063],[1482449342672,20.067],[1482449486672,20.074],[1482449630672,20.081],[1482449774672,20.086],[1482449918672,20.088],[1482450062672,20.095],[1482450206672,20.100],[1482450350672,20.108],[1482450494672,20.118],[1482450638672,20.127],[1482450782672,20.132],[1482450926672,20.142],[1482451070672,20.146],[1482451214672,20.152],[1482451358672,20.158],[1482451502672,20.166],[1482451646672,20.167],[1482451790672,20.173],[1482451934672,20.177],[1482452078672,20.180],[1482452222672,20.185],[1482452366672,20.189],[1482452510672,20.190],[1482452654672,20.194],[1482452798672,20.199],[1482452942672,20.199],[1482453086672,20.203],[1482453230672,20.211],[1482453374672,20.214],[1482453518672,20.225],[1482453662672,20.244],[1482453806672,20.249],[1482453950672,20.257],[1482454094672,20.267],[1482454238672,20.263],[1482454382672,20.266],[1482454526672,20.267],[1482454670672,20.271],[1482454814672,20.275],[1482454958672,20.277],[1482455102672,20.280],[1482455246672,20.288],[1482455390672,20.293],[1482455534672,20.298],[1482455678672,20.305]],\n  "min":[[1482412622672,19.493],[1482412766672,19.487],[1482412910672,19.486],[1482413054672,19.481],[1482413198672,19.477],[1482413342672,19.474],[1482413486672,19.470],[1482413630672,19.466],[1482413774672,19.462],[1482413918672,19.460],[1482414062672,19.455],[1482414206672,19.453],[1482414350672,19.449],[1482414494672,19.445],[1482414638672,19.442],[1482414782672,19.439],[1482414926672,19.434],[1482415070672,19.432],[1482415214672,19.429],[1482415358672,19.426],[1482415502672,19.421],[1482415646672,19.420],[1482415790672,19.416],[1482415934672,19.413],[1482416078672,19.410],[1482416222672,19.405],[1482416366672,19.401],[1482416510672,19.399],[1482416654672,19.396],[1482416798672,19.392],[1482416942672,19.390],[1482417086672,19.388],[1482417230672,19.383],[1482417374672,19.382],[1482417518672,19.378],[1482417662672,19.375],[1482417806672,19.372],[1482417950672,19.370],[1482418094672,19.365],[1482418238672,19.364],[1482418382672,19.360],[1482418526672,19.359],[1482418670672,19.356],[1482418814672,19.352],[1482418958672,19.348],[1482419102672,19.345],[1482419246672,19.343],[1482419390672,19.339],[1482419534672,19.337],[1482419678672,19.335],[1482419822672,19.331],[1482419966672,19.329],[1482420110672,19.328],[1482420254672,19.325],[1482420398672,19.323],[1482420542672,19.321],[1482420686672,19.320],[1482420830672,19.320],[1482420974672,19.318],[1482421118672,19.316],[1482421262672,19.313],[1482421406672,19.313],[1482421550672,19.311],[1482421694672,19.310],[1482421838672,19.308],[1482421982672,19.306],[1482422126672,19.306],[1482422270672,19.304],[1482422414672,19.302],[1482422558672,19.302],[1482422702672,19.301],[1482422846672,19.299],[1482422990672,19.299],[1482423134672,19.299],[1482423278672,19.297],[1482423422672,19.296],[1482423566672,19.296],[1482423710672,19.293],[1482423854672,19.293],[1482423998672,19.289],[1482424142672,19.288],[1482424286672,19.287],[1482424430672,19.288],[1482424574672,19.290],[1482424718672,19.290],[1482424862672,19.290],[1482425006672,19.290],[1482425150672,19.291],[1482425294672,19.287],[1482425438672,19.285],[1482425582672,19.280],[1482425726672,19.279],[1482425870672,19.276],[1482426014672,19.275],[1482426158672,19.274],[1482426302672,19.273],[1482426446672,19.272],[1482426590672,19.271],[1482426734672,19.270],[1482426878672,19.268],[1482427022672,19.266],[1482427166672,19.266],[1482427310672,19.264],[1482427454672,19.263],[1482427598672,19.263],[1482427742672,19.261],[1482427886672,19.259],[1482428030672,19.256],[1482428174672,19.257],[1482428318672,19.255],[1482428462672,19.254],[1482428606672,19.253],[1482428750672,19.255],[1482428894672,19.254],[1482429038672,19.254],[1482429182672,19.255],[1482429326672,19.254],[1482429470672,19.254],[1482429614672,19.255],[1482429758672,19.256],[1482429902672,19.254],[1482430046672,19.255],[1482430190672,19.255],[1482430334672,19.255],[1482430478672,19.255],[1482430622672,19.257],[1482430766672,19.257],[1482430910672,19.258],[1482431054672,19.260],[1482431198672,19.262],[1482431342672,19.264],[1482431486672,19.265],[1482431630672,19.266],[1482431774672,19.265],[1482431918672,19.267],[1482432062672,19.268],[1482432206672,19.272],[1482432350672,19.274],[1482432494672,19.278],[1482432638672,19.281],[1482432782672,19.287],[1482432926672,19.291],[1482433070672,19.295],[1482433214672,19.302],[1482433358672,19.304],[1482433502672,19.308],[1482433646672,19.312],[1482433790672,19.315],[1482433934672,19.319],[1482434078672,19.325],[1482434222672,19.327],[1482434366672,19.331],[1482434510672,19.336],[1482434654672,19.339],[1482434798672,19.342],[1482434942672,19.345],[1482435086672,19.352],[1482435230672,19.356],[1482435374672,19.360],[1482435518672,19.366],[1482435662672,19.370],[1482435806672,19.377],[1482435950672,19.381],[1482436094672,19.388],[1482436238672,19.394],[1482436382672,19.398],[1482436526672,19.403],[1482436670672,19.405],[1482436814672,19.411],[1482436958672,19.418],[1482437102672,19.420],[1482437246672,19.422],[1482437390672,19.427],[1482437534672,19.430],[1482437678672,19.437],[1482437822672,19.441],[1482437966672,19.446],[1482438110672,19.451],[1482438254672,19.457],[1482438398672,19.463],[1482438542672,19.466],[1482438686672,19.472],[1482438830672,19.480],[1482438974672,19.487],[1482439118672,19.489],[1482439262672,19.498],[1482439406672,19.502],[1482439550672,19.509],[1482439694672,19.513],[1482439838672,19.519],[1482439982672,19.526],[1482440126672,19.532],[1482440270672,19.537],[1482440414672,19.544],[1482440558672,19.549],[1482440702672,19.558],[1482440846672,19.563],[1482440990672,19.573],[1482441134672,19.578],[1482441278672,19.584],[1482441422672,19.591],[1482441566672,19.597],[1482441710672,19.604],[1482441854672,19.608],[1482441998672,19.614],[1482442142672,19.617],[1482442286672,19.624],[1482442430672,19.630],[1482442574672,19.639],[1482442718672,19.642],[1482442862672,19.650],[1482443006672,19.659],[1482443150672,19.665],[1482443294672,19.673],[1482443438672,19.675],[1482443582672,19.684],[1482443726672,19.689],[1482443870672,19.691],[1482444014672,19.695],[1482444158672,19.715],[1482444302672,19.778],[1482444446672,19.804],[1482444590672,19.823],[1482444734672,19.838],[1482444878672,19.857],[1482445022672,19.870],[1482445166672,19.877],[1482445310672,19.888],[1482445454672,19.893],[1482445598672,19.904],[1482445742672,19.907],[1482445886672,19.923],[1482446030672,19.942],[1482446174672,19.951],[1482446318672,19.966],[1482446462672,19.979],[1482446606672,19.987],[1482446750672,19.993],[1482446894672,19.996],[1482447038672,19.997],[1482447182672,19.997],[1482447326672,19.998],[1482447470672,19.998],[1482447614672,19.997],[1482447758672,19.997],[1482447902672,19.997],[1482448046672,19.993],[1482448190672,20.004],[1482448334672,20.010],[1482448478672,20.014],[1482448622672,20.022],[1482448766672,20.035],[1482448910672,20.044],[1482449054672,20.055],[1482449198672,20.060],[1482449342672,20.065],[1482449486672,20.070],[1482449630672,20.078],[1482449774672,20.083],[1482449918672,20.087],[1482450062672,20.092],[1482450206672,20.098],[1482450350672,20.106],[1482450494672,20.114],[1482450638672,20.124],[1482450782672,20.131],[1482450926672,20.140],[1482451070672,20.146],[1482451214672,20.151],[1482451358672,20.156],[1482451502672,20.162],[1482451646672,20.167],[1482451790672,20.172],[1482451934672,20.172],[1482452078672,20.178],[1482452222672,20.182],[1482452366672,20.187],[1482452510672,20.189],[1482452654672,20.192],[1482452798672,20.194],[1482452942672,20.199],[1482453086672,20.201],[1482453230672,20.210],[1482453374672,20.212],[1482453518672,20.225],[1482453662672,20.241],[1482453806672,20.249],[1482453950672,20.257],[1482454094672,20.260],[1482454238672,20.259],[1482454382672,20.262],[1482454526672,20.264],[1482454670672,20.266],[1482454814672,20.273],[1482454958672,20.275],[1482455102672,20.279],[1482455246672,20.285],[1482455390672,20.292],[1482455534672,20.297],[1482455678672,20.304]]\n}'
_trenddata_24hr_agitation_json = '{\n  "max":[[1482369566971,0.0000],[1482369854971,0.0000],[1482370142971,0.0000],[1482370430971,0.0000],[1482370718971,0.0000],[1482371006971,0.0000],[1482371294971,0.0000],[1482371582971,0.0000],[1482371870971,0.0000],[1482372158971,0.0000],[1482372446971,0.0000],[1482372734971,0.0000],[1482373022971,0.0000],[1482373310971,0.0000],[1482373598971,0.0000],[1482373886971,0.0000],[1482374174971,0.0000],[1482374462971,0.0000],[1482374750971,0.0000],[1482375038971,0.0000],[1482375326971,0.0000],[1482375614971,0.0000],[1482375902971,0.0000],[1482376190971,0.0000],[1482376478971,0.0000],[1482376766971,0.0000],[1482377054971,0.0000],[1482377342971,0.0000],[1482377630971,0.0000],[1482377918971,0.0000],[1482378206971,0.0000],[1482378494971,0.0000],[1482378782971,0.0000],[1482379070971,0.0000],[1482379358971,0.0000],[1482379646971,0.0000],[1482379934971,0.0000],[1482380222971,0.0000],[1482380510971,0.0000],[1482380798971,0.0000],[1482381086971,0.0000],[1482381374971,0.0000],[1482381662971,0.0000],[1482381950971,0.0000],[1482382238971,0.0000],[1482382526971,0.0000],[1482382814971,0.0000],[1482383102971,0.0000],[1482383390971,0.0000],[1482383678971,0.0000],[1482383966971,0.0000],[1482384254971,0.0000],[1482384542971,0.0000],[1482384830971,0.0000],[1482385118971,0.0000],[1482385406971,0.0000],[1482385694971,0.0000],[1482385982971,0.0000],[1482386270971,0.0000],[1482386558971,0.0000],[1482386846971,0.0000],[1482387134971,0.0000],[1482387422971,0.0000],[1482387710971,0.0000],[1482387998971,0.0000],[1482388286971,0.0000],[1482388574971,0.0000],[1482388862971,0.0000],[1482389150971,0.0000],[1482389438971,0.0000],[1482389726971,0.0000],[1482390014971,0.0000],[1482390302971,0.0000],[1482390590971,0.0000],[1482390878971,0.0000],[1482391166971,0.0000],[1482391454971,0.0000],[1482391742971,0.0000],[1482392030971,0.0000],[1482392318971,0.0000],[1482392606971,0.0000],[1482392894971,0.0000],[1482393182971,0.0000],[1482393470971,0.0000],[1482393758971,0.0000],[1482394046971,0.0000],[1482394334971,0.0000],[1482394622971,0.0000],[1482394910971,0.0000],[1482395198971,0.0000],[1482395486971,0.0000],[1482395774971,0.0000],[1482396062971,0.0000],[1482396350971,0.0000],[1482396638971,0.0000],[1482396926971,0.0000],[1482397214971,0.0000],[1482397502971,0.0000],[1482397790971,0.0000],[1482398078971,0.0000],[1482398366971,0.0000],[1482398654971,0.0000],[1482398942971,0.0000],[1482399230971,0.0000],[1482399518971,0.0000],[1482399806971,0.0000],[1482400094971,0.0000],[1482400382971,0.0000],[1482400670971,0.0000],[1482400958971,0.0000],[1482401246971,0.0000],[1482401534971,0.0000],[1482401822971,0.0000],[1482402110971,0.0000],[1482402398971,0.0000],[1482402686971,0.0000],[1482402974971,0.0000],[1482403262971,0.0000],[1482403550971,0.0000],[1482403838971,0.0000],[1482404126971,0.0000],[1482404414971,0.0000],[1482404702971,0.0000],[1482404990971,0.0000],[1482405278971,0.0000],[1482405566971,0.0000],[1482405854971,0.0000],[1482406142971,0.0000],[1482406430971,0.0000],[1482406718971,0.0000],[1482407006971,0.0000],[1482407294971,0.0000],[1482407582971,0.0000],[1482407870971,0.0000],[1482408158971,0.0000],[1482408446971,0.0000],[1482408734971,0.0000],[1482409022971,0.0000],[1482409310971,0.0000],[1482409598971,0.0000],[1482409886971,0.0000],[1482410174971,0.0000],[1482410462971,0.0000],[1482410750971,0.0000],[1482411038971,0.0000],[1482411326971,0.0000],[1482411614971,0.0000],[1482411902971,0.0000],[1482412190971,0.0000],[1482412478971,0.0000],[1482412766971,0.0000],[1482413054971,0.0000],[1482413342971,0.0000],[1482413630971,0.0000],[1482413918971,0.0000],[1482414206971,0.0000],[1482414494971,0.0000],[1482414782971,0.0000],[1482415070971,0.0000],[1482415358971,0.0000],[1482415646971,0.0000],[1482415934971,0.0000],[1482416222971,0.0000],[1482416510971,0.0000],[1482416798971,0.0000],[1482417086971,0.0000],[1482417374971,0.0000],[1482417662971,0.0000],[1482417950971,0.0000],[1482418238971,0.0000],[1482418526971,0.0000],[1482418814971,0.0000],[1482419102971,0.0000],[1482419390971,0.0000],[1482419678971,0.0000],[1482419966971,0.0000],[1482420254971,0.0000],[1482420542971,0.0000],[1482420830971,0.0000],[1482421118971,0.0000],[1482421406971,0.0000],[1482421694971,0.0000],[1482421982971,0.0000],[1482422270971,0.0000],[1482422558971,0.0000],[1482422846971,0.0000],[1482423134971,0.0000],[1482423422971,0.0000],[1482423710971,0.0000],[1482423998971,0.0000],[1482424286971,0.0000],[1482424574971,0.0000],[1482424862971,0.0000],[1482425150971,0.0000],[1482425438971,0.0000],[1482425726971,0.0000],[1482426014971,0.0000],[1482426302971,0.0000],[1482426590971,0.0000],[1482426878971,0.0000],[1482427166971,0.0000],[1482427454971,0.0000],[1482427742971,0.0000],[1482428030971,0.0000],[1482428318971,0.0000],[1482428606971,0.0000],[1482428894971,0.0000],[1482429182971,0.0000],[1482429470971,0.0000],[1482429758971,0.0000],[1482430046971,0.0000],[1482430334971,0.0000],[1482430622971,0.0000],[1482430910971,0.0000],[1482431198971,0.0000],[1482431486971,0.0000],[1482431774971,0.0000],[1482432062971,0.0000],[1482432350971,0.0000],[1482432638971,0.0000],[1482432926971,0.0000],[1482433214971,0.0000],[1482433502971,0.0000],[1482433790971,0.0000],[1482434078971,0.0000],[1482434366971,0.0000],[1482434654971,0.0000],[1482434942971,0.0000],[1482435230971,0.0000],[1482435518971,0.0000],[1482435806971,0.0000],[1482436094971,0.0000],[1482436382971,0.0000],[1482436670971,0.0000],[1482436958971,0.0000],[1482437246971,0.0000],[1482437534971,0.0000],[1482437822971,0.0000],[1482438110971,0.0000],[1482438398971,0.0000],[1482438686971,0.0000],[1482438974971,0.0000],[1482439262971,0.0000],[1482439550971,0.0000],[1482439838971,0.0000],[1482440126971,0.0000],[1482440414971,0.0000],[1482440702971,0.0000],[1482440990971,0.0000],[1482441278971,0.0000],[1482441566971,0.0000],[1482441854971,0.0000],[1482442142971,0.0000],[1482442430971,0.0000],[1482442718971,0.0000],[1482443006971,0.0000],[1482443294971,0.0000],[1482443582971,0.0000],[1482443870971,16.324],[1482444158971,20.032],[1482444446971,20.038],[1482444734971,20.039],[1482445022971,20.038],[1482445310971,20.042],[1482445598971,20.043],[1482445886971,0.0000],[1482446174971,0.0000],[1482446462971,0.0000],[1482446750971,0.0000],[1482447038971,0.0000],[1482447326971,0.0000],[1482447614971,0.0000],[1482447902971,12.057],[1482448190971,35.305],[1482448478971,36.028],[1482448766971,12.062],[1482449054971,12.061],[1482449342971,12.061],[1482449630971,12.062],[1482449918971,12.061],[1482450206971,12.062],[1482450494971,12.063],[1482450782971,12.060],[1482451070971,12.060],[1482451358971,12.060],[1482451646971,12.060],[1482451934971,12.063],[1482452222971,12.063],[1482452510971,12.061],[1482452798971,12.062],[1482453086971,12.061],[1482453374971,12.059],[1482453662971,0.0000],[1482453950971,8.4473],[1482454238971,12.058],[1482454526971,22.355],[1482454814971,36.042],[1482455102971,36.048],[1482455390971,36.048],[1482455678971,36.031]],\n  "last":[[1482369566971,0.0000],[1482369854971,0.0000],[1482370142971,0.0000],[1482370430971,0.0000],[1482370718971,0.0000],[1482371006971,0.0000],[1482371294971,0.0000],[1482371582971,0.0000],[1482371870971,0.0000],[1482372158971,0.0000],[1482372446971,0.0000],[1482372734971,0.0000],[1482373022971,0.0000],[1482373310971,0.0000],[1482373598971,0.0000],[1482373886971,0.0000],[1482374174971,0.0000],[1482374462971,0.0000],[1482374750971,0.0000],[1482375038971,0.0000],[1482375326971,0.0000],[1482375614971,0.0000],[1482375902971,0.0000],[1482376190971,0.0000],[1482376478971,0.0000],[1482376766971,0.0000],[1482377054971,0.0000],[1482377342971,0.0000],[1482377630971,0.0000],[1482377918971,0.0000],[1482378206971,0.0000],[1482378494971,0.0000],[1482378782971,0.0000],[1482379070971,0.0000],[1482379358971,0.0000],[1482379646971,0.0000],[1482379934971,0.0000],[1482380222971,0.0000],[1482380510971,0.0000],[1482380798971,0.0000],[1482381086971,0.0000],[1482381374971,0.0000],[1482381662971,0.0000],[1482381950971,0.0000],[1482382238971,0.0000],[1482382526971,0.0000],[1482382814971,0.0000],[1482383102971,0.0000],[1482383390971,0.0000],[1482383678971,0.0000],[1482383966971,0.0000],[1482384254971,0.0000],[1482384542971,0.0000],[1482384830971,0.0000],[1482385118971,0.0000],[1482385406971,0.0000],[1482385694971,0.0000],[1482385982971,0.0000],[1482386270971,0.0000],[1482386558971,0.0000],[1482386846971,0.0000],[1482387134971,0.0000],[1482387422971,0.0000],[1482387710971,0.0000],[1482387998971,0.0000],[1482388286971,0.0000],[1482388574971,0.0000],[1482388862971,0.0000],[1482389150971,0.0000],[1482389438971,0.0000],[1482389726971,0.0000],[1482390014971,0.0000],[1482390302971,0.0000],[1482390590971,0.0000],[1482390878971,0.0000],[1482391166971,0.0000],[1482391454971,0.0000],[1482391742971,0.0000],[1482392030971,0.0000],[1482392318971,0.0000],[1482392606971,0.0000],[1482392894971,0.0000],[1482393182971,0.0000],[1482393470971,0.0000],[1482393758971,0.0000],[1482394046971,0.0000],[1482394334971,0.0000],[1482394622971,0.0000],[1482394910971,0.0000],[1482395198971,0.0000],[1482395486971,0.0000],[1482395774971,0.0000],[1482396062971,0.0000],[1482396350971,0.0000],[1482396638971,0.0000],[1482396926971,0.0000],[1482397214971,0.0000],[1482397502971,0.0000],[1482397790971,0.0000],[1482398078971,0.0000],[1482398366971,0.0000],[1482398654971,0.0000],[1482398942971,0.0000],[1482399230971,0.0000],[1482399518971,0.0000],[1482399806971,0.0000],[1482400094971,0.0000],[1482400382971,0.0000],[1482400670971,0.0000],[1482400958971,0.0000],[1482401246971,0.0000],[1482401534971,0.0000],[1482401822971,0.0000],[1482402110971,0.0000],[1482402398971,0.0000],[1482402686971,0.0000],[1482402974971,0.0000],[1482403262971,0.0000],[1482403550971,0.0000],[1482403838971,0.0000],[1482404126971,0.0000],[1482404414971,0.0000],[1482404702971,0.0000],[1482404990971,0.0000],[1482405278971,0.0000],[1482405566971,0.0000],[1482405854971,0.0000],[1482406142971,0.0000],[1482406430971,0.0000],[1482406718971,0.0000],[1482407006971,0.0000],[1482407294971,0.0000],[1482407582971,0.0000],[1482407870971,0.0000],[1482408158971,0.0000],[1482408446971,0.0000],[1482408734971,0.0000],[1482409022971,0.0000],[1482409310971,0.0000],[1482409598971,0.0000],[1482409886971,0.0000],[1482410174971,0.0000],[1482410462971,0.0000],[1482410750971,0.0000],[1482411038971,0.0000],[1482411326971,0.0000],[1482411614971,0.0000],[1482411902971,0.0000],[1482412190971,0.0000],[1482412478971,0.0000],[1482412766971,0.0000],[1482413054971,0.0000],[1482413342971,0.0000],[1482413630971,0.0000],[1482413918971,0.0000],[1482414206971,0.0000],[1482414494971,0.0000],[1482414782971,0.0000],[1482415070971,0.0000],[1482415358971,0.0000],[1482415646971,0.0000],[1482415934971,0.0000],[1482416222971,0.0000],[1482416510971,0.0000],[1482416798971,0.0000],[1482417086971,0.0000],[1482417374971,0.0000],[1482417662971,0.0000],[1482417950971,0.0000],[1482418238971,0.0000],[1482418526971,0.0000],[1482418814971,0.0000],[1482419102971,0.0000],[1482419390971,0.0000],[1482419678971,0.0000],[1482419966971,0.0000],[1482420254971,0.0000],[1482420542971,0.0000],[1482420830971,0.0000],[1482421118971,0.0000],[1482421406971,0.0000],[1482421694971,0.0000],[1482421982971,0.0000],[1482422270971,0.0000],[1482422558971,0.0000],[1482422846971,0.0000],[1482423134971,0.0000],[1482423422971,0.0000],[1482423710971,0.0000],[1482423998971,0.0000],[1482424286971,0.0000],[1482424574971,0.0000],[1482424862971,0.0000],[1482425150971,0.0000],[1482425438971,0.0000],[1482425726971,0.0000],[1482426014971,0.0000],[1482426302971,0.0000],[1482426590971,0.0000],[1482426878971,0.0000],[1482427166971,0.0000],[1482427454971,0.0000],[1482427742971,0.0000],[1482428030971,0.0000],[1482428318971,0.0000],[1482428606971,0.0000],[1482428894971,0.0000],[1482429182971,0.0000],[1482429470971,0.0000],[1482429758971,0.0000],[1482430046971,0.0000],[1482430334971,0.0000],[1482430622971,0.0000],[1482430910971,0.0000],[1482431198971,0.0000],[1482431486971,0.0000],[1482431774971,0.0000],[1482432062971,0.0000],[1482432350971,0.0000],[1482432638971,0.0000],[1482432926971,0.0000],[1482433214971,0.0000],[1482433502971,0.0000],[1482433790971,0.0000],[1482434078971,0.0000],[1482434366971,0.0000],[1482434654971,0.0000],[1482434942971,0.0000],[1482435230971,0.0000],[1482435518971,0.0000],[1482435806971,0.0000],[1482436094971,0.0000],[1482436382971,0.0000],[1482436670971,0.0000],[1482436958971,0.0000],[1482437246971,0.0000],[1482437534971,0.0000],[1482437822971,0.0000],[1482438110971,0.0000],[1482438398971,0.0000],[1482438686971,0.0000],[1482438974971,0.0000],[1482439262971,0.0000],[1482439550971,0.0000],[1482439838971,0.0000],[1482440126971,0.0000],[1482440414971,0.0000],[1482440702971,0.0000],[1482440990971,0.0000],[1482441278971,0.0000],[1482441566971,0.0000],[1482441854971,0.0000],[1482442142971,0.0000],[1482442430971,0.0000],[1482442718971,0.0000],[1482443006971,0.0000],[1482443294971,0.0000],[1482443582971,0.0000],[1482443870971,0.0000],[1482444158971,16.985],[1482444446971,20.010],[1482444734971,20.013],[1482445022971,19.865],[1482445310971,20.011],[1482445598971,20.021],[1482445886971,0.0000],[1482446174971,0.0000],[1482446462971,0.0000],[1482446750971,0.0000],[1482447038971,0.0000],[1482447326971,0.0000],[1482447614971,0.0000],[1482447902971,0.0000],[1482448190971,12.055],[1482448478971,35.286],[1482448766971,12.060],[1482449054971,11.923],[1482449342971,11.833],[1482449630971,12.059],[1482449918971,12.047],[1482450206971,11.828],[1482450494971,12.046],[1482450782971,11.884],[1482451070971,12.058],[1482451358971,12.046],[1482451646971,12.059],[1482451934971,11.845],[1482452222971,12.050],[1482452510971,12.061],[1482452798971,11.995],[1482453086971,12.049],[1482453374971,11.838],[1482453662971,0.0000],[1482453950971,0.0000],[1482454238971,8.0702],[1482454526971,12.044],[1482454814971,0.0000],[1482455102971,36.009],[1482455390971,35.988],[1482455678971,36.006]],\n  "min":[[1482369566971,0.0000],[1482369854971,0.0000],[1482370142971,0.0000],[1482370430971,0.0000],[1482370718971,0.0000],[1482371006971,0.0000],[1482371294971,0.0000],[1482371582971,0.0000],[1482371870971,0.0000],[1482372158971,0.0000],[1482372446971,0.0000],[1482372734971,0.0000],[1482373022971,0.0000],[1482373310971,0.0000],[1482373598971,0.0000],[1482373886971,0.0000],[1482374174971,0.0000],[1482374462971,0.0000],[1482374750971,0.0000],[1482375038971,0.0000],[1482375326971,0.0000],[1482375614971,0.0000],[1482375902971,0.0000],[1482376190971,0.0000],[1482376478971,0.0000],[1482376766971,0.0000],[1482377054971,0.0000],[1482377342971,0.0000],[1482377630971,0.0000],[1482377918971,0.0000],[1482378206971,0.0000],[1482378494971,0.0000],[1482378782971,0.0000],[1482379070971,0.0000],[1482379358971,0.0000],[1482379646971,0.0000],[1482379934971,0.0000],[1482380222971,0.0000],[1482380510971,0.0000],[1482380798971,0.0000],[1482381086971,0.0000],[1482381374971,0.0000],[1482381662971,0.0000],[1482381950971,0.0000],[1482382238971,0.0000],[1482382526971,0.0000],[1482382814971,0.0000],[1482383102971,0.0000],[1482383390971,0.0000],[1482383678971,0.0000],[1482383966971,0.0000],[1482384254971,0.0000],[1482384542971,0.0000],[1482384830971,0.0000],[1482385118971,0.0000],[1482385406971,0.0000],[1482385694971,0.0000],[1482385982971,0.0000],[1482386270971,0.0000],[1482386558971,0.0000],[1482386846971,0.0000],[1482387134971,0.0000],[1482387422971,0.0000],[1482387710971,0.0000],[1482387998971,0.0000],[1482388286971,0.0000],[1482388574971,0.0000],[1482388862971,0.0000],[1482389150971,0.0000],[1482389438971,0.0000],[1482389726971,0.0000],[1482390014971,0.0000],[1482390302971,0.0000],[1482390590971,0.0000],[1482390878971,0.0000],[1482391166971,0.0000],[1482391454971,0.0000],[1482391742971,0.0000],[1482392030971,0.0000],[1482392318971,0.0000],[1482392606971,0.0000],[1482392894971,0.0000],[1482393182971,0.0000],[1482393470971,0.0000],[1482393758971,0.0000],[1482394046971,0.0000],[1482394334971,0.0000],[1482394622971,0.0000],[1482394910971,0.0000],[1482395198971,0.0000],[1482395486971,0.0000],[1482395774971,0.0000],[1482396062971,0.0000],[1482396350971,0.0000],[1482396638971,0.0000],[1482396926971,0.0000],[1482397214971,0.0000],[1482397502971,0.0000],[1482397790971,0.0000],[1482398078971,0.0000],[1482398366971,0.0000],[1482398654971,0.0000],[1482398942971,0.0000],[1482399230971,0.0000],[1482399518971,0.0000],[1482399806971,0.0000],[1482400094971,0.0000],[1482400382971,0.0000],[1482400670971,0.0000],[1482400958971,0.0000],[1482401246971,0.0000],[1482401534971,0.0000],[1482401822971,0.0000],[1482402110971,0.0000],[1482402398971,0.0000],[1482402686971,0.0000],[1482402974971,0.0000],[1482403262971,0.0000],[1482403550971,0.0000],[1482403838971,0.0000],[1482404126971,0.0000],[1482404414971,0.0000],[1482404702971,0.0000],[1482404990971,0.0000],[1482405278971,0.0000],[1482405566971,0.0000],[1482405854971,0.0000],[1482406142971,0.0000],[1482406430971,0.0000],[1482406718971,0.0000],[1482407006971,0.0000],[1482407294971,0.0000],[1482407582971,0.0000],[1482407870971,0.0000],[1482408158971,0.0000],[1482408446971,0.0000],[1482408734971,0.0000],[1482409022971,0.0000],[1482409310971,0.0000],[1482409598971,0.0000],[1482409886971,0.0000],[1482410174971,0.0000],[1482410462971,0.0000],[1482410750971,0.0000],[1482411038971,0.0000],[1482411326971,0.0000],[1482411614971,0.0000],[1482411902971,0.0000],[1482412190971,0.0000],[1482412478971,0.0000],[1482412766971,0.0000],[1482413054971,0.0000],[1482413342971,0.0000],[1482413630971,0.0000],[1482413918971,0.0000],[1482414206971,0.0000],[1482414494971,0.0000],[1482414782971,0.0000],[1482415070971,0.0000],[1482415358971,0.0000],[1482415646971,0.0000],[1482415934971,0.0000],[1482416222971,0.0000],[1482416510971,0.0000],[1482416798971,0.0000],[1482417086971,0.0000],[1482417374971,0.0000],[1482417662971,0.0000],[1482417950971,0.0000],[1482418238971,0.0000],[1482418526971,0.0000],[1482418814971,0.0000],[1482419102971,0.0000],[1482419390971,0.0000],[1482419678971,0.0000],[1482419966971,0.0000],[1482420254971,0.0000],[1482420542971,0.0000],[1482420830971,0.0000],[1482421118971,0.0000],[1482421406971,0.0000],[1482421694971,0.0000],[1482421982971,0.0000],[1482422270971,0.0000],[1482422558971,0.0000],[1482422846971,0.0000],[1482423134971,0.0000],[1482423422971,0.0000],[1482423710971,0.0000],[1482423998971,0.0000],[1482424286971,0.0000],[1482424574971,0.0000],[1482424862971,0.0000],[1482425150971,0.0000],[1482425438971,0.0000],[1482425726971,0.0000],[1482426014971,0.0000],[1482426302971,0.0000],[1482426590971,0.0000],[1482426878971,0.0000],[1482427166971,0.0000],[1482427454971,0.0000],[1482427742971,0.0000],[1482428030971,0.0000],[1482428318971,0.0000],[1482428606971,0.0000],[1482428894971,0.0000],[1482429182971,0.0000],[1482429470971,0.0000],[1482429758971,0.0000],[1482430046971,0.0000],[1482430334971,0.0000],[1482430622971,0.0000],[1482430910971,0.0000],[1482431198971,0.0000],[1482431486971,0.0000],[1482431774971,0.0000],[1482432062971,0.0000],[1482432350971,0.0000],[1482432638971,0.0000],[1482432926971,0.0000],[1482433214971,0.0000],[1482433502971,0.0000],[1482433790971,0.0000],[1482434078971,0.0000],[1482434366971,0.0000],[1482434654971,0.0000],[1482434942971,0.0000],[1482435230971,0.0000],[1482435518971,0.0000],[1482435806971,0.0000],[1482436094971,0.0000],[1482436382971,0.0000],[1482436670971,0.0000],[1482436958971,0.0000],[1482437246971,0.0000],[1482437534971,0.0000],[1482437822971,0.0000],[1482438110971,0.0000],[1482438398971,0.0000],[1482438686971,0.0000],[1482438974971,0.0000],[1482439262971,0.0000],[1482439550971,0.0000],[1482439838971,0.0000],[1482440126971,0.0000],[1482440414971,0.0000],[1482440702971,0.0000],[1482440990971,0.0000],[1482441278971,0.0000],[1482441566971,0.0000],[1482441854971,0.0000],[1482442142971,0.0000],[1482442430971,0.0000],[1482442718971,0.0000],[1482443006971,0.0000],[1482443294971,0.0000],[1482443582971,0.0000],[1482443870971,0.0000],[1482444158971,16.985],[1482444446971,19.812],[1482444734971,19.804],[1482445022971,19.819],[1482445310971,19.772],[1482445598971,0.0000],[1482445886971,0.0000],[1482446174971,0.0000],[1482446462971,0.0000],[1482446750971,0.0000],[1482447038971,0.0000],[1482447326971,0.0000],[1482447614971,0.0000],[1482447902971,0.0000],[1482448190971,0.0000],[1482448478971,11.854],[1482448766971,11.806],[1482449054971,11.801],[1482449342971,11.808],[1482449630971,11.803],[1482449918971,11.810],[1482450206971,11.821],[1482450494971,11.814],[1482450782971,11.810],[1482451070971,11.808],[1482451358971,11.820],[1482451646971,11.810],[1482451934971,11.820],[1482452222971,11.801],[1482452510971,11.791],[1482452798971,11.822],[1482453086971,11.827],[1482453374971,0.0000],[1482453662971,0.0000],[1482453950971,0.0000],[1482454238971,0.0000],[1482454526971,0.0000],[1482454814971,0.0000],[1482455102971,35.957],[1482455390971,35.956],[1482455678971,35.956]]\n}'
_trenddata_24hr_do_json = '{\n  "max":[[1482369567274,-0.23436],[1482369855274,-0.24195],[1482370143274,-0.24118],[1482370431274,-0.23443],[1482370719274,-0.23773],[1482371007274,-0.23583],[1482371295274,-0.24406],[1482371583274,-0.24309],[1482371871274,-0.24103],[1482372159274,-0.23593],[1482372447274,-0.23600],[1482372735274,-0.24510],[1482373023274,-0.24442],[1482373311274,-0.24489],[1482373599274,-0.24053],[1482373887274,-0.24520],[1482374175274,-0.24032],[1482374463274,-0.24400],[1482374751274,-0.24319],[1482375039274,-0.24227],[1482375327274,-0.24529],[1482375615274,-0.23972],[1482375903274,-0.23613],[1482376191274,-0.23644],[1482376479274,-0.24474],[1482376767274,-0.24472],[1482377055274,-0.24428],[1482377343274,-0.24039],[1482377631274,-0.24332],[1482377919274,-0.24392],[1482378207274,-0.24401],[1482378495274,-0.24386],[1482378783274,-0.24416],[1482379071274,-0.24340],[1482379359274,-0.24046],[1482379647274,-0.24243],[1482379935274,-0.23355],[1482380223274,-0.23743],[1482380511274,-0.23742],[1482380799274,-0.23574],[1482381087274,-0.23443],[1482381375274,-0.23358],[1482381663274,-0.24468],[1482381951274,-0.23310],[1482382239274,-0.24117],[1482382527274,-0.24289],[1482382815274,-0.24227],[1482383103274,-0.23321],[1482383391274,-0.24474],[1482383679274,-0.24149],[1482383967274,-0.22868],[1482384255274,-0.22833],[1482384543274,-0.24350],[1482384831274,-0.24018],[1482385119274,-0.23708],[1482385407274,-0.22298],[1482385695274,-0.24144],[1482385983274,-0.24413],[1482386271274,-0.24363],[1482386559274,-0.24387],[1482386847274,-0.24227],[1482387135274,-0.24193],[1482387423274,-0.24324],[1482387711274,-0.24482],[1482387999274,-0.23596],[1482388287274,-0.23867],[1482388575274,-0.24084],[1482388863274,-0.23211],[1482389151274,-0.23093],[1482389439274,-0.24138],[1482389727274,-0.24159],[1482390015274,-0.24513],[1482390303274,-0.24461],[1482390591274,-0.24480],[1482390879274,-0.24363],[1482391167274,-0.24382],[1482391455274,-0.24091],[1482391743274,-0.23323],[1482392031274,-0.23348],[1482392319274,-0.23879],[1482392607274,-0.24526],[1482392895274,-0.24152],[1482393183274,-0.24149],[1482393471274,-0.24482],[1482393759274,-0.24308],[1482394047274,-0.24411],[1482394335274,-0.24416],[1482394623274,-0.24217],[1482394911274,-0.24223],[1482395199274,-0.24002],[1482395487274,-0.24299],[1482395775274,-0.22967],[1482396063274,-0.23003],[1482396351274,-0.22619],[1482396639274,-0.22592],[1482396927274,-0.22855],[1482397215274,-0.24345],[1482397503274,-0.22927],[1482397791274,-0.23239],[1482398079274,-0.23349],[1482398367274,-0.22559],[1482398655274,-0.22994],[1482398943274,-0.23049],[1482399231274,-0.22782],[1482399519274,-0.23467],[1482399807274,-0.23039],[1482400095274,-0.23083],[1482400383274,-0.23202],[1482400671274,-0.22826],[1482400959274,-0.22721],[1482401247274,-0.23088],[1482401535274,-0.23376],[1482401823274,-0.22171],[1482402111274,-0.22231],[1482402399274,-0.23293],[1482402687274,-0.23757],[1482402975274,-0.22953],[1482403263274,-0.23800],[1482403551274,-0.23483],[1482403839274,-0.24217],[1482404127274,-0.24076],[1482404415274,-0.22596],[1482404703274,-0.23425],[1482404991274,-0.23051],[1482405279274,-0.23019],[1482405567274,-0.22796],[1482405855274,-0.23621],[1482406143274,-0.23602],[1482406431274,-0.23843],[1482406719274,-0.23736],[1482407007274,-0.23092],[1482407295274,-0.23674],[1482407583274,-0.22731],[1482407871274,-0.23426],[1482408159274,-0.24216],[1482408447274,-0.23809],[1482408735274,-0.23779],[1482409023274,-0.23668],[1482409311274,-0.24060],[1482409599274,-0.23519],[1482409887274,-0.23037],[1482410175274,-0.23948],[1482410463274,-0.23886],[1482410751274,-0.24417],[1482411039274,-0.24464],[1482411327274,-0.24370],[1482411615274,-0.24145],[1482411903274,-0.24400],[1482412191274,-0.23890],[1482412479274,-0.24241],[1482412767274,-0.23498],[1482413055274,-0.23413],[1482413343274,-0.24364],[1482413631274,-0.24403],[1482413919274,-0.23784],[1482414207274,-0.22149],[1482414495274,-0.24081],[1482414783274,-0.23937],[1482415071274,-0.23995],[1482415359274,-0.23199],[1482415647274,-0.23192],[1482415935274,-0.22646],[1482416223274,-0.22927],[1482416511274,-0.23644],[1482416799274,-0.23515],[1482417087274,-0.24356],[1482417375274,-0.23652],[1482417663274,-0.23727],[1482417951274,-0.23593],[1482418239274,-0.23351],[1482418527274,-0.23992],[1482418815274,-0.24016],[1482419103274,-0.22742],[1482419391274,-0.24019],[1482419679274,-0.22796],[1482419967274,-0.23277],[1482420255274,-0.23450],[1482420543274,-0.22789],[1482420831274,-0.22296],[1482421119274,-0.23510],[1482421407274,-0.23885],[1482421695274,-0.23475],[1482421983274,-0.23242],[1482422271274,-0.23409],[1482422559274,-0.24005],[1482422847274,-0.24273],[1482423135274,-0.23996],[1482423423274,-0.23490],[1482423711274,-0.22956],[1482423999274,-0.24042],[1482424287274,-0.22955],[1482424575274,-0.23586],[1482424863274,-0.23709],[1482425151274,-0.23063],[1482425439274,-0.23311],[1482425727274,-0.24546],[1482426015274,-0.23996],[1482426303274,-0.23906],[1482426591274,-0.23114],[1482426879274,-0.23283],[1482427167274,-0.24005],[1482427455274,-0.24105],[1482427743274,-0.23970],[1482428031274,-0.23154],[1482428319274,-0.23746],[1482428607274,-0.24067],[1482428895274,-0.23944],[1482429183274,-0.23627],[1482429471274,-0.24509],[1482429759274,-0.23769],[1482430047274,-0.24202],[1482430335274,-0.24492],[1482430623274,-0.23196],[1482430911274,-0.24010],[1482431199274,-0.24560],[1482431487274,-0.24539],[1482431775274,-0.24505],[1482432063274,-0.23972],[1482432351274,-0.23798],[1482432639274,-0.23803],[1482432927274,-0.23960],[1482433215274,-0.24430],[1482433503274,-0.24401],[1482433791274,-0.24255],[1482434079274,-0.24418],[1482434367274,-0.24076],[1482434655274,-0.23818],[1482434943274,-0.24088],[1482435231274,-0.23909],[1482435519274,-0.23454],[1482435807274,-0.23644],[1482436095274,-0.23996],[1482436383274,-0.24114],[1482436671274,-0.23982],[1482436959274,-0.23483],[1482437247274,-0.23735],[1482437535274,-0.24297],[1482437823274,-0.23787],[1482438111274,-0.23025],[1482438399274,-0.23225],[1482438687274,-0.23256],[1482438975274,-0.23504],[1482439263274,-0.23944],[1482439551274,-0.24016],[1482439839274,-0.23817],[1482440127274,-0.22725],[1482440415274,-0.23807],[1482440703274,-0.23665],[1482440991274,-0.24049],[1482441279274,-0.23715],[1482441567274,-0.22935],[1482441855274,-0.24368],[1482442143274,-0.24070],[1482442431274,-0.23433],[1482442719274,-0.22831],[1482443007274,-0.24318],[1482443295274,-0.23606],[1482443583274,-0.24243],[1482443871274,-0.24098],[1482444159274,-0.24417],[1482444447274,-0.22562],[1482444735274,-0.23560],[1482445023274,-0.24243],[1482445311274,-0.24318],[1482445599274,-0.24393],[1482445887274,-0.24295],[1482446175274,-0.24032],[1482446463274,-0.24289],[1482446751274,-0.24366],[1482447039274,-0.23764],[1482447327274,-0.24335],[1482447615274,-0.24062],[1482447903274,-0.23783],[1482448191274,-0.23004],[1482448479274,-0.24241],[1482448767274,-0.24355],[1482449055274,-0.24305],[1482449343274,-0.24305],[1482449631274,-0.24111],[1482449919274,-0.24250],[1482450207274,-0.22990],[1482450495274,-0.23638],[1482450783274,-0.23584],[1482451071274,-0.23475],[1482451359274,-0.23410],[1482451647274,-0.24267],[1482451935274,-0.24422],[1482452223274,-0.24468],[1482452511274,-0.24539],[1482452799274,-0.24464],[1482453087274,-0.24513],[1482453375274,-0.24437],[1482453663274,-0.24512],[1482453951274,-0.24478],[1482454239274,-0.24485],[1482454527274,-0.24393],[1482454815274,-0.24455],[1482455103274,-0.24364],[1482455391274,-0.24358],[1482455679274,-0.24403]],\n  "last":[[1482369567274,-0.24597],[1482369855274,-0.24731],[1482370143274,-0.24764],[1482370431274,-0.24603],[1482370719274,-0.24609],[1482371007274,-0.24693],[1482371295274,-0.24764],[1482371583274,-0.24681],[1482371871274,-0.25072],[1482372159274,-0.24488],[1482372447274,-0.24656],[1482372735274,-0.24751],[1482373023274,-0.24740],[1482373311274,-0.24767],[1482373599274,-0.24709],[1482373887274,-0.24608],[1482374175274,-0.24662],[1482374463274,-0.24733],[1482374751274,-0.24738],[1482375039274,-0.24641],[1482375327274,-0.24666],[1482375615274,-0.24812],[1482375903274,-0.24748],[1482376191274,-0.24771],[1482376479274,-0.24717],[1482376767274,-0.24752],[1482377055274,-0.24710],[1482377343274,-0.24614],[1482377631274,-0.24711],[1482377919274,-0.24663],[1482378207274,-0.24733],[1482378495274,-0.24972],[1482378783274,-0.24829],[1482379071274,-0.24762],[1482379359274,-0.24743],[1482379647274,-0.24816],[1482379935274,-0.23580],[1482380223274,-0.24564],[1482380511274,-0.24673],[1482380799274,-0.24741],[1482381087274,-0.24707],[1482381375274,-0.24842],[1482381663274,-0.24728],[1482381951274,-0.24676],[1482382239274,-0.24649],[1482382527274,-0.24638],[1482382815274,-0.24780],[1482383103274,-0.24764],[1482383391274,-0.24754],[1482383679274,-0.24703],[1482383967274,-0.24754],[1482384255274,-0.24566],[1482384543274,-0.24752],[1482384831274,-0.24747],[1482385119274,-0.24721],[1482385407274,-0.24707],[1482385695274,-0.24685],[1482385983274,-0.24679],[1482386271274,-0.24731],[1482386559274,-0.24584],[1482386847274,-0.24699],[1482387135274,-0.24716],[1482387423274,-0.24745],[1482387711274,-0.24754],[1482387999274,-0.24706],[1482388287274,-0.24685],[1482388575274,-0.24825],[1482388863274,-0.24515],[1482389151274,-0.24743],[1482389439274,-0.24685],[1482389727274,-0.24689],[1482390015274,-0.24757],[1482390303274,-0.24633],[1482390591274,-0.24867],[1482390879274,-0.24765],[1482391167274,-0.24612],[1482391455274,-0.24685],[1482391743274,-0.24750],[1482392031274,-0.26103],[1482392319274,-0.24679],[1482392607274,-0.24786],[1482392895274,-0.24693],[1482393183274,-0.24657],[1482393471274,-0.24690],[1482393759274,-0.24730],[1482394047274,-0.24760],[1482394335274,-0.24683],[1482394623274,-0.24775],[1482394911274,-0.24900],[1482395199274,-0.24681],[1482395487274,-0.24738],[1482395775274,-0.24667],[1482396063274,-0.24551],[1482396351274,-0.24345],[1482396639274,-0.24594],[1482396927274,-0.24493],[1482397215274,-0.24730],[1482397503274,-0.24665],[1482397791274,-0.24635],[1482398079274,-0.24805],[1482398367274,-0.24564],[1482398655274,-0.24706],[1482398943274,-0.23136],[1482399231274,-0.24727],[1482399519274,-0.24665],[1482399807274,-0.24598],[1482400095274,-0.24710],[1482400383274,-0.24380],[1482400671274,-0.24867],[1482400959274,-0.24713],[1482401247274,-0.24665],[1482401535274,-0.24816],[1482401823274,-0.24608],[1482402111274,-0.24090],[1482402399274,-0.24345],[1482402687274,-0.24711],[1482402975274,-0.24663],[1482403263274,-0.24718],[1482403551274,-0.24667],[1482403839274,-0.24693],[1482404127274,-0.24665],[1482404415274,-0.24734],[1482404703274,-0.24645],[1482404991274,-0.25024],[1482405279274,-0.24720],[1482405567274,-0.24731],[1482405855274,-0.25030],[1482406143274,-0.24614],[1482406431274,-0.24566],[1482406719274,-0.24728],[1482407007274,-0.24700],[1482407295274,-0.25159],[1482407583274,-0.24795],[1482407871274,-0.24714],[1482408159274,-0.24709],[1482408447274,-0.24655],[1482408735274,-0.24663],[1482409023274,-0.24799],[1482409311274,-0.24394],[1482409599274,-0.24735],[1482409887274,-0.24891],[1482410175274,-0.24601],[1482410463274,-0.24670],[1482410751274,-0.24737],[1482411039274,-0.24693],[1482411327274,-0.24618],[1482411615274,-0.24806],[1482411903274,-0.24551],[1482412191274,-0.24629],[1482412479274,-0.24795],[1482412767274,-0.24690],[1482413055274,-0.24704],[1482413343274,-0.24680],[1482413631274,-0.24635],[1482413919274,-0.24718],[1482414207274,-0.24689],[1482414495274,-0.24662],[1482414783274,-0.24656],[1482415071274,-0.24585],[1482415359274,-0.24667],[1482415647274,-0.24632],[1482415935274,-0.24643],[1482416223274,-0.24651],[1482416511274,-0.24812],[1482416799274,-0.24745],[1482417087274,-0.24653],[1482417375274,-0.24721],[1482417663274,-0.24866],[1482417951274,-0.24804],[1482418239274,-0.24623],[1482418527274,-0.24704],[1482418815274,-0.24723],[1482419103274,-0.24681],[1482419391274,-0.24686],[1482419679274,-0.24585],[1482419967274,-0.24643],[1482420255274,-0.24372],[1482420543274,-0.24603],[1482420831274,-0.24540],[1482421119274,-0.24618],[1482421407274,-0.24693],[1482421695274,-0.24608],[1482421983274,-0.24760],[1482422271274,-0.24685],[1482422559274,-0.24578],[1482422847274,-0.24683],[1482423135274,-0.24710],[1482423423274,-0.26289],[1482423711274,-0.24714],[1482423999274,-0.24690],[1482424287274,-0.25013],[1482424575274,-0.25142],[1482424863274,-0.24677],[1482425151274,-0.24709],[1482425439274,-0.24700],[1482425727274,-0.24734],[1482426015274,-0.24645],[1482426303274,-0.24704],[1482426591274,-0.24762],[1482426879274,-0.24844],[1482427167274,-0.24839],[1482427455274,-0.24694],[1482427743274,-0.24721],[1482428031274,-0.24758],[1482428319274,-0.24846],[1482428607274,-0.24758],[1482428895274,-0.24774],[1482429183274,-0.24628],[1482429471274,-0.24622],[1482429759274,-0.24897],[1482430047274,-0.24645],[1482430335274,-0.24677],[1482430623274,-0.24709],[1482430911274,-0.24607],[1482431199274,-0.24781],[1482431487274,-0.24768],[1482431775274,-0.24908],[1482432063274,-0.24646],[1482432351274,-0.24858],[1482432639274,-0.24669],[1482432927274,-0.24703],[1482433215274,-0.24770],[1482433503274,-0.24741],[1482433791274,-0.24703],[1482434079274,-0.24772],[1482434367274,-0.24642],[1482434655274,-0.24726],[1482434943274,-0.24738],[1482435231274,-0.24663],[1482435519274,-0.24619],[1482435807274,-0.24629],[1482436095274,-0.24709],[1482436383274,-0.24764],[1482436671274,-0.24641],[1482436959274,-0.24734],[1482437247274,-0.24652],[1482437535274,-0.24757],[1482437823274,-0.24617],[1482438111274,-0.24641],[1482438399274,-0.24669],[1482438687274,-0.24667],[1482438975274,-0.24740],[1482439263274,-0.24767],[1482439551274,-0.24856],[1482439839274,-0.24642],[1482440127274,-0.24792],[1482440415274,-0.24670],[1482440703274,-0.24648],[1482440991274,-0.24049],[1482441279274,-0.24523],[1482441567274,-0.24846],[1482441855274,-0.24486],[1482442143274,-0.24813],[1482442431274,-0.24730],[1482442719274,-0.24826],[1482443007274,-0.24981],[1482443295274,-0.26678],[1482443583274,-0.25503],[1482443871274,-0.24652],[1482444159274,-0.24660],[1482444447274,-0.24632],[1482444735274,-0.24503],[1482445023274,-0.24587],[1482445311274,-0.24762],[1482445599274,-0.24741],[1482445887274,-0.24666],[1482446175274,-0.24594],[1482446463274,-0.24583],[1482446751274,-0.24642],[1482447039274,-0.24553],[1482447327274,-0.24690],[1482447615274,-0.24843],[1482447903274,-0.24645],[1482448191274,-0.24559],[1482448479274,-0.24604],[1482448767274,-0.24643],[1482449055274,-0.24553],[1482449343274,-0.24776],[1482449631274,-0.24619],[1482449919274,-0.24717],[1482450207274,-0.24730],[1482450495274,-0.25102],[1482450783274,-0.24697],[1482451071274,-0.24844],[1482451359274,-0.23638],[1482451647274,-0.25005],[1482451935274,-0.24770],[1482452223274,-0.24556],[1482452511274,-0.24676],[1482452799274,-0.24750],[1482453087274,-0.24737],[1482453375274,-0.24747],[1482453663274,-0.24663],[1482453951274,-0.24686],[1482454239274,-0.24747],[1482454527274,-0.24770],[1482454815274,-0.24724],[1482455103274,-0.24796],[1482455391274,-0.24856],[1482455679274,-0.24842]],\n  "min":[[1482369567274,-0.26223],[1482369855274,-0.25279],[1482370143274,-0.25323],[1482370431274,-0.26743],[1482370719274,-0.25984],[1482371007274,-0.26228],[1482371295274,-0.25009],[1482371583274,-0.25666],[1482371871274,-0.25451],[1482372159274,-0.26271],[1482372447274,-0.25885],[1482372735274,-0.25591],[1482373023274,-0.25003],[1482373311274,-0.25279],[1482373599274,-0.25132],[1482373887274,-0.25064],[1482374175274,-0.25354],[1482374463274,-0.25455],[1482374751274,-0.25667],[1482375039274,-0.25504],[1482375327274,-0.25412],[1482375615274,-0.25533],[1482375903274,-0.25691],[1482376191274,-0.24985],[1482376479274,-0.25234],[1482376767274,-0.25252],[1482377055274,-0.25237],[1482377343274,-0.25109],[1482377631274,-0.25143],[1482377919274,-0.25238],[1482378207274,-0.24971],[1482378495274,-0.24972],[1482378783274,-0.25521],[1482379071274,-0.25700],[1482379359274,-0.24958],[1482379647274,-0.26295],[1482379935274,-0.25697],[1482380223274,-0.26423],[1482380511274,-0.26211],[1482380799274,-0.26584],[1482381087274,-0.25712],[1482381375274,-0.25920],[1482381663274,-0.25053],[1482381951274,-0.26787],[1482382239274,-0.26645],[1482382527274,-0.25099],[1482382815274,-0.25203],[1482383103274,-0.25935],[1482383391274,-0.26458],[1482383679274,-0.26160],[1482383967274,-0.26123],[1482384255274,-0.26614],[1482384543274,-0.25738],[1482384831274,-0.25827],[1482385119274,-0.25869],[1482385407274,-0.26906],[1482385695274,-0.25697],[1482385983274,-0.25534],[1482386271274,-0.25445],[1482386559274,-0.25055],[1482386847274,-0.26148],[1482387135274,-0.25465],[1482387423274,-0.25244],[1482387711274,-0.25060],[1482387999274,-0.26257],[1482388287274,-0.25014],[1482388575274,-0.25309],[1482388863274,-0.26409],[1482389151274,-0.26688],[1482389439274,-0.26573],[1482389727274,-0.25092],[1482390015274,-0.25128],[1482390303274,-0.24971],[1482390591274,-0.25439],[1482390879274,-0.24982],[1482391167274,-0.25613],[1482391455274,-0.25422],[1482391743274,-0.26678],[1482392031274,-0.26805],[1482392319274,-0.25616],[1482392607274,-0.25130],[1482392895274,-0.24973],[1482393183274,-0.24992],[1482393471274,-0.24951],[1482393759274,-0.25097],[1482394047274,-0.25149],[1482394335274,-0.25574],[1482394623274,-0.25867],[1482394911274,-0.25246],[1482395199274,-0.24956],[1482395487274,-0.25174],[1482395775274,-0.26068],[1482396063274,-0.26351],[1482396351274,-0.26267],[1482396639274,-0.26982],[1482396927274,-0.26526],[1482397215274,-0.25314],[1482397503274,-0.25939],[1482397791274,-0.26440],[1482398079274,-0.26472],[1482398367274,-0.26348],[1482398655274,-0.26279],[1482398943274,-0.27468],[1482399231274,-0.27035],[1482399519274,-0.26315],[1482399807274,-0.26481],[1482400095274,-0.26122],[1482400383274,-0.26446],[1482400671274,-0.26123],[1482400959274,-0.26655],[1482401247274,-0.27602],[1482401535274,-0.26460],[1482401823274,-0.27248],[1482402111274,-0.27155],[1482402399274,-0.26420],[1482402687274,-0.25963],[1482402975274,-0.26113],[1482403263274,-0.24901],[1482403551274,-0.25383],[1482403839274,-0.24886],[1482404127274,-0.24997],[1482404415274,-0.26787],[1482404703274,-0.25722],[1482404991274,-0.26672],[1482405279274,-0.26560],[1482405567274,-0.26089],[1482405855274,-0.26082],[1482406143274,-0.26237],[1482406431274,-0.25292],[1482406719274,-0.25035],[1482407007274,-0.26162],[1482407295274,-0.25585],[1482407583274,-0.26058],[1482407871274,-0.26450],[1482408159274,-0.25173],[1482408447274,-0.25365],[1482408735274,-0.25558],[1482409023274,-0.26195],[1482409311274,-0.26100],[1482409599274,-0.26525],[1482409887274,-0.26821],[1482410175274,-0.26189],[1482410463274,-0.25067],[1482410751274,-0.25537],[1482411039274,-0.25592],[1482411327274,-0.25034],[1482411615274,-0.24898],[1482411903274,-0.24939],[1482412191274,-0.25358],[1482412479274,-0.25432],[1482412767274,-0.25241],[1482413055274,-0.25827],[1482413343274,-0.25484],[1482413631274,-0.24938],[1482413919274,-0.26276],[1482414207274,-0.26219],[1482414495274,-0.25269],[1482414783274,-0.24959],[1482415071274,-0.26297],[1482415359274,-0.26487],[1482415647274,-0.26028],[1482415935274,-0.26383],[1482416223274,-0.25682],[1482416511274,-0.25521],[1482416799274,-0.25053],[1482417087274,-0.25346],[1482417375274,-0.26202],[1482417663274,-0.25310],[1482417951274,-0.25552],[1482418239274,-0.25210],[1482418527274,-0.25336],[1482418815274,-0.25732],[1482419103274,-0.25867],[1482419391274,-0.24928],[1482419679274,-0.26741],[1482419967274,-0.26530],[1482420255274,-0.25752],[1482420543274,-0.25976],[1482420831274,-0.26331],[1482421119274,-0.25371],[1482421407274,-0.25172],[1482421695274,-0.26069],[1482421983274,-0.25266],[1482422271274,-0.26131],[1482422559274,-0.25193],[1482422847274,-0.25414],[1482423135274,-0.25620],[1482423423274,-0.26289],[1482423711274,-0.26865],[1482423999274,-0.25775],[1482424287274,-0.27434],[1482424575274,-0.26856],[1482424863274,-0.25652],[1482425151274,-0.26032],[1482425439274,-0.25966],[1482425727274,-0.24997],[1482426015274,-0.25130],[1482426303274,-0.27155],[1482426591274,-0.25991],[1482426879274,-0.25770],[1482427167274,-0.25976],[1482427455274,-0.26447],[1482427743274,-0.26328],[1482428031274,-0.25647],[1482428319274,-0.25527],[1482428607274,-0.26893],[1482428895274,-0.25414],[1482429183274,-0.25824],[1482429471274,-0.25258],[1482429759274,-0.26104],[1482430047274,-0.25255],[1482430335274,-0.25330],[1482430623274,-0.26362],[1482430911274,-0.25504],[1482431199274,-0.24975],[1482431487274,-0.24915],[1482431775274,-0.25017],[1482432063274,-0.25776],[1482432351274,-0.25837],[1482432639274,-0.26430],[1482432927274,-0.24948],[1482433215274,-0.24898],[1482433503274,-0.24873],[1482433791274,-0.25432],[1482434079274,-0.24876],[1482434367274,-0.24904],[1482434655274,-0.25216],[1482434943274,-0.24920],[1482435231274,-0.25123],[1482435519274,-0.25256],[1482435807274,-0.25395],[1482436095274,-0.25752],[1482436383274,-0.24938],[1482436671274,-0.24936],[1482436959274,-0.25571],[1482437247274,-0.25003],[1482437535274,-0.25596],[1482437823274,-0.26079],[1482438111274,-0.26444],[1482438399274,-0.25137],[1482438687274,-0.25874],[1482438975274,-0.25983],[1482439263274,-0.26128],[1482439551274,-0.25633],[1482439839274,-0.25184],[1482440127274,-0.25449],[1482440415274,-0.25426],[1482440703274,-0.26011],[1482440991274,-0.25081],[1482441279274,-0.25099],[1482441567274,-0.25858],[1482441855274,-0.25097],[1482442143274,-0.26015],[1482442431274,-0.25619],[1482442719274,-0.26056],[1482443007274,-0.26386],[1482443295274,-0.26678],[1482443583274,-0.25503],[1482443871274,-0.25743],[1482444159274,-0.25097],[1482444447274,-0.25906],[1482444735274,-0.26321],[1482445023274,-0.26305],[1482445311274,-0.25646],[1482445599274,-0.25714],[1482445887274,-0.25714],[1482446175274,-0.25220],[1482446463274,-0.25087],[1482446751274,-0.25031],[1482447039274,-0.25515],[1482447327274,-0.25370],[1482447615274,-0.26542],[1482447903274,-0.26492],[1482448191274,-0.25270],[1482448479274,-0.25358],[1482448767274,-0.25139],[1482449055274,-0.25167],[1482449343274,-0.25125],[1482449631274,-0.25286],[1482449919274,-0.25118],[1482450207274,-0.26030],[1482450495274,-0.26422],[1482450783274,-0.26376],[1482451071274,-0.26519],[1482451359274,-0.26529],[1482451647274,-0.25190],[1482451935274,-0.25381],[1482452223274,-0.25006],[1482452511274,-0.24975],[1482452799274,-0.24962],[1482453087274,-0.25063],[1482453375274,-0.25005],[1482453663274,-0.25033],[1482453951274,-0.25060],[1482454239274,-0.24975],[1482454527274,-0.25005],[1482454815274,-0.25072],[1482455103274,-0.24990],[1482455391274,-0.25044],[1482455679274,-0.25035]]\n}'
_trenddata_24hr_ph_json = '{\n  "max":[[1482369567575,7.2730],[1482369855575,7.2748],[1482370143575,7.2748],[1482370431575,7.2730],[1482370719575,7.2723],[1482371007575,7.2718],[1482371295575,7.2723],[1482371583575,7.2737],[1482371871575,7.2735],[1482372159575,7.2721],[1482372447575,7.2734],[1482372735575,7.2722],[1482373023575,7.2722],[1482373311575,7.2743],[1482373599575,7.2743],[1482373887575,7.2730],[1482374175575,7.2724],[1482374463575,7.2752],[1482374751575,7.2761],[1482375039575,7.2761],[1482375327575,7.2775],[1482375615575,7.2761],[1482375903575,7.2761],[1482376191575,7.2760],[1482376479575,7.2761],[1482376767575,7.2774],[1482377055575,7.2774],[1482377343575,7.2768],[1482377631575,7.2761],[1482377919575,7.2762],[1482378207575,7.2768],[1482378495575,7.2779],[1482378783575,7.2763],[1482379071575,7.2750],[1482379359575,7.2751],[1482379647575,7.2751],[1482379935575,7.2752],[1482380223575,7.2760],[1482380511575,7.2762],[1482380799575,7.2751],[1482381087575,7.2761],[1482381375575,7.2760],[1482381663575,7.2761],[1482381951575,7.2778],[1482382239575,7.2777],[1482382527575,7.2774],[1482382815575,7.2774],[1482383103575,7.2766],[1482383391575,7.2770],[1482383679575,7.2788],[1482383967575,7.2787],[1482384255575,7.2780],[1482384543575,7.2778],[1482384831575,7.2782],[1482385119575,7.2781],[1482385407575,7.2803],[1482385695575,7.2807],[1482385983575,7.2775],[1482386271575,7.2787],[1482386559575,7.2789],[1482386847575,7.2789],[1482387135575,7.2800],[1482387423575,7.2814],[1482387711575,7.2813],[1482387999575,7.2802],[1482388287575,7.2807],[1482388575575,7.2801],[1482388863575,7.2801],[1482389151575,7.2817],[1482389439575,7.2814],[1482389727575,7.2804],[1482390015575,7.2813],[1482390303575,7.2813],[1482390591575,7.2817],[1482390879575,7.2840],[1482391167575,7.2828],[1482391455575,7.2827],[1482391743575,7.2827],[1482392031575,7.2821],[1482392319575,7.2815],[1482392607575,7.2828],[1482392895575,7.2827],[1482393183575,7.2816],[1482393471575,7.2815],[1482393759575,7.2816],[1482394047575,7.2827],[1482394335575,7.2845],[1482394623575,7.2842],[1482394911575,7.2828],[1482395199575,7.2829],[1482395487575,7.2827],[1482395775575,7.2839],[1482396063575,7.2853],[1482396351575,7.2854],[1482396639575,7.2833],[1482396927575,7.2819],[1482397215575,7.2829],[1482397503575,7.2827],[1482397791575,7.2841],[1482398079575,7.2839],[1482398367575,7.2834],[1482398655575,7.2819],[1482398943575,7.2818],[1482399231575,7.2817],[1482399519575,7.2816],[1482399807575,7.2831],[1482400095575,7.2847],[1482400383575,7.2847],[1482400671575,7.2851],[1482400959575,7.2843],[1482401247575,7.2845],[1482401535575,7.2855],[1482401823575,7.2857],[1482402111575,7.2849],[1482402399575,7.2848],[1482402687575,7.2848],[1482402975575,7.2851],[1482403263575,7.2855],[1482403551575,7.2865],[1482403839575,7.2857],[1482404127575,7.2843],[1482404415575,7.2851],[1482404703575,7.2843],[1482404991575,7.2840],[1482405279575,7.2851],[1482405567575,7.2844],[1482405855575,7.2843],[1482406143575,7.2843],[1482406431575,7.2843],[1482406719575,7.2847],[1482407007575,7.2865],[1482407295575,7.2861],[1482407583575,7.2836],[1482407871575,7.2836],[1482408159575,7.2836],[1482408447575,7.2833],[1482408735575,7.2844],[1482409023575,7.2851],[1482409311575,7.2845],[1482409599575,7.2833],[1482409887575,7.2834],[1482410175575,7.2831],[1482410463575,7.2834],[1482410751575,7.2844],[1482411039575,7.2844],[1482411327575,7.2822],[1482411615575,7.2822],[1482411903575,7.2826],[1482412191575,7.2821],[1482412479575,7.2837],[1482412767575,7.2847],[1482413055575,7.2823],[1482413343575,7.2821],[1482413631575,7.2819],[1482413919575,7.2822],[1482414207575,7.2830],[1482414495575,7.2836],[1482414783575,7.2832],[1482415071575,7.2826],[1482415359575,7.2825],[1482415647575,7.2819],[1482415935575,7.2814],[1482416223575,7.2832],[1482416511575,7.2836],[1482416799575,7.2815],[1482417087575,7.2804],[1482417375575,7.2808],[1482417663575,7.2798],[1482417951575,7.2811],[1482418239575,7.2809],[1482418527575,7.2806],[1482418815575,7.2802],[1482419103575,7.2804],[1482419391575,7.2805],[1482419679575,7.2808],[1482419967575,7.2811],[1482420255575,7.2796],[1482420543575,7.2793],[1482420831575,7.2790],[1482421119575,7.2790],[1482421407575,7.2782],[1482421695575,7.2784],[1482421983575,7.2795],[1482422271575,7.2795],[1482422559575,7.2784],[1482422847575,7.2794],[1482423135575,7.2775],[1482423423575,7.2776],[1482423711575,7.2782],[1482423999575,7.2782],[1482424287575,7.2769],[1482424575575,7.2721],[1482424863575,7.2726],[1482425151575,7.2721],[1482425439575,7.2731],[1482425727575,7.2732],[1482426015575,7.2719],[1482426303575,7.2720],[1482426591575,7.2727],[1482426879575,7.2721],[1482427167575,7.2692],[1482427455575,7.2681],[1482427743575,7.2682],[1482428031575,7.2692],[1482428319575,7.2693],[1482428607575,7.2695],[1482428895575,7.2797],[1482429183575,7.2718],[1482429471575,7.2705],[1482429759575,7.2795],[1482430047575,7.2795],[1482430335575,7.2693],[1482430623575,7.2720],[1482430911575,7.2710],[1482431199575,7.2705],[1482431487575,7.2696],[1482431775575,7.2692],[1482432063575,7.2693],[1482432351575,7.2714],[1482432639575,7.2705],[1482432927575,7.2693],[1482433215575,7.2688],[1482433503575,7.2675],[1482433791575,7.2680],[1482434079575,7.2761],[1482434367575,7.2756],[1482434655575,7.2735],[1482434943575,7.2743],[1482435231575,7.2768],[1482435519575,6.0015],[1482435807575,3.9425],[1482436095575,0.73273],[1482436383575,0.017210],[1482436671575,0.017431],[1482436959575,0.017790],[1482437247575,0.017806],[1482437535575,0.017988],[1482437823575,0.018183],[1482438111575,0.018731],[1482438399575,0.018860],[1482438687575,0.019060],[1482438975575,0.019202],[1482439263575,0.019471],[1482439551575,0.019711],[1482439839575,0.019990],[1482440127575,0.020407],[1482440415575,0.020621],[1482440703575,0.021007],[1482440991575,0.020912],[1482441279575,0.021283],[1482441567575,0.022072],[1482441855575,0.021767],[1482442143575,0.022001],[1482442431575,0.022268],[1482442719575,0.023213],[1482443007575,0.022713],[1482443295575,0.023007],[1482443583575,0.023459],[1482443871575,0.023374],[1482444159575,0.025576],[1482444447575,0.027156],[1482444735575,0.027309],[1482445023575,0.027623],[1482445311575,0.027989],[1482445599575,0.028288],[1482445887575,0.029061],[1482446175575,0.032843],[1482446463575,0.030843],[1482446751575,0.030618],[1482447039575,0.031850],[1482447327575,0.031288],[1482447615575,0.030380],[1482447903575,0.030471],[1482448191575,0.031396],[1482448479575,0.031086],[1482448767575,0.031504],[1482449055575,0.031906],[1482449343575,0.032100],[1482449631575,0.032426],[1482449919575,0.032523],[1482450207575,0.033189],[1482450495575,0.033595],[1482450783575,0.034278],[1482451071575,0.034500],[1482451359575,0.034691],[1482451647575,0.034555],[1482451935575,0.034759],[1482452223575,0.035084],[1482452511575,0.035156],[1482452799575,0.034962],[1482453087575,0.035379],[1482453375575,0.035803],[1482453663575,0.036785],[1482453951575,0.037906],[1482454239575,0.037912],[1482454527575,0.036772],[1482454815575,0.037427],[1482455103575,0.038098],[1482455391575,0.038118],[1482455679575,0.037669]],\n  "last":[[1482369567575,7.2724],[1482369855575,7.2724],[1482370143575,7.2734],[1482370431575,7.2703],[1482370719575,7.2697],[1482371007575,7.2696],[1482371295575,7.2700],[1482371583575,7.2710],[1482371871575,7.2710],[1482372159575,7.2696],[1482372447575,7.2695],[1482372735575,7.2712],[1482373023575,7.2683],[1482373311575,7.2712],[1482373599575,7.2698],[1482373887575,7.2709],[1482374175575,7.2718],[1482374463575,7.2712],[1482374751575,7.2745],[1482375039575,7.2736],[1482375327575,7.2749],[1482375615575,7.2721],[1482375903575,7.2735],[1482376191575,7.2721],[1482376479575,7.2737],[1482376767575,7.2729],[1482377055575,7.2748],[1482377343575,7.2762],[1482377631575,7.2735],[1482377919575,7.2735],[1482378207575,7.2758],[1482378495575,7.2746],[1482378783575,7.2756],[1482379071575,7.2735],[1482379359575,7.2727],[1482379647575,7.2748],[1482379935575,7.2747],[1482380223575,7.2737],[1482380511575,7.2736],[1482380799575,7.2734],[1482381087575,7.2736],[1482381375575,7.2735],[1482381663575,7.2761],[1482381951575,7.2750],[1482382239575,7.2774],[1482382527575,7.2759],[1482382815575,7.2757],[1482383103575,7.2761],[1482383391575,7.2757],[1482383679575,7.2760],[1482383967575,7.2773],[1482384255575,7.2764],[1482384543575,7.2761],[1482384831575,7.2773],[1482385119575,7.2761],[1482385407575,7.2773],[1482385695575,7.2799],[1482385983575,7.2774],[1482386271575,7.2774],[1482386559575,7.2774],[1482386847575,7.2781],[1482387135575,7.2763],[1482387423575,7.2783],[1482387711575,7.2788],[1482387999575,7.2794],[1482388287575,7.2794],[1482388575575,7.2787],[1482388863575,7.2776],[1482389151575,7.2774],[1482389439575,7.2810],[1482389727575,7.2788],[1482390015575,7.2800],[1482390303575,7.2787],[1482390591575,7.2801],[1482390879575,7.2793],[1482391167575,7.2817],[1482391455575,7.2808],[1482391743575,7.2814],[1482392031575,7.2812],[1482392319575,7.2810],[1482392607575,7.2788],[1482392895575,7.2802],[1482393183575,7.2799],[1482393471575,7.2803],[1482393759575,7.2788],[1482394047575,7.2802],[1482394335575,7.2801],[1482394623575,7.2820],[1482394911575,7.2801],[1482395199575,7.2814],[1482395487575,7.2802],[1482395775575,7.2802],[1482396063575,7.2806],[1482396351575,7.2845],[1482396639575,7.2814],[1482396927575,7.2794],[1482397215575,7.2795],[1482397503575,7.2816],[1482397791575,7.2802],[1482398079575,7.2822],[1482398367575,7.2818],[1482398655575,7.2816],[1482398943575,7.2814],[1482399231575,7.2802],[1482399519575,7.2803],[1482399807575,7.2807],[1482400095575,7.2815],[1482400383575,7.2816],[1482400671575,7.2834],[1482400959575,7.2817],[1482401247575,7.2816],[1482401535575,7.2817],[1482401823575,7.2842],[1482402111575,7.2821],[1482402399575,7.2829],[1482402687575,7.2839],[1482402975575,7.2821],[1482403263575,7.2817],[1482403551575,7.2828],[1482403839575,7.2842],[1482404127575,7.2842],[1482404415575,7.2822],[1482404703575,7.2841],[1482404991575,7.2823],[1482405279575,7.2823],[1482405567575,7.2830],[1482405855575,7.2811],[1482406143575,7.2831],[1482406431575,7.2830],[1482406719575,7.2819],[1482407007575,7.2840],[1482407295575,7.2845],[1482407583575,7.2817],[1482407871575,7.2812],[1482408159575,7.2808],[1482408447575,7.2817],[1482408735575,7.2811],[1482409023575,7.2830],[1482409311575,7.2829],[1482409599575,7.2806],[1482409887575,7.2826],[1482410175575,7.2807],[1482410463575,7.2805],[1482410751575,7.2803],[1482411039575,7.2831],[1482411327575,7.2806],[1482411615575,7.2806],[1482411903575,7.2792],[1482412191575,7.2810],[1482412479575,7.2810],[1482412767575,7.2818],[1482413055575,7.2792],[1482413343575,7.2818],[1482413631575,7.2807],[1482413919575,7.2803],[1482414207575,7.2803],[1482414495575,7.2798],[1482414783575,7.2820],[1482415071575,7.2794],[1482415359575,7.2804],[1482415647575,7.2815],[1482415935575,7.2794],[1482416223575,7.2794],[1482416511575,7.2815],[1482416799575,7.2784],[1482417087575,7.2781],[1482417375575,7.2795],[1482417663575,7.2781],[1482417951575,7.2792],[1482418239575,7.2794],[1482418527575,7.2768],[1482418815575,7.2777],[1482419103575,7.2781],[1482419391575,7.2765],[1482419679575,7.2794],[1482419967575,7.2794],[1482420255575,7.2794],[1482420543575,7.2756],[1482420831575,7.2782],[1482421119575,7.2768],[1482421407575,7.2756],[1482421695575,7.2764],[1482421983575,7.2753],[1482422271575,7.2788],[1482422559575,7.2756],[1482422847575,7.2768],[1482423135575,7.2761],[1482423423575,7.2738],[1482423711575,7.2747],[1482423999575,7.2762],[1482424287575,7.2751],[1482424575575,7.2698],[1482424863575,7.2705],[1482425151575,7.2705],[1482425439575,7.2698],[1482425727575,7.2717],[1482426015575,7.2692],[1482426303575,7.2705],[1482426591575,7.2704],[1482426879575,7.2704],[1482427167575,7.2654],[1482427455575,7.2666],[1482427743575,7.2667],[1482428031575,7.2679],[1482428319575,7.2673],[1482428607575,7.2677],[1482428895575,7.2687],[1482429183575,7.2705],[1482429471575,7.2684],[1482429759575,7.2679],[1482430047575,7.2795],[1482430335575,7.2680],[1482430623575,7.2692],[1482430911575,7.2692],[1482431199575,7.2679],[1482431487575,7.2691],[1482431775575,7.2668],[1482432063575,7.2687],[1482432351575,7.2666],[1482432639575,7.2695],[1482432927575,7.2671],[1482433215575,7.2679],[1482433503575,7.2653],[1482433791575,7.2646],[1482434079575,7.2678],[1482434367575,7.2744],[1482434655575,7.2719],[1482434943575,7.2726],[1482435231575,7.2717],[1482435519575,6.0015],[1482435807575,3.9425],[1482436095575,0.73273],[1482436383575,0.016828],[1482436671575,0.017114],[1482436959575,0.017312],[1482437247575,0.017588],[1482437535575,0.017589],[1482437823575,0.017736],[1482438111575,0.018018],[1482438399575,0.018406],[1482438687575,0.018652],[1482438975575,0.018939],[1482439263575,0.019027],[1482439551575,0.019309],[1482439839575,0.019245],[1482440127575,0.018980],[1482440415575,0.019969],[1482440703575,0.020046],[1482440991575,0.020288],[1482441279575,0.020274],[1482441567575,0.020305],[1482441855575,0.020929],[1482442143575,0.021376],[1482442431575,0.021272],[1482442719575,0.021432],[1482443007575,0.021807],[1482443295575,0.021178],[1482443583575,0.022365],[1482443871575,0.022803],[1482444159575,0.023146],[1482444447575,0.025539],[1482444735575,0.026247],[1482445023575,0.026845],[1482445311575,0.027507],[1482445599575,0.027731],[1482445887575,0.028547],[1482446175575,0.028737],[1482446463575,0.029595],[1482446751575,0.029717],[1482447039575,0.030066],[1482447327575,0.029933],[1482447615575,0.029839],[1482447903575,0.029896],[1482448191575,0.030283],[1482448479575,0.030350],[1482448767575,0.030988],[1482449055575,0.031388],[1482449343575,0.031443],[1482449631575,0.032111],[1482449919575,0.032100],[1482450207575,0.032202],[1482450495575,0.032501],[1482450783575,0.033093],[1482451071575,0.033502],[1482451359575,0.034259],[1482451647575,0.033850],[1482451935575,0.034233],[1482452223575,0.034458],[1482452511575,0.034465],[1482452799575,0.034721],[1482453087575,0.034736],[1482453375575,0.035010],[1482453663575,0.035868],[1482453951575,0.036075],[1482454239575,0.036946],[1482454527575,0.036214],[1482454815575,0.036430],[1482455103575,0.036627],[1482455391575,0.036783],[1482455679575,0.036953]],\n  "min":[[1482369567575,7.2695],[1482369855575,7.2695],[1482370143575,7.2694],[1482370431575,7.2691],[1482370719575,7.2690],[1482371007575,7.2682],[1482371295575,7.2680],[1482371583575,7.2682],[1482371871575,7.2671],[1482372159575,7.2670],[1482372447575,7.2667],[1482372735575,7.2681],[1482373023575,7.2680],[1482373311575,7.2683],[1482373599575,7.2685],[1482373887575,7.2683],[1482374175575,7.2681],[1482374463575,7.2683],[1482374751575,7.2707],[1482375039575,7.2708],[1482375327575,7.2721],[1482375615575,7.2709],[1482375903575,7.2706],[1482376191575,7.2709],[1482376479575,7.2710],[1482376767575,7.2722],[1482377055575,7.2722],[1482377343575,7.2716],[1482377631575,7.2714],[1482377919575,7.2712],[1482378207575,7.2722],[1482378495575,7.2714],[1482378783575,7.2730],[1482379071575,7.2722],[1482379359575,7.2722],[1482379647575,7.2717],[1482379935575,7.2722],[1482380223575,7.2722],[1482380511575,7.2721],[1482380799575,7.2722],[1482381087575,7.2726],[1482381375575,7.2731],[1482381663575,7.2734],[1482381951575,7.2737],[1482382239575,7.2746],[1482382527575,7.2747],[1482382815575,7.2745],[1482383103575,7.2735],[1482383391575,7.2735],[1482383679575,7.2744],[1482383967575,7.2741],[1482384255575,7.2742],[1482384543575,7.2749],[1482384831575,7.2751],[1482385119575,7.2752],[1482385407575,7.2757],[1482385695575,7.2760],[1482385983575,7.2746],[1482386271575,7.2760],[1482386559575,7.2761],[1482386847575,7.2761],[1482387135575,7.2761],[1482387423575,7.2783],[1482387711575,7.2774],[1482387999575,7.2774],[1482388287575,7.2774],[1482388575575,7.2772],[1482388863575,7.2770],[1482389151575,7.2772],[1482389439575,7.2770],[1482389727575,7.2774],[1482390015575,7.2775],[1482390303575,7.2775],[1482390591575,7.2786],[1482390879575,7.2788],[1482391167575,7.2790],[1482391455575,7.2788],[1482391743575,7.2793],[1482392031575,7.2793],[1482392319575,7.2787],[1482392607575,7.2788],[1482392895575,7.2788],[1482393183575,7.2785],[1482393471575,7.2788],[1482393759575,7.2788],[1482394047575,7.2788],[1482394335575,7.2788],[1482394623575,7.2799],[1482394911575,7.2800],[1482395199575,7.2800],[1482395487575,7.2793],[1482395775575,7.2788],[1482396063575,7.2787],[1482396351575,7.2805],[1482396639575,7.2769],[1482396927575,7.2773],[1482397215575,7.2785],[1482397503575,7.2789],[1482397791575,7.2793],[1482398079575,7.2802],[1482398367575,7.2781],[1482398655575,7.2788],[1482398943575,7.2785],[1482399231575,7.2778],[1482399519575,7.2783],[1482399807575,7.2784],[1482400095575,7.2799],[1482400383575,7.2812],[1482400671575,7.2811],[1482400959575,7.2809],[1482401247575,7.2808],[1482401535575,7.2807],[1482401823575,7.2795],[1482402111575,7.2805],[1482402399575,7.2808],[1482402687575,7.2811],[1482402975575,7.2813],[1482403263575,7.2815],[1482403551575,7.2818],[1482403839575,7.2798],[1482404127575,7.2813],[1482404415575,7.2808],[1482404703575,7.2804],[1482404991575,7.2803],[1482405279575,7.2804],[1482405567575,7.2803],[1482405855575,7.2802],[1482406143575,7.2803],[1482406431575,7.2801],[1482406719575,7.2807],[1482407007575,7.2815],[1482407295575,7.2815],[1482407583575,7.2804],[1482407871575,7.2804],[1482408159575,7.2803],[1482408447575,7.2802],[1482408735575,7.2804],[1482409023575,7.2814],[1482409311575,7.2801],[1482409599575,7.2793],[1482409887575,7.2791],[1482410175575,7.2797],[1482410463575,7.2793],[1482410751575,7.2792],[1482411039575,7.2791],[1482411327575,7.2792],[1482411615575,7.2786],[1482411903575,7.2782],[1482412191575,7.2792],[1482412479575,7.2781],[1482412767575,7.2784],[1482413055575,7.2780],[1482413343575,7.2781],[1482413631575,7.2777],[1482413919575,7.2779],[1482414207575,7.2779],[1482414495575,7.2790],[1482414783575,7.2784],[1482415071575,7.2774],[1482415359575,7.2768],[1482415647575,7.2781],[1482415935575,7.2769],[1482416223575,7.2768],[1482416511575,7.2779],[1482416799575,7.2769],[1482417087575,7.2765],[1482417375575,7.2766],[1482417663575,7.2766],[1482417951575,7.2756],[1482418239575,7.2768],[1482418527575,7.2765],[1482418815575,7.2756],[1482419103575,7.2759],[1482419391575,7.2764],[1482419679575,7.2756],[1482419967575,7.2768],[1482420255575,7.2755],[1482420543575,7.2751],[1482420831575,7.2755],[1482421119575,7.2751],[1482421407575,7.2742],[1482421695575,7.2744],[1482421983575,7.2750],[1482422271575,7.2747],[1482422559575,7.2751],[1482422847575,7.2743],[1482423135575,7.2735],[1482423423575,7.2738],[1482423711575,7.2737],[1482423999575,7.2735],[1482424287575,7.2678],[1482424575575,7.2681],[1482424863575,7.2686],[1482425151575,7.2681],[1482425439575,7.2686],[1482425727575,7.2691],[1482426015575,7.2691],[1482426303575,7.2690],[1482426591575,7.2689],[1482426879575,7.2650],[1482427167575,7.2641],[1482427455575,7.2641],[1482427743575,7.2647],[1482428031575,7.2651],[1482428319575,7.2653],[1482428607575,7.2660],[1482428895575,7.2665],[1482429183575,7.2658],[1482429471575,7.2666],[1482429759575,7.2663],[1482430047575,7.2665],[1482430335575,7.2665],[1482430623575,7.2665],[1482430911575,7.2665],[1482431199575,7.2653],[1482431487575,7.2666],[1482431775575,7.2664],[1482432063575,7.2655],[1482432351575,7.2666],[1482432639575,7.2663],[1482432927575,7.2665],[1482433215575,7.2641],[1482433503575,7.2639],[1482433791575,7.2643],[1482434079575,7.2653],[1482434367575,7.2716],[1482434655575,7.2711],[1482434943575,7.2704],[1482435231575,6.0072],[1482435519575,3.9512],[1482435807575,0.74650],[1482436095575,0.016045],[1482436383575,0.016141],[1482436671575,0.016380],[1482436959575,0.016360],[1482437247575,0.016792],[1482437535575,0.016895],[1482437823575,0.016978],[1482438111575,0.017107],[1482438399575,0.017664],[1482438687575,0.017424],[1482438975575,0.017852],[1482439263575,0.017464],[1482439551575,0.018341],[1482439839575,0.018624],[1482440127575,0.018750],[1482440415575,0.019104],[1482440703575,0.019330],[1482440991575,0.019686],[1482441279575,0.019996],[1482441567575,0.019902],[1482441855575,0.020579],[1482442143575,0.020441],[1482442431575,0.021018],[1482442719575,0.021177],[1482443007575,0.021444],[1482443295575,0.021178],[1482443583575,0.022238],[1482443871575,0.022271],[1482444159575,0.023146],[1482444447575,0.025184],[1482444735575,0.025907],[1482445023575,0.026492],[1482445311575,0.027039],[1482445599575,0.027466],[1482445887575,0.028008],[1482446175575,0.028301],[1482446463575,0.024311],[1482446751575,0.029522],[1482447039575,0.024054],[1482447327575,0.029175],[1482447615575,0.029019],[1482447903575,0.029214],[1482448191575,0.029801],[1482448479575,0.030022],[1482448767575,0.030615],[1482449055575,0.031107],[1482449343575,0.031309],[1482449631575,0.031589],[1482449919575,0.031800],[1482450207575,0.031794],[1482450495575,0.032310],[1482450783575,0.032534],[1482451071575,0.032784],[1482451359575,0.033331],[1482451647575,0.033751],[1482451935575,0.033841],[1482452223575,0.033703],[1482452511575,0.034236],[1482452799575,0.034418],[1482453087575,0.034519],[1482453375575,0.034926],[1482453663575,0.035401],[1482453951575,0.035149],[1482454239575,0.035597],[1482454527575,0.036134],[1482454815575,0.036133],[1482455103575,0.035953],[1482455391575,0.036115],[1482455679575,0.036953]]\n}'
_trenddata_24hr_temperature_json = '{\n  "max":[[1482369567885,20.179],[1482369855885,20.179],[1482370143885,20.182],[1482370431885,20.184],[1482370719885,20.184],[1482371007885,20.186],[1482371295885,20.188],[1482371583885,20.186],[1482371871885,20.188],[1482372159885,20.189],[1482372447885,20.192],[1482372735885,20.194],[1482373023885,20.195],[1482373311885,20.197],[1482373599885,20.199],[1482373887885,20.201],[1482374175885,20.200],[1482374463885,20.202],[1482374751885,20.201],[1482375039885,20.202],[1482375327885,20.204],[1482375615885,20.204],[1482375903885,20.204],[1482376191885,20.204],[1482376479885,20.200],[1482376767885,20.195],[1482377055885,20.194],[1482377343885,20.192],[1482377631885,20.189],[1482377919885,20.189],[1482378207885,20.186],[1482378495885,20.184],[1482378783885,20.181],[1482379071885,20.180],[1482379359885,20.180],[1482379647885,20.180],[1482379935885,20.179],[1482380223885,20.178],[1482380511885,20.177],[1482380799885,20.176],[1482381087885,20.174],[1482381375885,20.172],[1482381663885,20.173],[1482381951885,20.172],[1482382239885,20.169],[1482382527885,20.170],[1482382815885,20.170],[1482383103885,20.169],[1482383391885,20.167],[1482383679885,20.164],[1482383967885,20.162],[1482384255885,20.158],[1482384543885,20.154],[1482384831885,20.156],[1482385119885,20.152],[1482385407885,20.146],[1482385695885,20.143],[1482385983885,20.139],[1482386271885,20.136],[1482386559885,20.140],[1482386847885,20.135],[1482387135885,20.131],[1482387423885,20.124],[1482387711885,20.120],[1482387999885,20.113],[1482388287885,20.107],[1482388575885,20.101],[1482388863885,20.094],[1482389151885,20.086],[1482389439885,20.078],[1482389727885,20.068],[1482390015885,20.058],[1482390303885,20.049],[1482390591885,20.039],[1482390879885,20.032],[1482391167885,20.027],[1482391455885,20.023],[1482391743885,20.023],[1482392031885,20.018],[1482392319885,20.013],[1482392607885,20.009],[1482392895885,20.002],[1482393183885,19.996],[1482393471885,19.993],[1482393759885,19.988],[1482394047885,19.981],[1482394335885,19.977],[1482394623885,19.974],[1482394911885,19.967],[1482395199885,19.959],[1482395487885,19.951],[1482395775885,19.943],[1482396063885,19.938],[1482396351885,19.927],[1482396639885,19.920],[1482396927885,19.915],[1482397215885,19.909],[1482397503885,19.899],[1482397791885,19.893],[1482398079885,19.887],[1482398367885,19.879],[1482398655885,19.874],[1482398943885,19.868],[1482399231885,19.860],[1482399519885,19.853],[1482399807885,19.848],[1482400095885,19.838],[1482400383885,19.830],[1482400671885,19.824],[1482400959885,19.818],[1482401247885,19.810],[1482401535885,19.802],[1482401823885,19.792],[1482402111885,19.782],[1482402399885,19.773],[1482402687885,19.766],[1482402975885,19.760],[1482403263885,19.753],[1482403551885,19.747],[1482403839885,19.741],[1482404127885,19.733],[1482404415885,19.726],[1482404703885,19.717],[1482404991885,19.709],[1482405279885,19.700],[1482405567885,19.695],[1482405855885,19.685],[1482406143885,19.678],[1482406431885,19.671],[1482406719885,19.663],[1482407007885,19.654],[1482407295885,19.646],[1482407583885,19.639],[1482407871885,19.631],[1482408159885,19.626],[1482408447885,19.617],[1482408735885,19.609],[1482409023885,19.598],[1482409311885,19.591],[1482409599885,19.583],[1482409887885,19.573],[1482410175885,19.564],[1482410463885,19.557],[1482410751885,19.550],[1482411039885,19.545],[1482411327885,19.537],[1482411615885,19.529],[1482411903885,19.523],[1482412191885,19.514],[1482412479885,19.504],[1482412767885,19.497],[1482413055885,19.490],[1482413343885,19.482],[1482413631885,19.473],[1482413919885,19.466],[1482414207885,19.458],[1482414495885,19.452],[1482414783885,19.445],[1482415071885,19.438],[1482415359885,19.433],[1482415647885,19.427],[1482415935885,19.422],[1482416223885,19.413],[1482416511885,19.408],[1482416799885,19.400],[1482417087885,19.393],[1482417375885,19.387],[1482417663885,19.382],[1482417951885,19.375],[1482418239885,19.369],[1482418527885,19.365],[1482418815885,19.358],[1482419103885,19.352],[1482419391885,19.347],[1482419679885,19.342],[1482419967885,19.337],[1482420255885,19.331],[1482420543885,19.328],[1482420831885,19.326],[1482421119885,19.322],[1482421407885,19.321],[1482421695885,19.317],[1482421983885,19.314],[1482422271885,19.310],[1482422559885,19.308],[1482422847885,19.306],[1482423135885,19.304],[1482423423885,19.302],[1482423711885,19.302],[1482423999885,19.295],[1482424287885,19.294],[1482424575885,19.296],[1482424863885,19.296],[1482425151885,19.298],[1482425439885,19.290],[1482425727885,19.285],[1482426015885,19.282],[1482426303885,19.279],[1482426591885,19.278],[1482426879885,19.274],[1482427167885,19.272],[1482427455885,19.269],[1482427743885,19.267],[1482428031885,19.263],[1482428319885,19.262],[1482428607885,19.262],[1482428895885,19.259],[1482429183885,19.261],[1482429471885,19.262],[1482429759885,19.261],[1482430047885,19.261],[1482430335885,19.261],[1482430623885,19.264],[1482430911885,19.267],[1482431199885,19.270],[1482431487885,19.273],[1482431775885,19.273],[1482432063885,19.277],[1482432351885,19.286],[1482432639885,19.292],[1482432927885,19.308],[1482433215885,19.312],[1482433503885,19.318],[1482433791885,19.328],[1482434079885,19.335],[1482434367885,19.344],[1482434655885,19.352],[1482434943885,19.358],[1482435231885,19.370],[1482435519885,19.380],[1482435807885,19.393],[1482436095885,19.402],[1482436383885,19.413],[1482436671885,19.421],[1482436959885,19.429],[1482437247885,19.435],[1482437535885,19.445],[1482437823885,19.453],[1482438111885,19.466],[1482438399885,19.475],[1482438687885,19.491],[1482438975885,19.500],[1482439263885,19.510],[1482439551885,19.524],[1482439839885,19.535],[1482440127885,19.547],[1482440415885,19.560],[1482440703885,19.575],[1482440991885,19.586],[1482441279885,19.600],[1482441567885,19.613],[1482441855885,19.622],[1482442143885,19.635],[1482442431885,19.647],[1482442719885,19.662],[1482443007885,19.674],[1482443295885,19.687],[1482443583885,19.696],[1482443871885,19.715],[1482444159885,19.806],[1482444447885,19.842],[1482444735885,19.871],[1482445023885,19.890],[1482445311885,19.908],[1482445599885,19.923],[1482445887885,19.952],[1482446175885,19.980],[1482446463885,19.994],[1482446751885,20.001],[1482447039885,20.003],[1482447327885,20.005],[1482447615885,20.004],[1482447903885,20.007],[1482448191885,20.020],[1482448479885,20.037],[1482448767885,20.057],[1482449055885,20.069],[1482449343885,20.080],[1482449631885,20.089],[1482449919885,20.099],[1482450207885,20.118],[1482450495885,20.133],[1482450783885,20.147],[1482451071885,20.158],[1482451359885,20.169],[1482451647885,20.181],[1482451935885,20.187],[1482452223885,20.193],[1482452511885,20.199],[1482452799885,20.204],[1482453087885,20.216],[1482453375885,20.243],[1482453663885,20.258],[1482453951885,20.275],[1482454239885,20.269],[1482454527885,20.277],[1482454815885,20.282],[1482455103885,20.294],[1482455391885,20.308],[1482455679885,20.311]],\n  "last":[[1482369567885,20.172],[1482369855885,20.174],[1482370143885,20.175],[1482370431885,20.178],[1482370719885,20.179],[1482371007885,20.183],[1482371295885,20.183],[1482371583885,20.183],[1482371871885,20.183],[1482372159885,20.185],[1482372447885,20.189],[1482372735885,20.191],[1482373023885,20.192],[1482373311885,20.192],[1482373599885,20.195],[1482373887885,20.197],[1482374175885,20.196],[1482374463885,20.195],[1482374751885,20.198],[1482375039885,20.198],[1482375327885,20.201],[1482375615885,20.199],[1482375903885,20.203],[1482376191885,20.204],[1482376479885,20.193],[1482376767885,20.192],[1482377055885,20.192],[1482377343885,20.188],[1482377631885,20.187],[1482377919885,20.183],[1482378207885,20.183],[1482378495885,20.182],[1482378783885,20.178],[1482379071885,20.175],[1482379359885,20.177],[1482379647885,20.175],[1482379935885,20.175],[1482380223885,20.174],[1482380511885,20.172],[1482380799885,20.171],[1482381087885,20.172],[1482381375885,20.170],[1482381663885,20.167],[1482381951885,20.168],[1482382239885,20.167],[1482382527885,20.165],[1482382815885,20.163],[1482383103885,20.165],[1482383391885,20.165],[1482383679885,20.163],[1482383967885,20.155],[1482384255885,20.156],[1482384543885,20.152],[1482384831885,20.151],[1482385119885,20.149],[1482385407885,20.145],[1482385695885,20.142],[1482385983885,20.138],[1482386271885,20.134],[1482386559885,20.136],[1482386847885,20.131],[1482387135885,20.130],[1482387423885,20.122],[1482387711885,20.115],[1482387999885,20.113],[1482388287885,20.106],[1482388575885,20.100],[1482388863885,20.092],[1482389151885,20.085],[1482389439885,20.077],[1482389727885,20.063],[1482390015885,20.057],[1482390303885,20.049],[1482390591885,20.034],[1482390879885,20.031],[1482391167885,20.024],[1482391455885,20.023],[1482391743885,20.020],[1482392031885,20.018],[1482392319885,20.011],[1482392607885,20.006],[1482392895885,20.001],[1482393183885,19.994],[1482393471885,19.989],[1482393759885,19.985],[1482394047885,19.981],[1482394335885,19.973],[1482394623885,19.971],[1482394911885,19.964],[1482395199885,19.957],[1482395487885,19.951],[1482395775885,19.941],[1482396063885,19.935],[1482396351885,19.924],[1482396639885,19.920],[1482396927885,19.912],[1482397215885,19.905],[1482397503885,19.899],[1482397791885,19.890],[1482398079885,19.885],[1482398367885,19.877],[1482398655885,19.869],[1482398943885,19.865],[1482399231885,19.860],[1482399519885,19.851],[1482399807885,19.845],[1482400095885,19.837],[1482400383885,19.829],[1482400671885,19.821],[1482400959885,19.818],[1482401247885,19.809],[1482401535885,19.798],[1482401823885,19.791],[1482402111885,19.778],[1482402399885,19.770],[1482402687885,19.765],[1482402975885,19.759],[1482403263885,19.751],[1482403551885,19.745],[1482403839885,19.739],[1482404127885,19.728],[1482404415885,19.726],[1482404703885,19.716],[1482404991885,19.708],[1482405279885,19.700],[1482405567885,19.691],[1482405855885,19.683],[1482406143885,19.677],[1482406431885,19.669],[1482406719885,19.658],[1482407007885,19.654],[1482407295885,19.644],[1482407583885,19.637],[1482407871885,19.630],[1482408159885,19.623],[1482408447885,19.617],[1482408735885,19.609],[1482409023885,19.596],[1482409311885,19.591],[1482409599885,19.580],[1482409887885,19.573],[1482410175885,19.563],[1482410463885,19.553],[1482410751885,19.550],[1482411039885,19.545],[1482411327885,19.533],[1482411615885,19.526],[1482411903885,19.523],[1482412191885,19.513],[1482412479885,19.503],[1482412767885,19.495],[1482413055885,19.489],[1482413343885,19.480],[1482413631885,19.472],[1482413919885,19.463],[1482414207885,19.457],[1482414495885,19.452],[1482414783885,19.444],[1482415071885,19.435],[1482415359885,19.430],[1482415647885,19.423],[1482415935885,19.416],[1482416223885,19.413],[1482416511885,19.408],[1482416799885,19.396],[1482417087885,19.392],[1482417375885,19.386],[1482417663885,19.377],[1482417951885,19.373],[1482418239885,19.365],[1482418527885,19.362],[1482418815885,19.358],[1482419103885,19.349],[1482419391885,19.346],[1482419679885,19.339],[1482419967885,19.333],[1482420255885,19.329],[1482420543885,19.328],[1482420831885,19.324],[1482421119885,19.322],[1482421407885,19.314],[1482421695885,19.313],[1482421983885,19.308],[1482422271885,19.308],[1482422559885,19.304],[1482422847885,19.305],[1482423135885,19.303],[1482423423885,19.298],[1482423711885,19.296],[1482423999885,19.293],[1482424287885,19.291],[1482424575885,19.292],[1482424863885,19.292],[1482425151885,19.293],[1482425439885,19.289],[1482425727885,19.284],[1482426015885,19.282],[1482426303885,19.277],[1482426591885,19.276],[1482426879885,19.272],[1482427167885,19.272],[1482427455885,19.269],[1482427743885,19.266],[1482428031885,19.261],[1482428319885,19.260],[1482428607885,19.257],[1482428895885,19.258],[1482429183885,19.258],[1482429471885,19.257],[1482429759885,19.260],[1482430047885,19.259],[1482430335885,19.258],[1482430623885,19.258],[1482430911885,19.263],[1482431199885,19.265],[1482431487885,19.266],[1482431775885,19.271],[1482432063885,19.273],[1482432351885,19.275],[1482432639885,19.283],[1482432927885,19.292],[1482433215885,19.305],[1482433503885,19.308],[1482433791885,19.315],[1482434079885,19.327],[1482434367885,19.331],[1482434655885,19.340],[1482434943885,19.350],[1482435231885,19.358],[1482435519885,19.367],[1482435807885,19.380],[1482436095885,19.391],[1482436383885,19.403],[1482436671885,19.412],[1482436959885,19.420],[1482437247885,19.429],[1482437535885,19.431],[1482437823885,19.441],[1482438111885,19.452],[1482438399885,19.465],[1482438687885,19.474],[1482438975885,19.488],[1482439263885,19.499],[1482439551885,19.511],[1482439839885,19.521],[1482440127885,19.532],[1482440415885,19.546],[1482440703885,19.558],[1482440991885,19.574],[1482441279885,19.587],[1482441567885,19.598],[1482441855885,19.611],[1482442143885,19.617],[1482442431885,19.632],[1482442719885,19.643],[1482443007885,19.660],[1482443295885,19.674],[1482443583885,19.684],[1482443871885,19.695],[1482444159885,19.715],[1482444447885,19.807],[1482444735885,19.838],[1482445023885,19.871],[1482445311885,19.889],[1482445599885,19.904],[1482445887885,19.923],[1482446175885,19.951],[1482446463885,19.980],[1482446751885,19.994],[1482447039885,20.000],[1482447327885,20.000],[1482447615885,20.001],[1482447903885,20.000],[1482448191885,20.006],[1482448479885,20.018],[1482448767885,20.035],[1482449055885,20.058],[1482449343885,20.067],[1482449631885,20.081],[1482449919885,20.088],[1482450207885,20.100],[1482450495885,20.118],[1482450783885,20.132],[1482451071885,20.146],[1482451359885,20.158],[1482451647885,20.167],[1482451935885,20.177],[1482452223885,20.185],[1482452511885,20.190],[1482452799885,20.199],[1482453087885,20.203],[1482453375885,20.214],[1482453663885,20.244],[1482453951885,20.257],[1482454239885,20.263],[1482454527885,20.267],[1482454815885,20.275],[1482455103885,20.280],[1482455391885,20.293],[1482455679885,20.305]],\n  "min":[[1482369567885,20.169],[1482369855885,20.171],[1482370143885,20.174],[1482370431885,20.177],[1482370719885,20.177],[1482371007885,20.180],[1482371295885,20.180],[1482371583885,20.178],[1482371871885,20.180],[1482372159885,20.183],[1482372447885,20.185],[1482372735885,20.185],[1482373023885,20.187],[1482373311885,20.189],[1482373599885,20.193],[1482373887885,20.191],[1482374175885,20.193],[1482374463885,20.193],[1482374751885,20.193],[1482375039885,20.194],[1482375327885,20.196],[1482375615885,20.196],[1482375903885,20.197],[1482376191885,20.193],[1482376479885,20.189],[1482376767885,20.187],[1482377055885,20.185],[1482377343885,20.183],[1482377631885,20.182],[1482377919885,20.180],[1482378207885,20.178],[1482378495885,20.177],[1482378783885,20.174],[1482379071885,20.174],[1482379359885,20.173],[1482379647885,20.173],[1482379935885,20.172],[1482380223885,20.170],[1482380511885,20.169],[1482380799885,20.168],[1482381087885,20.167],[1482381375885,20.166],[1482381663885,20.166],[1482381951885,20.165],[1482382239885,20.162],[1482382527885,20.163],[1482382815885,20.161],[1482383103885,20.162],[1482383391885,20.159],[1482383679885,20.155],[1482383967885,20.153],[1482384255885,20.151],[1482384543885,20.148],[1482384831885,20.147],[1482385119885,20.144],[1482385407885,20.138],[1482385695885,20.135],[1482385983885,20.130],[1482386271885,20.129],[1482386559885,20.130],[1482386847885,20.127],[1482387135885,20.119],[1482387423885,20.114],[1482387711885,20.108],[1482387999885,20.103],[1482388287885,20.097],[1482388575885,20.089],[1482388863885,20.083],[1482389151885,20.076],[1482389439885,20.062],[1482389727885,20.053],[1482390015885,20.045],[1482390303885,20.035],[1482390591885,20.027],[1482390879885,20.023],[1482391167885,20.018],[1482391455885,20.016],[1482391743885,20.015],[1482392031885,20.009],[1482392319885,20.003],[1482392607885,19.999],[1482392895885,19.992],[1482393183885,19.987],[1482393471885,19.985],[1482393759885,19.976],[1482394047885,19.972],[1482394335885,19.967],[1482394623885,19.964],[1482394911885,19.955],[1482395199885,19.947],[1482395487885,19.939],[1482395775885,19.934],[1482396063885,19.923],[1482396351885,19.916],[1482396639885,19.909],[1482396927885,19.903],[1482397215885,19.897],[1482397503885,19.889],[1482397791885,19.882],[1482398079885,19.877],[1482398367885,19.869],[1482398655885,19.863],[1482398943885,19.856],[1482399231885,19.849],[1482399519885,19.842],[1482399807885,19.833],[1482400095885,19.826],[1482400383885,19.820],[1482400671885,19.815],[1482400959885,19.807],[1482401247885,19.798],[1482401535885,19.787],[1482401823885,19.779],[1482402111885,19.768],[1482402399885,19.762],[1482402687885,19.756],[1482402975885,19.750],[1482403263885,19.742],[1482403551885,19.737],[1482403839885,19.727],[1482404127885,19.724],[1482404415885,19.715],[1482404703885,19.705],[1482404991885,19.698],[1482405279885,19.689],[1482405567885,19.682],[1482405855885,19.674],[1482406143885,19.668],[1482406431885,19.659],[1482406719885,19.652],[1482407007885,19.644],[1482407295885,19.636],[1482407583885,19.628],[1482407871885,19.621],[1482408159885,19.613],[1482408447885,19.606],[1482408735885,19.595],[1482409023885,19.587],[1482409311885,19.577],[1482409599885,19.570],[1482409887885,19.560],[1482410175885,19.554],[1482410463885,19.548],[1482410751885,19.541],[1482411039885,19.532],[1482411327885,19.525],[1482411615885,19.516],[1482411903885,19.511],[1482412191885,19.501],[1482412479885,19.493],[1482412767885,19.486],[1482413055885,19.477],[1482413343885,19.470],[1482413631885,19.462],[1482413919885,19.455],[1482414207885,19.449],[1482414495885,19.442],[1482414783885,19.434],[1482415071885,19.429],[1482415359885,19.421],[1482415647885,19.416],[1482415935885,19.410],[1482416223885,19.401],[1482416511885,19.396],[1482416799885,19.390],[1482417087885,19.383],[1482417375885,19.378],[1482417663885,19.372],[1482417951885,19.365],[1482418239885,19.360],[1482418527885,19.356],[1482418815885,19.348],[1482419103885,19.343],[1482419391885,19.337],[1482419679885,19.331],[1482419967885,19.328],[1482420255885,19.323],[1482420543885,19.320],[1482420831885,19.318],[1482421119885,19.313],[1482421407885,19.311],[1482421695885,19.308],[1482421983885,19.306],[1482422271885,19.302],[1482422559885,19.301],[1482422847885,19.299],[1482423135885,19.297],[1482423423885,19.296],[1482423711885,19.293],[1482423999885,19.288],[1482424287885,19.287],[1482424575885,19.290],[1482424863885,19.290],[1482425151885,19.287],[1482425439885,19.280],[1482425727885,19.276],[1482426015885,19.274],[1482426303885,19.272],[1482426591885,19.270],[1482426879885,19.266],[1482427167885,19.264],[1482427455885,19.263],[1482427743885,19.259],[1482428031885,19.256],[1482428319885,19.254],[1482428607885,19.253],[1482428895885,19.254],[1482429183885,19.254],[1482429471885,19.254],[1482429759885,19.254],[1482430047885,19.255],[1482430335885,19.255],[1482430623885,19.257],[1482430911885,19.258],[1482431199885,19.262],[1482431487885,19.265],[1482431775885,19.265],[1482432063885,19.268],[1482432351885,19.274],[1482432639885,19.281],[1482432927885,19.291],[1482433215885,19.302],[1482433503885,19.308],[1482433791885,19.315],[1482434079885,19.325],[1482434367885,19.331],[1482434655885,19.339],[1482434943885,19.345],[1482435231885,19.356],[1482435519885,19.366],[1482435807885,19.377],[1482436095885,19.388],[1482436383885,19.398],[1482436671885,19.405],[1482436959885,19.418],[1482437247885,19.422],[1482437535885,19.430],[1482437823885,19.441],[1482438111885,19.451],[1482438399885,19.463],[1482438687885,19.472],[1482438975885,19.487],[1482439263885,19.498],[1482439551885,19.509],[1482439839885,19.519],[1482440127885,19.532],[1482440415885,19.544],[1482440703885,19.558],[1482440991885,19.573],[1482441279885,19.584],[1482441567885,19.597],[1482441855885,19.608],[1482442143885,19.617],[1482442431885,19.630],[1482442719885,19.642],[1482443007885,19.659],[1482443295885,19.673],[1482443583885,19.684],[1482443871885,19.691],[1482444159885,19.715],[1482444447885,19.804],[1482444735885,19.838],[1482445023885,19.870],[1482445311885,19.888],[1482445599885,19.904],[1482445887885,19.923],[1482446175885,19.951],[1482446463885,19.979],[1482446751885,19.993],[1482447039885,19.997],[1482447327885,19.998],[1482447615885,19.997],[1482447903885,19.993],[1482448191885,20.004],[1482448479885,20.014],[1482448767885,20.035],[1482449055885,20.055],[1482449343885,20.065],[1482449631885,20.078],[1482449919885,20.087],[1482450207885,20.098],[1482450495885,20.114],[1482450783885,20.131],[1482451071885,20.146],[1482451359885,20.156],[1482451647885,20.167],[1482451935885,20.172],[1482452223885,20.182],[1482452511885,20.189],[1482452799885,20.194],[1482453087885,20.201],[1482453375885,20.212],[1482453663885,20.241],[1482453951885,20.257],[1482454239885,20.259],[1482454527885,20.264],[1482454815885,20.273],[1482455103885,20.279],[1482455391885,20.292],[1482455679885,20.304]]\n}'
_trenddata_72hr_agitation_json = '{\n  "max":[[1482339040188,0.0000],[1482339904188,0.0000],[1482340768188,0.0000],[1482341632188,0.0000],[1482342496188,0.0000],[1482343360188,0.0000],[1482344224188,0.0000],[1482345088188,0.0000],[1482345952188,0.0000],[1482346816188,0.0000],[1482347680188,0.0000],[1482348544188,0.0000],[1482349408188,0.0000],[1482350272188,0.0000],[1482351136188,0.0000],[1482352000188,0.0000],[1482352864188,0.0000],[1482353728188,0.0000],[1482354592188,0.0000],[1482355456188,0.0000],[1482356320188,0.0000],[1482357184188,0.0000],[1482358048188,0.0000],[1482358912188,0.0000],[1482359776188,0.0000],[1482360640188,0.0000],[1482361504188,0.0000],[1482362368188,0.0000],[1482363232188,0.0000],[1482364096188,0.0000],[1482364960188,0.0000],[1482365824188,0.0000],[1482366688188,0.0000],[1482367552188,0.0000],[1482368416188,0.0000],[1482369280188,0.0000],[1482370144188,0.0000],[1482371008188,0.0000],[1482371872188,0.0000],[1482372736188,0.0000],[1482373600188,0.0000],[1482374464188,0.0000],[1482375328188,0.0000],[1482376192188,0.0000],[1482377056188,0.0000],[1482377920188,0.0000],[1482378784188,0.0000],[1482379648188,0.0000],[1482380512188,0.0000],[1482381376188,0.0000],[1482382240188,0.0000],[1482383104188,0.0000],[1482383968188,0.0000],[1482384832188,0.0000],[1482385696188,0.0000],[1482386560188,0.0000],[1482387424188,0.0000],[1482388288188,0.0000],[1482389152188,0.0000],[1482390016188,0.0000],[1482390880188,0.0000],[1482391744188,0.0000],[1482392608188,0.0000],[1482393472188,0.0000],[1482394336188,0.0000],[1482395200188,0.0000],[1482396064188,0.0000],[1482396928188,0.0000],[1482397792188,0.0000],[1482398656188,0.0000],[1482399520188,0.0000],[1482400384188,0.0000],[1482401248188,0.0000],[1482402112188,0.0000],[1482402976188,0.0000],[1482403840188,0.0000],[1482404704188,0.0000],[1482405568188,0.0000],[1482406432188,0.0000],[1482407296188,0.0000],[1482408160188,0.0000],[1482409024188,0.0000],[1482409888188,0.0000],[1482410752188,0.0000],[1482411616188,0.0000],[1482412480188,0.0000],[1482413344188,0.0000],[1482414208188,0.0000],[1482415072188,0.0000],[1482415936188,0.0000],[1482416800188,0.0000],[1482417664188,0.0000],[1482418528188,0.0000],[1482419392188,0.0000],[1482420256188,0.0000],[1482421120188,0.0000],[1482421984188,0.0000],[1482422848188,0.0000],[1482423712188,0.0000],[1482424576188,0.0000],[1482425440188,0.0000],[1482426304188,0.0000],[1482427168188,0.0000],[1482428032188,0.0000],[1482428896188,0.0000],[1482429760188,0.0000],[1482430624188,0.0000],[1482431488188,0.0000],[1482432352188,0.0000],[1482433216188,0.0000],[1482434080188,0.0000],[1482434944188,0.0000],[1482435808188,0.0000],[1482436672188,0.0000],[1482437536188,0.0000],[1482438400188,0.0000],[1482439264188,0.0000],[1482440128188,0.0000],[1482440992188,0.0000],[1482441856188,0.0000],[1482442720188,0.0000],[1482443584188,20.032],[1482444448188,20.039],[1482445312188,20.043],[1482446176188,0.0000],[1482447040188,0.0000],[1482447904188,36.028],[1482448768188,12.062],[1482449632188,12.062],[1482450496188,12.063],[1482451360188,12.063],[1482452224188,12.063],[1482453088188,12.061],[1482453952188,22.355],[1482454816188,36.048],[1482455680188,36.035]],\n  "last":[[1482339040188,0.0000],[1482339904188,0.0000],[1482340768188,0.0000],[1482341632188,0.0000],[1482342496188,0.0000],[1482343360188,0.0000],[1482344224188,0.0000],[1482345088188,0.0000],[1482345952188,0.0000],[1482346816188,0.0000],[1482347680188,0.0000],[1482348544188,0.0000],[1482349408188,0.0000],[1482350272188,0.0000],[1482351136188,0.0000],[1482352000188,0.0000],[1482352864188,0.0000],[1482353728188,0.0000],[1482354592188,0.0000],[1482355456188,0.0000],[1482356320188,0.0000],[1482357184188,0.0000],[1482358048188,0.0000],[1482358912188,0.0000],[1482359776188,0.0000],[1482360640188,0.0000],[1482361504188,0.0000],[1482362368188,0.0000],[1482363232188,0.0000],[1482364096188,0.0000],[1482364960188,0.0000],[1482365824188,0.0000],[1482366688188,0.0000],[1482367552188,0.0000],[1482368416188,0.0000],[1482369280188,0.0000],[1482370144188,0.0000],[1482371008188,0.0000],[1482371872188,0.0000],[1482372736188,0.0000],[1482373600188,0.0000],[1482374464188,0.0000],[1482375328188,0.0000],[1482376192188,0.0000],[1482377056188,0.0000],[1482377920188,0.0000],[1482378784188,0.0000],[1482379648188,0.0000],[1482380512188,0.0000],[1482381376188,0.0000],[1482382240188,0.0000],[1482383104188,0.0000],[1482383968188,0.0000],[1482384832188,0.0000],[1482385696188,0.0000],[1482386560188,0.0000],[1482387424188,0.0000],[1482388288188,0.0000],[1482389152188,0.0000],[1482390016188,0.0000],[1482390880188,0.0000],[1482391744188,0.0000],[1482392608188,0.0000],[1482393472188,0.0000],[1482394336188,0.0000],[1482395200188,0.0000],[1482396064188,0.0000],[1482396928188,0.0000],[1482397792188,0.0000],[1482398656188,0.0000],[1482399520188,0.0000],[1482400384188,0.0000],[1482401248188,0.0000],[1482402112188,0.0000],[1482402976188,0.0000],[1482403840188,0.0000],[1482404704188,0.0000],[1482405568188,0.0000],[1482406432188,0.0000],[1482407296188,0.0000],[1482408160188,0.0000],[1482409024188,0.0000],[1482409888188,0.0000],[1482410752188,0.0000],[1482411616188,0.0000],[1482412480188,0.0000],[1482413344188,0.0000],[1482414208188,0.0000],[1482415072188,0.0000],[1482415936188,0.0000],[1482416800188,0.0000],[1482417664188,0.0000],[1482418528188,0.0000],[1482419392188,0.0000],[1482420256188,0.0000],[1482421120188,0.0000],[1482421984188,0.0000],[1482422848188,0.0000],[1482423712188,0.0000],[1482424576188,0.0000],[1482425440188,0.0000],[1482426304188,0.0000],[1482427168188,0.0000],[1482428032188,0.0000],[1482428896188,0.0000],[1482429760188,0.0000],[1482430624188,0.0000],[1482431488188,0.0000],[1482432352188,0.0000],[1482433216188,0.0000],[1482434080188,0.0000],[1482434944188,0.0000],[1482435808188,0.0000],[1482436672188,0.0000],[1482437536188,0.0000],[1482438400188,0.0000],[1482439264188,0.0000],[1482440128188,0.0000],[1482440992188,0.0000],[1482441856188,0.0000],[1482442720188,0.0000],[1482443584188,0.0000],[1482444448188,20.010],[1482445312188,20.011],[1482446176188,0.0000],[1482447040188,0.0000],[1482447904188,0.0000],[1482448768188,12.060],[1482449632188,12.059],[1482450496188,12.046],[1482451360188,12.046],[1482452224188,12.050],[1482453088188,12.049],[1482453952188,0.0000],[1482454816188,0.0000],[1482455680188,36.006]],\n  "min":[[1482339040188,0.0000],[1482339904188,0.0000],[1482340768188,0.0000],[1482341632188,0.0000],[1482342496188,0.0000],[1482343360188,0.0000],[1482344224188,0.0000],[1482345088188,0.0000],[1482345952188,0.0000],[1482346816188,0.0000],[1482347680188,0.0000],[1482348544188,0.0000],[1482349408188,0.0000],[1482350272188,0.0000],[1482351136188,0.0000],[1482352000188,0.0000],[1482352864188,0.0000],[1482353728188,0.0000],[1482354592188,0.0000],[1482355456188,0.0000],[1482356320188,0.0000],[1482357184188,0.0000],[1482358048188,0.0000],[1482358912188,0.0000],[1482359776188,0.0000],[1482360640188,0.0000],[1482361504188,0.0000],[1482362368188,0.0000],[1482363232188,0.0000],[1482364096188,0.0000],[1482364960188,0.0000],[1482365824188,0.0000],[1482366688188,0.0000],[1482367552188,0.0000],[1482368416188,0.0000],[1482369280188,0.0000],[1482370144188,0.0000],[1482371008188,0.0000],[1482371872188,0.0000],[1482372736188,0.0000],[1482373600188,0.0000],[1482374464188,0.0000],[1482375328188,0.0000],[1482376192188,0.0000],[1482377056188,0.0000],[1482377920188,0.0000],[1482378784188,0.0000],[1482379648188,0.0000],[1482380512188,0.0000],[1482381376188,0.0000],[1482382240188,0.0000],[1482383104188,0.0000],[1482383968188,0.0000],[1482384832188,0.0000],[1482385696188,0.0000],[1482386560188,0.0000],[1482387424188,0.0000],[1482388288188,0.0000],[1482389152188,0.0000],[1482390016188,0.0000],[1482390880188,0.0000],[1482391744188,0.0000],[1482392608188,0.0000],[1482393472188,0.0000],[1482394336188,0.0000],[1482395200188,0.0000],[1482396064188,0.0000],[1482396928188,0.0000],[1482397792188,0.0000],[1482398656188,0.0000],[1482399520188,0.0000],[1482400384188,0.0000],[1482401248188,0.0000],[1482402112188,0.0000],[1482402976188,0.0000],[1482403840188,0.0000],[1482404704188,0.0000],[1482405568188,0.0000],[1482406432188,0.0000],[1482407296188,0.0000],[1482408160188,0.0000],[1482409024188,0.0000],[1482409888188,0.0000],[1482410752188,0.0000],[1482411616188,0.0000],[1482412480188,0.0000],[1482413344188,0.0000],[1482414208188,0.0000],[1482415072188,0.0000],[1482415936188,0.0000],[1482416800188,0.0000],[1482417664188,0.0000],[1482418528188,0.0000],[1482419392188,0.0000],[1482420256188,0.0000],[1482421120188,0.0000],[1482421984188,0.0000],[1482422848188,0.0000],[1482423712188,0.0000],[1482424576188,0.0000],[1482425440188,0.0000],[1482426304188,0.0000],[1482427168188,0.0000],[1482428032188,0.0000],[1482428896188,0.0000],[1482429760188,0.0000],[1482430624188,0.0000],[1482431488188,0.0000],[1482432352188,0.0000],[1482433216188,0.0000],[1482434080188,0.0000],[1482434944188,0.0000],[1482435808188,0.0000],[1482436672188,0.0000],[1482437536188,0.0000],[1482438400188,0.0000],[1482439264188,0.0000],[1482440128188,0.0000],[1482440992188,0.0000],[1482441856188,0.0000],[1482442720188,0.0000],[1482443584188,0.0000],[1482444448188,19.804],[1482445312188,0.0000],[1482446176188,0.0000],[1482447040188,0.0000],[1482447904188,0.0000],[1482448768188,11.801],[1482449632188,11.803],[1482450496188,11.808],[1482451360188,11.810],[1482452224188,11.791],[1482453088188,0.0000],[1482453952188,0.0000],[1482454816188,0.0000],[1482455680188,35.956]]\n}'
_trenddata_72hr_do_json = '{\n  "max":[[1482339040387,-0.22758],[1482339904387,-0.23430],[1482340768387,-0.22293],[1482341632387,-0.23623],[1482342496387,-0.24430],[1482343360387,-0.24212],[1482344224387,-0.23703],[1482345088387,-0.23952],[1482345952387,-0.24122],[1482346816387,-0.22973],[1482347680387,-0.22500],[1482348544387,-0.23099],[1482349408387,-0.23166],[1482350272387,-0.22674],[1482351136387,-0.24231],[1482352000387,-0.23124],[1482352864387,-0.23753],[1482353728387,-0.23161],[1482354592387,-0.23056],[1482355456387,-0.22960],[1482356320387,-0.22957],[1482357184387,-0.24249],[1482358048387,-0.24304],[1482358912387,-0.24561],[1482359776387,-0.23944],[1482360640387,-0.23307],[1482361504387,-0.22778],[1482362368387,-0.23887],[1482363232387,-0.24253],[1482364096387,-0.24231],[1482364960387,-0.24029],[1482365824387,-0.23883],[1482366688387,-0.24468],[1482367552387,-0.24450],[1482368416387,-0.24158],[1482369280387,-0.23075],[1482370144387,-0.23443],[1482371008387,-0.23583],[1482371872387,-0.23593],[1482372736387,-0.24442],[1482373600387,-0.24032],[1482374464387,-0.24227],[1482375328387,-0.23613],[1482376192387,-0.23644],[1482377056387,-0.24039],[1482377920387,-0.24386],[1482378784387,-0.24046],[1482379648387,-0.23355],[1482380512387,-0.23443],[1482381376387,-0.23310],[1482382240387,-0.24117],[1482383104387,-0.23321],[1482383968387,-0.22833],[1482384832387,-0.22298],[1482385696387,-0.24144],[1482386560387,-0.24193],[1482387424387,-0.23596],[1482388288387,-0.23211],[1482389152387,-0.23093],[1482390016387,-0.24461],[1482390880387,-0.24091],[1482391744387,-0.23323],[1482392608387,-0.24149],[1482393472387,-0.24308],[1482394336387,-0.24217],[1482395200387,-0.22967],[1482396064387,-0.22592],[1482396928387,-0.22855],[1482397792387,-0.22559],[1482398656387,-0.22782],[1482399520387,-0.23039],[1482400384387,-0.22721],[1482401248387,-0.22171],[1482402112387,-0.22231],[1482402976387,-0.22953],[1482403840387,-0.22596],[1482404704387,-0.23019],[1482405568387,-0.22796],[1482406432387,-0.23092],[1482407296387,-0.22731],[1482408160387,-0.23779],[1482409024387,-0.23519],[1482409888387,-0.23037],[1482410752387,-0.24370],[1482411616387,-0.23890],[1482412480387,-0.23413],[1482413344387,-0.23784],[1482414208387,-0.22149],[1482415072387,-0.23192],[1482415936387,-0.22646],[1482416800387,-0.23515],[1482417664387,-0.23351],[1482418528387,-0.22742],[1482419392387,-0.22796],[1482420256387,-0.22296],[1482421120387,-0.23475],[1482421984387,-0.23242],[1482422848387,-0.23490],[1482423712387,-0.22955],[1482424576387,-0.23063],[1482425440387,-0.23311],[1482426304387,-0.23114],[1482427168387,-0.23970],[1482428032387,-0.23154],[1482428896387,-0.23627],[1482429760387,-0.23769],[1482430624387,-0.23196],[1482431488387,-0.23972],[1482432352387,-0.23798],[1482433216387,-0.24255],[1482434080387,-0.23818],[1482434944387,-0.23454],[1482435808387,-0.23644],[1482436672387,-0.23483],[1482437536387,-0.23025],[1482438400387,-0.23225],[1482439264387,-0.23817],[1482440128387,-0.22725],[1482440992387,-0.22935],[1482441856387,-0.23433],[1482442720387,-0.22831],[1482443584387,-0.24098],[1482444448387,-0.22562],[1482445312387,-0.24295],[1482446176387,-0.24032],[1482447040387,-0.23764],[1482447904387,-0.23004],[1482448768387,-0.24305],[1482449632387,-0.22990],[1482450496387,-0.23475],[1482451360387,-0.23410],[1482452224387,-0.24464],[1482453088387,-0.24437],[1482453952387,-0.24393],[1482454816387,-0.24358],[1482455680387,-0.24403]],\n  "last":[[1482339040387,-0.24649],[1482339904387,-0.24922],[1482340768387,-0.24743],[1482341632387,-0.24754],[1482342496387,-0.24743],[1482343360387,-0.24931],[1482344224387,-0.24700],[1482345088387,-0.24686],[1482345952387,-0.24724],[1482346816387,-0.24767],[1482347680387,-0.24714],[1482348544387,-0.25072],[1482349408387,-0.24862],[1482350272387,-0.24704],[1482351136387,-0.24679],[1482352000387,-0.24876],[1482352864387,-0.24492],[1482353728387,-0.24747],[1482354592387,-0.25489],[1482355456387,-0.24751],[1482356320387,-0.24862],[1482357184387,-0.24670],[1482358048387,-0.24717],[1482358912387,-0.24854],[1482359776387,-0.24924],[1482360640387,-0.24798],[1482361504387,-0.25006],[1482362368387,-0.24720],[1482363232387,-0.24873],[1482364096387,-0.24730],[1482364960387,-0.24799],[1482365824387,-0.24764],[1482366688387,-0.24796],[1482367552387,-0.24723],[1482368416387,-0.24788],[1482369280387,-0.24718],[1482370144387,-0.24764],[1482371008387,-0.24693],[1482371872387,-0.25072],[1482372736387,-0.24751],[1482373600387,-0.24709],[1482374464387,-0.24733],[1482375328387,-0.24666],[1482376192387,-0.24771],[1482377056387,-0.24710],[1482377920387,-0.24663],[1482378784387,-0.24829],[1482379648387,-0.24816],[1482380512387,-0.24673],[1482381376387,-0.24842],[1482382240387,-0.24649],[1482383104387,-0.24764],[1482383968387,-0.24754],[1482384832387,-0.24747],[1482385696387,-0.24685],[1482386560387,-0.24584],[1482387424387,-0.24745],[1482388288387,-0.24685],[1482389152387,-0.24743],[1482390016387,-0.24757],[1482390880387,-0.24765],[1482391744387,-0.24750],[1482392608387,-0.24786],[1482393472387,-0.24690],[1482394336387,-0.24683],[1482395200387,-0.24681],[1482396064387,-0.24551],[1482396928387,-0.24493],[1482397792387,-0.24635],[1482398656387,-0.24706],[1482399520387,-0.24665],[1482400384387,-0.24380],[1482401248387,-0.24665],[1482402112387,-0.24090],[1482402976387,-0.24663],[1482403840387,-0.24693],[1482404704387,-0.24645],[1482405568387,-0.24731],[1482406432387,-0.24566],[1482407296387,-0.25159],[1482408160387,-0.24709],[1482409024387,-0.24799],[1482409888387,-0.24891],[1482410752387,-0.24737],[1482411616387,-0.24806],[1482412480387,-0.24795],[1482413344387,-0.24680],[1482414208387,-0.24689],[1482415072387,-0.24585],[1482415936387,-0.24643],[1482416800387,-0.24745],[1482417664387,-0.24866],[1482418528387,-0.24704],[1482419392387,-0.24686],[1482420256387,-0.24372],[1482421120387,-0.24618],[1482421984387,-0.24760],[1482422848387,-0.24683],[1482423712387,-0.24714],[1482424576387,-0.25142],[1482425440387,-0.24700],[1482426304387,-0.24704],[1482427168387,-0.24839],[1482428032387,-0.24758],[1482428896387,-0.24774],[1482429760387,-0.24897],[1482430624387,-0.24709],[1482431488387,-0.24768],[1482432352387,-0.24858],[1482433216387,-0.24770],[1482434080387,-0.24772],[1482434944387,-0.24738],[1482435808387,-0.24629],[1482436672387,-0.24641],[1482437536387,-0.24757],[1482438400387,-0.24669],[1482439264387,-0.24767],[1482440128387,-0.24792],[1482440992387,-0.24049],[1482441856387,-0.24486],[1482442720387,-0.24826],[1482443584387,-0.25503],[1482444448387,-0.24632],[1482445312387,-0.24762],[1482446176387,-0.24594],[1482447040387,-0.24553],[1482447904387,-0.24645],[1482448768387,-0.24643],[1482449632387,-0.24619],[1482450496387,-0.25102],[1482451360387,-0.23638],[1482452224387,-0.24556],[1482453088387,-0.24737],[1482453952387,-0.24686],[1482454816387,-0.24724],[1482455680387,-0.24842]],\n  "min":[[1482339040387,-0.26504],[1482339904387,-0.26814],[1482340768387,-0.26834],[1482341632387,-0.26832],[1482342496387,-0.25060],[1482343360387,-0.25527],[1482344224387,-0.26109],[1482345088387,-0.25541],[1482345952387,-0.25361],[1482346816387,-0.26765],[1482347680387,-0.26775],[1482348544387,-0.26069],[1482349408387,-0.27119],[1482350272387,-0.26155],[1482351136387,-0.26020],[1482352000387,-0.26345],[1482352864387,-0.26442],[1482353728387,-0.27430],[1482354592387,-0.26800],[1482355456387,-0.26907],[1482356320387,-0.26791],[1482357184387,-0.26045],[1482358048387,-0.25643],[1482358912387,-0.25473],[1482359776387,-0.26229],[1482360640387,-0.26448],[1482361504387,-0.26500],[1482362368387,-0.26970],[1482363232387,-0.25442],[1482364096387,-0.25993],[1482364960387,-0.26509],[1482365824387,-0.26341],[1482366688387,-0.25201],[1482367552387,-0.25431],[1482368416387,-0.25530],[1482369280387,-0.26223],[1482370144387,-0.26743],[1482371008387,-0.26228],[1482371872387,-0.26271],[1482372736387,-0.25591],[1482373600387,-0.25354],[1482374464387,-0.25667],[1482375328387,-0.25691],[1482376192387,-0.25252],[1482377056387,-0.25237],[1482377920387,-0.25238],[1482378784387,-0.25700],[1482379648387,-0.26423],[1482380512387,-0.26584],[1482381376387,-0.26787],[1482382240387,-0.26645],[1482383104387,-0.26458],[1482383968387,-0.26614],[1482384832387,-0.26906],[1482385696387,-0.25697],[1482386560387,-0.26148],[1482387424387,-0.26257],[1482388288387,-0.26409],[1482389152387,-0.26688],[1482390016387,-0.25439],[1482390880387,-0.25613],[1482391744387,-0.26805],[1482392608387,-0.25130],[1482393472387,-0.25149],[1482394336387,-0.25867],[1482395200387,-0.26068],[1482396064387,-0.26982],[1482396928387,-0.26526],[1482397792387,-0.26472],[1482398656387,-0.27468],[1482399520387,-0.26481],[1482400384387,-0.26655],[1482401248387,-0.27602],[1482402112387,-0.27155],[1482402976387,-0.26113],[1482403840387,-0.26787],[1482404704387,-0.26672],[1482405568387,-0.26237],[1482406432387,-0.26162],[1482407296387,-0.26450],[1482408160387,-0.25558],[1482409024387,-0.26525],[1482409888387,-0.26821],[1482410752387,-0.25592],[1482411616387,-0.25358],[1482412480387,-0.25827],[1482413344387,-0.26276],[1482414208387,-0.26219],[1482415072387,-0.26487],[1482415936387,-0.26383],[1482416800387,-0.26202],[1482417664387,-0.25552],[1482418528387,-0.25867],[1482419392387,-0.26741],[1482420256387,-0.26331],[1482421120387,-0.26069],[1482421984387,-0.26131],[1482422848387,-0.26289],[1482423712387,-0.27434],[1482424576387,-0.26856],[1482425440387,-0.25966],[1482426304387,-0.27155],[1482427168387,-0.26447],[1482428032387,-0.26893],[1482428896387,-0.25824],[1482429760387,-0.26104],[1482430624387,-0.26362],[1482431488387,-0.25776],[1482432352387,-0.26430],[1482433216387,-0.25432],[1482434080387,-0.25216],[1482434944387,-0.25256],[1482435808387,-0.25752],[1482436672387,-0.25571],[1482437536387,-0.26444],[1482438400387,-0.25983],[1482439264387,-0.26128],[1482440128387,-0.26011],[1482440992387,-0.25858],[1482441856387,-0.26015],[1482442720387,-0.26678],[1482443584387,-0.25743],[1482444448387,-0.26321],[1482445312387,-0.25714],[1482446176387,-0.25220],[1482447040387,-0.26542],[1482447904387,-0.26492],[1482448768387,-0.25167],[1482449632387,-0.26030],[1482450496387,-0.26519],[1482451360387,-0.26529],[1482452224387,-0.25006],[1482453088387,-0.25063],[1482453952387,-0.25060],[1482454816387,-0.25072],[1482455680387,-0.25035]]\n}'
_trenddata_72hr_ph_json = '{\n  "max":[[1482339040695,7.2874],[1482339904695,7.2710],[1482340768695,7.2833],[1482341632695,7.2808],[1482342496695,7.2803],[1482343360695,7.2775],[1482344224695,7.2789],[1482345088695,7.2793],[1482345952695,7.2788],[1482346816695,7.2762],[1482347680695,7.2778],[1482348544695,7.2789],[1482349408695,7.2738],[1482350272695,7.2827],[1482351136695,7.2813],[1482352000695,7.2748],[1482352864695,7.2729],[1482353728695,7.2735],[1482354592695,7.2713],[1482355456695,7.2729],[1482356320695,7.2721],[1482357184695,7.2748],[1482358048695,7.2734],[1482358912695,7.2762],[1482359776695,7.2771],[1482360640695,7.2838],[1482361504695,7.2825],[1482362368695,7.2734],[1482363232695,7.2726],[1482364096695,7.2718],[1482364960695,7.2711],[1482365824695,7.2738],[1482366688695,7.2748],[1482367552695,7.2761],[1482368416695,7.2749],[1482369280695,7.2748],[1482370144695,7.2748],[1482371008695,7.2737],[1482371872695,7.2735],[1482372736695,7.2743],[1482373600695,7.2743],[1482374464695,7.2761],[1482375328695,7.2775],[1482376192695,7.2774],[1482377056695,7.2774],[1482377920695,7.2779],[1482378784695,7.2763],[1482379648695,7.2760],[1482380512695,7.2762],[1482381376695,7.2778],[1482382240695,7.2777],[1482383104695,7.2788],[1482383968695,7.2787],[1482384832695,7.2803],[1482385696695,7.2807],[1482386560695,7.2800],[1482387424695,7.2814],[1482388288695,7.2807],[1482389152695,7.2817],[1482390016695,7.2817],[1482390880695,7.2840],[1482391744695,7.2827],[1482392608695,7.2828],[1482393472695,7.2827],[1482394336695,7.2845],[1482395200695,7.2839],[1482396064695,7.2854],[1482396928695,7.2829],[1482397792695,7.2841],[1482398656695,7.2819],[1482399520695,7.2847],[1482400384695,7.2851],[1482401248695,7.2857],[1482402112695,7.2849],[1482402976695,7.2865],[1482403840695,7.2857],[1482404704695,7.2851],[1482405568695,7.2844],[1482406432695,7.2865],[1482407296695,7.2861],[1482408160695,7.2844],[1482409024695,7.2851],[1482409888695,7.2834],[1482410752695,7.2844],[1482411616695,7.2826],[1482412480695,7.2847],[1482413344695,7.2822],[1482414208695,7.2836],[1482415072695,7.2826],[1482415936695,7.2836],[1482416800695,7.2815],[1482417664695,7.2811],[1482418528695,7.2806],[1482419392695,7.2811],[1482420256695,7.2796],[1482421120695,7.2790],[1482421984695,7.2795],[1482422848695,7.2794],[1482423712695,7.2782],[1482424576695,7.2726],[1482425440695,7.2732],[1482426304695,7.2727],[1482427168695,7.2692],[1482428032695,7.2695],[1482428896695,7.2797],[1482429760695,7.2795],[1482430624695,7.2720],[1482431488695,7.2696],[1482432352695,7.2714],[1482433216695,7.2688],[1482434080695,7.2761],[1482434944695,7.2768],[1482435808695,3.9425],[1482436672695,0.017806],[1482437536695,0.018731],[1482438400695,0.019202],[1482439264695,0.019990],[1482440128695,0.021007],[1482440992695,0.022072],[1482441856695,0.022268],[1482442720695,0.023213],[1482443584695,0.025576],[1482444448695,0.027623],[1482445312695,0.029061],[1482446176695,0.032843],[1482447040695,0.031850],[1482447904695,0.031396],[1482448768695,0.032100],[1482449632695,0.033189],[1482450496695,0.034500],[1482451360695,0.034759],[1482452224695,0.035156],[1482453088695,0.036785],[1482453952695,0.037912],[1482454816695,0.038118],[1482455680695,0.037669]],\n  "last":[[1482339040695,7.2775],[1482339904695,7.2684],[1482340768695,7.2674],[1482341632695,7.2675],[1482342496695,7.2750],[1482343360695,7.2752],[1482344224695,7.2763],[1482345088695,7.2750],[1482345952695,7.2711],[1482346816695,7.2736],[1482347680695,7.2729],[1482348544695,7.2745],[1482349408695,7.2672],[1482350272695,7.2734],[1482351136695,7.2696],[1482352000695,7.2709],[1482352864695,7.2695],[1482353728695,7.2690],[1482354592695,7.2689],[1482355456695,7.2687],[1482356320695,7.2683],[1482357184695,7.2698],[1482358048695,7.2710],[1482358912695,7.2722],[1482359776695,7.2771],[1482360640695,7.2698],[1482361504695,7.2696],[1482362368695,7.2684],[1482363232695,7.2700],[1482364096695,7.2696],[1482364960695,7.2706],[1482365824695,7.2635],[1482366688695,7.2728],[1482367552695,7.2710],[1482368416695,7.2725],[1482369280695,7.2709],[1482370144695,7.2734],[1482371008695,7.2696],[1482371872695,7.2710],[1482372736695,7.2712],[1482373600695,7.2698],[1482374464695,7.2712],[1482375328695,7.2749],[1482376192695,7.2721],[1482377056695,7.2748],[1482377920695,7.2735],[1482378784695,7.2756],[1482379648695,7.2748],[1482380512695,7.2736],[1482381376695,7.2735],[1482382240695,7.2774],[1482383104695,7.2761],[1482383968695,7.2773],[1482384832695,7.2773],[1482385696695,7.2799],[1482386560695,7.2774],[1482387424695,7.2783],[1482388288695,7.2794],[1482389152695,7.2774],[1482390016695,7.2800],[1482390880695,7.2793],[1482391744695,7.2814],[1482392608695,7.2788],[1482393472695,7.2803],[1482394336695,7.2801],[1482395200695,7.2814],[1482396064695,7.2806],[1482396928695,7.2794],[1482397792695,7.2802],[1482398656695,7.2816],[1482399520695,7.2803],[1482400384695,7.2816],[1482401248695,7.2816],[1482402112695,7.2821],[1482402976695,7.2821],[1482403840695,7.2842],[1482404704695,7.2841],[1482405568695,7.2830],[1482406432695,7.2830],[1482407296695,7.2845],[1482408160695,7.2808],[1482409024695,7.2830],[1482409888695,7.2826],[1482410752695,7.2803],[1482411616695,7.2806],[1482412480695,7.2810],[1482413344695,7.2818],[1482414208695,7.2803],[1482415072695,7.2794],[1482415936695,7.2794],[1482416800695,7.2784],[1482417664695,7.2781],[1482418528695,7.2768],[1482419392695,7.2765],[1482420256695,7.2794],[1482421120695,7.2768],[1482421984695,7.2753],[1482422848695,7.2768],[1482423712695,7.2747],[1482424576695,7.2698],[1482425440695,7.2698],[1482426304695,7.2705],[1482427168695,7.2654],[1482428032695,7.2679],[1482428896695,7.2687],[1482429760695,7.2679],[1482430624695,7.2692],[1482431488695,7.2691],[1482432352695,7.2666],[1482433216695,7.2679],[1482434080695,7.2678],[1482434944695,7.2726],[1482435808695,3.9425],[1482436672695,0.017114],[1482437536695,0.017589],[1482438400695,0.018406],[1482439264695,0.019027],[1482440128695,0.018980],[1482440992695,0.020288],[1482441856695,0.020929],[1482442720695,0.021432],[1482443584695,0.022365],[1482444448695,0.025539],[1482445312695,0.027507],[1482446176695,0.028737],[1482447040695,0.030066],[1482447904695,0.029896],[1482448768695,0.030988],[1482449632695,0.032111],[1482450496695,0.032501],[1482451360695,0.034259],[1482452224695,0.034458],[1482453088695,0.034736],[1482453952695,0.036075],[1482454816695,0.036430],[1482455680695,0.036953]],\n  "min":[[1482339040695,7.2671],[1482339904695,7.2667],[1482340768695,7.2667],[1482341632695,7.2666],[1482342496695,7.2722],[1482343360695,7.2721],[1482344224695,7.2723],[1482345088695,7.2709],[1482345952695,7.2709],[1482346816695,7.2708],[1482347680695,7.2711],[1482348544695,7.2658],[1482349408695,7.2658],[1482350272695,7.2683],[1482351136695,7.2678],[1482352000695,7.2672],[1482352864695,7.2671],[1482353728695,7.2662],[1482354592695,7.2670],[1482355456695,7.2658],[1482356320695,7.2679],[1482357184695,7.2684],[1482358048695,7.2686],[1482358912695,7.2695],[1482359776695,7.2684],[1482360640695,7.2683],[1482361504695,7.2663],[1482362368695,7.2672],[1482363232695,7.2658],[1482364096695,7.2658],[1482364960695,7.2620],[1482365824695,7.2631],[1482366688695,7.2697],[1482367552695,7.2708],[1482368416695,7.2696],[1482369280695,7.2695],[1482370144695,7.2690],[1482371008695,7.2680],[1482371872695,7.2667],[1482372736695,7.2680],[1482373600695,7.2681],[1482374464695,7.2683],[1482375328695,7.2706],[1482376192695,7.2709],[1482377056695,7.2714],[1482377920695,7.2712],[1482378784695,7.2722],[1482379648695,7.2717],[1482380512695,7.2721],[1482381376695,7.2731],[1482382240695,7.2745],[1482383104695,7.2735],[1482383968695,7.2741],[1482384832695,7.2751],[1482385696695,7.2746],[1482386560695,7.2761],[1482387424695,7.2774],[1482388288695,7.2770],[1482389152695,7.2770],[1482390016695,7.2775],[1482390880695,7.2788],[1482391744695,7.2787],[1482392608695,7.2785],[1482393472695,7.2788],[1482394336695,7.2788],[1482395200695,7.2788],[1482396064695,7.2769],[1482396928695,7.2773],[1482397792695,7.2781],[1482398656695,7.2778],[1482399520695,7.2783],[1482400384695,7.2809],[1482401248695,7.2795],[1482402112695,7.2805],[1482402976695,7.2813],[1482403840695,7.2798],[1482404704695,7.2803],[1482405568695,7.2802],[1482406432695,7.2801],[1482407296695,7.2804],[1482408160695,7.2802],[1482409024695,7.2793],[1482409888695,7.2791],[1482410752695,7.2791],[1482411616695,7.2782],[1482412480695,7.2780],[1482413344695,7.2777],[1482414208695,7.2779],[1482415072695,7.2768],[1482415936695,7.2768],[1482416800695,7.2765],[1482417664695,7.2756],[1482418528695,7.2756],[1482419392695,7.2756],[1482420256695,7.2751],[1482421120695,7.2742],[1482421984695,7.2747],[1482422848695,7.2735],[1482423712695,7.2678],[1482424576695,7.2681],[1482425440695,7.2686],[1482426304695,7.2650],[1482427168695,7.2641],[1482428032695,7.2651],[1482428896695,7.2658],[1482429760695,7.2663],[1482430624695,7.2653],[1482431488695,7.2655],[1482432352695,7.2663],[1482433216695,7.2639],[1482434080695,7.2653],[1482434944695,3.9512],[1482435808695,0.016045],[1482436672695,0.016360],[1482437536695,0.016895],[1482438400695,0.017424],[1482439264695,0.017464],[1482440128695,0.018750],[1482440992695,0.019686],[1482441856695,0.020441],[1482442720695,0.021177],[1482443584695,0.022238],[1482444448695,0.025184],[1482445312695,0.027039],[1482446176695,0.024311],[1482447040695,0.024054],[1482447904695,0.029214],[1482448768695,0.030615],[1482449632695,0.031589],[1482450496695,0.032310],[1482451360695,0.033331],[1482452224695,0.033703],[1482453088695,0.034519],[1482453952695,0.035149],[1482454816695,0.035953],[1482455680695,0.036953]]\n}'
_trenddata_72hr_temperature_json = '{\n  "max":[[1482339040995,20.011],[1482339904995,20.016],[1482340768995,20.011],[1482341632995,20.010],[1482342496995,20.013],[1482343360995,20.019],[1482344224995,20.027],[1482345088995,20.037],[1482345952995,20.051],[1482346816995,20.109],[1482347680995,20.125],[1482348544995,20.140],[1482349408995,20.155],[1482350272995,20.169],[1482351136995,20.183],[1482352000995,20.195],[1482352864995,20.198],[1482353728995,20.199],[1482354592995,20.199],[1482355456995,20.195],[1482356320995,20.181],[1482357184995,20.176],[1482358048995,20.179],[1482358912995,20.173],[1482359776995,20.165],[1482360640995,20.159],[1482361504995,20.156],[1482362368995,20.147],[1482363232995,20.151],[1482364096995,20.152],[1482364960995,20.155],[1482365824995,20.155],[1482366688995,20.154],[1482367552995,20.162],[1482368416995,20.170],[1482369280995,20.179],[1482370144995,20.184],[1482371008995,20.188],[1482371872995,20.192],[1482372736995,20.197],[1482373600995,20.201],[1482374464995,20.202],[1482375328995,20.204],[1482376192995,20.204],[1482377056995,20.194],[1482377920995,20.189],[1482378784995,20.181],[1482379648995,20.180],[1482380512995,20.177],[1482381376995,20.173],[1482382240995,20.170],[1482383104995,20.169],[1482383968995,20.162],[1482384832995,20.156],[1482385696995,20.143],[1482386560995,20.140],[1482387424995,20.124],[1482388288995,20.107],[1482389152995,20.086],[1482390016995,20.058],[1482390880995,20.032],[1482391744995,20.023],[1482392608995,20.009],[1482393472995,19.993],[1482394336995,19.977],[1482395200995,19.959],[1482396064995,19.938],[1482396928995,19.915],[1482397792995,19.893],[1482398656995,19.874],[1482399520995,19.853],[1482400384995,19.830],[1482401248995,19.810],[1482402112995,19.782],[1482402976995,19.760],[1482403840995,19.741],[1482404704995,19.717],[1482405568995,19.695],[1482406432995,19.671],[1482407296995,19.646],[1482408160995,19.626],[1482409024995,19.598],[1482409888995,19.573],[1482410752995,19.550],[1482411616995,19.529],[1482412480995,19.504],[1482413344995,19.482],[1482414208995,19.458],[1482415072995,19.438],[1482415936995,19.422],[1482416800995,19.400],[1482417664995,19.382],[1482418528995,19.365],[1482419392995,19.347],[1482420256995,19.331],[1482421120995,19.322],[1482421984995,19.314],[1482422848995,19.306],[1482423712995,19.302],[1482424576995,19.298],[1482425440995,19.290],[1482426304995,19.279],[1482427168995,19.272],[1482428032995,19.263],[1482428896995,19.262],[1482429760995,19.261],[1482430624995,19.270],[1482431488995,19.277],[1482432352995,19.308],[1482433216995,19.328],[1482434080995,19.352],[1482434944995,19.380],[1482435808995,19.413],[1482436672995,19.435],[1482437536995,19.466],[1482438400995,19.500],[1482439264995,19.535],[1482440128995,19.575],[1482440992995,19.613],[1482441856995,19.647],[1482442720995,19.687],[1482443584995,19.806],[1482444448995,19.890],[1482445312995,19.952],[1482446176995,20.001],[1482447040995,20.005],[1482447904995,20.037],[1482448768995,20.080],[1482449632995,20.118],[1482450496995,20.158],[1482451360995,20.187],[1482452224995,20.204],[1482453088995,20.258],[1482453952995,20.277],[1482454816995,20.308],[1482455680995,20.311]],\n  "last":[[1482339040995,20.004],[1482339904995,20.007],[1482340768995,20.011],[1482341632995,20.005],[1482342496995,20.004],[1482343360995,20.010],[1482344224995,20.015],[1482345088995,20.025],[1482345952995,20.032],[1482346816995,20.048],[1482347680995,20.107],[1482348544995,20.122],[1482349408995,20.138],[1482350272995,20.152],[1482351136995,20.169],[1482352000995,20.182],[1482352864995,20.193],[1482353728995,20.195],[1482354592995,20.195],[1482355456995,20.195],[1482356320995,20.177],[1482357184995,20.173],[1482358048995,20.173],[1482358912995,20.170],[1482359776995,20.162],[1482360640995,20.158],[1482361504995,20.156],[1482362368995,20.141],[1482363232995,20.147],[1482364096995,20.145],[1482364960995,20.147],[1482365824995,20.150],[1482366688995,20.150],[1482367552995,20.146],[1482368416995,20.160],[1482369280995,20.169],[1482370144995,20.175],[1482371008995,20.183],[1482371872995,20.183],[1482372736995,20.191],[1482373600995,20.195],[1482374464995,20.195],[1482375328995,20.201],[1482376192995,20.204],[1482377056995,20.192],[1482377920995,20.183],[1482378784995,20.178],[1482379648995,20.175],[1482380512995,20.172],[1482381376995,20.170],[1482382240995,20.167],[1482383104995,20.165],[1482383968995,20.155],[1482384832995,20.151],[1482385696995,20.142],[1482386560995,20.136],[1482387424995,20.122],[1482388288995,20.106],[1482389152995,20.085],[1482390016995,20.057],[1482390880995,20.031],[1482391744995,20.020],[1482392608995,20.006],[1482393472995,19.989],[1482394336995,19.973],[1482395200995,19.957],[1482396064995,19.935],[1482396928995,19.912],[1482397792995,19.890],[1482398656995,19.869],[1482399520995,19.851],[1482400384995,19.829],[1482401248995,19.809],[1482402112995,19.778],[1482402976995,19.759],[1482403840995,19.739],[1482404704995,19.716],[1482405568995,19.691],[1482406432995,19.669],[1482407296995,19.644],[1482408160995,19.623],[1482409024995,19.596],[1482409888995,19.573],[1482410752995,19.550],[1482411616995,19.526],[1482412480995,19.503],[1482413344995,19.480],[1482414208995,19.457],[1482415072995,19.435],[1482415936995,19.416],[1482416800995,19.396],[1482417664995,19.377],[1482418528995,19.362],[1482419392995,19.346],[1482420256995,19.329],[1482421120995,19.322],[1482421984995,19.308],[1482422848995,19.305],[1482423712995,19.296],[1482424576995,19.292],[1482425440995,19.289],[1482426304995,19.277],[1482427168995,19.272],[1482428032995,19.261],[1482428896995,19.258],[1482429760995,19.260],[1482430624995,19.258],[1482431488995,19.266],[1482432352995,19.275],[1482433216995,19.305],[1482434080995,19.327],[1482434944995,19.350],[1482435808995,19.380],[1482436672995,19.412],[1482437536995,19.431],[1482438400995,19.465],[1482439264995,19.499],[1482440128995,19.532],[1482440992995,19.574],[1482441856995,19.611],[1482442720995,19.643],[1482443584995,19.684],[1482444448995,19.807],[1482445312995,19.889],[1482446176995,19.951],[1482447040995,20.000],[1482447904995,20.000],[1482448768995,20.035],[1482449632995,20.081],[1482450496995,20.118],[1482451360995,20.158],[1482452224995,20.185],[1482453088995,20.203],[1482453952995,20.257],[1482454816995,20.275],[1482455680995,20.305]],\n  "min":[[1482339040995,20.000],[1482339904995,20.006],[1482340768995,20.003],[1482341632995,20.000],[1482342496995,20.000],[1482343360995,20.007],[1482344224995,20.014],[1482345088995,20.022],[1482345952995,20.031],[1482346816995,20.047],[1482347680995,20.106],[1482348544995,20.120],[1482349408995,20.136],[1482350272995,20.149],[1482351136995,20.165],[1482352000995,20.179],[1482352864995,20.190],[1482353728995,20.190],[1482354592995,20.190],[1482355456995,20.175],[1482356320995,20.169],[1482357184995,20.168],[1482358048995,20.168],[1482358912995,20.161],[1482359776995,20.154],[1482360640995,20.151],[1482361504995,20.138],[1482362368995,20.135],[1482363232995,20.142],[1482364096995,20.141],[1482364960995,20.144],[1482365824995,20.146],[1482366688995,20.144],[1482367552995,20.144],[1482368416995,20.158],[1482369280995,20.166],[1482370144995,20.174],[1482371008995,20.178],[1482371872995,20.180],[1482372736995,20.185],[1482373600995,20.191],[1482374464995,20.193],[1482375328995,20.196],[1482376192995,20.187],[1482377056995,20.182],[1482377920995,20.177],[1482378784995,20.173],[1482379648995,20.170],[1482380512995,20.167],[1482381376995,20.165],[1482382240995,20.161],[1482383104995,20.155],[1482383968995,20.148],[1482384832995,20.138],[1482385696995,20.129],[1482386560995,20.119],[1482387424995,20.103],[1482388288995,20.083],[1482389152995,20.053],[1482390016995,20.027],[1482390880995,20.016],[1482391744995,20.003],[1482392608995,19.987],[1482393472995,19.972],[1482394336995,19.955],[1482395200995,19.934],[1482396064995,19.909],[1482396928995,19.889],[1482397792995,19.869],[1482398656995,19.849],[1482399520995,19.826],[1482400384995,19.807],[1482401248995,19.779],[1482402112995,19.756],[1482402976995,19.737],[1482403840995,19.715],[1482404704995,19.689],[1482405568995,19.668],[1482406432995,19.644],[1482407296995,19.621],[1482408160995,19.595],[1482409024995,19.570],[1482409888995,19.548],[1482410752995,19.525],[1482411616995,19.501],[1482412480995,19.477],[1482413344995,19.455],[1482414208995,19.434],[1482415072995,19.416],[1482415936995,19.396],[1482416800995,19.378],[1482417664995,19.360],[1482418528995,19.343],[1482419392995,19.328],[1482420256995,19.318],[1482421120995,19.308],[1482421984995,19.301],[1482422848995,19.296],[1482423712995,19.287],[1482424576995,19.287],[1482425440995,19.274],[1482426304995,19.266],[1482427168995,19.259],[1482428032995,19.253],[1482428896995,19.254],[1482429760995,19.254],[1482430624995,19.257],[1482431488995,19.265],[1482432352995,19.274],[1482433216995,19.302],[1482434080995,19.325],[1482434944995,19.345],[1482435808995,19.377],[1482436672995,19.405],[1482437536995,19.430],[1482438400995,19.463],[1482439264995,19.498],[1482440128995,19.532],[1482440992995,19.573],[1482441856995,19.608],[1482442720995,19.642],[1482443584995,19.684],[1482444448995,19.804],[1482445312995,19.888],[1482446176995,19.951],[1482447040995,19.997],[1482447904995,19.993],[1482448768995,20.035],[1482449632995,20.078],[1482450496995,20.114],[1482451360995,20.156],[1482452224995,20.182],[1482453088995,20.201],[1482453952995,20.257],[1482454816995,20.273],[1482455680995,20.304]]\n}'
_trenddata_7day_agitation_json = '{\n  "max":[[1482340769296,0.0000],[1482342785296,0.0000],[1482344801296,0.0000],[1482346817296,0.0000],[1482348833296,0.0000],[1482350849296,0.0000],[1482352865296,0.0000],[1482354881296,0.0000],[1482356897296,0.0000],[1482358913296,0.0000],[1482360929296,0.0000],[1482362945296,0.0000],[1482364961296,0.0000],[1482366977296,0.0000],[1482368993296,0.0000],[1482371009296,0.0000],[1482373025296,0.0000],[1482375041296,0.0000],[1482377057296,0.0000],[1482379073296,0.0000],[1482381089296,0.0000],[1482383105296,0.0000],[1482385121296,0.0000],[1482387137296,0.0000],[1482389153296,0.0000],[1482391169296,0.0000],[1482393185296,0.0000],[1482395201296,0.0000],[1482397217296,0.0000],[1482399233296,0.0000],[1482401249296,0.0000],[1482403265296,0.0000],[1482405281296,0.0000],[1482407297296,0.0000],[1482409313296,0.0000],[1482411329296,0.0000],[1482413345296,0.0000],[1482415361296,0.0000],[1482417377296,0.0000],[1482419393296,0.0000],[1482421409296,0.0000],[1482423425296,0.0000],[1482425441296,0.0000],[1482427457296,0.0000],[1482429473296,0.0000],[1482431489296,0.0000],[1482433505296,0.0000],[1482435521296,0.0000],[1482437537296,0.0000],[1482439553296,0.0000],[1482441569296,0.0000],[1482443585296,20.039],[1482445601296,20.043],[1482447617296,36.028],[1482449633296,12.063],[1482451649296,12.063],[1482453665296,36.042],[1482455681296,36.048]],\n  "last":[[1482340769296,0.0000],[1482342785296,0.0000],[1482344801296,0.0000],[1482346817296,0.0000],[1482348833296,0.0000],[1482350849296,0.0000],[1482352865296,0.0000],[1482354881296,0.0000],[1482356897296,0.0000],[1482358913296,0.0000],[1482360929296,0.0000],[1482362945296,0.0000],[1482364961296,0.0000],[1482366977296,0.0000],[1482368993296,0.0000],[1482371009296,0.0000],[1482373025296,0.0000],[1482375041296,0.0000],[1482377057296,0.0000],[1482379073296,0.0000],[1482381089296,0.0000],[1482383105296,0.0000],[1482385121296,0.0000],[1482387137296,0.0000],[1482389153296,0.0000],[1482391169296,0.0000],[1482393185296,0.0000],[1482395201296,0.0000],[1482397217296,0.0000],[1482399233296,0.0000],[1482401249296,0.0000],[1482403265296,0.0000],[1482405281296,0.0000],[1482407297296,0.0000],[1482409313296,0.0000],[1482411329296,0.0000],[1482413345296,0.0000],[1482415361296,0.0000],[1482417377296,0.0000],[1482419393296,0.0000],[1482421409296,0.0000],[1482423425296,0.0000],[1482425441296,0.0000],[1482427457296,0.0000],[1482429473296,0.0000],[1482431489296,0.0000],[1482433505296,0.0000],[1482435521296,0.0000],[1482437537296,0.0000],[1482439553296,0.0000],[1482441569296,0.0000],[1482443585296,0.0000],[1482445601296,19.865],[1482447617296,0.0000],[1482449633296,11.923],[1482451649296,12.058],[1482453665296,12.049],[1482455681296,36.009]],\n  "min":[[1482340769296,0.0000],[1482342785296,0.0000],[1482344801296,0.0000],[1482346817296,0.0000],[1482348833296,0.0000],[1482350849296,0.0000],[1482352865296,0.0000],[1482354881296,0.0000],[1482356897296,0.0000],[1482358913296,0.0000],[1482360929296,0.0000],[1482362945296,0.0000],[1482364961296,0.0000],[1482366977296,0.0000],[1482368993296,0.0000],[1482371009296,0.0000],[1482373025296,0.0000],[1482375041296,0.0000],[1482377057296,0.0000],[1482379073296,0.0000],[1482381089296,0.0000],[1482383105296,0.0000],[1482385121296,0.0000],[1482387137296,0.0000],[1482389153296,0.0000],[1482391169296,0.0000],[1482393185296,0.0000],[1482395201296,0.0000],[1482397217296,0.0000],[1482399233296,0.0000],[1482401249296,0.0000],[1482403265296,0.0000],[1482405281296,0.0000],[1482407297296,0.0000],[1482409313296,0.0000],[1482411329296,0.0000],[1482413345296,0.0000],[1482415361296,0.0000],[1482417377296,0.0000],[1482419393296,0.0000],[1482421409296,0.0000],[1482423425296,0.0000],[1482425441296,0.0000],[1482427457296,0.0000],[1482429473296,0.0000],[1482431489296,0.0000],[1482433505296,0.0000],[1482435521296,0.0000],[1482437537296,0.0000],[1482439553296,0.0000],[1482441569296,0.0000],[1482443585296,0.0000],[1482445601296,0.0000],[1482447617296,0.0000],[1482449633296,11.801],[1482451649296,11.791],[1482453665296,0.0000],[1482455681296,35.956]]\n}'
_trenddata_7day_do_json = '{\n  "max":[[1482340769497,-0.22293],[1482342785497,-0.23824],[1482344801497,-0.23703],[1482346817497,-0.22973],[1482348833497,-0.22500],[1482350849497,-0.22674],[1482352865497,-0.23124],[1482354881497,-0.22960],[1482356897497,-0.22957],[1482358913497,-0.24304],[1482360929497,-0.22778],[1482362945497,-0.23887],[1482364961497,-0.23883],[1482366977497,-0.24023],[1482368993497,-0.23075],[1482371009497,-0.23443],[1482373025497,-0.23600],[1482375041497,-0.23613],[1482377057497,-0.24039],[1482379073497,-0.23355],[1482381089497,-0.23310],[1482383105497,-0.22833],[1482385121497,-0.22298],[1482387137497,-0.23596],[1482389153497,-0.23093],[1482391169497,-0.23323],[1482393185497,-0.24149],[1482395201497,-0.22619],[1482397217497,-0.22559],[1482399233497,-0.22782],[1482401249497,-0.22171],[1482403265497,-0.22596],[1482405281497,-0.22796],[1482407297497,-0.22731],[1482409313497,-0.23037],[1482411329497,-0.23890],[1482413345497,-0.22149],[1482415361497,-0.22646],[1482417377497,-0.23351],[1482419393497,-0.22742],[1482421409497,-0.22296],[1482423425497,-0.22955],[1482425441497,-0.23063],[1482427457497,-0.23154],[1482429473497,-0.23196],[1482431489497,-0.23798],[1482433505497,-0.23818],[1482435521497,-0.23454],[1482437537497,-0.23025],[1482439553497,-0.22725],[1482441569497,-0.22831],[1482443585497,-0.22562],[1482445601497,-0.24032],[1482447617497,-0.23004],[1482449633497,-0.22990],[1482451649497,-0.23410],[1482453665497,-0.24393],[1482455681497,-0.24358]],\n  "last":[[1482340769497,-0.24605],[1482342785497,-0.24772],[1482344801497,-0.24700],[1482346817497,-0.24757],[1482348833497,-0.24862],[1482350849497,-0.24704],[1482352865497,-0.24784],[1482354881497,-0.24629],[1482356897497,-0.24862],[1482358913497,-0.24847],[1482360929497,-0.24782],[1482362945497,-0.24720],[1482364961497,-0.24809],[1482366977497,-0.24764],[1482368993497,-0.24788],[1482371009497,-0.24603],[1482373025497,-0.24656],[1482375041497,-0.24733],[1482377057497,-0.24717],[1482379073497,-0.24972],[1482381089497,-0.24673],[1482383105497,-0.24638],[1482385121497,-0.24752],[1482387137497,-0.24584],[1482389153497,-0.24825],[1482391169497,-0.24867],[1482393185497,-0.24786],[1482395201497,-0.24775],[1482397217497,-0.24594],[1482399233497,-0.24706],[1482401249497,-0.24867],[1482403265497,-0.24711],[1482405281497,-0.24645],[1482407297497,-0.24728],[1482409313497,-0.24663],[1482411329497,-0.24737],[1482413345497,-0.24690],[1482415361497,-0.24656],[1482417377497,-0.24745],[1482419393497,-0.24723],[1482421409497,-0.24540],[1482423425497,-0.24683],[1482425441497,-0.24677],[1482427457497,-0.24844],[1482429473497,-0.24774],[1482431489497,-0.24607],[1482433505497,-0.24703],[1482435521497,-0.24738],[1482437537497,-0.24734],[1482439553497,-0.24740],[1482441569497,-0.24049],[1482443585497,-0.24981],[1482445601497,-0.24587],[1482447617497,-0.24553],[1482449633497,-0.24553],[1482451649497,-0.24844],[1482453665497,-0.24737],[1482455681497,-0.24796]],\n  "min":[[1482340769497,-0.26834],[1482342785497,-0.26274],[1482344801497,-0.26109],[1482346817497,-0.26775],[1482348833497,-0.27119],[1482350849497,-0.26345],[1482352865497,-0.27430],[1482354881497,-0.26907],[1482356897497,-0.26791],[1482358913497,-0.26229],[1482360929497,-0.26500],[1482362945497,-0.26970],[1482364961497,-0.26509],[1482366977497,-0.26341],[1482368993497,-0.26223],[1482371009497,-0.26743],[1482373025497,-0.25885],[1482375041497,-0.25691],[1482377057497,-0.25252],[1482379073497,-0.26423],[1482381089497,-0.26787],[1482383105497,-0.26614],[1482385121497,-0.26906],[1482387137497,-0.26257],[1482389153497,-0.26688],[1482391169497,-0.26805],[1482393185497,-0.25574],[1482395201497,-0.26351],[1482397217497,-0.26982],[1482399233497,-0.27468],[1482401249497,-0.27602],[1482403265497,-0.26787],[1482405281497,-0.26672],[1482407297497,-0.26450],[1482409313497,-0.26821],[1482411329497,-0.25592],[1482413345497,-0.26276],[1482415361497,-0.26487],[1482417377497,-0.26202],[1482419393497,-0.26741],[1482421409497,-0.26331],[1482423425497,-0.27434],[1482425441497,-0.27155],[1482427457497,-0.26893],[1482429473497,-0.26362],[1482431489497,-0.26430],[1482433505497,-0.25432],[1482435521497,-0.25752],[1482437537497,-0.26444],[1482439553497,-0.26128],[1482441569497,-0.26056],[1482443585497,-0.26678],[1482445601497,-0.26305],[1482447617497,-0.26542],[1482449633497,-0.26422],[1482451649497,-0.26529],[1482453665497,-0.25072],[1482455681497,-0.25044]]\n}'
_trenddata_7day_ph_json = '{\n  "max":[[1482340769799,7.2833],[1482342785799,7.2803],[1482344801799,7.2793],[1482346817799,7.2788],[1482348833799,7.2789],[1482350849799,7.2827],[1482352865799,7.2735],[1482354881799,7.2729],[1482356897799,7.2748],[1482358913799,7.2771],[1482360929799,7.2838],[1482362945799,7.2734],[1482364961799,7.2722],[1482366977799,7.2761],[1482368993799,7.2749],[1482371009799,7.2737],[1482373025799,7.2743],[1482375041799,7.2775],[1482377057799,7.2774],[1482379073799,7.2779],[1482381089799,7.2778],[1482383105799,7.2788],[1482385121799,7.2807],[1482387137799,7.2814],[1482389153799,7.2817],[1482391169799,7.2840],[1482393185799,7.2845],[1482395201799,7.2854],[1482397217799,7.2841],[1482399233799,7.2847],[1482401249799,7.2857],[1482403265799,7.2865],[1482405281799,7.2851],[1482407297799,7.2865],[1482409313799,7.2851],[1482411329799,7.2844],[1482413345799,7.2847],[1482415361799,7.2836],[1482417377799,7.2815],[1482419393799,7.2811],[1482421409799,7.2795],[1482423425799,7.2794],[1482425441799,7.2732],[1482427457799,7.2721],[1482429473799,7.2797],[1482431489799,7.2714],[1482433505799,7.2761],[1482435521799,7.2768],[1482437537799,0.019060],[1482439553799,0.021007],[1482441569799,0.023213],[1482443585799,0.027309],[1482445601799,0.032843],[1482447617799,0.031850],[1482449633799,0.034278],[1482451649799,0.035156],[1482453665799,0.037912],[1482455681799,0.038118]],\n  "last":[[1482340769799,7.2697],[1482342785799,7.2687],[1482344801799,7.2763],[1482346817799,7.2712],[1482348833799,7.2736],[1482350849799,7.2734],[1482352865799,7.2708],[1482354881799,7.2708],[1482356897799,7.2683],[1482358913799,7.2709],[1482360929799,7.2710],[1482362945799,7.2684],[1482364961799,7.2696],[1482366977799,7.2709],[1482368993799,7.2725],[1482371009799,7.2703],[1482373025799,7.2695],[1482375041799,7.2712],[1482377057799,7.2737],[1482379073799,7.2746],[1482381089799,7.2736],[1482383105799,7.2759],[1482385121799,7.2761],[1482387137799,7.2774],[1482389153799,7.2787],[1482391169799,7.2801],[1482393185799,7.2788],[1482395201799,7.2820],[1482397217799,7.2814],[1482399233799,7.2816],[1482401249799,7.2834],[1482403265799,7.2839],[1482405281799,7.2841],[1482407297799,7.2819],[1482409313799,7.2811],[1482411329799,7.2803],[1482413345799,7.2818],[1482415361799,7.2820],[1482417377799,7.2784],[1482419393799,7.2777],[1482421409799,7.2782],[1482423425799,7.2768],[1482425441799,7.2705],[1482427457799,7.2704],[1482429473799,7.2687],[1482431489799,7.2692],[1482433505799,7.2671],[1482435521799,7.2726],[1482437537799,0.017312],[1482439553799,0.018939],[1482441569799,0.020288],[1482443585799,0.021807],[1482445601799,0.026845],[1482447617799,0.030066],[1482449633799,0.031388],[1482451649799,0.033502],[1482453665799,0.034736],[1482455681799,0.036627]],\n  "min":[[1482340769799,7.2666],[1482342785799,7.2672],[1482344801799,7.2709],[1482346817799,7.2708],[1482348833799,7.2658],[1482350849799,7.2678],[1482352865799,7.2671],[1482354881799,7.2658],[1482356897799,7.2679],[1482358913799,7.2684],[1482360929799,7.2663],[1482362945799,7.2658],[1482364961799,7.2620],[1482366977799,7.2680],[1482368993799,7.2694],[1482371009799,7.2670],[1482373025799,7.2667],[1482375041799,7.2683],[1482377057799,7.2710],[1482379073799,7.2714],[1482381089799,7.2721],[1482383105799,7.2735],[1482385121799,7.2746],[1482387137799,7.2761],[1482389153799,7.2770],[1482391169799,7.2786],[1482393185799,7.2785],[1482395201799,7.2787],[1482397217799,7.2769],[1482399233799,7.2778],[1482401249799,7.2795],[1482403265799,7.2798],[1482405281799,7.2801],[1482407297799,7.2802],[1482409313799,7.2791],[1482411329799,7.2781],[1482413345799,7.2777],[1482415361799,7.2768],[1482417377799,7.2756],[1482419393799,7.2751],[1482421409799,7.2742],[1482423425799,7.2678],[1482425441799,7.2681],[1482427457799,7.2641],[1482429473799,7.2658],[1482431489799,7.2653],[1482433505799,7.2639],[1482435521799,0.016045],[1482437537799,0.016360],[1482439553799,0.017464],[1482441569799,0.019686],[1482443585799,0.021178],[1482445601799,0.024311],[1482447617799,0.024054],[1482449633799,0.031107],[1482451649799,0.032784],[1482453665799,0.034519],[1482455681799,0.035953]]\n}'
_trenddata_7day_temperature_json = '{\n  "max":[[1482340770000,20.016],[1482342786000,20.019],[1482344802000,20.043],[1482346818000,20.122],[1482348834000,20.155],[1482350850000,20.189],[1482352866000,20.199],[1482354882000,20.199],[1482356898000,20.181],[1482358914000,20.179],[1482360930000,20.161],[1482362946000,20.151],[1482364962000,20.155],[1482366978000,20.162],[1482368994000,20.182],[1482371010000,20.189],[1482373026000,20.201],[1482375042000,20.204],[1482377058000,20.200],[1482379074000,20.184],[1482381090000,20.177],[1482383106000,20.170],[1482385122000,20.156],[1482387138000,20.140],[1482389154000,20.101],[1482391170000,20.039],[1482393186000,20.009],[1482395202000,19.974],[1482397218000,19.920],[1482399234000,19.874],[1482401250000,19.824],[1482403266000,19.766],[1482405282000,19.717],[1482407298000,19.663],[1482409314000,19.609],[1482411330000,19.550],[1482413346000,19.497],[1482415362000,19.445],[1482417378000,19.400],[1482419394000,19.358],[1482421410000,19.326],[1482423426000,19.306],[1482425442000,19.298],[1482427458000,19.274],[1482429474000,19.264],[1482431490000,19.292],[1482433506000,19.352],[1482435522000,19.421],[1482437538000,19.491],[1482439554000,19.575],[1482441570000,19.662],[1482443586000,19.871],[1482445602000,20.001],[1482447618000,20.057],[1482449634000,20.147],[1482451650000,20.204],[1482453666000,20.282],[1482455682000,20.311]],\n  "last":[[1482340770000,20.012],[1482342786000,20.004],[1482344802000,20.015],[1482346818000,20.040],[1482348834000,20.120],[1482350850000,20.152],[1482352866000,20.186],[1482354882000,20.195],[1482356898000,20.177],[1482358914000,20.177],[1482360930000,20.157],[1482362946000,20.141],[1482364962000,20.142],[1482366978000,20.151],[1482368994000,20.160],[1482371010000,20.178],[1482373026000,20.189],[1482375042000,20.195],[1482377058000,20.193],[1482379074000,20.182],[1482381090000,20.172],[1482383106000,20.165],[1482385122000,20.152],[1482387138000,20.136],[1482389154000,20.100],[1482391170000,20.034],[1482393186000,20.006],[1482395202000,19.971],[1482397218000,19.920],[1482399234000,19.869],[1482401250000,19.821],[1482403266000,19.765],[1482405282000,19.716],[1482407298000,19.658],[1482409314000,19.609],[1482411330000,19.550],[1482413346000,19.495],[1482415362000,19.444],[1482417378000,19.396],[1482419394000,19.358],[1482421410000,19.324],[1482423426000,19.305],[1482425442000,19.292],[1482427458000,19.272],[1482429474000,19.258],[1482431490000,19.263],[1482433506000,19.292],[1482435522000,19.350],[1482437538000,19.420],[1482439554000,19.488],[1482441570000,19.574],[1482443586000,19.660],[1482445602000,19.871],[1482447618000,20.000],[1482449634000,20.058],[1482451650000,20.146],[1482453666000,20.203],[1482455682000,20.280]],\n  "min":[[1482340770000,20.000],[1482342786000,20.000],[1482344802000,20.014],[1482346818000,20.036],[1482348834000,20.115],[1482350850000,20.149],[1482352866000,20.185],[1482354882000,20.175],[1482356898000,20.168],[1482358914000,20.155],[1482360930000,20.138],[1482362946000,20.135],[1482364962000,20.142],[1482366978000,20.144],[1482368994000,20.158],[1482371010000,20.177],[1482373026000,20.185],[1482375042000,20.193],[1482377058000,20.178],[1482379074000,20.170],[1482381090000,20.162],[1482383106000,20.151],[1482385122000,20.129],[1482387138000,20.097],[1482389154000,20.035],[1482391170000,20.003],[1482393186000,19.967],[1482395202000,19.916],[1482397218000,19.869],[1482399234000,19.820],[1482401250000,19.762],[1482403266000,19.715],[1482405282000,19.659],[1482407298000,19.606],[1482409314000,19.548],[1482411330000,19.493],[1482413346000,19.442],[1482415362000,19.396],[1482417378000,19.356],[1482419394000,19.320],[1482421410000,19.301],[1482423426000,19.287],[1482425442000,19.270],[1482427458000,19.253],[1482429474000,19.254],[1482431490000,19.258],[1482433506000,19.291],[1482435522000,19.345],[1482437538000,19.418],[1482439554000,19.487],[1482441570000,19.573],[1482443586000,19.659],[1482445602000,19.870],[1482447618000,19.993],[1482449634000,20.055],[1482451650000,20.146],[1482453666000,20.201],[1482455682000,20.279]]\n}'
_trends = [
    ('15min', 'agitation', _trenddata_15min_agitation_json),
    ('2hr', 'agitation', _trenddata_2hr_agitation_json),
    ('12hr', 'agitation', _trenddata_12hr_agitation_json),
    ('24hr', 'agitation', _trenddata_24hr_agitation_json),
    ('72hr', 'agitation', _trenddata_72hr_agitation_json),
    ('7day', 'agitation', _trenddata_7day_agitation_json),
    ('15min', 'ph', _trenddata_15min_ph_json),
    ('2hr', 'ph', _trenddata_2hr_ph_json),
    ('12hr', 'ph', _trenddata_12hr_ph_json),
    ('24hr', 'ph', _trenddata_24hr_ph_json),
    ('72hr', 'ph', _trenddata_72hr_ph_json),
    ('7day', 'ph', _trenddata_7day_ph_json),
    ('15min', 'do', _trenddata_15min_do_json),
    ('2hr', 'do', _trenddata_2hr_do_json),
    ('12hr', 'do', _trenddata_12hr_do_json),
    ('24hr', 'do', _trenddata_24hr_do_json),
    ('72hr', 'do', _trenddata_72hr_do_json),
    ('7day', 'do', _trenddata_7day_do_json),
    ('15min', 'temperature', _trenddata_15min_temperature_json),
    ('2hr', 'temperature', _trenddata_2hr_temperature_json),
    ('12hr', 'temperature', _trenddata_12hr_temperature_json),
    ('24hr', 'temperature', _trenddata_24hr_temperature_json),
    ('72hr', 'temperature', _trenddata_72hr_temperature_json),
    ('7day', 'temperature', _trenddata_7day_temperature_json),
]
_trenddata = OrderedDict()
for _s, _g, _d in _trends:
    _trenddata[_g] = _trenddata.get(_g) or OrderedDict()
    _jd = json.loads(_d)
    _trenddata[_g][_s] = _trenddata[_g].get(_s) or {}
    for _type in _jd:
        _trenddata[_g][_s][_type] = _jd[_type]
del _s, _g, _d, _jd, _type

In [8]:
from collections import OrderedDict
groups = "agitation ph do temperature".split()
spans = "15min 2hr 12hr 24hr 72hr 7day".split()
strings = []
jdata = OrderedDict()
for g in groups:
    for s in spans:
        key = '_trenddata_%s_%s_json' % (s, g)
        #data = globals()[key]
        s = "    (%r, %r, %s)," % (s, g, key)
        strings.append(s)
        #jdata[g] = jdata.get(g) or OrderedDict()
        #jdata[g][s] = jdata[g].get(s) or OrderedDict()
        #jdata[g][s] = data
s2 = "\n".join(strings)
print(s2)
import clipboard
clipboard.copy(s2)

    ('15min', 'agitation', _trenddata_15min_agitation_json),
    ('2hr', 'agitation', _trenddata_2hr_agitation_json),
    ('12hr', 'agitation', _trenddata_12hr_agitation_json),
    ('24hr', 'agitation', _trenddata_24hr_agitation_json),
    ('72hr', 'agitation', _trenddata_72hr_agitation_json),
    ('7day', 'agitation', _trenddata_7day_agitation_json),
    ('15min', 'ph', _trenddata_15min_ph_json),
    ('2hr', 'ph', _trenddata_2hr_ph_json),
    ('12hr', 'ph', _trenddata_12hr_ph_json),
    ('24hr', 'ph', _trenddata_24hr_ph_json),
    ('72hr', 'ph', _trenddata_72hr_ph_json),
    ('7day', 'ph', _trenddata_7day_ph_json),
    ('15min', 'do', _trenddata_15min_do_json),
    ('2hr', 'do', _trenddata_2hr_do_json),
    ('12hr', 'do', _trenddata_12hr_do_json),
    ('24hr', 'do', _trenddata_24hr_do_json),
    ('72hr', 'do', _trenddata_72hr_do_json),
    ('7day', 'do', _trenddata_7day_do_json),
    ('15min', 'temperature', _trenddata_15min_temperature_json),
    ('2hr', 'temperature', _trenddata_2hr

In [102]:
_unregistered = []
def _dbg_events_(events, ev):
    if ev not in events.handlers:
        _unregistered.append(ev)
        print("Unregistered event:", ev)

class EventDispatcher():
    def __init__(self):
        self.handlers = {}
        
    def create_event(self, ev):
        self.handlers[ev] = self.handlers.get(ev, [])
        
    def register(self, ev, handler):
        if ev not in self.handlers:
            self.create_event(ev)
        self.handlers[ev].append(handler)
        
    def unregister(self, ev, handler):
        self.handlers[ev].remove(handler)
        
    def fire(self, ev, ob, e, *args):
        _dbg_events_(self, ev)
        for h in self.handlers.get(ev, ()):
            h(ev, ob, e, *args)
            

In [131]:
STICKY_ALL = (tk.N, tk.S, tk.E, tk.W)

def set_entry_widths(self, w):
    for it in self.items():
        it.entry.config(width=w)

class FrameWidget1(tk.Frame):
    ITEM_EVENT = "ITEM_VALUE_MODIFIED"
    CALL_NAME = "DEFAULT"
    COLUMNS = 3
    def __init__(self, master, events, **kw):
        super().__init__(master, **kw)
        self.events = events
        self.events.create_event(self.ITEM_EVENT)
        self.events.register(self.ITEM_EVENT, self.on_item_value_modified)
        self.init_data()
        self.frames = OrderedDict()
        self.setup_frames()
        self.configure_items()
        self.configure_frames()
        
    def configure_items(self):
        for it in self.items():
            it.set_fmt(str, "%s")
        
    def configure_frames(self):
        lengths = [0,0,0]
        lengths2 = [0,0,0]

        # set the column width to the widest
        # of individual widths determined by
        # len(text)
        for f in self.frames.values():
            col = f.frame.grid_info()['column']
            for it in f.items.values():
                w = len(it.label['text'])
                if w > lengths[col]:
                    lengths[col] = w
                w2 = len(it.indicator['text'])
                w2 += 2  # just in case
                if w2 > lengths2[col]:
                    lengths2[col] = w2
        for f in self.frames.values():
            col = f.frame.grid_info()['column']
            for it in f.items.values():
                it.label.config(width=lengths[col])   
                it.indicator.config(width=lengths2[col])
    
    def init_data(self):
        self.data = {}
        
    def sync(self):
        for k in self.data:
            f = self.frames[k]
            for k2 in self.data[k2]:
                it = f.items[k2]
                self.data[k][k2] = it.get_value()
                
    def setup_frames(self):
        max_rows = max(len(c) for c in self.data)
        for i, k1 in enumerate(sorted(self.data)):
            r = i // self.COLUMNS
            c = i % self.COLUMNS
            f = ItemFrame(self, self.events, k1)
            for i2, k2 in enumerate(self.data[k1]):
                it = f.add_item(k2, self.data[k1][k2])
                item_key = (k1, k2)
                it.set_event(self.ITEM_EVENT, item_key)
            self.frames[k1] = f
            f.grid(rowspan=max_rows, column=c, row=r*max_rows, sticky=STICKY_ALL)
            
    def jsonify(self):
        return json.dumps(self.data, indent=4)
    
    def xmlify(self):
        return hmu.create_hello_xml(self.data, "Message")
    
    def on_item_value_modified(self, event, item, e, key, value):
        k1, k2 = key
        self.data[k1][k2] = value
    
    # Utility functions 
    
    def on_err(self, v):
        print("Invalid value for '%s %s':" % (self.text, name), repr(v))
        
    def items(self):
        for frame in self.frames.values():
            yield from frame.items.values()
            
    def set_entry_widths(self, w):
        for it in self.items():
            it.entry.config(width=w)
     
class _TrendDisplayFrame(tk.Frame):
    def __init__(self, master, events, group, span, typ, dwidth=30, **kw):
        super().__init__(master, **kw)
        self.group = group
        self.span = span
        self.type = typ
        self.events = events
        self.main_label = tk.Label(self, text="%s %s %s" % (group, span, typ))
        self.display = tk.Text(self, wrap=tk.CHAR, width=dwidth)
        self.scrollbar = tk.Scrollbar(self)
        self.scrollbar.config(command=self.display.yview)
        self.display.config(yscrollcommand=self.scrollbar.set)
        self.time_button = ttk.Button(self, text="Set Time to Now", command=self.time_btn_click)
        self.fx_label = tk.Label(self, text="f(x):")
        self.fx_entry = tk.Entry(self)
        self.fx_apply = tk.Button(self, text="Apply", command=self.apply_btn_click)
        self.fx_entry.bind("<Return>", lambda *_: self.fx_apply.invoke())

    def show(self):
        self.pack()
        self.main_label.config(width=20)
        self.main_label.grid(row=0, column=0)
        self.display.grid(row=0, column=1, rowspan=20, columnspan=3)
        self.scrollbar.grid(row=0, column=4, rowspan=20, sticky="ns")
        self.time_button.grid(row=0, column=5)
        self.fx_label.grid(row=1, column=5)
        self.fx_entry.grid(row=2, column=5, columnspan=2, sticky="ew")
        self.fx_apply.grid(row=3, column=5)
        
    def get_data(self):
        return self.display.get(1.0, tk.END)

    def time_btn_click(self):
        self.events.fire("TREND_DATA_TIME_RESET", self, None, self.group, self.span, self.type)

    def apply_btn_click(self):
        self.events.fire("TREND_DATA_APPLY_FUNCTION", self, None, self.group, self.span, self.type)

    def set_text(self, text):
        self.display.delete(1.0, tk.END)
        self.display.insert(tk.INSERT, text)

    def set_data(self, data):
        self.set_text(self.data_to_text(data))

    def data_to_text(self, data):
        return "".join("%s,%.5f\n" % (int(a),float(b)) for a, b in data)

    def bindall_subwidget(self, ev, func):
        widgets = (
            self,
            self.main_label,
            self.time_button,
            self.fx_label,
            self.fx_entry,
            self.fx_apply,
        )
        for w in widgets:
            w.bind(ev, func)
    
_fn_src = \
"""
from math import *
def func(x):
    return %s
"""

def _build_function(contents):
    fn = _fn_src % contents
    ns = {}
    exec(fn, ns, ns)
    return ns["func"]
            
class GTDWidget(tk.Frame):
    _groups_ = "agitation ph do temperature".split()
    _spans_ = "15min 2hr 12hr 24hr 72hr 7day".split()
    _types_ = "max min last".split()
    
    def __init__(self, master, events, **kw):
        super().__init__(master, **kw)
        self.events = events
        self.events.register("TREND_DATA_TIME_RESET", self._reset_time)
        self.events.register("TREND_DATA_APPLY_FUNCTION", self._apply_function)
        self.init_data()
        self.frames = OrderedDict()
        self.setup_frames()

    def setup_frames(self):
        for g in self._groups_:
            for s in self._spans_:
                for t in self._types_:
                    d = self.data[g][s][t]
                    frame = _TrendDisplayFrame(self, self.events, g, s, t)
                    frame.set_data(d)
                    frame.show()
                    self.frames[(g, s, t)] = frame
                    
    def sync_data(self, g, s, t):
        f = self.frames[(g, s, t)]
        t = f.get_data()
        data = [[float(a), float(b)] for a, b in s.split(",") for s in t.splitlines()]
        self.data[g][s][t] = data
        
    def sync_all_data(self):
        for g, s, t in self.frames:  # iterate over keys
            self.sync_data(g, s, t)
    
    def _reset_time(self, ev, frame, e, g, s, t):
        # Each array is a 300 element array of evenly
        # spaced data points. This maps the "span"
        # to the spacing interval. 
        tts = {
            "15min": 15*60 / 300,
            "2hr": 2*60*60 / 300,
            "12hr": 12*60*60 / 300,
            "24hr": 24*60*60 / 300,
            "72hr": 72*60*60 / 300,
            "7day": 7*24*60*60 / 300
        } 
        now = time.time()
        interval = tts[s]
        d = self.data[g][s][t]
        for i in range(len(d)):
            d[i][0] = (now + interval * i) * 1000
        frame.set_data(d)

    def _apply_function(self, ev, frame, e, g, s, t):
        d = self.data[g][s][t]
        contents = frame.fx_entry.get()
        fn = self._build_function(contents)
        t1 = d[0][0]
        for i in range(len(d)):
            x = d[i][0] - t1
            try:
                y = f(x/1000)  # ms -> s
            except Exception as e:
                print(e)
                break
            else:
                if isinstance(y, (int, float)):
                    d[i][1] = y
                else:
                    print("Error: f(x) function returned non-numeric value %r" % type(y))
                    break
        frame.set_data(d)
        
    def bindall_subwidget(self, ev, func):
        for w in self.frames.values():
            w.bindall_subwidget(ev, func)
        
    def init_data(self):
        self.data = copy.deepcopy(_trenddata)

In [130]:
import json
root = tk.Tk()
sf = ScrollFrame(root, width=550, height=600, xscroll=True)

w = GTDWidget(sf.inner_frame, EventDispatcher())
w.pack()
w.bindall_subwidget("<MouseWheel>", sf.on_mouse_scroll)
sf.grid()
root.mainloop()

WARNING, NOT 300 VALUES: agitation 72hr last
WARNING, NOT 300 VALUES: agitation 72hr max
WARNING, NOT 300 VALUES: agitation 72hr min
WARNING, NOT 300 VALUES: agitation 7day last
WARNING, NOT 300 VALUES: agitation 7day max
WARNING, NOT 300 VALUES: agitation 7day min
WARNING, NOT 300 VALUES: ph 72hr last
WARNING, NOT 300 VALUES: ph 72hr max
WARNING, NOT 300 VALUES: ph 72hr min
WARNING, NOT 300 VALUES: ph 7day last
WARNING, NOT 300 VALUES: ph 7day max
WARNING, NOT 300 VALUES: ph 7day min
WARNING, NOT 300 VALUES: do 72hr last
WARNING, NOT 300 VALUES: do 72hr max
WARNING, NOT 300 VALUES: do 72hr min
WARNING, NOT 300 VALUES: do 7day last
WARNING, NOT 300 VALUES: do 7day max
WARNING, NOT 300 VALUES: do 7day min
WARNING, NOT 300 VALUES: temperature 72hr last
WARNING, NOT 300 VALUES: temperature 72hr max
WARNING, NOT 300 VALUES: temperature 72hr min
WARNING, NOT 300 VALUES: temperature 7day last
WARNING, NOT 300 VALUES: temperature 7day max
WARNING, NOT 300 VALUES: temperature 7day min
